In [1]:
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException

# ====== 함수 정의 ======
# 화면 스크롤 하는 함수
def scroll_to(height):
    current_scroll_position = driver.execute_script("return window.pageYOffset;")
    new_scroll_position = current_scroll_position + height
    driver.execute_script(f'window.scrollTo(0,{new_scroll_position});')
    time.sleep(1)

In [2]:
path = 'https://korean.visitkorea.or.kr/list/travelinfo.do?service=ms#ms^0^All^All^e6875575-2cc2-43ba-9651-28d31a7b3e23,651c5b95-a5b3-11e8-8165-020027310001,c24d515f-3202-45e5-834e-1a091901aeff,d3fd4d9f-fbd4-430f-b5d5-291b4d9920be,3f36ca4b-6f45-45cb-9042-265c96a4868c,23bc02b8-da01-41bf-8118-af882436cd3c,2d4f4e06-2d37-4e54-ad5c-172add6e6680,9668f0f1-8afe-4526-8007-503bd02fd6d8,0f29b431-75ac-4ab4-a892-b247d516b31d,640d3489-8fc3-11e8-8165-020027310001,1601b0a3-144e-40b7-95b4-b946e537a25b,1c981ad4-7834-11e8-82c8-020027310001,266bf7a0-cbab-4bb4-b800-d7edd5642180,cdd12d65-1f38-4829-a1be-bf235a0fb3f2^39^^1^#%EC%A0%84%EC%B2%B4'
path = 'https://korean.visitkorea.or.kr/list/travelinfo.do?service=ms#ms^0^All^All^e6875575-2cc2-43ba-9651-28d31a7b3e23,651c5b95-a5b3-11e8-8165-020027310001,c24d515f-3202-45e5-834e-1a091901aeff,d3fd4d9f-fbd4-430f-b5d5-291b4d9920be,3f36ca4b-6f45-45cb-9042-265c96a4868c,23bc02b8-da01-41bf-8118-af882436cd3c,2d4f4e06-2d37-4e54-ad5c-172add6e6680,9668f0f1-8afe-4526-8007-503bd02fd6d8,0f29b431-75ac-4ab4-a892-b247d516b31d,640d3489-8fc3-11e8-8165-020027310001,1601b0a3-144e-40b7-95b4-b946e537a25b,1c981ad4-7834-11e8-82c8-020027310001,266bf7a0-cbab-4bb4-b800-d7edd5642180,cdd12d65-1f38-4829-a1be-bf235a0fb3f2^590^^1^#%EC%A0%84%EC%B2%B4'

In [3]:
tour = []

service = webdriver.chrome.service.Service('chromedriver.exe')
driver = webdriver.Chrome(service=service)
wait = WebDriverWait(driver, 10)

driver.get(path)
time.sleep(5)

wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="totalCnt"]')))
total_results = int(driver.find_element(By.XPATH,'//*[@id="totalCnt"]').text.replace(",",""))
total_pages = (total_results // 10) + (1 if total_results % 10!=0 else 0)


for _ in range(total_pages-51):
    # 현재 페이지의 카드 수(동적으로 매번 계산)
    time.sleep(3)
    locations = driver.find_elements(By.CSS_SELECTOR, 'ul.list_thumType.place > li' )

    # 결과물 1개씩 클릭해서 정보 크롤링
    for idx, location in enumerate(locations):
        # 매번 재조회(리스트 DOM 갱신 대비)
        location_data = {}
        # 장소이름
        time.sleep(2)
        try:
            location_data['place']=location.find_element(By.CSS_SELECTOR, "div.area_txt.catchphrase > div > a").text
        except:
            location_data['place']='오류'

        # 장소 짤막한 설명
        try:
            info=location.find_element(By.CSS_SELECTOR, "p.phrase").text
            location_data['info'] = info
        except:
            location_data['info']=''

        try:
            tags = location.find_elements(By.CSS_SELECTOR, 'div.area_txt.catchphrase > p.tag > span')
            tags_list = []
            
            for tag in tags:
                tags_list.append(tag.text.replace("#",""))
            location_data['태그'] = " ".join(tags_list)
        except (StaleElementReferenceException, TimeoutException):
            location_data['태그'] = " ".join(tags_list)
        
        tour.append(location_data)
        print(location_data)
        time.sleep(1)
        scroll_to(110)


        
    time.sleep(1)
    page_button_path = driver.find_elements(By.CSS_SELECTOR, '.page_box > a')
    if len(page_button_path) >= 2:
        try:
            # '다음' 또는 '>'가 있으면 우선 클릭, 없으면 기존 방식(-2)로
            next_btn = None
            for a in reversed(page_button_path):
                t = a.text.strip()
                if t in ('다음', '>', '›', 'Next'):
                    next_btn = a
                    break
            if next_btn is None:
                next_btn = page_button_path[-2]
            next_btn.click()
            time.sleep(2)
            # wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, contents_path)))
        except Exception:
            break
    else:
        break

# ====== 저장 & 종료 ======
pd.DataFrame(tour).to_csv(f'추가 데이터.csv', index=False, encoding='utf-8')
time.sleep(1)
driver.close()


{'place': '보광사(속초)', 'info': '영랑호의 뛰어난 경관을 품고 있는 사찰', '태그': '관광지 불교 불교문화 불교사찰 사찰 사찰여행 역사 역사관광지         '}
{'place': '낙산묘각사', 'info': '서울 중심인 낙산 동쪽에 자리 잡은 사찰', '태그': '관광지 대한불교관음종 불교 불교문화 사찰 사찰여행 산사체험프로그램             '}
{'place': '파로호(화천)', 'info': '대한민국 최북단에 위치하는 인공호수', '태그': '가족여행 강원권 강원도_여행지_추천 경치여행 경치좋은곳 관광지      '}
{'place': '김희수아트센터', 'info': '수림문화재단 설립자의 교육, 문화 후원 정신을 기리는 공간', '태그': '김희수기념관 김희수아트센터 문화시설 서울미술관 서울아트갤러리 서울아트센터 '}
{'place': '창렬서원', 'info': '사육신 중 한 사람인 하위지를 제향하기 위해 건립된 곳', '태그': '관광지 사육신 안동관광 안동여행 안동유적지 안동창렬서원 역사여행'}
{'place': '성북동고택북촌산책길', 'info': '옛 정취를 느낄 수 있는 역사·문화의 길', '태그': '걷기좋은길 길상사 레포츠 북촌 서울산책길 서울여행 성북동고택북촌산책길 '}
{'place': '평창힐링스테이', 'info': '깊은 산 속에서 즐기는 꿈 같은 하루', '태그': '숙박 자연 자연환경 펜션 평창 평창가볼만한곳 평창펜션'}
{'place': '대각사(서울)', 'info': '나라의 독립을 소망했던 자취가 남아있는 사찰', '태그': '3.1운동독립만세 관광지 귀국봉영회 대각사상 대각회 백용성스님 불교              '}
{'place': '둔덕시골농촌체험센터', 'info': '자연속에서 다양하게 시골농촌을 체험할 수 있는 곳', '태그': '거제둔덕농촌체험 관광지 농촌마을 농촌체험마을 단체MT 단체농산물체험    '}
{'place': '해마루오토캠핑장', 'info'

In [ ]:
pd.DataFrame(tour).to_csv('추가데이터5.csv',index=False)

In [12]:
tour = [{'place': '또봇정크아트뮤지엄', 'info': '거대한 또봇을 눈앞에서 만날 수 있는 곳', '태그': '가족여행 경주가볼만한곳 경주엑스포대공원 경주여행 또봇 또봇정크아트뮤지엄   '}
,{'place': '경주 남산 불곡 마애여래좌상', 'info': '보물, 경상북도 경주시 인왕동에 있는 삼국시대의 화강석제 불상', '태그': '관광지 역사 역사관광지 역사를품은곳 역사문화재 역사속 역사유적지'}
,{'place': '보은 서원리 소나무', 'info': '우아한 모습의 정이품송 부인 소나무', '태그': '관광지 보은서원리소나무 서원리소나무 소나무 역사공부 역사관광지         '}
,{'place': '허균·허난설헌 기념공원', 'info': '허균과 허난설헌을 만나다', '태그': '강릉 강릉여행 경포해변 관광명소 관광지 허균허난설헌기념관'}
,{'place': '하동 코리아 짚와이어', 'info': '다도해의 아름다운 경치를 하늘에서 짜릿하게!', '태그': '2024_여행가는달과함께하는_발도장여행지 꿀잼_액티비티 레포츠 레포츠여행        '}
,{'place': '한국대중음악박물관', 'info': '한국대중음악의 찬란한 역사를 기록한 공간', '태그': '가족과함께 경주가볼만한곳 대중음악박물관 문화시설 문화행사 박물관    '}
,{'place': '다산초당', 'info': '실학을 집대성한 대학자 정약용 선생이 유배생활을 했던 다산초당', '태그': '2024_여행가는달과함께하는_발도장여행지 가족여행 관광지 교과서속여행                '}
,{'place': '춘천김정은가옥', 'info': '춘천의 옛이야기와 정갈한 전통미가 살아 숨 쉬는 20세기 초 한옥문화공간', '태그': '강원도문화재자료 관광지 역사 역사공부 역사관광지 역사를품은곳 역사문화재        '}
,{'place': '환희컵박물관', 'info': '세계의 다양한 컵들을 만날 수 있는 곳', '태그': '강릉가볼만한곳 강릉실내관광지 강릉여행 문화시설 미세먼지_피하는_실내여행    '}
,{'place': '등명낙가사(강릉)', 'info': '동해 일출을 감상할 수 있는 사찰', '태그': '관광지 불교 불교사찰 사찰 사찰여행 역사 역사공부 역사관광지       '}
,{'place': '강진 병영마을 옛 담장', 'info': '돌과 흙을 번갈아 쌓은 토석담장', '태그': '관광지 산책길 산책하기 산책하기좋은곳 역사 역사공부 역사관광지            '}
,{'place': '솔거미술관', 'info': '사람과 공간 자연이 만나는 미술관', '태그': '가족과함께 경주가볼만한곳 경주미술관 경주엑스포대공원 나들이 문화시설  '}
,{'place': '강문해변', 'info': '반려동물과 백사장을 달려볼 수 있는 해변', '태그': '가족여행 강문해변 강원_가볼만한곳 강원권 강원권반려동물여행지 강원도_여행지_추천                '}
,{'place': '당항포관광지', 'info': '대표적인 가족형 다목적 관광지', '태그': '23_24한국관광100선 가족과함께 경상권 관광지 남녀노소 당항포          '}
,{'place': '오류', 'info': '', '태그': ''}
,{'place': '취묵헌서예관', 'info': '세대를 넘어 예술의 가치를 나누는 열린 문화예술 공간', '태그': '문화시설 문화예술 서예관 전시관 천안가볼만한곳 천안관광 천안시 '}
,{'place': '천마의 궁전', 'info': '유네스코세계문화유산을 모티브로 한 미디어 아트', '태그': '가족과함께 경주1박2일 경주가볼만한곳 경주엑스포 경주엑스포대공원    '}
,{'place': '정동진해변', 'info': '세계에서 바다와 가장 가까운 간이역이 있는 해수욕장', '태그': '강원_가볼만한곳 강원권 강원도_여행지_추천 관광지 바다내음 바다풍경          '}
,{'place': '병영성홍교', 'info': '호남을 지켜온 군사 요충지의 관문', '태그': '관광지 역사 역사공부 역사관광지 역사를품은곳 역사문화재 역사속       '}
,{'place': '정동진시간박물관', 'info': '시간을 주제로 한 재미있고 독특한 전시공간', '태그': '강릉가볼만한곳 강릉여행 강원권 문화시설 박물관 산불피해지역 아이와함께    '}
,{'place': '정동진시간박물관', 'info': '시간을 주제로 한 재미있고 독특한 전시공간', '태그': '강릉가볼만한곳 강릉여행 강원권 문화시설 박물관 산불피해지역 아이와함께    '}
,{'place': '정동진 레일바이크', 'info': '청정 바다 동해와 함께하는 정동진 레일바이크', '태그': '강릉가볼만한곳 강원권반려동물여행지 경치좋은곳 레포츠 바다경치 바다내음          '}
,{'place': '안인해변', 'info': '바닷물이 맑고 수심이 얕아 물놀이를 하기 좋은 해수욕장', '태그': '가족여행 강릉바다 강원권 관광지 기차여행 나들이 당일치기여행              '}
,{'place': '양평 용문사 은행나무', 'info': '동양 최고의 은행나무', '태그': '관광지 양평가볼만한곳 양평관광지추천 용문사 용문사은행나무 은행나무 '}
,{'place': '살롱헤리티지', 'info': '국내 최초의 세계유산 미디어 홍보관', '태그': 'OWHC OWHC-AP 경주1박2일 경주가볼만한곳 경주나들이 경주아이랑가볼만한곳      '}
,{'place': '안인항', 'info': '푸른 강릉의 바다가 아름다운 안인항', '태그': '경치여행 경치좋은곳 관광지 바다경치 바다내음 자연속으로 자연여행   '}
,{'place': '판운캠핑장(다그린캠핑장)', 'info': '영월 주천면 판운리에 있는 캠핑장', '태그': '가족 그늘이 맑은 깨끗한 레포츠 여유있는 친절한'}
,{'place': '등명해변(등명해수욕장)', 'info': '소나무와 해변이 어우러져 아름다운 곳', '태그': '가족여행 강릉바닷가 강원권 강원도해수욕장 관광지 기차여행 나들이                '}
,{'place': '주문진해변', 'info': '고운 모래의 백사장, 매년 오징어 축제가 열리는 주문진해변', '태그': '가족여행 강원_가볼만한곳 강원권 강원도_여행지_추천 관광지 기차여행                '}
,{'place': '어민수산시장', 'info': '신선한 해산물을 저렴한 가격에 만날 수 있는 곳', '태그': '강릉관광 쇼핑 수산시장 월간_사진_제보_이벤트 재래시장 주문진관광 '}
,{'place': '강릉 임영관 삼문', 'info': '고려시대 관리나 내빈을 대접하고 묵게 하던 시설', '태그': '관광지 역사 역사공부 역사관광지 역사를품은곳 역사문화재 역사속      '}
,{'place': '강릉 월화거리', 'info': '옛것과 새로운 것의 경계를 허물다', '태그': 'IoT 강릉가볼만한곳 강릉여행 강릉월화거리 관광지 데이트코스 사진찍기         '}
,{'place': '옥계해변', 'info': '깨끗하고 넓은 백사장과 오래된 송림지역을 자랑하는 곳', '태그': '강릉바다 강릉해변 강릉해수욕장 강원권 강원도캠핑장 관광지 바다풍경     '}
,{'place': '숲사랑홍보관', 'info': '산림의 중요성을 홍보하는 곳', '태그': '강릉가볼만한곳 강릉여행 강원권 교과서속여행 문화시설 사계절 숲사랑홍보관    '}
,{'place': '강릉 서부시장', 'info': '시장 상인들과 로컬 크리에이터가 함께 만든 트렌디한 시장', '태그': 'CCC프로젝트 가족여행 강릉전통시장 강원권 기차여행 나들이 나홀로여행            '}
,{'place': '강릉 동부시장', 'info': '입소문난 맛집이 즐비한 상가건물형 시장', '태그': '가족여행 강릉동부시장 강릉시장 강원권 기차여행 나들이 나홀로여행           '}
,{'place': '강릉 대기리마을', 'info': '푸른 고원 마을', '태그': '관광지 농촌마을 농촌여행 농촌체험마을 아이와함께 이색체험 자연마을        '}
,{'place': '대관령박물관', 'info': '역사를 배우고 추억을 담아가는 박물관', '태그': '가족여행 강원권 기차여행 나들이 대관령박물관 문화관광해설사 문화시설        '}
,{'place': '강릉 농산물도매시장', 'info': '영동지역 농산물 유통의 중심', '태그': '가족여행 강릉농산물도매시장 강원권 기차여행 나들이 나홀로여행 데이트코스      '}
,{'place': '안반데기', 'info': '쏟아지는 별빛과 함께 떠나는 은하수 여행', '태그': '2024_여행가는달과함께하는_발도장여행지 INFP낭만여행 가볼래터 가족과함께           '}
,{'place': '당사해양낚시공원', 'info': '낚시를 좋아하는 사람이라면 한 번쯤 꼭 와봐야 할 곳', '태그': '가볼래터 관광지 당사해양낚시공원 동해바다 여행구독 울산가볼만한곳    '}
,{'place': '노추산 모정탑길', 'info': '간절한 소원을 빌면 이루어지는 길', '태그': '3000개의돌 강릉모정탑길 강릉트래킹 강원권 강원권여행 노추산 노추산모정탑        '}
,{'place': '한양대학교 박물관', 'info': '각 시대의 다양한 유물로 선조들의 삶을 볼 수 있는 곳', '태그': '교과서속여행 문화시설 박물관 서울가볼만한곳 수도권 아이와함께 왕십리박물관    '}
,{'place': '남항진해변(남항진해수욕장)', 'info': '크고 작은 바위와 어우러진 맑은 바다가 아름다운 해변', '태그': '가족여행 강원권 관광지 기차여행 나들이 남항진해변 남항진해수욕장               '}
,{'place': '수청저수지', 'info': '오염없는 깨끗한 공기와 물을 느낄 수 있는 곳', '태그': '레포츠 민물낚시 수청저수지 전북권 정읍낚시터 정읍저수지'}
,{'place': '남한강출렁다리', 'info': '산책하기 좋은 관광명소', '태그': '관광지 남한강출렁다리 데이트장소 명소 여주가볼만한곳 여주관광명소  '}
,{'place': '금진해변', 'info': '조용하고 아늑한 분위기가 매력적인 해변', '태그': '가족여행 강원권 관광지 금진해변 바다풍경여행 친구와함께'}
,{'place': '금진항(강릉)', 'info': '아름다운 바다뷰를 감상할 수 있는 해안 드라이브 명소', '태그': '관광지'}
,{'place': '경포호(철새도래지)', 'info': '경포해변이 품은 아름다운 호수', '태그': '가족여행 강릉가볼만한곳 강원권 강원도_여행지_추천 경포호 경포호수       '}
,{'place': '경포가시연습지', 'info': '생물 다양성의 보고 ‘경포습지’', '태그': '경치좋은곳 관광지 산책하기좋은곳 자연관광지 자연발생 자연속으로      '}
,{'place': '강릉 중앙시장', 'info': '강릉 도심 상권을 이루는 종합형 전통시장, 중앙시장', '태그': '2024_여행가는달과함께하는_발도장여행지 강릉중앙시장 강원권 먹거리  '}
,{'place': '강릉 솔향수목원', 'info': '계절별로 변화하는 다양한 풍경을 가진 곳', '태그': '가을단풍축제 강원권 강원도_여행지_추천 관광지 꽃여행 눈꽃&설경여행               '}
,{'place': '강릉단오제전수교육관', 'info': '전통문화를 보존하고 계승하는 시민 문화공간', '태그': '강릉단오제 공연장대관 교육공간 교육프로그램 문화공간 문화관광 문화시설     '}
,{'place': '강릉교육문화관', 'info': '교육, 문화 배움터이자 독서, 문화 활동 실현의 장', '태그': '강릉도서관 강릉도서대출 강릉문화강좌 강릉문화센터 강릉스마트도서관  '}
,{'place': '화산산성', 'info': '화산 일대에 외적의 침입을 막기 위해 축성된 산성', '태그': '경상권 경상북도기념물 관광지 군위바람개비언덕 군위화산산성 대구근교가볼만한곳               '}
,{'place': '혜원의 집', 'info': '시골 감성이 주는 힐링을 몸소 느끼다', '태그': 'SNS감성사진 감성사진 관광지 리틀포레스트 사진찍기 사진찍기좋은곳    '}
,{'place': '열두개울', 'info': '기암절벽과 맑은 계곡물의 아름다운 선경', '태그': '경기권 경치좋은곳 관광지 당일여행 서울근교여행 연천 연천가볼만한곳    '}
,{'place': '백의리층 (한탄강 유네스코 세계지질공원)', 'info': '옛 한탄강과 지금의 한탄강이 만나는 곳', '태그': '경치좋은곳 관광지 국가지질공원 자연 자연관광지 자연발생 자연속으로      '}
,{'place': '연천역 급수탑', 'info': '일제강점기의 역사가 남아있는 곳', '태그': '경기권 관광지 급수탑 서울근교여행 역사 역사공부 역사관광지 역사속     '}
,{'place': '고대산', 'info': '북녘 땅을 바라볼 수 있는 산', '태그': '가족과함께 가족여행 관광지 등산 서울근교여행 수도권 연천 연천가볼만한곳   '}
,{'place': '궁평해송군락지', 'info': '솔향을 맡으며 걷는 산책로', '태그': '관광지 데이트코스 드라이브코스 산책하기좋은 서해안가볼만한곳 일몰맛집  '}
,{'place': '매향항선착장', 'info': '90년대 초반 화옹지구 간척사업으로 만든 선착장이 지금까지 이어진 곳', '태그': '관광지 농섬 당일치기여행 매향리선착장 매향항선착장 서해드라이브코스  '}
,{'place': '백미항', 'info': '갯벌이 인상적인 아름다운 어촌', '태그': '감투섬 갯벌체험 관광지 백미리체험마을 서해일몰 일몰명소'}
,{'place': '선창포구', 'info': '싱싱한 해산물을 저렴하게 구입할 수 있는 곳', '태그': '감성여행 관광지 선창포구 주곡리 화성시가볼만한곳 화옹방조제'}
,{'place': '제부도선착장', 'info': '빨간 등대와 일몰이 아름다운 선착장', '태그': '가족나들이 관광지 데이트코스 드라이브코스 등대 서해가볼만한곳 일몰명소'}
,{'place': '서울대공원 산림욕장', 'info': '푸르른 내음이 가득한 곳', '태그': '관광지 나들이 둘레길 산림욕장 산책 서울대공원 휴식 힐링'}
,{'place': '과천문화원', 'info': '과천시민들이 자유롭게 참여할 수 있는 생활문화공간', '태그': '과천 과천무동답교놀이 과천문화원 문화강좌 문화시설 문화원 전통교양강좌  '}
,{'place': '연주암', 'info': '관악산 연주봉 남쪽에 자리하고 있는 사찰', '태그': '관광지 불교 불교문화 불교사찰 사찰 사찰여행 역사 자연속으로       '}
,{'place': '온온사', 'info': '정조가 머무르던 객사', '태그': '경기도유형문화재 관광지 문화관광해설사 서울근교여행 수도권 역사            '}
,{'place': '충주산방', 'info': '다양한 작품과 생활 자기를 만날 수 있는 공간', '태그': '나주 도자기 도자기판매장 쇼핑 충주산방 향토도자기'}
,{'place': '과천야생화자연학습장', 'info': '자연 속에서 즐기는 힐링', '태그': '관광지 아이와함께 자연 자연관광지 자연속으로 자연여행 자연좋은곳       '}
,{'place': '국립한국자생식물원', 'info': '아름다운 우리의 꽃과 나무들을 만날 수 있는 곳', '태그': '가족과함께 강원도가볼만한곳 관광지 식물원 아이와함께 자연 자연환경  '}
,{'place': '한천서원', 'info': '충렬공 전이갑과 충강공 전의갑 형제를 배향하고 있는 조선 후기 서원', '태그': '경상권 고려개국공신사당 관광지 교과서속여행 달성한천서원 대구한천서원                  '}
,{'place': '렛츠런파크 서울', 'info': '사계절 내내 이벤트가 가득한 가족공원', '태그': '가족과함께 경마장 레포츠 렛츠런파크 렛츠런파크서울 말박물관 이색체험 '}
,{'place': '렛츠런파크 서울 포니랜드', 'info': '렛츠런파크 서울 경주로 안에 위치한 가족공원', '태그': '경마 경마공원 공원 관광지 나들이 렛츠런파크 승마체험 어린이승마체험'}
,{'place': '달성 태고정', 'info': '사육신 박팽년의 손자 박일산이 세운 별당 건물', '태그': '관광지 대구태고정 사육신박팽년 사육신사당 역사 역사공부 역사관광지            '}
,{'place': '달성 하목정', 'info': '조선 사대부의 생활양식을 볼 수 있는 곳', '태그': '가볼래터 경상권 경상북도유형문화재 관광지 노을과따오기 달성하목정            '}
,{'place': '렛츠런파크 서울 말박물관', 'info': '말의 역사와 문화를 볼 수 있는 말 전문박물관', '태그': '과천말박물관 나들이 렛츠런파크 말박물관 문화시설 박물관 서울경마공원          '}
,{'place': '엄마아빠어렸을적에', 'info': '1960, 70년대를 체험할 수 있는 박물관', '태그': '경북권 관광지 엄마아빠어렸을적에 역사전시관'}
,{'place': '미호박물관', 'info': '자연을 통한 힐링, 남양주 한강뷰 뮤지엄 카페', '태그': '남양주 남양주박물관 남양주아이와함께가는박물관 남양주여행 문화시설    '}
,{'place': '산들소리', 'info': '숲에서 숨 쉬는 산들소리', '태그': '가볼래터 가족여행 관광지 데이트코스 브런치카페 사계절 산들소리수목원        '}
,{'place': '춘천별빛캠핑장', 'info': '산 정상에 위치하여 탁 트인 전망을 자랑하는 캠핑장', '태그': '레포츠 춘천가볼만한곳 춘천별빛캠핑장 춘천여행 춘천캠핑장'}
,{'place': '울산양떼목장', 'info': '초원 위에서 아이들과 함께하는 동물 먹이 주기 체험장', '태그': '관광지 양떼목장 양먹이주기체험 울산가볼만한곳 울산아이와함께 울산여행   '}
,{'place': '광덕사(대구)', 'info': '부처님 오신 날에 방문하기 좋은 사찰', '태그': '관광지 광덕사 대구가볼만한곳 대구사찰 대구여행 대구힐링'}
,{'place': '왈츠와 닥터만 커피박물관', 'info': '커피를 체험하고 소통하는 공간', '태그': '가족여행 경기권 문화시설 서울근교여행 왈츠와닥터만커피박물관 이색체험  '}
,{'place': '원호루', 'info': '조선 유생들의 생활단면을 볼 수 있는 누각', '태그': '경상권 관광지 역사 역사공부 역사를품은곳 역사문화재 역사속으로     '}
,{'place': '도곡재', 'info': '묘골마을에 위치한 조선 후기에 세운 사랑채', '태그': '관광지 달성군 대구 대구광역시 역사 역사공부 역사관광지 역사를품은곳       '}
,{'place': '일제강점기 군산역사관', 'info': '잊어서는 안 될 기억을 보존한 곳', '태그': '관광지 군산 역사관광지 일제강점기 군산역사관 전북'}
,{'place': '어달항', 'info': '알록달록 눈이 즐거운 아담한 항구', '태그': '가볼래터 관광지 동해 동해여행 어달항 여행구독'}
,{'place': '구 일본우선주식회사 인천지점', 'info': '항만 운송 사업을 장악하고자 설립된 일제강점기 회사', '태그': '건축물 건축여행 관광지 근대건축물 인천가볼만한곳 일본우선주식회사   '}
,{'place': '구룡포 일본인 가옥거리', 'info': '100여 년 전 일본 가옥이 남아있는 곳', '태그': '2024_여행가는달과함께하는_발도장여행지 걷기좋은길 경상권 관광지              '}
,{'place': '운정체육공원', 'info': '파주시의 시민 체육시설', '태그': '관광지 시민체육시설 와동동 운정신도시 인조잔디축구장 파주시 풋살구장'}
,{'place': '전북에어포스 스쿨', 'info': '유명 아이돌도 즐긴 완주 액티비티 체험', '태그': 'BTS bts여행 bts투어 가볼래터 가족여행 관광지 비행체험 여행구독     '}
,{'place': '물무산 행복숲', 'info': '온몸으로 느껴보는 황토', '태그': '관광지'}
,{'place': '함평해수찜', 'info': '천연 해수를 데운 물로 하는 찜질', '태그': '2023_설날_온천 가족과함께 건강여행 건강한힐링여행 관광지 부모님과함께           '}
,{'place': '부산 구 백제병원', 'info': '1922년에 지어진 최초의 근대식 개인종합병원', '태그': '관광지 브라운핸즈백제 역사 역사공부 역사관광지 역사를품은곳 역사문화재         '}
,{'place': '배너미오토캠핑장', 'info': '수려한 자연환경과 함께하는 자연의 쉼터', '태그': '대구시근교 레포츠 장박하기좋은 청도운문사인근 청도캠핑장 청정계곡캠핑 '}
,{'place': '부산시민회관', 'info': '친근하게 다가가는 시민들의 문화쉼터', '태그': '가족과함께 가족여행 공연장 기차여행 나들이 데이트코스 문화시설               '}
,{'place': '난지한강공원물놀이장', 'info': '음악분수가 있는 한강변의 가족형 물놀이장', '태그': '난지수영장 난지캠핑장 레포츠 월간_사진_제보_이벤트 음악분수 한강물놀이장  '}
,{'place': '충렬사', 'info': '조선시대 충신들의 위패가 봉안된 사당', '태그': '강화충렬사 관광지 수도권 역사 역사공부 역사관광지 역사를품은곳           '}
,{'place': '국립생태원(서천)', 'info': '다양한 생태계를 한 번에 볼 수 있는 곳', '태그': '2024_여행가는달과함께하는_발도장여행지 가족여행 관광지 무더위_피해_실내여행          '}
,{'place': '롯데리조트속초', 'info': '설악과 동해, 자연의 감성을 담은 공간', '태그': '강원도여행 관광지 롯데리조트 바다전망 속초숙소 속초여행 숙박'}
,{'place': '롯데리조트부여', 'info': '백제의 신비로운 유적지에서 펼쳐지는 새로운 개념의 휴양', '태그': '가족여행 관광지 롯데부여리조트 부여숙소 숙박 온천&스파 워터파크 '}
,{'place': '노루용추 계곡 (청송 국가지질공원)', 'info': '작은 폭포 아래에 형성되어 있는 폭호', '태그': '관광지'}
,{'place': '옛 용산신학교와 원효로 예수성심성당', 'info': '한국 가톨릭의 역사가 살아 숨 쉬는 곳', '태그': '관광지 서울용산신학교 서울용산신학교와원효로예수성심성당 역사 역사공부             '}
,{'place': '윤동주 하숙집 터', 'info': '시인의 발자취를 좇는 여행', '태그': '관광지 서울 서울가볼만한곳 역사 역사문화 윤동주 윤동주하숙집'}
,{'place': '매원마을', 'info': '조선시대 영남 3대 양반촌', '태그': '관광지 능소화명소 마을 양반촌 칠곡가볼만한곳 칠곡관광지 칠곡군 '}
,{'place': '통영대교', 'info': '통영 8경 중 하나인 아름다운 야경 명소', '태그': '건축물 경상권 경치좋은곳 관광지 데이트코스 미륵도 산책하기좋은곳           '}
,{'place': '수락리 주상절리 (청송 국가지질공원)', 'info': '화산재가 만들어낸 주상절리', '태그': '관광지'}
,{'place': '밀양 월연정', 'info': '수려한 풍광 속 고즈넉한 한옥의 멋', '태그': '관광지 역사공부 역사관광지 역사문화 역사문화재 역사속 역사속으로     '}
,{'place': '파천 구상 화강암 (청송 국가지질공원)', 'info': '공같이 둥근 구조를 포함한 화강암', '태그': '관광지 국가지질공원 자연 자연관광지 자연발생 자연속으로 자연여행      '}
,{'place': '추사 김정희 선생 고택', 'info': '추사 김정희 선생 관련 유물을 만나볼 수 있는 곳', '태그': '관광지 김정희선생고택 김정희선생묘 역사 역사공부 역사관광지 역사를품은곳             '}
,{'place': '쇠소깍', 'info': '신비한 계곡에 온 기분을 느낄 수 있는 쇠소깍', '태그': '15_16한국관광100선 17_18한국관광100선 2024_여행가는달과함께하는_발도장여행지                '}
,{'place': '엉또폭포', 'info': '비가 많이 오면 꼭 가보아야 할 제주의 숨은 비경', '태그': '가족여행 관광지 서귀포엉또폭포 엉또폭포 자연 자연속으로 자연여행            '}
,{'place': '청송 후송당 고택', 'info': '우리 민족의 역사를 반영하는 건축물', '태그': '경상권 관광지 교과서속여행 역사 역사공부 역사를품은곳 역사문화재         '}
,{'place': '충주 미륵대원지', 'info': '옛 절터에 서린 미륵의 바람, 충주 미륵대원지', '태그': '가족여행 가족체험여행 관광지 교과서속여행 아이와함께 역사 연인과함께  '}
,{'place': '김해 분산성', 'info': '김해 드넒은 평야가 내려다보이는 가야시대의 산성', '태그': '가볼래터 가족여행 경상권 관광지 김해해운사 만장대 분산성 여행구독           '}
,{'place': '따라비 오름', 'info': '넓은 평원과 하얀 억새가 채워진 오름', '태그': '가족여행 관광지 산책코스 오름 자연 자연여행 자연좋은곳 자연풍경         '}
,{'place': '안동 고성이씨 탑동파 종택', 'info': '조선 중기 양반집의 요소를 두루 갖추고 있는 주택', '태그': '경상권 고성이씨탑동파종택 관광지 교과서속여행 문화재 역사 역사공부          '}
,{'place': '중앙탑공원(충주)', 'info': '충주 시민의 위락 시설로 자리 잡은 공원', '태그': '2022_추석_별빛_가득_야간관광 2024_여행가는달과함께하는_발도장여행지                 '}
,{'place': '경북 청송 주왕산면·파천면 [슬로시티]', 'info': '자연경관과 한국의 전통고택들의 조화가 어우러진 곳', '태그': '경상권 고택체험 관광지 남녀노소 덕천마을 선비마을 슬로시티 역사            '}
,{'place': '진해해양공원', 'info': '최대 높이, 최대규모의 태양광 발전시설', '태그': '공원 관광지 문화관광해설사 창원 창원솔라타워 해양공원'}
,{'place': '한생연 휴먼탐구과학관', 'info': '과학을 재미있게 배우는 공간', '태그': '과학박물관 문화시설 어린이체험박물관 인체과학박물관 인체교육 한생연  '}
,{'place': '죽계구곡', 'info': '물이 흐르는 소리가 노래소리같이 아름다운 죽계구곡', '태그': '경상권 경치좋은곳 관광지 여름여행 자연 자연&숲 자연좋은곳 피서여행 '}
,{'place': '한라생태숲', 'info': '숲속 걷기를 통해 힐링이 되는 공간', '태그': '가족여행 걷기길 걷기좋은 관광지 등산 숲 숲체험 자연 자연속으로              '}
,{'place': '창녕 우포늪', 'info': '대한민국 최대 규모의 자연 내륙 습지', '태그': '2024_여행가는달과함께하는_발도장여행지 21_22한국관광100선 25_26한국관광100선                     '}
,{'place': '관곡지', 'info': '연꽃의 은은한 향기가 퍼지는 연꽃 명소', '태그': '관곡지 관광지 문화관광해설사 문화재답사 수도권 시흥가볼만한곳 시흥관곡지       '}
,{'place': '추억촌', 'info': '강원도 강릉 조용한 캠핑장', '태그': '가성비글램핑장 가을단풍명소 강릉시캠핑장 강원도캠핑장 대관령둘레길    '}
,{'place': '보광농촌체험휴양마을', 'info': '초등학교의 정취와 자연 속 즐거움을 느낄 수 있는 야영 체험지', '태그': '가을단풍명소 강릉 강릉가볼만한곳 강릉시캠핑장 강원도캠핑장 겨울눈꽃명소 '}
,{'place': '김포만화도서관', 'info': '다양한 연령층을 위한 만화 도서관', '태그': '김포가볼만한곳 김포도서관 김포만화도서관 김포시 도서관 만화도서관 '}
,{'place': '강동숲속도서관', 'info': '숲과 과학이 함께하는 질문과 상상의 숲, 강동숲속도서관', '태그': '강동구도서관 강동숲속도서관 도서관 문화시설 서울도서관 아이와함께'}
,{'place': '파주출판단지근린공원', 'info': '출판문화 특성을 살려 조성된 공원', '태그': '관광지 근린공원 도심속힐링 산책하기좋은 파주나들이 파주출판단지'}
,{'place': '용암사(파주)', 'info': '거대한 불상이 우람하게 서 있는 사찰', '태그': '관광지 불교 불교문화 불교사찰 사찰 사찰여행 역사 자연좋은곳       '}
,{'place': '파주 삼릉 [유네스코 세계유산]', 'info': '공릉·순릉·영릉을 통칭하는 조선시대 왕릉', '태그': '관광지 수도권 아이와함께 역사 역사공부 역사관광지 역사를품은곳          '}
,{'place': '감악산(파주)', 'info': '출렁다리 명소가 있는 감악산', '태그': '걷기좋은곳 관광지 등산 등산객 등산로 산 수도권 양주감악산 자연         '}
,{'place': '금촌통일시장', 'info': '구경하며 골라사는 재미가 있는 전통시장', '태그': '5일장 가족여행 금촌5일장 금촌오일장 금촌장 민속장 사계절 쇼핑          '}
,{'place': '황희선생유적지·반구정', 'info': '조선 명재상 청백리의 귀감 황희 선생 유적지', '태그': '경기도기념물 경기도문화재자료 관광지 반구정 수도권 역사 역사공부          '}
,{'place': '화석정', 'info': '율곡 이이의 이야기가 있는 정자', '태그': '경기도유형문화재 관광지 수도권 역사 역사유적지 임진강화석정 파주화석정 '}
,{'place': '도라전망대', 'info': '북한의 생활을 바라볼 수 있는 전망대', '태그': 'DMZ안보견학 관광지 남녀노소 도라전망대 아이와함께 안보관광 안보여행       '}
,{'place': '장산전망대', 'info': '파주 평화누리길의 숨은 명소', '태그': '관광지 노지캠핑 반구정길 초평도 평화누리8길 평화누리길8코스'}
,{'place': '제3땅굴', 'info': '분단의 역사를 보여주는 제3땅굴', '태그': 'DMZ여행 가족과함께 관광지 아이와함께 역사 역사관광지 역사탐방    '}
,{'place': '운정건강공원', 'info': '다양한 체육시설이 가득한 건강공원', '태그': '가온 공원 관광지 운정건강공원 운정다목적체육관 운정신도시 파주  '}
,{'place': '강릉경포카라반파크', 'info': '바다의 풍광이 캠핑의 즐거움을 더해주는 곳', '태그': '강릉아이들과가볼만한곳 강릉카라반 강릉캠핑장 레포츠 아이들과강릉여행'}
,{'place': '관호산성', 'info': '아름다운 뷰를 만끽할 수 있는 칠곡 명소', '태그': '관광지 둘레길 산책하기좋은곳 칠곡가볼만한곳 칠곡관광지 칠곡군 칠곡명소'}
,{'place': '교하중앙공원', 'info': '다양한 공간으로 구성된 교하중앙공원', '태그': '7개국정원 관광지 교하중앙공원 외국공원축소 테마공원 파주나들이  '}
,{'place': '심학산 둘레길', 'info': '천천히 걷기 좋은 둘레길', '태그': '둘레길 레포츠 심학산 파주등산 파주명소 파주여행'}
,{'place': '창경궁', 'info': '그 시절부터 우리가 사랑한 서울의 다섯 궁궐 ④', '태그': '15_16한국관광100선 17_18한국관광100선 19_20한국관광100선 2024_여행가는달과함께하는_발도장여행지                      '}
,{'place': '강릉 노추산힐링캠프', 'info': '캠핑장 앞 송천 견지낚시 메기낚시를 경험해보자', '태그': '가을단풍명소 강릉시캠핑장 강원도캠핑장 금강송 레포츠 여름물놀이'}
,{'place': '고송정지', 'info': '단종을 향한 3대의 충절 고송정지', '태그': '관광지 역사 역사공부 역사관광지 역사를품은곳 역사문화재 역사속       '}
,{'place': '오정각', 'info': '5대에 걸친 충신 정문 효자 정문', '태그': '관광지 역사 역사공부 역사관광지 역사를품은곳 역사문화재 역사속      '}
,{'place': '사세충렬문', 'info': '선조들의 충심과 열녀정신을 엿볼 수 있는 곳', '태그': '관광지 역사 역사공부 역사관광지 역사를품은곳 역사속 역사속으로      '}
,{'place': '이익선생묘', 'info': '실학의 대가 성호 이익선생의 묘', '태그': '관광지 역사 역사공부 역사관광지 역사기행 역사문화 역사문화재        '}
,{'place': '새암공원', 'info': '문학과 자연이 어우러져 있는 공원', '태그': '관광지 문학공원 새암공원 시비 운정산책길 운정새암공원 운정운동코스 '}
,{'place': '교하향교', 'info': '도심 속 숨겨진 벚꽃 명소로 사랑받는 향교', '태그': '경기도문화재자료 관광지 교하향교 수도권 역사 역사공부 역사관광지              '}
,{'place': '화산서원(완주)', 'info': '조선시대 학문과 유학 정신을 계승한 사액서원', '태그': '관광지 교육기관 서원 역사 역사공부 역사관광지 역사를품은곳 역사문화재        '}
,{'place': '대덕승마장', 'info': '누구나 쉽게 즐길 수 있는 가족형 승마장', '태그': '대구 대구광역시 대구달서구 대구승마장 대덕승마장 레포츠 승마 '}
,{'place': '자운서원', 'info': '율곡 이이의 학문과 정신을 기리기 위해 세워진 서원', '태그': '관광지 기념관 역사관광지 유적지 율곡기념관 율곡사당 율곡이이    '}
,{'place': '파산서원', 'info': '기호학파의 거두 성혼을 배향한 파산서원', '태그': '경기도문화재자료 관광지 서원 수도권 역사 역사공부 역사관광지                 '}
,{'place': '고양 서삼릉 [유네스코 세계유산]', 'info': '3기의 왕릉과 원, 묘, 태실이 모여있는 곳', '태그': '가족여행 고양서삼릉 관광지 교과서속여행 나들이 당일치기여행 드라이브여행             '}
,{'place': '고흥향교', 'info': '전라도의 유교 문화를 보존하고 전승하는 교육의 장', '태그': '고흥아문 고흥존심당 고흥향교 관광지 역사 역사관광지 역사를품은곳              '}
,{'place': '재동서원', 'info': '송간과 김시습의 기품있는 절개', '태그': '고흥서동사 고흥역사 고흥의서원 관광지 김시습 매월당 서동사 서원          '}
,{'place': '수도암(고흥)', 'info': '운암산 중턱에 자리잡은 조용한 사찰', '태그': '관광지 불교 불교문화 불교사찰 사찰 사찰여행 역사 자연속으로       '}
,{'place': '송광암', 'info': '고려의 국사인 보조국사 지눌스님이 조각한 나무새가 날아와 불사의 터를 잡다', '태그': '거금도사찰 거금도작은사찰 고흥섬 고흥송광암 관광지 기도도량 불교               '}
,{'place': '장안공원', 'info': '오랜 세월의 고고한 자태를 뽐내다', '태그': '1978년에조성된공원 관광지 도심속공원 수원성곽옆공원 장안문 화서문과서북공심돈을품은공원'}
,{'place': '수리산도립공원', 'info': '군포시를 병풍처럼 둘러싸고 있는 진산', '태그': '경치좋은곳 관광지 등산로 서울근교여행 수도권 수리산도립공원 자연   '}
,{'place': '갈미한글공원', 'info': '국어학자 이희승 박사를 기념하다', '태그': '공원 관광지 나들이 데이트 둘레길 산책 의왕시공원 휴식'}
,{'place': '서수원체육공원', 'info': '다양한 체육활동을 즐길 수 있는 체육공원', '태그': '관광지 수원 수원체육공원 수원체육관 수원체육시설 유소년축구장 체육공원    '}
,{'place': '국립무형유산원', 'info': '우리의 과거와 현재, 그리고 미래를 담은 무형유산의 보고(寶庫)', '태그': '공연 국립무형유산원 문화시설 문화유산 전시 전주가볼만한곳'}
,{'place': '공주 중학동 구 선교사 가옥', 'info': '영명학교가 교육을 시작한 상징적 건물', '태그': '공주가볼만한곳 공주당일코스 공주여행 관광지 선교사 역사 역사공부    '}
,{'place': '소전미술관', 'info': '서예 대가 소전 손재형 선생의 작품을 볼 수 있는 곳', '태그': '문화시설 문화예술 소전미술관 실내데이트 진도군 진도명소 진도소전미술관'}
,{'place': '망원한강공원', 'info': '원효대교와 성산대교 사이의 야경 명소 공원', '태그': '25_26한국관광100선 가족과_함께 걷기길 관광지 나들이 도심여행            '}
,{'place': '완주 고산 놀토피아', 'info': '모험놀이 콘텐츠를 갖춘 국내 실내형 체험시설', '태그': '가족과함께 관광지 놀이기구 레포츠 미세먼지_피하는_실내여행 실내관광지        '}
,{'place': '서울함공원', 'info': '세 척의 퇴역 군함을 이용하여 조성한 함상 테마파크', '태그': '가족여행 관광지 마포공원 서울함 서울함공원 아이와함께 역사이야기      '}
,{'place': '영산포풍물시장(5, 10일)', 'info': '제철 농산물과 다양한 수산물을 즐길 수 있는 곳', '태그': '나주 쇼핑 시장 영산포풍물시장 오일장 전라권 전통시장'}
,{'place': '노을공원(서울)', 'info': '아름다운 석양이 펼쳐지는 공원', '태그': '관광지 노을공원 노을전망 누에체험장 서울가볼만한곳 자연체험 전망데크   '}
,{'place': '웰리힐리파크 워터플래닛', 'info': '물의 행성에서 만나는 새로운 스릴과 즐거움이 가득한 곳', '태그': '관광지 아이와함께 여름가볼만한곳 여름나들이 여름물놀이 여름방학      '}
,{'place': '제주 서귀포 산방산', 'info': '거대한 조각 작품을 보는 듯한 웅장한 산방산', '태그': '가족여행 걷기길 관광지 사진찍기좋은곳 산방산(제주) 연인과함께 자연        '}
,{'place': '스몹 대전', 'info': '세계 최초 어른들의 놀이터', '태그': '가족과함께 대구가볼만한곳 레포츠 실내스포츠 실내액티비티 액티비티 '}
,{'place': '안동 법흥사지 칠층전탑', 'info': '통일신라시대부터 자리를 지켜온 역사의 흔적', '태그': '가족여행 경상권 관광지 안동법흥사지칠층전탑 역사 역사유적지 친구와함께'}
,{'place': '소양강스카이워크', 'info': '하늘 위를 걷는 스릴감, 오색 조명의 아름다움 모두 갖춘 스카이워크', '태그': '2024_여행가는달과함께하는_발도장여행지 감성사진 강원권 강원도_여행지_추천              '}
,{'place': '구인사(단양)', 'info': '소백산 기슭에 길게 펼쳐져 있는 구인사', '태그': '2024_여행가는달과함께하는_발도장여행지 관광지 구인사(단양) 구인사대법당                          '}
,{'place': '도예공방 필', 'info': '도자기 인형을 만날 수 있는 곳', '태그': '나전칠기 도예공방 도예체험 도자기 도자기체험 쇼핑 원데이클래스    '}
,{'place': '영천생태지구공원', 'info': '금호강변을 따라 조성된 사계절 꽃과 힐링 공간', '태그': '관광지 꽃구경 꽃구경명소 꽃구경하기좋은곳 도심속자연공원 영천공원  '}
,{'place': '용화사(통영)', 'info': '한려해상 다도해가 한눈에 내려다 보이는 유서 깊은 사찰', '태그': '관광지 도솔암 미륵산사찰 불교 불교사찰 사찰 사찰여행 역사 자연좋은곳           '}
,{'place': '단양 보발재', 'info': '아름다운 단풍을 즐길 수 있는 드라이브 코스', '태그': '2024_여행가는달과함께하는_발도장여행지 가족여행 관광지 단양가볼만한곳                  '}
,{'place': '세기알해변', 'info': '제주의 아름다운 에메랄드빛 바다', '태그': '가볼래터 관광지 바다여행 바다풍경 세기알해변 여행구독 제주도_가볼만한곳     '}
,{'place': '필운동 홍건익 가옥', 'info': '고즈넉한 서울시 공공한옥 역사가옥', '태그': '공공한옥 관광지 서울_여행지_추천 서울가볼만한곳 서울권 전통한옥'}
,{'place': '불모산 노을 전망대', 'info': '해넘이 해돋이가 아름다운 전망대', '태그': '관광지 불모산 연인과함께 일몰명소 일출일몰 전망대 창원가볼만한곳  '}
,{'place': '충청남도역사박물관', 'info': '충남의 역사와 문화를 보존하고 계승하는 박물관', '태그': '공주가볼만한곳 공주박물관 문화시설 박물관 아이와함께 역사공부 역사관광지                 '}
,{'place': '왕이메오름', 'info': '크고 작은 봉우리가 하나의 산을 이루고 있는 오름', '태그': '경치좋은곳 관광지 남녀노소 레포츠 서귀포가볼만한곳 오름 자연좋은곳      '}
,{'place': '경주 계림', 'info': '김알지의 탄생 설화가 깃든 숲', '태그': '경상권 관광지 교과서속여행 산책길 산책하기 산책하기좋은곳 아이와함께          '}
,{'place': '성주 한개마을', 'info': '한옥 보존 마을과 돌담길', '태그': '경상권 관광지 교과서속여행 문화관광해설사 역사 역사공부 역사관광지    '}
,{'place': '장생포고래문화특구', 'info': '고래의 역사와 문화를 간직한 해양관광특구', '태그': '관광지'}
,{'place': '창원 고현리 공룡발자국 화석', 'info': '능지물 바닷가 바위 위에 공룡의 발자국', '태그': '경치좋은곳 공룡발자국 공룡화석지 관광지 바다경치 아이와함께 역사탐험   '}
,{'place': '진천 농다리', 'info': '우리나라에서 가장 길고 오래된 돌다리', '태그': '2024_여행가는달과함께하는_발도장여행지 가족여행 관광지 교과서속여행            '}
,{'place': '구병산', 'info': '아홉개의 봉우리로 이루어진 가을 단풍 명소', '태그': '가을명소 관광지 등산 보은가볼만한곳 보은구병산 산 자연 자연좋은곳   '}
,{'place': '연천재인폭포오토캠핑장', 'info': '한탄강댐을 배경으로 펼쳐진 광활한 대지 위 캠핑장', '태그': '경기도캠핑장 레포츠 신탄리역 연천군캠핑장 연천재인폭포오토캠핑장        '}
,{'place': '천안 봉선홍경사 갈기비', 'info': '천안의 유일한 국보이자 역사 문화유산', '태그': '가족여행 관광지 교과서속여행 아이와함께 역사공부 역사관광지 역사를품은곳        '}
,{'place': '합덕성당', 'info': '100년 역사를 자랑하는 당진의 성당', '태그': '관광지 당진문화재 당진합덕성당 성가정순례자의집 성지순례 역사 역사공부                 '}
,{'place': '꽃지해수욕장', 'info': '아름다운 해당화가 가득한 해수욕장', '태그': '2024_여행가는달과함께하는_발도장여행지 21_22한국관광100선 23_24한국관광100선           '}
,{'place': '지경리해변', 'info': '잔잔한 파도와 부드러운 모래사장이 매력적인 해변', '태그': '가족여행 강원권 강원권반려동물여행지 관광지 기차여행 나들이 당일치기여행               '}
,{'place': '안목해변', 'info': '향긋한 커피향과 아름다운 바다가 어우러진 명소', '태그': '2024_여행가는달과함께하는_발도장여행지 INFP혼행 강원_가볼만한곳           '}
,{'place': '송암스페이스센터', 'info': '꿈과 도전을 이끌어가는 우주교육기관', '태그': '관광지 사계절 서울근교여행 송암스페이스센터 수도권 아이와함께 자연  '}
,{'place': '수도권매립지 드림파크 야생화단지', 'info': '꽃과 함께하는 사계절 산책로, 마음을 쉬게 하는 도시 숲', '태그': '2024_여행가는달과함께하는_발도장여행지 가족여행 관광지 남녀노소              '}
,{'place': '태릉국제스케이트장', 'info': '국내 유일 400m 실내 링크장', '태그': '겨울스포츠 레포츠 복합레저스포츠 스케이트 스케이트강습 스케이트장       '}
,{'place': '합덕제', 'info': '조선 3대 저수지 중의 하나인 곳', '태그': '가볼래터 관광지 생태 여행구독 연인과밤밤곡곡 자연 자연풍경'}
,{'place': '곡교천 은행나무길', 'info': '전국의 아름다운 10대 가로수길', '태그': '가을여행지 걷기길 곡교천 곡교천은행나무길 관광지 남녀노소 반려견과함께            '}
,{'place': '대포주상절리', 'info': '가슴이 탁 트이는 시원한 절경', '태그': '가족여행 관광지 대포동지삿개 데이트코스 바다풍경 연인과함께 연중무휴    '}
,{'place': '삼성궁', 'info': '웅장하고 아기자기한 돌탑이 아름다운 곳', '태그': '2024_여행가는달과함께하는_발도장여행지 가족과함께 가족여행 경치좋은곳                '}
,{'place': '솔뫼성지', 'info': '한국 최초의 사제인 성 김대건 안드레아 신부의 탄생지', '태그': '가족여행 관광지 근대문화유산 문화관광해설사 솔뫼성지 아이와함께      '}
,{'place': '함허동천야영장', 'info': '아름다운 산수에 둘러싸여 즐기는 캠핑', '태그': '2024_여행가는달과함께하는_발도장여행지 가족과함께 레포츠 야영장      '}
,{'place': '광덕사(천안)', 'info': '우리나라에서 가장 오래된 호두나무가 있는 사찰', '태그': '관광지 광덕사 불교 불교문화 사찰 사찰여행 역사 역사유적지 연중무휴          '}
,{'place': '종묘 [유네스코 세계유산]', 'info': '왕과 왕비의 신주를 모시는 국가 사당', '태그': '가족과함께 가족여행 관광지 교과서속여행 당일치기여행 서울근교여행           '}
,{'place': '도둑골캠핑장', 'info': '가족단위 캠퍼들에게 즐길 거리가 가득한 공간', '태그': '겨울썰매장 경상남도캠핑장 레포츠 방방이 불멍캠핑 수영장캠핑 여름물놀이  '}
,{'place': '승월교 소원의 다리', 'info': '소원이 많이 달려있는 다리', '태그': '가족과함께 관광지 남원가볼만한곳 남원야경명소 소원의다리 승월교   '}
,{'place': '혼인지', 'info': '제주도 시조의 자취를 더듬어 보기 좋은 곳', '태그': '2024_여행가는달과함께하는_발도장여행지 가족여행 가족체험여행 걷기여행          '}
,{'place': '밀양 만어산 암괴류', 'info': '경쇠소리 나는 물고기 형상의 크고 작은 반석들', '태그': '관광지 독특한반석 만어사 만어사경석 만어산 밀양가볼만한곳 밀양여행  '}
,{'place': '2018 평창동계올림픽대회 및 동계패럴림픽대회 기념관', 'info': '뜨거웠던 올림픽의 순간을 재조명하는 공간', '태그': '문화관광해설사 문화시설'}
,{'place': '북한산 사기막야영장', 'info': '북한산국립공원 안에서 편안한 휴식을 즐길수 있는 국내 최초 탄소중립 야영장', '태그': '레포츠 북한산국립공원 사기막야영장 쉬기좋은곳 야영장 야영장추천 '}
,{'place': '요천수경 음악분수', 'info': '남원의 밤, 요천 위에서 피어나는 빛과 음악의 축제', '태그': '관광지 남원가볼만한곳 남원야간명소 분수 야간명소 요천수경음악분수  '}
,{'place': '네츄럴밸리', 'info': '화려한 수영장과 캠핑의 즐거움을 모두 즐길 수 있는 최고의 풀램핑장', '태그': '개별수영장 개별수영장숙소 글램핑 레포츠 쉬기좋은숙소 온수풀 풀램핑장'}
,{'place': '각원사', 'info': '천안의 새로운 관광명소로 떠오르고 있는 각원사', '태그': '각원사 관광지 교과서속여행 기차여행 나들이 사찰&산사 아이와함께       '}
,{'place': '충무교와 통영운하', 'info': '통영의 아름답고 신비한 야경을 감상할 수 있는 곳', '태그': '가볼래터 경치좋은곳 관광지 데이트 데이트장소추천 산책길 산책코스              '}
,{'place': '당포성지', 'info': '왜구의 침입을 막아낸 임진왜란 승전지 당포성', '태그': '관광지 당포승첩 망루터 석축진성 역사 역사관광지 역사를품은곳           '}
,{'place': '표충사(밀양)', 'info': '사명대사의 충훈을 추모하며', '태그': '가족여행 관광지 불교 불교문화 사찰&산사 아이와함께 역사 역사유적지        '}
,{'place': '케이팝 스퀘어 홍대', 'info': 'K-POP 팬이라면 놓칠 수 없는 즐거움이 가득한 공간', '태그': 'K-POP K-POP공간 서울가볼만한곳 쇼핑 친구와함께 케이팝스퀘어홍대 '}
,{'place': '경주 월성', 'info': '임금이 살았던 초승달을 닮은 성', '태그': '경상권 경주월성 관광지 반월성 역사 역사유적지 연중무휴'}
,{'place': '부천천문과학관', 'info': '단순한 1회성 체험보다 깊이 있는 천문학 교육에 중점을 둔 시민천문대', '태그': '과학관 관광지 부천 부천가볼만한곳 부천천문과학관 아이들과함께 천문대'}
,{'place': '천축사(서울)', 'info': '천년이 넘는 역사를 가진 사찰', '태그': '관광지 도심속 불교 불교문화 사찰 사찰여행 수도권 역사 자연환경        '}
,{'place': '서울시립보라매청소년센터', 'info': '행복한 성장의 주인공, 청소년을 위한 공간', '태그': '교육프로그램 동작구 레포츠 보라매청소년수련관 청소년수련원'}
,{'place': '규봉암(화순)', 'info': '자연이 그려낸 풍경산수화 속 고즈넉한 사찰', '태그': '관광지 불교 불교사찰 사찰 사찰여행 역사 역사공부 역사관광지               '}
,{'place': '영광풍력발전단지', 'info': '푸른 바다와 하늘 배경의 햐얀 풍력 발전기', '태그': '관광지 영광여행 영광영화촬영지 영광일몰명소 영화독전 영화독전촬영지  '}
,{'place': '배봉산', 'info': '걷기에 최적화된 무장애 둘레길을 가진 동대문구 대표 산', '태그': '가벼운등산 관광지 남녀노소 동대문구산 동대문구산책길 배봉산둘레길              '}
,{'place': '용암서원(합천)', 'info': '남명 조식 선생의 기운이 깃든 곳', '태그': '경상권 관광지 역사 역사공부 역사관광지 역사를품은곳 역사문화재           '}
,{'place': '세종마을 음식문화거리', 'info': '골라 먹는 재미가 있는 시장 맛집 골목', '태그': '가족과함께 관광지 맛집 먹자골목 종로 친구와함께 핫플레이스'}
,{'place': '켄싱턴 프렌치 가든', 'info': '국내 최대 규모의 프랑스식 정원', '태그': '가족과함께 강원가볼만한곳 강원여행 관광지 사진찍기좋은곳 산책 월간_사진_제보_이벤트    '}
,{'place': '클레이아크 김해미술관', 'info': '전통과 현대, 순수미술과 응용미술이 교류하는 곳', '태그': '가족여행 경상권 기념품판매 김해가볼만한곳 문화시설 미술관 사계절     '}
,{'place': '[효석문학100리길 1코스] 문학의 길', 'info': '문학과 자연이 어우러진 평창의 대표 걷기 여행길', '태그': '강원 강원평창군 관광지 문학 문학자연 효석문학100리길 효석문학100리길(1구간)'}
,{'place': '도안사', 'info': '자연과 조화를 이루어 아늑하게 자리 잡은 도심 속 사찰', '태그': '108 108산사 108산사순례 관광지 도선사 도안사 불교 불교문화                 '}
,{'place': '용산공원 부분개방부지', 'info': '역사를 품은 도심 속 감성 휴양지', '태그': 'SNS감성핫플레이스 SNS속_인생샷명소 가볼래터 관광지 데이트코스          '}
,{'place': '계방산(평창)', 'info': '사계절 자연의 아름다움을 품은 오대산국립공원의 최고봉', '태그': '강원권 관광지 등산 산 자연 자연속으로 자연여행 자연좋은곳 자연풍경    '}
,{'place': '대장마을', 'info': '지리산 자락에 위치한 벽화가 아름다운 마을', '태그': '가족과함께 관광지 대장마을 레프팅 마을관광지 벽화 산청가볼만한곳  '}
,{'place': '광운대학교 아이스링크', 'info': '국내 대학 최초의 국제 규격 아이스링크', '태그': '겨울레포츠 광운대스케이트장 광운대아이스링크 교육프로그램 남녀노소        '}
,{'place': '증평민속체험박물관', 'info': '증평의 역사와 전통문화를 만나는 곳', '태그': '가족체험여행 공예체험 공예품 농촌체험 목공예 문화관광해설사 문화시설             '}
,{'place': '탄금호 피크닉공원', 'info': '아름다운 물결이 일렁이는 탄금호에서 즐기는 피크닉', '태그': '공원 관광지 충주 충주탄금호피크닉공원 탄금호 탄금호피크닉공원'}
,{'place': '충주고구려비전시관', 'info': '남한강 유역까지 진출한 고구려의 비석 기념관', '태그': '고구려비 문화시설 박물관 아이와함께 역사공부 역사관광지 역사를품은곳           '}
,{'place': '상원사(오대산)', 'info': '문수보살 전설과 국보 동종을 품은 오대산의 천년 고찰', '태그': '가볼래터 가족여행 강원권 강원도사찰 관광지 기차여행 나들이 당일치기여행            '}
,{'place': '과천향교', 'info': '현재도 다양한 유교체험을 할 수 있는 조선시대 공립교육기관', '태그': '경기도문화재자료 과천향교 관광지 문화관광해설사 서울근교여행 수도권              '}
,{'place': '원주 반계리 은행나무', 'info': '800년의 세월을 보낸 은행나무', '태그': '감성사진 관광지 사진찍기 사진찍기좋은곳 사진촬영 사진촬영명소 원주가볼만한곳      '}
,{'place': '하눌저수지', 'info': '한적하게 산책하기 좋은 곳', '태그': '경북권 레포츠 산책길 자연속으로 자연여행 자연좋은곳 자연풍경 '}
,{'place': '알펜시아 알파인코스터', 'info': '시속 40㎞로 달리는 알펜시아의 짜릿한 알파인코스터', '태그': '겨울레포츠 관광지 레포츠 스피드즐기기 아이와가기좋은 알파인코스터  '}
,{'place': '호산공원', 'info': '아름다운 숲길을 거닐 수 있는 공원', '태그': '관광지 대구공원 대구나들이 대구산책 대구여행 대구힐링여행'}
,{'place': '제주민속촌', 'info': '역사와 문화가 살아 숨 쉬는 곳!', '태그': '가족과함께 교육여행 문화시설 반려동물 반려동물동반여행지 사계절           '}
,{'place': '해운대 로데오거리', 'info': '쇼핑과 관광을 모두 즐길 수 있는 거리', '태그': '경상권 관광지 부산관광 부산로데오거리 쇼핑 연인과함께 친구와함께  '}
,{'place': '오대산 선재길', 'info': '오대산의 자연과 문화가 어우러진 숲길', '태그': '2024_여행가는달과함께하는_발도장여행지 강원권 걷기여행 레포츠 산림욕   '}
,{'place': '기장향교', 'info': '조선 광해군 때 지어진 교육기관', '태그': '경상권 관광지 교과서속여행 기장가볼만한곳 기장향교 역사 역사관광지  '}
,{'place': '제부도워터워크', 'info': '제부도의 아름다운 자연경관을 볼 수 있는 곳', '태그': '관광지 일몰명소 제부도바다열림길 제부도여행 제부도워터워크 제비꼬리길 '}
,{'place': '섬진강 대나무숲길', 'info': '섬진강 풍경이 아름다운 대나무 수', '태그': '걷기길 대나무 대나무숲 레포츠 섬진강대나무숲길'}
,{'place': '고령 주산성', 'info': '신라군에 대항하기 위해 축조된 가야의 산성', '태그': '경상권 고령관광 고령여행 고령유적 관광지 사적 성 역사 역사공부           '}
,{'place': '은하사(김해)', 'info': '아름다운 별이 쏟아지는 신어산의 사찰', '태그': '경상권 관광지 김해은하사 불교 불교문화 사찰 산사 역사 역사유적지        '}
,{'place': '한반도섬', 'info': '국토 정중앙에 위치한 작은 한반도', '태그': '강원권 관광지 섬 양구가볼만한곳 한반도섬'}
,{'place': '선샤인랜드', 'info': '도전과 모험, 즐거움과 휴식이 어우러지는 선샤인 랜드', '태그': '2024_여행가는달과함께하는_발도장여행지 가족나들이 관광지 논산가볼만한곳     '}
,{'place': '송골매 패러글라이딩 스쿨', 'info': '완전한 자연을 느낄 수 있는 패러글라이딩 체험', '태그': '레포츠 스쿨 용인시 패러글라이더 활공장'}
,{'place': '구봉산 전망대', 'info': '광양만의 장엄한 파노라마', '태그': '가을밤의낭만곡곡 경치여행 경치좋은곳 관광지 광양만야경 광양야경               '}
,{'place': '남대천(강릉)', 'info': '강릉단오제가 열려 지역주민 축제의 장이 되는 곳', '태그': '강릉단오제 강릉자연관광지 강릉철교 관광지 남대천산책 남대천축제  '}
,{'place': '어답산관광지', 'info': '임금도 들렀던 청정 계곡이 있는 휴양관광지', '태그': '가족과함께 관광지 부모님과함께 아이와함께 연인과함께 자연관광지      '}
,{'place': '아침고요수목원', 'info': '싱그러운 꽃향기가 가득한 수목원', '태그': '17_18한국관광100선 19_20한국관광100선 2024_여행가는달과함께하는_발도장여행지                   '}
,{'place': '의성 사촌마을', 'info': '뛰어난 학자가 많이 배출된 의성지역 양반마을', '태그': '가족여행 경상권 관광지 교과서속여행 아이와함께 역사 역사를품은곳          '}
,{'place': '[북한산 둘레길 12구간] 충의길', 'info': '산세가 아름다운 숲을 보며 걷는 길', '태그': '레포츠 북한산트래킹 서울트래킹 트래킹 트래킹코스'}
,{'place': '마운틴코스터', 'info': '자연 속에서 즐기는 스릴만점 익스트림 스포츠', '태그': '강원권 강원도_여행지_추천 관광지 레포츠 마운틴코스터 액티비티  '}
,{'place': '부여 정림사지 오층석탑 [유네스코 세계유산]', 'info': '백제의 사찰이 있던 터에 있는 장중 하면서도 우아한 5층석탑', '태그': '가족여행 가족체험여행 관광지 교과서속여행 부여 아이와함께 역사         '}
,{'place': '카포레', 'info': '다양한 문화가 있는 숲속의 캐비닛', '태그': '관광지 남녀노소 양평가볼만한곳 양평여행 연인과함께 이색체험 휴식공간    '}
,{'place': '외도 보타니아', 'info': '아름다운 자연을 품고 있는 지상낙원, 외도 보타니아', '태그': '19_20한국관광100선 가족여행 경남 경상권 관광지 데이트장소추천       '}
,{'place': '팔복예술공장', 'info': '폐산업시설에서 예술인의 창작시설로 변신한 곳', '태그': '가볼래터 뉴트로_감성여행 문화공간 문화시설 문화예술 문화예술창작공간            '}
,{'place': '육모정', 'info': '전통 있는 원동향약 관련 유적지', '태그': '관광지 구룡계곡 남원시 여행지 육모정 정자 지리산'}
,{'place': '매헌 윤봉길의사 기념관', 'info': '윤봉길 의사의 삶과 업적을 담은 공간', '태그': '교과서속여행 기념관 매헌윤봉길의사기념관 문화시설 미세먼지_피하는_실내여행                '}
,{'place': '사직단', 'info': '곡신의 신에게 제사를 올리던 조선시대 제단', '태그': '관광지 광주사직단 사직공원 사직단 사직단광주 아이와함께 역사         '}
,{'place': '캠핑라운드', 'info': '아이들과 함께가기 좋은 곳', '태그': '그늘이많은 레포츠 부산시인근 숲속캠핑 야경이예쁜 양산캠핑장 키즈캠핑'}
,{'place': '안지랑 곱창골목', 'info': '대구여행 필수 먹거리, 안지랑 곱창', '태그': '2024_여행가는달과함께하는_발도장여행지 가족여행 경상권 곱창골목           '}
,{'place': '뚝섬한강공원', 'info': '사계절 내내 다양하게 즐길 수 있는 공원', '태그': '25_26한국관광100선 가족과_함께 걷기길 관광지 나들이 도심여행              '}
,{'place': '주작산별소리캠핑장', 'info': '새소리와 물소리, 밤하늘 별들이 가득한 캠핑장', '태그': '강진캠핑장 계곡물놀이 레포츠 물놀이캠핑장 수영장캠핑장 아이와가볼만한곳'}
,{'place': '남산서울타워', 'info': '대를 이어온 데이트의 추억', '태그': '19_20한국관광100선 2024_여행가는달과함께하는_발도장여행지 21_22한국관광100선                         '}
,{'place': '문경도자기박물관', 'info': '전통 있는 문경 도자기와 그 역사를 전시한 박물관', '태그': '가족과함께 남녀노소 문경가볼만한곳 문경도자기전시관 문화시설 미세먼지_피하는_실내여행   '}
,{'place': '김해가야테마파크', 'info': '우리 민족의 찬란한 유산인 가야의 역사를 배우는 놀이공간', '태그': '2024_여행가는달과함께하는_발도장여행지 23_24한국관광100선 가볼래터                '}
,{'place': '클럽프리모', 'info': '아름다운 자연 속 리조트 풍의 글램핑장', '태그': '계곡캠핑장 레포츠 물놀이캠핑장 분위기좋은캠핑장 수영장캠핑장 횡성캠핑장'}
,{'place': '대구제일교회', 'info': '대구광역시에서 가장 오래된 기독교회 건물 대구유형문화유산', '태그': '경상권 관광지 교과서속여행 근대문화유산 남문안예배당 대구기독교역사관                   '}
,{'place': '하늘바다캠핑장', 'info': '여름에 가기 좋은 캠핑장', '태그': '걷기길 낚시 레포츠 바다캠핑 수영장 신안캠핑장 안좌도 안좌도오토캠핑장  '}
,{'place': '하늘구름길캠핑장', 'info': '냇가 그네에서 인생샷 남기기 좋은 캠핑장', '태그': '가을단풍명소 걷기길 레포츠 방갈로 봄꽃여행 산속캠핑장 임실오토캠핑장 '}
,{'place': '백악산', 'info': '자연미를 간직한 백개의 봉우리로 이루어진 산', '태그': '걷기좋은 관광지 괴산가볼만한곳 백개의봉우리 백악산 자연 자연풍경 '}
,{'place': '캠프안단테', 'info': '숲속 캠핑을 즐길 수 있는 곳', '태그': '경상북도캠핑장 레포츠 물놀이캠핑장 벚꽃캠핑 수영장캠핑장 청도캠핑장'}
,{'place': '하느제캠핑장', 'info': '산 내음과 함께하는 캠핑장', '태그': '걷기길 낙안읍성 레포츠 벌교시장 보성여관 보성차밭 보성캠핑장 '}
,{'place': '흥인지문', 'info': '서울성곽 8개의 문 중에서 동쪽에 있는 문', '태그': '가족여행 관광지 교과서속여행 당일치기여행 서울근교여행 서울성곽                 '}
,{'place': '도째비골스카이밸리', 'info': '동해시의 푸른 하늘과 바다를 배경으로 짜릿한 체험을 즐겨보자!', '태그': '2024_여행가는달과함께하는_발도장여행지 23_24한국관광100선 25_26한국관광100선          '}
,{'place': '통제영오토캠핑장', 'info': '한산도 섬에 위치한 조용하고 아늑한 캠핑장', '태그': '레포츠 봉돌해수욕장 여름물놀이 일몰명소 통영캠핑장 통제영캠핑 한산대첩기념비 '}
,{'place': '에코캠프오토캠핑장', 'info': '그림같이 펼쳐지는 풍경 속 달콤한 휴식', '태그': '계곡캠핑장 레포츠 수영장캠핑장 숲속캠핑장 완주캠핑장 전북특별자치도캠핑장'}
,{'place': '경복궁', 'info': '조선 건국 후 창건한 조선왕조 제일의 법궁', '태그': '15_16한국관광100선 17_18한국관광100선 19_20한국관광100선 2024_여행가는달과함께하는_발도장여행지                            '}
,{'place': '신선봉', 'info': '경치 좋은 암릉을 자량하는 괴산 신선봉', '태그': '가족과함께 걷기좋은 관광지 괴산 괴산가볼만한곳 신선봉 자연 자연풍경 '}
,{'place': '피아골오토캠핑장', 'info': '자연을 벗삼아 힐링!', '태그': '걷기길 레포츠 봄꽃여행 여름물놀이 연곡사 오토캠핑장 지리산캠핑  '}
,{'place': '푸른고래카라반', 'info': '푸른 바다, 시원한 파도 소리가 기다리는 감성 캠핑', '태그': '감포항 갯바위낚시 걷기길 레포츠 바다캠핑장 불국사 여름물놀이   '}
,{'place': '여미지식물원', 'info': '희귀식물이 있는 온실과 특색 있는 야외정원을 관람할 수 있는 아름다운 곳', '태그': '2024_여행가는달과함께하는_발도장여행지 가족여행 관광지 꽃여행 데이트코스              '}
,{'place': '영양고추홍보전시관', 'info': '영양고추의 가치를 높이고 이해를 도모하기 위한 공간', '태그': '문화시설 박물관 산업관광 아이와함께 영양가볼만한곳 영양고추 영양고추홍보전시관   '}
,{'place': '태백스피드웨이', 'info': '고원지대에서 스피드 스포츠를 즐기며 일상 탈출하기', '태그': '레포츠 소도야영장 스포츠주행 태백레이싱 태백모터스포츠 태백바이크 '}
,{'place': '돌산공원', 'info': '여수의 모습을 한 눈에 담을 수 있는 돌산공원', '태그': '2024_여행가는달과함께하는_발도장여행지 관광지 돌산공원 돌산대교                '}
,{'place': '관음사(제주)', 'info': '아름다운 제주의 자연을 품은 고즈넉한 사찰', '태그': '관광지 불교 불교문화 불교사찰 사찰 사찰여행 역사 자연속으로       '}
,{'place': '가곡유황족욕체험장', 'info': '발끝에서 시작되는 건강한 힐링', '태그': '가곡유황족욕체험장 가족여행 관광지 삼척가볼만한곳 삼척가족여행 삼척여행  '}
,{'place': '동광리농촌체험마을', 'info': '크고 작은 오름 사이에서 제주 농촌을 체험하는 마을', '태그': '가족여행 관광지 농촌길 농촌여행코스 농촌체험 농촌체험 농촌체험마을      '}
,{'place': '화순 고인돌군 유적 [유네스코 세계유산]', 'info': '유네스코에 등재된 세계 유일의 고인돌 군락지', '태그': '고인돌군유적 관광지 교과서속여행 아이와함께 역사 역사공부 역사관광지             '}
,{'place': '가파도', 'info': '돌담과 어우러져 아름다운 풍경을 자아내는 가파도 청보리', '태그': '가족과함께 가파도 가파도청보리 관광지 산책 산책코스 아이와함께    '}
,{'place': '서운암', 'info': '다양한 야생화와 도자 대장경을 만날 수 있는 사찰', '태그': '가족과함께 관광지 사찰여행 삼천불전 서운암 십육만도자대장경 암자기행   '}
,{'place': '장생포고래박물관', 'info': '고래 인형과 역사를 알 수 있는 박물관', '태그': '고래박물관 쇼핑 울산남구 울산박물관 장생포 장생포고래박물관 장생포박물관'}
,{'place': '내소사(부안)', 'info': '사계절 색다른 아름다움으로 가득한 사찰', '태그': '가족여행 관광지 교과서속여행 내소사 불교문화 사찰&산사 아이와함께          '}
,{'place': '미황사', 'info': '달마산의 암봉을 병풍으로 두른 아름다운 사찰', '태그': '19_20한국관광100선 2024_여행가는달과함께하는_발도장여행지 관광지              '}
,{'place': '경주 첨성대', 'info': '동양 최고(最古)의 천문대', '태그': '2024_여행가는달과함께하는_발도장여행지 21_22한국관광100선 25_26한국관광100선                              '}
,{'place': '도산서원 [유네스코 세계유산]', 'info': '퇴계이황의 학문과 덕행을 기리고 추모하는 도산서원', '태그': '2024_여행가는달과함께하는_발도장여행지 가볼래터 가족여행 관광지                         '}
,{'place': '진하해수욕장', 'info': '잔잔한 파도에 몸과 마음이 힐링 되는 곳', '태그': '2024_여행가는달과함께하는_발도장여행지 가볼래터 가족여행 관광지            '}
,{'place': '몬스터파크 부산명지점', 'info': '대형 에어바운스가 있는 부산의 실내테마파크', '태그': '관광지 대형키즈카페 부산키즈카페 실내테마파크 아이와함께 에어바운스'}
,{'place': '월포해수욕장', 'info': '어종이 풍부해 방파제에서 바다낚시를 즐길 수 있는 해변', '태그': '가족여행 관광지 바다풍경 아이와함께 여름방학가볼만한곳 여름여행      '}
,{'place': '해운대해수욕장', 'info': '사시사철 젊은 열기로 뜨거운 해수욕장', '태그': '15_16한국관광100선 19_20한국관광100선 2024_여행가는달과함께하는_발도장여행지                      '}
,{'place': '다대포해수욕장', 'info': '오랜 풍화작용으로 희고 부드러운 모래가 특징인 다대포해수욕장', '태그': '2024_여행가는달과함께하는_발도장여행지 21_22한국관광100선 가족여행                '}
,{'place': '선유동계곡(문경)', 'info': '하나의 자연 작품을 보는 듯한 계곡', '태그': '계곡 관광지 문경 문경선유동계곡 여름피서지 자연'}
,{'place': '외암민속마을', 'info': '조상의 슬기와 숨결이 살아 숨 쉬는 마을', '태그': '2024_여행가는달과함께하는_발도장여행지 21_22한국관광100선 가족여행                         '}
,{'place': '청주 상당산성', 'info': '천년고도의 위용을 드러내다', '태그': '2024_여행가는달과함께하는_발도장여행지 가족과함께 가족여행 관광지              '}
,{'place': '추성산촌생태마을', 'info': '칠선계곡을 안고 있는 마을', '태그': '관광지 지리산 지리산계곡 천왕봉 칠선계곡 함양여행'}
,{'place': '수옥폭포', 'info': '시원하게 쏟아지는 물줄기로 힐링하는 곳', '태그': '3단폭포 가족여행 관광지 괴산가볼만한곳 수옥폭포 여름가볼만한곳         '}
,{'place': '화암사(고성)', 'info': '천년이 넘는 역사를 간직하고 있는 화암사', '태그': '2024_여행가는달과함께하는_발도장여행지 가족과함께 강원권 고성화암사                 '}
,{'place': '쌍곡구곡', 'info': '웅장한 산에 둘러싸인 자연경관', '태그': '가족여행 경치좋은곳 관광지 괴산가볼만한곳 쌍곡구곡 여름가볼만한곳     '}
,{'place': '백양사', 'info': '사계절 변화 있는 아름다운 경치를 보여주는 백양사', '태그': '2024_여행가는달과함께하는_발도장여행지 가족여행 관광지 교과서속여행              '}
,{'place': '용추폭포(괴산)', 'info': '흰 바위들과 어울려 흐르는 폭포', '태그': '가족과함께 관광지 괴산가볼만한곳 괴산여행 숲트레킹 여름피서지 자연풍경  '}
,{'place': '선유구곡', 'info': '산과 물 바위가 잘 어우러진 절묘한 계곡', '태그': '가족과함께 관광지 괴산가볼만한곳 선유구곡 선유동계곡 여름피서지   '}
,{'place': '하조대해수욕장', 'info': '바다 앞 기암절벽과 하조대 정자가 어우러져 아름다운 절경을 자랑하는 곳', '태그': '가족여행 강원권 강원도_여행지_추천 관광지 기차여행 나들이 당일치기여행               '}
,{'place': '안양예술공원', 'info': '예술작품과 자연이 어우러진 공공예술 특화공원', '태그': '가족여행 공공예술 관광지 나들이 당일치기여행 드라이브여행 서울근교여행         '}
,{'place': '용지호수공원', 'info': '창원의 아름다운 야경을 감상할 수 있는 곳', '태그': '가족과함께 관광지 남녀노소 데이트명소 산책하기 아이와함께 야경명소       '}
,{'place': '광한루원', 'info': '우주관을 표현한 한국 제일의 정원', '태그': '2024_여행가는달과함께하는_발도장여행지 25_26한국관광100선 가볼래터                        '}
,{'place': '괴산향교', 'info': '지방민의 교육을 위해 나라에서 지은 교육기관', '태그': '가족여행 관광지 괴산가볼만한곳 괴산향교 역사 역사관광지 충청권  '}
,{'place': '상효원수목원', 'info': '제주도의 녹색 관광지 만들기 사업 참여 관광지', '태그': '관광지 비밀정원 상효원 스몰웨딩 제주도꽃정원 제주도수국명소 제주도캠핑장 '}
,{'place': '관방제림', 'info': '거대한 풍치림을 이루고 있는 제방', '태그': '2024_여행가는달과함께하는_발도장여행지 25_26한국관광100선 걷기길           '}
,{'place': '함안 말이산 고분군', 'info': '아라가야 문화를 알 수 있는 유적', '태그': '가야고분군 가족여행 경상권 관광지 사적 역사 역사관광지 역사관광지           '}
,{'place': '공림사', 'info': '낙영산과 어우러지는 신라의 천년 사찰', '태그': '공림사 관광지 괴산가볼만한곳 괴산공림사 괴산사찰 불교문화 사찰       '}
,{'place': '휘닉스 평창 루지랜드', 'info': '평창의 자연 속에서 즐기는 카트라이더 !', '태그': '관광지 남녀노소 레포츠 루지랜드 월간_사진_제보_이벤트 이색체험       '}
,{'place': '괴산 둔율 올갱이마을', 'info': '청정놀이가 가득한 둔율 올갱이마을', '태그': '가족여행 관광지 괴산가볼만한곳 농촌마을 농촌체험 올갱이 자연풍경   '}
,{'place': '칠보산(괴산)', 'info': '일곱 개의 봉우리가 보석처럼 아름답다는 칠보산', '태그': '경치좋은곳 관광지 괴산가볼만한곳 괴산칠보산 등산 봉우리 자연  '}
,{'place': '효석문화마을 메밀음식거리', 'info': '소설 속 이야기와 맛있는 전통 음식이 가득한 곳', '태그': '가족과함께 관광지 메밀국수 메밀꽃 메밀밭 메밀음식 메밀전병 봉평막국수       '}
,{'place': '구만서원', 'info': '조선시대 4현의 덕행을 추모하기 위한 서원', '태그': '관광지 구만서원 봉화가볼만한곳 서원 역사 역사관광지 혼자가도좋은'}
,{'place': '봉화향교', 'info': '봉화군민의 유학교육과 교화를 위해 지은 교육기관', '태그': '관광지 봉화가볼만한곳 역사 역사공부 역사관광지 향교 혼자가도좋은'}
,{'place': '노원문화원', 'info': '내일이 기대되는 문화도시를 위한 공간', '태그': '노원구 노원문화원 문화생활 문화시설 복합문화공간 서울가볼만한곳  '}
,{'place': '참샘허브나라', 'info': '농부가 그린 숲속 꿈의 정원', '태그': '관광지 꽃새미마을 농원 밀양가볼만한곳 밀양여행 아이와함께 여름수영장  '}
,{'place': '상운사(고양)', 'info': '수려한 봉우리들에 둘러싸인 북한산의 대표적인 사찰', '태그': '고양상운사 관광지 북한산 북한산사찰 불교 불교문화 사찰 사찰여행            '}
,{'place': '갱매폭포', 'info': '청계산 자연 속에 있는 맑은 폭포', '태그': '관광지 여름피서지 자연 자연여행 자연좋은곳 자연풍경 자연환경  '}
,{'place': '송정제방길', 'info': '사계절 아름다운 길', '태그': '레포츠 산책길 산책로 산책하기 산책하기좋은곳 성동구산책길 성동구산책로     '}
,{'place': '올림픽체조경기장(KSPO DOME)', 'info': '생동하는 문화 공간, 대형공연이 늘 함께하는 곳', '태그': 'K-POP공연장 KSPO DOME 공연장 레포츠 올림픽공연장 올림픽공원콘서트장  '}
,{'place': '익산 미륵사지 [유네스코 세계유산]', 'info': '모든 백성들의 구원을 이루려는 간절한 염원', '태그': '2020 2020한국관광의별 2024_여행가는달과함께하는_발도장여행지 21_22한국관광100선                    '}
,{'place': '오크밸리카라반캠핑장', 'info': '오크밸리에서 즐겨보는 힐링 캠핑', '태그': '럭셔리캠핑 레포츠 리조트시설 바베큐즐기기 불멍하기좋은 오크밸리 '}
,{'place': '무안군 오승우미술관', 'info': '문화예술의 발전을 위한 열린 미술관', '태그': '무안가볼만한곳 무안문화공간 무안여행 무안오승우미술관 문화시설 미술체험교실   '}
,{'place': '캠프ING', 'info': '경북 경주시 강동면에 위치한 캠프 ing', '태그': '가족캠핑여행 경주캠핑장 놀이터캠핑장 레포츠 벚꽃캠핑 아이와가볼만한곳'}
,{'place': '청간정카라반', 'info': '두 눈에 동해를 담고 진정한 쉼을 얻어가는 시간', '태그': '가족캠핑여행 고성가볼만한곳 고성카라반캠핑장 레포츠 바닷가캠핑장 '}
,{'place': '국립해양유산연구소', 'info': '소중한 해양 문화유산을 찾고 알리는 곳', '태그': '가족여행 국립해양박물관 목포가볼만한곳 목포여행 목포해양유물전시관     '}
,{'place': '지리산국립공원(구례)', 'info': '반달가슴곰을 비롯한 다양한 생명들을 품어 키워내는 생명의 산', '태그': '가족여행 관광지 노고단 등산 생태관광지 연중무휴 자연 자연좋은곳    '}
,{'place': '통영편백숲길글램핑캠핑장', 'info': '통영의 화려한 비경과 피톤치드를 즐길 수 있는 캠핑장', '태그': '가족캠핑여행 레포츠 수영장캠핑장 아이들놀기좋은캠핑장 통영캠핑장 '}
,{'place': '강구안', 'info': '상쾌하고 활기찬 항구', '태그': '강구안 경상권 관광지 빛예술밤밤곡곡 연중무휴 자연 항구'}
,{'place': '목포 요트 마리나', 'info': '이국적인 풍경이 멋진 해양레저의 모든 것', '태그': '가족과함께 레포츠 목포가볼만한곳 목포요트마리나 목포요트체험 요트  '}
,{'place': '강진 전라병영성', 'info': '하멜이 8년여 동안을 억류생활 했던 곳', '태그': '관광지 역사 역사공부 역사관광지 역사를품은곳 역사문화재 역사속       '}
,{'place': '신성리 갈대밭', 'info': '영화와 드라마에 등장하여 친숙한 갈대밭', '태그': '2024_여행가는달과함께하는_발도장여행지 가족여행 갈대밭 갈대체험관                   '}
,{'place': '영모정', 'info': '효자 신의연의 효행을 기리고 본받기 위해 세운 누각', '태그': '관광지 누각 문화재 문화재자료 역사 역사공부 역사관광지 역사를품은곳           '}
,{'place': '얼음골까투리오토캠핑장', 'info': '반려견 동반도 가능한 여름철 떠나기 좋은 시원한 캠핑장', '태그': '가족캠핑여행 계곡물놀이캠핑장 그늘좋은캠핑장 나무그늘캠핑장 레포츠 '}
,{'place': '화순적벽', 'info': '철옹산성이 천혜의 장관을 연출하는 곳', '태그': '관광지 역사 역사유적지 전라권 전라남도기념물 화순가볼만한곳'}
,{'place': '진안 학동마을', 'info': '자연과 사람이 어우러진 체험휴양지 씨없는 곶감마을', '태그': '관광지 전북 전북가볼만한곳 전북특별자치도 진안 진안가볼만한곳 학동마을'}
,{'place': '나주 목사내아', 'info': '팽나무가 지키고 있는 나주목사의 살림집', '태그': '고택 관광지 나주가볼만한곳 목사내아 역사 역사관광지 역사유적지   '}
,{'place': '인천학생과학관', 'info': '다양한 분야의 과학과 기술을 체험할 수 있는 과학관', '태그': '가족여행 과학관 문화시설 실내관광지 아이와함께 인천가볼만한곳 인천과학관    '}
,{'place': '갑사', 'info': '삼국시대에 아도화상이 창건한 유서 깊은 사찰', '태그': '계룡산국립공원 관광지 불교 불교사찰 사찰 사찰여행 역사 자연속으로        '}
,{'place': '나주 금성관', 'info': '조선 시대 나주목의 객사 정청', '태그': '가족여행 가족체험여행 관광지 금성관 나주가볼만한곳 아이와함께 역사        '}
,{'place': '이중섭 미술관', 'info': '천재화가 이중섭 화백의 작품과 발자취를 담은 미술관', '태그': '기획전시 나홀로여행 문화시설 미술관 예술 이중섭미술관 제주권     '}
,{'place': '옥구향교', 'info': '배롱나무 꽃이 아름다운 조선시대 향교', '태그': '가족여행 관광지 군산여행 문화재 역사 역사공부 역사관광지 역사를품은곳         '}
,{'place': '능가사', 'info': '바다를 바라보며 마음의 번뇌를 씻다', '태그': '가족여행 관광지 불교 불교문화 사찰 사찰&산사 사찰여행 아이와함께           '}
,{'place': '옥정호 출렁다리 및 붕어섬생태공원', 'info': '휴식을 즐길 수 있는 섬 위의 정원', '태그': '관광지 붕어섬생태공원 옥정호출렁다리 임실'}
,{'place': '테이블에이 앤 로우캣', 'info': '동물들과 교감할 수 있는 실내 체험 동물원', '태그': '관광지 동물먹이주기 송도데이트코스 아이와함께 알파카 이색동물원   '}
,{'place': '관촉사(논산)', 'info': '귀중하고 독특한 유물이 많은 논산의 사찰', '태그': '가족여행 관광지 관촉사 교과서속여행 국보 논산문화재 논산사찰                 '}
,{'place': '금산사(김제)', 'info': '후백제왕 견훤의 한이 서린 1400년 고찰', '태그': '가족여행 관광지 김제여행 모악산 모악산도립공원 사찰&산사 아이와함께         '}
,{'place': '폭풍속으로 촬영지', 'info': '한 폭의 그림 같은 드라마 촬영지', '태그': '2024_여행가는달과함께하는_발도장여행지 SNS감성사진 가족과함께 감성사진                '}
,{'place': '저녁노을미술관', 'info': '푸른 바다정원을 배경으로 붉은 노을이 아름다운 미술관', '태그': '문화공간 문화시설 문화예술공간 신안관광 신안여행 연인과함께 열린문화공간    '}
,{'place': '문방저수지', 'info': '아산의 힐링 명소', '태그': '관광지 낚시장소추천 낚시터 낚시포인트 문방저수지 아산저수지 저수지'}
,{'place': '히어리캠핑장', 'info': '자연과 더불어 숨 쉴 수 있는 자연 친화적 캠핑장', '태그': '계곡캠핑장 김삿갓계곡캠핑장 레포츠 물놀이캠핑장 수영장캠핑장 영월캠핑장'}
,{'place': '대관령솔내음오토캠프장', 'info': '아름다운 자연과 솔내음을 느낄 수 있는 곳', '태그': '가을단풍명소 걷기길 겨울눈꽃명소 대관령양떼목장 대관령오토캠핑장     '}
,{'place': '해돋이도서관', 'info': '자연과 책이 공존하는 해돋이 도서관', '태그': '남녀노소 도서관 독서여행 문화공간 문화시설 복합문화공간 아이와함께  '}
,{'place': '폴라리스야영장', 'info': '푸른 바다가 보이는 전망 좋은 캠핑장', '태그': '고성글램핑 고성오토캠핑 동계캠핑 레포츠 바다캠핑 에어컨 온수제공 '}
,{'place': '난정리 해바라기마을 정원', 'info': '아름다운 해바라기 천국', '태그': '관광지 교동가볼만한곳 교동해바라기축제 난정리해바라기마을 난정리해바라기마을정원   '}
,{'place': '안녕오토캠핑', 'info': '아름다운 바다 경치와 함께 즐기는 캠핑', '태그': '경상남도캠핑장 레포츠 바다뷰오토캠핑장 바다캠핑장 애견동반오토캠핑장 '}
,{'place': '조계사(서울)', 'info': '한국 불교의 중심 사찰', '태그': '관광지 나홀로여행 불교 불교문화 사찰 산사 서울사찰 수도권사찰           '}
,{'place': '아이비캠프오토캠핑장', 'info': '아이들과 함께 감성캠핑을 즐길 수 있는 캠핑장', '태그': '강변캠핑장 레포츠 물놀이캠핑장 밀양캠핑장 숲속캠핑장 아이들과가볼만한곳'}
,{'place': '청령포 (강원고생대 국가지질공원)', 'info': '삼면이 강에 둘러싸여 마치 섬과 같은 곳', '태그': '가족과함께 가족여행 강원고생대_국가지질공원 관광지 교과서속여행                             '}
,{'place': '망미정 우송당', 'info': '용전천 기암절벽 위에 세워진 정자', '태그': '관광지 기암절벽 문화재 산책길 역사 역사공부 역사관광지 역사를품은곳         '}
,{'place': '아리솔캠핑장', 'info': '숲과 계곡 속에서 힐링할 수 있는 캠핑장', '태그': '계곡캠핑장 곡성캠핑장 레포츠 물놀이가능한캠핑장 반려동물동반캠핑장 '}
,{'place': '수리덤오토캠핑장', 'info': '청정 자연환경 속 아늑한 휴식 시간', '태그': '계곡캠핑장 레포츠 물놀이캠핑장 숲길탐방로캠핑장 아이들과가볼만한곳 '}
,{'place': '박수근미술관', 'info': '화가 박수근의 예술혼을 엿볼 수 있는 공간', '태그': 'DMZ 강원권 기차여행 나홀로여행 단체관람 당일치기여행 문화시설            '}
,{'place': '나리농원', 'info': '천만송이 천일홍 축제의 장', '태그': '경기권반려동물여행지 관광지 꽃여행 반려동물 반려동물동반여행지 산업관광  '}
,{'place': '수달캠핑장', 'info': '경북 청송군 안덕면에 위치한 수달캠핑', '태그': '레포츠 수영장캠핑장 아이들과가볼만한곳 잔디캠핑장 청송캠핑장 체험프로그램캠핑장'}
,{'place': '소래바다낚시터', 'info': '도심 속 자연친화적인 바다낚시터', '태그': '가족과함께 낚시 랍스터낚시 레포츠 소래바다 소래바다낚시터 인천가볼만한곳 '}
,{'place': '백담고을오토캠핑장', 'info': '맑고 시원한 계곡과 하나되는 최고의 쉼터', '태그': '가을단풍명소 공기가너무좋은 깊고깨끗한계곡 레포츠 물놀이하기좋은  '}
,{'place': '율곡수목원', 'info': '파주시의 자랑스러운 산림복합문화공간', '태그': '관광지 산책하기좋은곳 자연관광지 자연속으로 자연여행 자연좋은곳      '}
,{'place': '용왕산근린공원', 'info': '서울 시내를 조망할 수 있는 전망이 뛰어난 공원', '태그': '걷기길 관광지 나들이 아이와함께 용왕산근린공원 주말추천여행지 친구와함께'}
,{'place': '비로사(영주)', 'info': '소백산 비로봉 남쪽 기슭에 위치한 통일신라시대 사찰', '태그': '경상권 관광지 교과서속여행 문화재 불교 불교문화 사찰 사찰여행           '}
,{'place': '남양성모성지', 'info': '1866년 병인박해 때 순교한 무명 순교자들의 순교지', '태그': '가볼래터 관광지 산책길 산책하기 산책하기좋은곳 성지 여행구독          '}
,{'place': '덕유산국립공원(본소,적상분소)', 'info': '자연과 균형을 이룬 지상의 낙원, 덕이 많은 너그러운 모산', '태그': '가족여행 관광지 꽃여행 덕유산국립공원 등산 등산로 아이와함께           '}
,{'place': '옛서당캠핑농원', 'info': '넓은 잔디밭과 트리 하우스가 아름다운 캠핑장', '태그': '광주시인근 레포츠 별보기좋은 아이들이놀기좋은 장성캠핑장 폐교캠핑장'}
,{'place': '가리왕산케이블카', 'info': '올림픽의 감동과 가리왕산의 자연이 어우러진 케이블카', '태그': '가리왕산 가리왕산케이블카 가족과함께 강원도 강원도가볼만한곳 강원도여행       '}
,{'place': '보헤미안 박이추커피', 'info': '레전드라 불리는 감성커피공장', '태그': '강릉여행 강릉카페 관광지 남녀노소 로스팅커피 보헤미안박이추커피       '}
,{'place': '천문대오토캠핑장', 'info': '상쾌한 공기와 조용한 산자락에서 즐기는 캠핑', '태그': '경상남도캠핑장 김해캠핑장 레포츠 사이트넓은캠핑장 아이들놀기좋은캠핑장 '}
,{'place': '삼길포항', 'info': '싱싱한 회와 대호방조제의 풍경이 어우러진 항구', '태그': '2024_여행가는달과함께하는_발도장여행지 겨울철새 관광지 낚시 대호방조제            '}
,{'place': '기차왕국박물관카페', 'info': '아이들의 흥미를 자극하는 기차왕국', '태그': '기차박물관 기차왕국박물관 기차왕국박물관카페 모형카페 문화시설 아이와함께   '}
,{'place': '풍전 저수지', 'info': '서산시가지와 가까운 민물낚시 포인트', '태그': '레포츠 서산낚시 서산저수지 서산풍전저수지 저수지 충남낚시 충남저수지'}
,{'place': '영학관광농원야영장', 'info': '자연 속 아늑하고 평화롭게 쉬어가는 공간', '태그': '가족캠핑 레포츠 조용하고깨끗한 창원캠핑장 탁트인뷰 텃밭이있는'}
,{'place': '삼길포 선상어시장', 'info': '서해안의 미항에서 싱싱한 회를', '태그': '서산먹거리 서산볼거리 서산산책 서산시장 서산여행 쇼핑 시장'}
,{'place': '홍지문 및 탕춘대성', 'info': '한양도성과 북한산성이 유기적으로 연결된 도성', '태그': '가족여행 관광지 서울여행 역사 역사관광지 역사탐험 연중무휴 탕춘대성 '}
,{'place': '영천시오감공예체험장', 'info': '자연 속에서 창의력을 키우는 곳', '태그': '100리벚꽃이길 다양한공예체험 대구시근교 뛰어놀기좋은 레포츠 아이랑가기좋은  '}
,{'place': '노송지대', 'info': '푸른 노송들이 줄지어 있는 곳', '태그': '공원 관광지 노송공원 산책 산책로 산책로추천 수원 수원공원 수원공원산책 '}
,{'place': '영천드림랜드오토캠핑장', 'info': '수영장, 눈썰매장이 있는 캠핑장', '태그': '가족과함께 대구시근교 대형수영장 레포츠 불가마찜질 영천캠핑장'}
,{'place': '창원휴양림오토캠핑장', 'info': '다양한 즐거움을 선사하다', '태그': '둘레길캠핑장 레포츠 물놀이가능한캠핑장 수영장캠핑장 숲속캠핑장 창원캠핑장'}
,{'place': '찬경루', 'info': '암반 위에 독특하게 입지한 청송의 누각', '태그': '경상권 경상북도유형문화재 관광지 역사 역사관광지 역사를품은곳 역사문화재      '}
,{'place': '청년미디어타워', 'info': '다양한 문화생활을 할 수 있는 공간', '태그': '문화생활 문화시설 미디어타워 스튜디오 인천가볼만한곳 인천여행 전망대  '}
,{'place': '백도해수욕장오토캠핑장', 'info': '해수욕과 캠핑을 동시에 즐길 수 있는 곳', '태그': '고성캠핑장 넓은데크 레포츠 백도해수욕장캠핑 속초인근 여름에도시원한  '}
,{'place': '청송재래시장', 'info': '청송에서 가장 큰 규모를 자랑하는 오일장', '태그': '가족여행 경상권 나들이 나홀로여행 먹거리 쇼핑 시장맛집 아이와함께       '}
,{'place': '밀양표충오토캠핑장', 'info': '산과 계곡을 충분히 느낄 수 있는 캠핑장', '태그': '관리가잘된캠핑장 깨끗한계곡 레포츠 밀양캠핑장 벚꽃이놀이하기좋은 '}
,{'place': '청도 범곡리 지석묘군', 'info': '세 군데에 나누어져 있는 청동기 시대 지배층의 무덤떼', '태그': '경상권 관광지 기념물 역사 역사공부 역사를품은곳 역사문화재 역사속      '}
,{'place': '미르오토캠핑장', 'info': '자연과 문화가 어우러지는 캠핑장', '태그': '가족캠핑 레포츠 물놀이하기좋은 밀양캠핑장 봄벚꽃캠핑 워터슬라이스 '}
,{'place': '미루밸리캠핑장', 'info': '계곡과 폭포를 품에 안은 청정 지역에서의 힐링 시간', '태그': '가족캠핑 대구시근교 레포츠 산속캠핑 성주캠핑장 시원한계곡이있는'}
,{'place': '개평한옥마을', 'info': '조선시대 한옥 고택이 잘 보존된 마을', '태그': '경상권 고택 관광지 문화재 역사 역사공부 역사관광지 역사를품은곳          '}
,{'place': '그랑블루 글램핑야영장', 'info': '자연과 함께 하는 설렘이 있는 곳.', '태그': '강원도캠핑장 레포츠 서핑 설악해수욕장 애견동반캠핑장 양양군캠핑장  '}
,{'place': '들소리캠핑장', 'info': '어린이들을 위한 키즈 글램핑 시설', '태그': '나주글램핑 나주숙박 나주캠핑 레포츠 무박글램핑 영암숙박 영암캠핑 '}
,{'place': '구성공원', 'info': '성을 축조했던 자리에 위치한 시민공원', '태그': '가족여행 걷기여행 관광지 나들이 산책길 산책하기 산책하기좋은곳          '}
,{'place': '담양 명옥헌 원림', 'info': '백일홍과 연못과 정자가 아름다운 민간정원', '태그': '관광지 담양문화재 담양힐링스팟 명승 명옥헌원림 반려동물 반려동물동반여행지      '}
,{'place': '흑석사(영주)', 'info': '국보와 보물을 품은 의상이 창건하였다는 사찰', '태그': '관광지 불교 불교사찰 사찰 사찰여행 역사 자연속으로 자연좋은곳       '}
,{'place': '밀양알프스캠핑장', 'info': '천혜의 자연조건과 함께 즐기는 캠핑', '태그': '가을단풍 겨울얼음썰매 계곡캠핑 나무그늘캠핑장 레포츠 밀양캠핑장 '}
,{'place': '배재학당역사박물관', 'info': '유용한 인재를 기르고 배우는 집', '태그': '문화시설 배재학당 배재학당역사박물관 서울가볼만한곳 아이와 함께    '}
,{'place': '드림캠핑장', 'info': '뷰가 아름다운 곳', '태그': '낚시 레포츠 보성캠핑장 보성펜션 비봉공룡공원 비봉마리나 수상레저   '}
,{'place': '진월사(영주)', 'info': '대한불교조계종 제16교구 본사인 고운사 말사', '태그': '관광지 불교 불교문화 사찰 사찰여행 역사 자연속으로 자연환경       '}
,{'place': '모명재', 'info': '임진왜란 당시 명나라 장수로 귀화한 두사충을 기리는 사당', '태그': '경상권 관광지 두사충 모명재다례한복 사당모명재 역사 역사공부              '}
,{'place': '천성사', 'info': '문수산에 자리잡은 이화성스님이 세운 근세의 사찰', '태그': '관광지 불교 불교문화 사찰 사찰여행 역사 자연속으로 자연환경       '}
,{'place': '밀양강캠프스쿨', 'info': '울창한 나무 덕분에 그늘이 많은 캠핑장', '태그': '레포츠 밀양캠핑장 부산시인근 아이가놀기좋은 울창한숲속캠핑 유수풀이있는 '}
,{'place': '계양문화회관', 'info': '남녀노소 즐기는 계양 문화예술의 중심지', '태그': '공연장 문화시설 복합문화공간 아이와함께 예술 음악회 인천가볼만한곳   '}
,{'place': '밀양사계절오토캠핑장', 'info': '체험 활동과 캠핑을 동시에 즐기는 곳', '태그': '가족캠핑 계곡물놀이 관리가잘된캠핑장 다양한이벤트가진행되는 레포츠  '}
,{'place': '봉선사(남양주)', 'info': '활짝 핀 연꽃을 볼 수 있는 사찰', '태그': '가족여행 관광지 교과서속여행 당일치기여행 봉선사(경기) 사찰 산사       '}
,{'place': '방호정 감입곡류천', 'info': '부드러운 곡선을 그리며 흘러가는 하천의 모습이 장관', '태그': '관광지 국가지질공원 역사 역사를품은곳 역사문화재 역사속으로 역사탐험        '}
,{'place': '돌하르방캠핑장', 'info': '넓은 잔디밭과 아늑한 캠핑 공간에서 힐링하기', '태그': '레포츠 수영장캠핑 오설록 유리의성 제주도방갈로 제주도캠핑 카라반야영   '}
,{'place': '돌담캠핑장글램핑', 'info': '캠핑하며 즐겨보는 여행', '태그': '나무많은캠핑장 낚시 농장캠핑 레포츠 마산글램핑 마산캠핑장 창원글램핑 '}
,{'place': '갑곶순교성지', 'info': '숭고한 정신을 기리는 순교성지', '태그': 'DMZ여행 가족과함께 갑곶순교성지 강화가볼만한곳 관광지 성지순례   '}
,{'place': '정몽주선생묘', 'info': '고려 말 단심가로 사람들에게 널리 알려진 정몽주 선생의 묘', '태그': '경모사 관광지 모현당 산책코스 아이와가볼만한 역사여행지 정몽주선생묘 '}
,{'place': '덕유캠프농장', 'info': '전북 무주 덕유산 자락에 위치한 오토캠핑장', '태그': '겨울눈꽃명소 덕유산리조트 덕유산캠핑 레포츠 무주반디랜드 무주와인동굴 '}
,{'place': '설악향기로', 'info': '설악의 절경을 담은 힐링 산책로', '태그': '관광지 산책길 설악향기로 속초가볼만한곳 속초산책길 스카이워크 출렁다리'}
,{'place': '놀터캠핑장', 'info': '오션뷰를 바라보고 있는 캠핑장', '태그': '고성오토캠핑장 고성카라반 고성캠핑장 공룡캠핑장 레포츠 바다캠핑장 '}
,{'place': '솔향강릉오토캠핑장', 'info': '안목해변 근처에 있는 캠핑장', '태그': '강릉오토캠핑장 경찰수련원캠핑장 레포츠 바닷가캠핑장 안목해변캠핑장 '}
,{'place': '송하루오토캠핑장', 'info': '다양한 이벤트와 즐거움이 가득한 캠핑장', '태그': '가족캠핑여행 구미캠핑장 레포츠 수영장캠핑장 실내놀이터캠핑장 아이들과가볼만한곳'}
,{'place': '송정솔바람해변캠핑장', 'info': '탁 트인 남해 바다를 한눈에 담을 수 있는 캠핑장', '태그': '남해캠핑장 레포츠 물놀이캠핑장 소나무숲캠핑장 송정솔바람해변캠핑장 '}
,{'place': '회룡사(의정부)', 'info': '회룡사에 깃든 태조 이성계와 무학대사의 이야기', '태그': '관광지 불교 불교문화 사찰 사찰여행 역사 의정부 의정부가볼만한곳            '}
,{'place': '전주 풍남문', 'info': '전주부성의 사대문 중 남쪽에 있는 문', '태그': '가족여행 가족체험여행 관광지 교과서속여행 명견루 아이와함께 역사      '}
,{'place': '슈터스클럽', 'info': '도심 속에서 즐기는 짜릿한 슈팅장', '태그': '강남사격장 강남슈터스클럽 강남에어소프트건샵 레포츠 사격장 실내사격장 '}
,{'place': '[충효길코스 1코스] 고구동산길', 'info': '도심 속 자연과 하나가 될 수 있는 길', '태그': '동작구산책 레포츠 서울산책길 서울트레킹 충효고구동산길 트레킹 트레킹코스'}
,{'place': '오목대와 이목대', 'info': '조선의 역사가 시작된 곳', '태그': '가족여행 가족체험여행 관광지 교과서속여행 문화관광해설사 아이와함께          '}
,{'place': '동덕여자대학교 박물관', 'info': '역사와 전통을 자랑하는 동덕대학교 박물관', '태그': '교과서속여행 동덕여자대학교박물관 문화시설 박물관 사계절 수도권   '}
,{'place': '대구 계산동성당', 'info': '대구 도심에 위치한 영남 최초의 고딕양식 성당', '태그': '2024_여행가는달과함께하는_발도장여행지 경상권 계산성당미사 계산성당야경                      '}
,{'place': '제이퍼블릭컨트리클럽', 'info': '파주의 수려한 자연 환경을 담고 있는 골프장', '태그': '골프 골프리조트 골프장 골프클럽 레저스포츠 레포츠 복합레저스포츠          '}
,{'place': '보현사(청도)', 'info': '황금불상이 있는 사찰', '태그': '경상권 관광지 불교 불교문화 사찰 사찰여행 역사 연중무휴 자연속으로        '}
,{'place': '전주향교', 'info': '조선시대 지방교육의 요람, 전주향교', '태그': '가볼래터 가족여행 관광지 사적 숙박대전_가볼만한곳 숙박대전_추천기사              '}
,{'place': '무의도', 'info': '해안절벽과 기암괴석의 자연경관이 뛰어난 무의도', '태그': '가볼래터 가족여행 관광지 당일치기여행 무의도세렝게티 사진명소 여행구독 '}
,{'place': '솔섬오토캠핑장', 'info': '일몰이 아름다운 캠핑장', '태그': '경상남도 경상남도캠핑장 레포츠 바다캠핑장 사진명소캠핑장 사천시   '}
,{'place': '중앙호수공원 카페거리', 'info': '중앙호수공원에서 커피내음과 함께 여유를 즐기다', '태그': '관광지 산책로 산책하기 산책하기좋은곳 아이와함께 연인과함께 이색거리       '}
,{'place': '수타사', 'info': '아름다운 자연과 소중한 문화재를 품은 강원도 대표 천년고찰', '태그': '가볼래터 강원권 강원도_여행지_추천 관광지 문화관광해설사 사찰      '}
,{'place': '구도선착장', 'info': '잔잔한 바다 풍경을 감상할 수 있는 작은 항구', '태그': '관광지 서산낚시 서산데이트코스 서산산책 서산선착장 서산시 서산여행'}
,{'place': '왕산포구', 'info': '자연 속 다양한 즐거움이 가득한 곳', '태그': '관광지 서산데이트코스 서산산책 서산여행 서산여행가볼만한곳 서산포구 '}
,{'place': '장성호수변공원', 'info': '탁 트인 넓은 시야와 함께 웅장한 장성호의 전경', '태그': '관광지 대한민국대표걷는길 장성군가볼만한곳 장성산책하기좋은곳 장성여행   '}
,{'place': '강화전쟁박물관', 'info': '강화도 항쟁의 역사를 볼 수 있는 전시관', '태그': '강화가볼만한곳 강화전쟁박물관 교육여행 문화시설 아이와함께 역사체험   '}
,{'place': '벌말선착장', 'info': '가로림만 한가운데 선착장에서 즐기는 낚시', '태그': '낚시 레포츠 바다낚시 벌말 벌말항 서산벌말 충청권'}
,{'place': '창조자연사박물관', 'info': '국내 최초 창조론 중심의 자연사박물관', '태그': '교과서속여행 문화시설 박물관 사계절 수도권 실내여행지 아이와함께   '}
,{'place': '대구근대역사관', 'info': '대구 근현대사를 한눈에 조명할 수 있는 공간', '태그': '경상권 교과서속여행 근대문화유산 기념관 나홀로여행 대구근대박물관          '}
,{'place': '경희궁공원', 'info': '고즈넉한 풍경의 공원', '태그': '가족여행 고궁 관광지 교과서속여행 사계절 수도권 아이와함께 연인과함께       '}
,{'place': '자작도캠핑장', 'info': '삼포해변 아래 위치한 해변옆 송림 캠핑장', '태그': '고성군캠핑장 레포츠 여름물놀이 자작도캠핑장 캠핑장'}
,{'place': '속리산풍경펜션&캠핑장', 'info': '물놀이로 한여름 무더위를 날리기 좋은 곳', '태그': '레포츠 속리산캠핑장 속리산풍경캠핑장 속리산풍경펜션 속리산풍경펜션&캠핑장 '}
,{'place': '베니키아 산과바다 주문진리조트', 'info': '주문진 해수욕장 뷰가 예술적인 리조트', '태그': '강릉숙소 관광지 숙박'}
,{'place': '오렌지캠프', 'info': '깔끔함이 돋보이는 전망 좋은 캠핑장', '태그': '레포츠 보은캠핑장 여름물놀이 오렌지캠프 캠핑장'}
,{'place': '월미공원', 'info': '인천 도심 속 아름다운 자연공원', '태그': '가족여행 걷기길 관광지 산책코스 산책하기좋은곳 서울근교여행 수도권       '}
,{'place': '덕동계곡 초록꿈체험마을 오토캠핑장', 'info': '깔끔함이 돋보이는 덕동계곡 인근 캠핑장', '태그': '덕동계곡 덕동계곡초록꿈체험마을오토캠핑장 레포츠 여유있는캠핑장 오토캠핑장  '}
,{'place': '아트팩토리 참기름', 'info': '강화도에서 만나는 예술의 새로운 물결', '태그': '가볼래터 강화가볼만한곳 강화군 문화 문화시설 미디어아트 아트팩토리    '}
,{'place': '명파리해변야영장', 'info': '우리나라 최북단 해수욕장에 위치한 캠핑장', '태그': '고성 고성야영장 레포츠 명파리해변야영장 일출명소'}
,{'place': '정수루', 'info': '옛 조선시대의 나주 관아 누각', '태그': '관광지 광주양림동역사문화마을 역사 역사관광지 역사를품은곳 역사문화재        '}
,{'place': '약목장(3, 8일)', 'info': '참외로 유명한 칠곡의 전통시장', '태그': '가족여행 경상권 기차여행 나들이 나홀로여행 먹거리 쇼핑 시장맛집         '}
,{'place': '낭추골캠핑장', 'info': '썰매장 등 다양한 부대시설을 갖춘 곳', '태그': '낭추골 낭추골캠핑장 레포츠 여름물놀이 청주시 청주시캠핑장 청주캠핑장 '}
,{'place': '스카이리프트', 'info': '서울대공원 하늘 산책', '태그': '과천 과천가볼만한곳 과천서울대공원 레포츠 리프트 서울대공원 스카이리프트'}
,{'place': '더오창호', 'info': '오창호의 풍광이 눈앞에 펼쳐지는 곳', '태그': '글램핑 더오창호 더오창호글램핑 레포츠 일몰명소 청주글램핑'}
,{'place': '강화문화원', 'info': '향토 문화 연구와 문화센터 역할을 수행하는 곳', '태그': '강화가볼만한곳 강화문화원 문화강좌 문화시설 열린문화공간 인천강화미술관   '}
,{'place': '천일염전', 'info': '자연이 인간에게 주는 선물', '태그': '가족여행 경치좋은곳 관광지 생태체험 아이와함께 자연 전라권 천일염전 '}
,{'place': '운봉관', 'info': '군수 하담이 찬경루와 함께 건축한 청송군의 객사', '태그': '관광지 역사 역사공부 역사관광지 역사를품은곳 역사문화재 역사속      '}
,{'place': '청송 홍원리 개오동나무', 'info': '천연기념물로 지정된 노거수', '태그': '개오동나무 관광지 노거수 자연관광지 자연발생 자연속으로 자연여행      '}
,{'place': '명파리해변야영장', 'info': '우리나라 최북단 해수욕장에 위치한 캠핑장', '태그': '고성 고성야영장 레포츠 명파리해변야영장 일출명소'}
,{'place': '정수루', 'info': '옛 조선시대의 나주 관아 누각', '태그': '관광지 광주양림동역사문화마을 역사 역사관광지 역사를품은곳 역사문화재        '}
,{'place': '약목장(3, 8일)', 'info': '참외로 유명한 칠곡의 전통시장', '태그': '가족여행 경상권 기차여행 나들이 나홀로여행 먹거리 쇼핑 시장맛집         '}
,{'place': '낭추골캠핑장', 'info': '썰매장 등 다양한 부대시설을 갖춘 곳', '태그': '낭추골 낭추골캠핑장 레포츠 여름물놀이 청주시 청주시캠핑장 청주캠핑장 '}
,{'place': '스카이리프트', 'info': '서울대공원 하늘 산책', '태그': '과천 과천가볼만한곳 과천서울대공원 레포츠 리프트 서울대공원 스카이리프트'}
,{'place': '더오창호', 'info': '오창호의 풍광이 눈앞에 펼쳐지는 곳', '태그': '글램핑 더오창호 더오창호글램핑 레포츠 일몰명소 청주글램핑'}
,{'place': '강화문화원', 'info': '향토 문화 연구와 문화센터 역할을 수행하는 곳', '태그': '강화가볼만한곳 강화문화원 문화강좌 문화시설 열린문화공간 인천강화미술관   '}
,{'place': '천일염전', 'info': '자연이 인간에게 주는 선물', '태그': '가족여행 경치좋은곳 관광지 생태체험 아이와함께 자연 전라권 천일염전 '}
,{'place': '운봉관', 'info': '군수 하담이 찬경루와 함께 건축한 청송군의 객사', '태그': '관광지 역사 역사공부 역사관광지 역사를품은곳 역사문화재 역사속      '}
,{'place': '청송 홍원리 개오동나무', 'info': '천연기념물로 지정된 노거수', '태그': '개오동나무 관광지 노거수 자연관광지 자연발생 자연속으로 자연여행      '}
,{'place': '목계솔밭', 'info': '푸르른 소나무 군락지가 만들어내는 멋진 풍광', '태그': '관광지 산책명소 소나무군락지 솔밭 자연 청송소나무 힐링명소'}
,{'place': '에스엠비 웰니스 센터', 'info': '나에게 집중하는 진정한 회복의 순간', '태그': '관광지 부산 부산가볼만한곳 뷰티.스파 우수웰니스관광지 웰니스관광 '}
,{'place': '하동 야생차박물관 체험관(야생차체험)', 'info': '경상남도 하동군 화개면에 위치한 차 전문 박물관', '태그': '경남 경상남도 관광지 우수웰니스관광지 하동 하동가볼만한곳 하동군    '}
,{'place': '트리비움', 'info': '나를 더욱 깊게 마주할 수 있는 공간', '태그': '문화공간 가볼래터 건강 건강한 경기도 경기도관광 공간 공연장대관                         '}
,{'place': '금능해수욕장 야영장', 'info': '금능해변 야자수 아래서 이국적 캠핑', '태그': '금능해수욕장야영장 레포츠 일몰명소 제주도캠핑 제주도캠핑장 제주캠핑 '}
,{'place': '대흥사 [유네스코 세계유산]', 'info': '호국과 차의 성지인 천년사찰', '태그': '가족여행 관광지 교과서속여행 대흥사 불교 불교문화 불교사찰 사찰                     '}
,{'place': '해비치 호텔&리조트 제주', 'info': '제주 자연 그대로를 품은 종합휴양리조트', '태그': '관광지 서귀포숙소 숙박 스테이 우수웰니스관광지 장흥 장흥가볼만한곳        '}
,{'place': '약석원', 'info': '강화도 사자발쑥이 선사하는 건강한 힐링코스', '태그': '관광지 뷰티.스파 스테이 우수웰니스관광지 인천 인천가볼만한곳 자연치유   '}
,{'place': '전라남도 마음건강치유센터', 'info': '몸과 마음을 돌보는 한방 기반 웰니스 체험', '태그': '관광지 스테이 우수웰니스관광지 웰니스관광지 자연치유 전남 전남가볼만한곳    '}
,{'place': '신성리 공룡발자국', 'info': '공룡발자국 화석의 모습을 볼 수 있는 곳', '태그': '경상권 관광지 국가지질공원 신성계곡 역사공부 역사를품은곳 역사속         '}
,{'place': '청송 덕천마을', 'info': '한옥스테이와 전통문화 체험을 할 수 있는 농촌 마을', '태그': '관광지 남녀노소 아이와함께 자연마을 자연여행 자연좋은곳 자연환경       '}
,{'place': '산청 배산서원', 'info': '이원과 죽각 이광우를 모시는 서원', '태그': '경상권 관광지 배산서원 산청가볼만한곳 산청배산서원 역사 역사관광지  '}
,{'place': '국립신시도자연휴양림', 'info': '해, 별, 달을 품은 친환경 휴식처', '태그': '관광지 군산여행 바다경치 바다내음 신시도 자연속으로 자연여행       '}
,{'place': '소헌공원', 'info': '문화재가 있는 공원', '태그': '공원 공원나들이 관광지 문화재 역사문화 청송가볼만한곳 청송군관광지'}
,{'place': '팔영산 편백 치유의 숲', 'info': '천혜의 자연경관과 편백숲의 힐링공간', '태그': '가족과함께 고흥가볼만한곳 고흥여행 관광지 자연풍경 팔영산편백숲  '}
,{'place': '생말공원', 'info': '군산 시민들을 위한 휴식 및 체력 증진의 공간', '태그': '관광지 군산공원 군산근린공원 군산축구장 산책공원 산책로 생말공원 '}
,{'place': '네이스트 프라이빗', 'info': '풍경 멋지고 깔끔한 글램핑&캠핑장', '태그': '글램핑 레포츠 여유있는캠핑장 음성캠핑장 인기캠핑장2024 충북캠핑장 '}
,{'place': '삼척해상케이블카', 'info': '케이블카를 타고 에메랄드빛 바다 위 여행', '태그': '가족과함께 강원권 관광지 데이트코스 삼척가볼만한곳 삼척해상케이블카     '}
,{'place': '에프앤에프 캠핑', 'info': '계곡 물놀이가 가능한 깔끔한 캠핑장', '태그': '레포츠 여름물놀이'}
,{'place': '왕산·필봉산', 'info': '다양한 산행코스를 즐길 수 있는 산', '태그': '가족과함께 경치좋은곳 관광지 산청가볼만한곳 산청여행 왕산필봉산   '}
,{'place': '충주 비내오토캠핑장', 'info': '수목이 우거진 감성적인 캠핑장', '태그': '레포츠 여름물놀이'}
,{'place': '하비비캠핑장', 'info': '탄금호가 눈앞에 펼쳐지는 캠핑 바비큐장', '태그': '레포츠 여름물놀이'}
,{'place': '남해 노랑카라반 & 고카트', 'info': '한려수도 앵강만의 아름다운 경치와 함께 신나는 레이싱을 할 수 있는 곳', '태그': '가볼래터 남해 레포츠 액티비티여행 여행구독 카라반 카트체험'}
,{'place': '실상사(남원)', 'info': '지리산이 연꽃처럼 감싸안은 절', '태그': '가족여행 관광지 교과서속여행 남원실상사 불교 불교문화 사찰 사찰&산사              '}
,{'place': '대저수문 생태공원', 'info': '자연과 어우러진 공원', '태그': '가족나들이 관광지 낙동강 대저수문 대저수문생태공원 생태공원 피크닉'}
,{'place': '담양금성오토캠핑장', 'info': '금성산성에 자리한 아늑한 휴식처', '태그': '가을단풍명소 금성산성 담양오토캠핑 담양캠핑장 레포츠 수상레저 오토캠핑장 '}
,{'place': '달천공원오토캠핑장', 'info': '천주산 자락의 기운을 마음껏 느낄 수 있는 천혜의 자연 쉼터', '태그': '걷기길 레포츠 봄꽃여행 여름물놀이 일몰명소 일출명소 창원시티투어 '}
,{'place': '백운계곡캠핑장', 'info': '숲과 계곡 속에서 보내는 힐링의 시간', '태그': '경상남도캠핑장 레포츠 백운계곡둘레길 산청군캠핑장 여름물놀이 이동갈비'}
,{'place': '풍각장 (1, 6일)', 'info': '풍각 지역 상권의 중심지 역할을 해오던 곳', '태그': '가족여행 경상권 기차여행 나들이 나홀로여행 먹거리 쇼핑 시장맛집       '}
,{'place': '백경펜션오토캠핑장', 'info': '포항에서 사시사철 즐기는 캠핑', '태그': '가족캠핑 낚시가능 레포츠 바닷가캠핑 일출명소 파도소리들으며 포항캠핑장 '}
,{'place': '아트앤체험캠프', 'info': '가족단위 함께 즐길 수 있는 놀잇감이 많은 캠핑장소', '태그': '가을단풍명소 걷기길 낚시 액티비티 레포츠 봄꽃여행'}
,{'place': '산청 법계사 삼층석탑', 'info': '천왕봉 동쪽 중턱에 자리잡은 법계사의 석탑', '태그': '경남권 관광지 법계사 사찰여행 산청가볼만한곳 삼층석탑 역사 역사관광지 '}
,{'place': '성혈사(영주)', 'info': '소백산 기슭에 의상이 창건한 사찰', '태그': '경상권 관광지 불교 불교문화 사찰 사찰여행 성혈사 역사 연중무휴         '}
,{'place': '무주수목원캠핑장', 'info': '편하게 즐겨보는 캠핑', '태그': '겨울눈꽃명소 덕유산 레포츠 무주구천동계곡 수목원캠핑장 스키장인근 '}
,{'place': '무릉도원오토캠핑장', 'info': '300년 된 소나무들이 있는 자연 그대로의 캠핑장', '태그': '계곡이있는 깨끗하고관리잘된 대구시근교캠핑장 레포츠 벚꽃명소 조용한캠핑장'}
,{'place': '보정동카페거리', 'info': '이국적인 분위기의 카페 거리', '태그': '관광지 당일치기여행 데이트코스 보정동카페거리 브런치카페 서울근교여행       '}
,{'place': '교동도 대룡시장', 'info': '황해도 연백시장의 모습을 재현한 골목시장', '태그': '2024_여행가는달과함께하는_발도장여행지 가볼래터 가족여행 강화가볼만한곳                 '}
,{'place': '경포해수욕장', 'info': '울창한 송림병풍과 해변의 조화가 아름다운 곳', '태그': '2024_여행가는달과함께하는_발도장여행지 강릉경포해수욕장 강원권 강원도_여행지_추천             '}
,{'place': '금강소나무생태경영림', 'info': '소나무 피톤치드를 맡을 수 있는 자연생태탐방로', '태그': '500년대왕소나무 경북자연생태탐방로 관광지 금강소나무숲 본신리금강소나무숲  '}
,{'place': '숲속애캠핑장', 'info': '경치가 아름다운 함양 캠핑장', '태그': '경상남도캠핑장 눈썰매장캠핑장 레포츠 물놀이가능한캠핑장 수영장캠핑장 '}
,{'place': '수풀로 영문리', 'info': '누구나 걷기 좋은 산책 공간', '태그': '관광지 맨발걷기 산책코스 생태공원 생태복원지 생태체험 수풀로        '}
,{'place': '선바위관광지', 'info': '낙동정맥을 따라 흐르는 사계절 맑은 물과 환상의 절경', '태그': '가족과함께 가족체험여행 관광지 나무와옻칠협동조합 산업관광 아이와함께    '}
,{'place': '물안개오토캠핑장', 'info': '천혜의 자연환경 속 편안한 휴식', '태그': '대구시근교캠핑 레포츠 물놀이하기좋은 물멍하기좋은 밀양캠핑장 뷰가좋은캠핑'}
,{'place': '운암뜰 한식거리', 'info': '다양한 향토음식을 맛볼 수 있는 먹거리 단지', '태그': '가족과함께 건강식맛집 관광지 맛집 맛집탐방 숨은맛집 연인과함께        '}
,{'place': '영양 주실마을', 'info': '진취적인 문화를 간직한 매우 유서 깊은 마을', '태그': '관광지 역사 역사공부 역사관광지 역사를품은곳 역사문화 역사문화재         '}
,{'place': '웅석봉군립공원', 'info': '지리산을 가장 잘 볼 수 있는 산', '태그': '가족과함께 경치좋은곳 관광지 군립공원 산청가볼만한곳 웅석봉 자연  '}
,{'place': '영양 두들마을', 'info': '언덕 위에 있는 마을', '태그': '가족체험 관광지 농촌마을 농촌여행 농촌체험 아이와함께 역사 역사관광지         '}
,{'place': '더하루캠핑파크', 'info': '지친 일상에서 벗어나 색다른 휴식을 취할 수 있는 곳', '태그': '걷기길 레포츠 마산글램핑 마산캠핑장 수상레저 액티비티 여름물놀이  '}
,{'place': '오창 에딘버러CC', 'info': '12만평 규모의 자연친화적인 골프 클럽', '태그': '골프관광 골프여행 골프장 국내골프장 레포츠 오창에딘버러CC 청주 '}
,{'place': '봉화 북지리 마애여래좌상', 'info': '부처님의 위엄을 지닌 신라시대 거대 석불', '태그': '경상권 관광지 국보 봉화북지리마애여래좌상 역사 역사문화재 역사속으로 '}
,{'place': '봉평재래시장 (2, 7 일)', 'info': '자리를 묵묵히 지켜온 뚝심 있는 장터', '태그': '2024_여행가는달과함께하는_발도장여행지 가족여행 강원권 강원웰니스관광지                   '}
,{'place': '문장대오토캠핑장', 'info': '청량감 가득한 자연친화적 캠핑을 즐겨보자', '태그': '가족캠핑 레포츠 물놀이하기좋은 상주캠핑장 용유계곡 장각폭포'}
,{'place': '봉화산사', 'info': '연꽃밭이 인상적인 작은 사찰', '태그': '경북여행 관광지 봉화가볼만한곳 봉화사찰 봉화산사 봉화여행 봉화홍도라지조청'}
,{'place': '문경히든벨리오토캠핑장', 'info': '물놀이와 캠핑을 함께', '태그': '가족캠핑 계단식사이트 둔덕산 레포츠 문경새재 문경캠핑장 조용한캠핑장'}
,{'place': '석천계곡', 'info': '태백 산지에서 발원한 물이 문수산 자락에 흘러온 계곡', '태그': '가족여행 경상권 경치좋은곳 계곡 관광지 단풍명소 명승 봉화계곡        '}
,{'place': '문경수예리오토캠핑장', 'info': '시원한 폭포와 함께 즐기는 캠핑', '태그': '레포츠 문경새재 문경캠핑장 사이트가넓은 어린이가놀기좋은 키즈캠핑 '}
,{'place': '우도마을', 'info': '소가 누워있는 형상을 하고 있는 제주의 가장 큰 부속섬', '태그': '관광지 우도 우도관광지 자연 제주 제주가볼만한곳 제주관광 제주관광지 '}
,{'place': '백운동계곡(산청)', 'info': '조식의 체취가 많이 남아있는 계곡', '태그': '가족과함께 경상권 계곡 관광지 산청가볼만한곳 산청백운동계곡 여름피서지  '}
,{'place': '대성글램핑', 'info': '계절마다 다양한 볼거리가 있는 캠핑장', '태그': '가을단풍명소 걷기길 구미캠핑장 대구시캠핑장 레포츠 여름물놀이 칠곡캠핑장'}
,{'place': '대가야캠프타운', 'info': '대자연이 선사하는 즐거운 휴식', '태그': '경상북도카라반 고령대가야체험축제 고령대장간 대가야수목원 대가야역사테마관광지    '}
,{'place': '족은노꼬메오름', 'info': '숲길과 오름을 함께 즐길 수 있는 곳', '태그': '궷물오름 노꼬메 노꼬메오름 레포츠 오름여행 오름추천 제주가볼만한곳      '}
,{'place': '대충캠핑장', 'info': '고성의 아름다운 광경을 누릴 수 있는 캠핑장', '태그': '가을단풍명소 걷기길 고성캠핑장 레포츠 봄꽃여행 여름물놀이 천왕산 '}
,{'place': '춘양목송이 정보화마을', 'info': '즐거운 체험으로 가득한 경북 최북단에 있는 마을', '태그': '관광지 농촌 농촌체험 농촌체험마을 농촌체험여행 농촌체험학습 아이와함께       '}
,{'place': '보리사(경주)', 'info': '남산에서 가장 규모가 큰 통일신라시대 사찰', '태그': '관광지 배반동고찰 보리사마애석불 불교 불교문화 비구니스님 사찰           '}
,{'place': '당항포오토캠핑장', 'info': '낭만이 가득한 오토 캠핑장', '태그': '가을단풍명소 걷기길 경상남도고성공룡세계엑스포 당항포관광지 당항포요트    '}
,{'place': '경주 정혜사지 십삼층석탑', 'info': '정혜사터에 남은 특이한 형태의 통일신라 석조 불탑', '태그': '13층석탑 경상권 경주가볼만한곳 경주석탑 관광지 국보 역사 역사문화재       '}
,{'place': '경주 성동시장', 'info': '한식뷔페촌이 있어 맛있는 음식을 저렴하게 즐길 수 있는 성동시장', '태그': '가족여행 경상권 경주성동시장 나들이 나홀로여행 데이트코스 먹거리           '}
,{'place': '하동케이블카', 'info': '청정 남해 한려해상국립공원의 절경을 한눈에 볼 수 있는 하동 최고의 랜드마크', '태그': '가족과갈만한곳 관광지 역사탐방 하동케이블카 한려해상 힐링여행'}
,{'place': '달궁자동차야영장', 'info': '청정 지리산 속에서 즐기는 휴식', '태그': '가을단풍명소 국립공원캠핑 레포츠 산속캠핑장 지리산야영장 지리산카라반 '}
,{'place': '하동 편백자연휴양림', 'info': '빽빽한 편백나무 숲에서 피톤치드를 마음껏 들이마시자!', '태그': '경치여행 경치좋은곳 관광지 산책하기좋은곳 자연속으로 자연좋은곳     '}
,{'place': '도심다원', 'info': '녹차밭 카페로 유명한 하동 힐링지', '태그': '가볼래터 가족체험 경치좋은곳 관광지 녹차밭 농촌체험 사진찍기             '}
,{'place': '솔내음물댄동산', 'info': '다양한 부대시설이 있는 솔내음물댄동산', '태그': '가족캠핑여행 광주시근교캠핑장 레포츠 물놀이가능한캠핑장 아이들과가볼만한곳 '}
,{'place': '봉화 서동리 동·서 삼층석탑', 'info': '한쌍이 마주보고 있는 통일신라 석탑', '태그': '경상권 관광지 문화재 서동리동서삼층석탑 신라시대유물 쌍둥이탑 역사       '}
,{'place': '한라산둘레길 돌오름길 (보림농장삼거리~거린사슴오름)', 'info': '모두가 자연의 신비를 느낄 수 있는 환상적인 숲길', '태그': '걷기 걷기여행길 둘레길 레포츠 산책 제주 제주가볼만한곳'}
,{'place': '국제리더스클럽', 'info': '제주바다를 즐길 수 있는 사계절 해양레포츠 테마파크', '태그': '국제리더스클럽 레포츠 반잠수정 윈드서핑 제주도반잠수함 제주도아이들과가볼만한곳     '}
,{'place': '천아숲길［한라산둘레길 1구간］', 'info': '가을 단풍이 빼어나게 아름다운 트레킹 코스', '태그': '레포츠 산책길 산책로 산책하기 산책하기좋은곳 자연속으로 자연환경         '}
,{'place': '용담바다낚시터', 'info': '바다가 바로 옆에 있어 청정한 수질의 낚시터', '태그': '가족과함께 낚시체험 레포츠 바다낚시터 옹진가볼만한곳 용담바다낚시터  '}
,{'place': '운암사(문경)', 'info': '마르지 않는 샘 안의 돌부처가 암자가 된 사찰', '태그': '관광지 불교 불교문화 사찰 사찰여행 역사 자연속으로 자연환경       '}
,{'place': '위봉사(완주)', 'info': '봉황이 품고 있는 거대한 사찰', '태그': '관광지 문화관광해설사 불교 불교사찰 사찰 사찰여행 역사 자연속으로        '}
,{'place': '연동어촌체험마을', 'info': '아이와 함께 가기 좋은 경주 동해안 어촌마을', '태그': '가족과함께 경주어촌체험 관광지 남녀노소 어촌체험 연동마을 염동마을          '}
,{'place': '보문숲머리먹거리촌', 'info': '다양한 먹거리를 즐길 수 있는 봄꽃 명소', '태그': '경북권 경상북도_여행지_추천 경주가볼만한곳 관광지 먹거리촌 벚꽃길 '}
,{'place': '삼부연폭포 (한탄강 유네스코 세계지질공원)', 'info': '철원 9경에 속하는 아름다운 폭포', '태그': '강원권 관광지 국가지질공원 세계지질공원 자연 자연관광지 자연속으로        '}
,{'place': '지리산국립공원(산청)', 'info': '가장 넓은 면적의 산악형 국립공원', '태그': '가족과함께 관광지 국립공원 산청가볼만한곳 야영장 자연 자연관광지  '}
,{'place': '리썸카라반', 'info': '감성 가득한 캠핑', '태그': '가족 깨끗한 레포츠 사이트 간격이 넓은 아이들 놀기 좋은 여유있는      '}
,{'place': '연화도(통영)', 'info': '용머리바위와 수국이 있는 아름다운 섬', '태그': '2021_휴가철찾아가고싶은_33섬 가볼래터 가족과함께 관광지 수국명소            '}
,{'place': '송악산둘레길', 'info': '산방산과 한라산, 마라도를 모두 조망할 수 있는 제주둘레길', '태그': '레포츠 반려동물 반려동물동반여행지 서귀포애견동반여행지 송악산전망대      '}
,{'place': '사성암(구례)', 'info': '아슬아슬한 기암절벽을 벽 삼아 서 있는 부처의 세계', '태그': '2024_여행가는달과함께하는_발도장여행지 건축여행 경치좋은곳 관광지                             '}
,{'place': '수협 활어직판장', 'info': '싱싱함을 그대로', '태그': '경주가볼만한곳 경주수산시장 경주시가볼만한곳 경주여행 쇼핑 수협활어직판장 '}
,{'place': '산청 단성향교', 'info': '독특한 구성을 보여주는 지방민 교육 관리 향교', '태그': '관광지 교육기관 산청가볼만한곳 산청단성향교 산청여행 역사 역사관광지  '}
,{'place': '창꼼', 'info': '구멍난 돌을 통해 사진을 찍는 인증샷 명소', '태그': '관광지 우영우촬영지 제주도 제주도가볼만한곳 제주도창꼼 창꼼 창꼼바위'}
,{'place': '하동 범왕리 푸조나무', 'info': '기념물로 지정된 500년 수령의 푸조나무', '태그': '경남범왕리푸조나무 경남푸조나무 경상권 경상남도기념물 관광지 자연       '}
,{'place': '강화 삼랑성', 'info': '단군왕검의 세 아들이 세운 삼랑성', '태그': '강화가볼만한곳 강화삼랑성 강화정족산성 관광지 삼랑성(정족산성) 서울근교여행             '}
,{'place': '하동 삼신녹차마을', 'info': '향긋한 녹차향이 가득 차있는 마을', '태그': '가족체험 가족체험여행 경치좋은곳 관광지 녹차 녹차밭 농촌체험           '}
,{'place': '고배기동산', 'info': '제주도 한라산 아래 첫 마을', '태그': '걷기 걷기여행길 고배기동산 관광지 동산 제주 제주가볼만한곳 제주문화'}
,{'place': '가탄마을', 'info': '신선이 살면서 아름다운 여울에 낚싯대를 담갔다 하여 가탄', '태그': '가탄마을 관광지 십리벚꽃길 지리산둘레길 지리산둘레길14구간 지리산둘레길15구간  '}
,{'place': '섬진강기차마을', 'info': '옛 역사를 품고있는 곡성 기차마을', '태그': '19_20한국관광100선 2014 2024_여행가는달과함께하는_발도장여행지                  '}
,{'place': '불란지야시장', 'info': '중문 오일장 초입에 자리하고 있는 곳', '태그': '불란지야시장 쇼핑 제주가볼만한곳 제주관광명소 제주먹거리 제주야시장   '}
,{'place': '중산리계곡', 'info': '백운산 천왕봉에서 비롯된 계곡', '태그': '가족과함께 계곡 관광지 산청가볼만한곳 여름피서지 자연 자연관광지  '}
,{'place': '술상 전어마을', 'info': '싱싱하고 맛있는 전어가 가득 잡히는 곳', '태그': '관광지 술상전어마을 술상전어축제 전어마을 하동군 하동어촌마을 하동전어'}
,{'place': '서남신시장', 'info': '먹거리가 가득한 대구의 시장', '태그': '대구나들이 대구맛집 대구시장 대구여행 대구족발 쇼핑'}
,{'place': '대원사계곡길', 'info': '지리산 단풍을 보며 걷기 좋은 트레킹 코스', '태그': '국립공원 단풍명소 대원사 레포츠 산청가볼만한곳 산청계곡길 산청트레킹   '}
,{'place': '대명시장', 'info': '친절하고 행복한 대명시장', '태그': '대구나들이 대구쇼핑 대구시장 대구시장여행 대구여행 쇼핑'}
,{'place': '덕산사(산청)', 'info': '계절마다 아름다운 분위기를 뽐내는 사찰 덕산사', '태그': '관광지 덕산사 불교사찰 사찰여행 산청가볼만한곳 역사 역사관광지      '}
,{'place': '거림계곡', 'info': '지리산 여러 지류가 모여 이룬 아름다운 계곡', '태그': '관광지 산청가볼만한곳 산청거림계곡 산청여행 여름피서지 자연 자연풍경   '}
,{'place': '황매산(산청)', 'info': '사계절 다채로움이 끝없이 펼쳐지는 산', '태그': '가족과함께 가족여행 경상권 관광지 등산 산청가볼만한곳 산청황매산   '}
,{'place': '초월생활체육공원', 'info': '일상생활에서 즐기는 생활체육공원', '태그': '경기광주체육공원 경기도체육공원 곤지암천 관광지 너른광주체육공원  '}
,{'place': '대황강출렁다리', 'info': '곡성을 내다볼 수 있는 출렁다리', '태그': '가족과함께 경치좋은곳 관광지 산책코스 이색체험 출렁다리 트래킹코스'}
,{'place': '청석공원', 'info': '경안천 옆에 자리한 광주시민의 쉼터', '태그': '경기광주체육공원 경기도체육공원 관광지 너른광주체육공원 산책코스  '}
,{'place': '동정호', 'info': '아름다운 풍경 즐기며 하염없이 걷기 좋은 호숫가', '태그': '경치좋은곳 관광지 사진찍기좋은곳 산책길 산책하기 산책하기좋은곳     '}
,{'place': '중대물빛공원', 'info': '시민들에게 편안한 휴식공간', '태그': '경치좋은곳 관광지 남녀노소 산책길 산책로 산책하기 산책하기좋은곳         '}
,{'place': '광주시 호국보훈공원', 'info': '나라를 위한 희생을 기억하고 일상 속 활력을 주는 도심 속 휴식처', '태그': '가벼운걷기 경안공원31기념탑 경안공원현충탑 관광지 산책 산책하기좋은곳  '}
,{'place': '통영요트학교', 'info': '통영 바다 위에서 즐기는 특별한 요트 체험', '태그': '레포츠 바다풍경 요트 요트관광 이색체험 자연관광 친구와함께 통영 '}
,{'place': '상의자동차야영장', 'info': '안전하고 쾌적한 캠핑을 할 수 있는 자동차 야영장', '태그': '가족여행 남녀노소 레포츠 야영장 야외캠핑장 캠핑 캠핑장 휴식공간   '}
,{'place': '서울숲', 'info': '숲과 문화가 어우러지는 서울의 대표적인 녹색쉼터', '태그': '2024_여행가는달과함께하는_발도장여행지 23_24한국관광100선 25_26한국관광100선                    '}
,{'place': '무갑리계곡', 'info': '시원한 여름을 보내는 계곡', '태그': '경기도광주계곡 관광지 물놀이계곡 물놀이하기좋은곳 서을근교계곡 초월읍계곡'}
,{'place': '호국생활체육공원', 'info': '다양한 체육활동을 즐길 수 있는 공원', '태그': '곤지암체육시설 관광지 광주체육시설 주민들휴식처 체육공원'}
,{'place': '다온숲', 'info': '좋은 모든 일들이 다 온다는 뜻의 즐거움 가득한 글램핑 공간', '태그': '레포츠 바베큐장 세종시캠핑장 여름물놀이 워터풀 전동면캠핑장'}
,{'place': '고천체육공원', 'info': '의왕시민들의 여가 활동 최적지', '태그': '공원 관광지 물방울놀이터 산책 운동 의왕시공원 체육공원 힐링'}
,{'place': '분옥담과 선녀탕', 'info': '자연환경이 잘 지켜지고 있는 자연의 욕조', '태그': '관광지 분옥담 선녀탕 자연 전라권 풍경여행'}
,{'place': '내소힐링캠프', 'info': '산과 바다 그리고 계곡을 동시에 즐길 수 있는 곳', '태그': '격포해수욕장 계곡캠핑장 낚시 내소사 레포츠 바다캠핑장 산캠핑장 '}
,{'place': '애기봉평화생태공원', 'info': '민족의 한과 통일의 염원이 깃든 애기봉', '태그': '경치여행 경치좋은곳 관광지 수도권 안보관광 애기봉 자연 자연좋은곳   '}
,{'place': '남해초전마을캠핑장', 'info': '작은 캠핑장에서 보내는 평화롭고 아늑한 시간', '태그': '가을단풍명소 낚시 남해캠핑장 레포츠 봄꽃여행 수상레저 여름물놀이  '}
,{'place': '꿈애오토캠핑장', 'info': '도심에서 벗어나 좋은 기운을 느끼자', '태그': '가을단풍명소 걷기길 구미오토캠핑장 대구시오토캠핑장 레포츠 봄꽃여행  '}
,{'place': '국립인천해양박물관', 'info': '바다로 열어가는 미래 공간 인천해양박물관', '태그': '가볼래터 국립해양박물관 문화시설 아이와함께 여행구독 월미도 인천가볼만한곳  '}
,{'place': '내리계곡솔밭캠핑장', 'info': '울창한 소나무 아래 깨끗한 계곡과 시원하게 캠핑 즐기기', '태그': '김삿갓캠핑장 내리계곡 레포츠 여름물놀이 영월물놀이 영월캠핑장'}
,{'place': '내장야영장', 'info': '있는 그대로의 자연 속에서 즐기는 힐링', '태그': '가을단풍명소 걷기길 국립공원야영장 내장산 내장산야영장 레포츠'}
,{'place': '영월 섶다리 마을', 'info': '섶다리를 만날 수 있는 이색체험', '태그': '가족여행 강원권 강원권반려동물여행지 경치좋은곳 관광지 반려동물                 '}
,{'place': '종현농어촌체험휴양마을', 'info': '자연과 함께 다양한 체험을 즐길 수 있는 마을', '태그': '가족과함께 가족여행 갯벌체험 관광지 서울근교여행 수도권 아이와함께      '}
,{'place': '강원특별자치도 세계잼버리 수련장', 'info': '천혜의 자연경관을 가지고 있는 사계절 수련원', '태그': '2024_여행가는달과함께하는_발도장여행지 강원권 강원권반려동물여행지                '}
,{'place': '담양 소쇄원', 'info': '우리나라의 대표적인 민간 원림', '태그': '2024_여행가는달과함께하는_발도장여행지 가족여행 관광지 교과서속여행        '}
,{'place': '수우원', 'info': '블루베리에 관한 다양한 체험을 할 수 있는 농원', '태그': '가족과함께 경기도농장 관광지 블루베리 블루베리농장 아이와함께 이색체험          '}
,{'place': '홍난파 생가', 'info': '우리나라 현대음악의 여명기를 개척한 난파 홍영후의 생가', '태그': '관광지 역사 역사공부 역사관광지 홍난파선생 홍영후'}
,{'place': '솔가람 아트홀', 'info': '최상의 음향시설로 즐기는 클래식공연', '태그': '경기권 공연장 문화시설 솔가람아트홀 운정신도시공연장 클래식공연장  '}
,{'place': '팔당호', 'info': '한적하게 시간을 보내기 좋은 팔당호', '태그': '경치좋은곳 관광지 산책길 산책하기 산책하기좋은곳 자연 자연속으로     '}
,{'place': '내장산국민여가캠핑장', 'info': '한적한 분위기 속 캠핑할 수 있는 곳', '태그': '가을단풍명소 겨울눈꽃명소 국립공원야영장 내장산 내장산캠핑 레포츠 '}
,{'place': '서울액션스쿨 (마샬아트센터)', 'info': '세계적인 액션의 진수를 배우는 액션스쿨', '태그': '남녀노소 레포츠 마샬아트센터 문화시설 서울액션스쿨 수도권 연인과함께      '}
,{'place': '산속호수마을', 'info': '파로호를 품은 고요한 산속호수마을 동촌리', '태그': '강원권 관광지 농촌마을 농촌여행 농촌체험 농촌체험마을 농촌체험여행      '}
,{'place': '설뫼충효테마파크오토캠핑장', 'info': '자연 속에서 힐링을', '태그': '가족캠핑여행 레포츠 물놀이장캠핑 물놀이하기좋은캠핑장 아이들과놀기좋은캠핑장 '}
,{'place': '이사부독도기념관', 'info': '독도와 동해 수호의 중심지 삼척, 새로운 복합문화공간', '태그': '문화시설 복합문화공간 삼척가볼만한곳 삼척관광 삼척복합문화공간 아이와함께 '}
,{'place': '가상플레이그라운드', 'info': '온가족이 무료로 즐기는 가상 스포츠 체험 공간', '태그': '가상현실 관광지 드론 무료체험 스포츠체험 아이와함께 의성가볼만한곳  '}
,{'place': '상주보오토캠핑장', 'info': '시원한 수상레저와 절경을 만끽하는 캠핑', '태그': '레포츠 볼거리많은캠핑장 상주오토캠핑장 수상레저캠핑장 수상스키캠핑장 '}
,{'place': '서당골오토캠핑장', 'info': '고즈넉한 산골에 자리한 아름다운 캠핑장', '태그': '레포츠 밀양캠핑장 부산시근교캠핑장 뷰맛집캠핑장 숲속캠핑장 애견동반캠핑장'}
,{'place': '가시오름', 'info': '가파도와 마라도 조망이 가능한 오름', '태그': '관광지 서귀포가시오름 서귀포오름 오름 자연 제주가시오름 제주도오름'}
,{'place': '폭낭오름', 'info': '제주의 아름다운 자연을 느낄 수 있는 오름', '태그': '관광지 오름 자연 자연속으로 자연여행 자연좋은곳 자연풍경 자연환경      '}
,{'place': '설레미캠핑장', 'info': '일상에 지친 여러분에게 다정한 벗과 같은 공간', '태그': '레포츠 바닷가캠핑 신안캠핑장 증도캠핑장 해변데크캠핑 해수욕장캠핑'}
,{'place': '한탄강 횃불전망대', 'info': '철원의 역사적 의미를 현대적이고 공학적인 디자인으로 풀어낸 상징적인 구조물', '태그': '관광지 전망대 철원가볼만한곳 철원관광 철원관광명소 철원관광지 한탄강횃불전망대 '}
,{'place': '상주은모래비치캠핑장', 'info': '아름다운 바다 뷰를 감상하며 즐기는 캠핑', '태그': '남해캠핑장 레포츠 바다물놀이가능한캠핑장 상주은모래비치캠핑장 아름다운캠핑장 '}
,{'place': '원효암(양산)', 'info': '바다가 내려다보이는 경관을 간직한 사찰', '태그': '관광지 불교 불교문화 사찰여행 양산가볼만한곳 역사 종교 한국불교   '}
,{'place': '몽돌바다캠핑장', 'info': '해수욕장을 바라보며 즐기는 캠핑', '태그': '레포츠 몽돌해수욕장 바다캠핑장 신안캠핑 암태도캠핑 전용해변 천사대교'}
,{'place': '비토해양낚시공원', 'info': '천혜의 자연경관에 별주부전 전설을 가득 담은 공원', '태그': '관광지 바다생태체험 비토해양낚시공원 사천가볼만한곳 사천낚시명소  '}
,{'place': '상주우산오토캠핑장', 'info': '폐교를 활용한 캠핑장', '태그': '가족캠핑 레포츠 물놀이가능캠핑장 상주캠핑장 청주근교캠핑장 폐교캠핑장'}
,{'place': '신흥사(양산)', 'info': '영축산 기슭에 있는 아름다운 사찰', '태그': '관광지 불교 사찰여행 양산가볼만한곳 역사 자연속으로 한국불교  '}
,{'place': '여여정사', 'info': '아름다운 동굴 법당을 만날 수 있는 사찰', '태그': '관광지 목조과음보살좌상 밀양시 사찰 여여정사 여행지 절'}
,{'place': '성주자연오토캠핑장', 'info': '반려견과 함께하는 힐링 캠핑', '태그': '계곡캠핑장 대구시근교캠핑장 레포츠 성주캠핑장 아이와가볼만한곳 애견동반캠핑장'}
,{'place': '옥산성지', 'info': '조선시대 후기 거제의 읍치에 축조한 산성', '태그': '가족여행 거제옥산성지 경상권 경상남도기념물 관광지 역사 역사공부            '}
,{'place': '신라고분정보센터', 'info': '신라의 역사와 문화유산을 배울 수 있는 공간', '태그': '경주가볼만한곳 문화시설 신라고분정보센터 아이와가볼만한곳 아이와함께       '}
,{'place': '선야봉캠핑장', 'info': '산과 계곡, 물과 들이 어우러진 캠핑장', '태그': '계곡물놀이캠핑장 레포츠 물놀이캠핑장 수영장캠핑장 운주계곡캠핑장 '}
,{'place': '거제현 관아', 'info': '조선 후기 거제의 행정 중심 관청', '태그': '거제관아 거제문화재 거제역사여행 경남거제현관아 관광지 역사 역사공부           '}
,{'place': '제주아이브리조트', 'info': '편안한 쉼과 여유로운 추억', '태그': '관광지 숙박 제주가볼만한곳 제주리조트 제주숙박 제주아이브리조트   '}
,{'place': '거제도 포로수용소 유적공원', 'info': '6·25전쟁 포로를 주제로 한 국내 유일의 역사문화공간', '태그': '2024_여행가는달과함께하는_발도장여행지 가족과함께 거제포로수용소              '}
,{'place': '고현성', 'info': '조선시대 전기 거제의 읍치', '태그': '거제가볼만한곳 거제고현성 거제문화재 거제역사여행 경상남도지정문화재                 '}
,{'place': '상주보캠프존휴', 'info': '자연친화적인 캠핑장', '태그': '낙동강뷰캠핑장 레포츠 물놀이캠핑장 상주캠핑장 수영장캠핑장 아이들과가볼만한곳'}
,{'place': '비스타리조트', 'info': '제주 하늘과 바다, 한라산이 파노라믹한 비경을 이루는 곳', '태그': '관광지 노을명소 바다조망 비스타리조트 숙박 오션뷰리조트 이호테우해수욕장     '}
,{'place': '제주문학관', 'info': '제주문학의 정체성을 담은 도심 속 자연', '태그': '문화시설 전시관 제주가볼만한곳 제주도 제주문학 지역문학활성화 친구와함께'}
,{'place': '모꼬지캠핑장', 'info': '산책하기 좋은 둘레길이 있는 자연 속 캠핑장', '태그': '가족캠핑장 금평저수지캠핑장 김제캠핑장 레포츠 뷰가좋은 산책하기좋은'}
,{'place': '호도해수욕장', 'info': '조용하고 호젓한 섬 호도에서 만나는 해수욕장', '태그': '가족과함께 관광지 바다풍경 보령가볼만한곳 보령호도 섬여행 여름피서지   '}
,{'place': '석양마을주포오토캠핑장펜션', 'info': '붉은 태양과 바다 풍경을 감상하며 캠핑을 즐기다', '태그': '가족캠핑여행 레포츠 물놀이캠핑장 바다캠핑장 아이들과가볼만한곳 함평캠핑장'}
,{'place': '장목진객사', 'info': '조선 초기 왜구의 침략을 방어하기 위한 군진', '태그': '거제장목진객사 경남거제장목진객사 경남장목진객사 경상권 경상남도유형문화재            '}
,{'place': '춘향골 공설시장', 'info': '남원에서 가장 큰 재래시장', '태그': '5일장 공설시장 남원 남원시장 남원여행 문화관광형시장 쇼핑 전통시장  '}
,{'place': '뱀사골힐링야영장', 'info': '지리산 내에 위치한 캠핑장', '태그': '가성비캠핑장 계곡물놀이 국립공원 남원캠핑장 레포츠 전북특별자치도캠핑장 '}
,{'place': '산청둘레길오토캠핑장', 'info': '웅장한 지리산 경관을 온몸으로 느끼는 캠핑', '태그': '가족캠핑여행 계곡캠핑장 레포츠 물놀이캠핑장 산청오토캠핑장 캠핑장둘레길산책'}
,{'place': '별빛두레펜션', 'info': '맑고 시원한 계곡에서 즐기는 휴식', '태그': '걷기길 계곡물놀이 레포츠 여름물놀이 전북특별자치도캠핑장 진안 진안군캠핑장'}
,{'place': '루트7', 'info': '폐교에서 놀거리 가득한 캠핑장으로!', '태그': '경상북도캠핑장 데크가넓은 레포츠 펜션가능한 폐교캠핑장 포항캠핑장 '}
,{'place': '달궁힐링야영장', 'info': '자연이 주는 시원함 속 즐기는 캠핑', '태그': '걷기길 남원시캠핑장 레포츠 여름물놀이 완주 전북특별자치도캠핑장 '}
,{'place': '다람쥐 쉼터', 'info': '운주계곡의 끝자락에 위치한 곳', '태그': '가을단풍명소 레포츠 여름물놀이 완주 완주군캠핑장 완주여행 전북특별자치도캠핑장'}
,{'place': '거제식물원', 'info': '꽃과 나무의 숨결 속에서 자연과 함께하는 공간', '태그': '2024_여행가는달과함께하는_발도장여행지 가족과함께 관광지 아이와함께   '}
,{'place': '산내별빛오토캠핑장', 'info': '자연과 함께 캠핑을', '태그': '가족캠핑여행 경주캠핑장 레포츠 물놀이캠핑장 아이들과가볼만한곳 키즈캠'}
,{'place': '그날카라반', 'info': '산자락에서의 캠핑', '태그': '가을단풍명소 겨울눈꽃명소 레포츠 무주 무주군캠핑장 스키 전북특별자치도캠핑장'}
,{'place': '지리산산내들펜션야영장', 'info': '지리산 천왕봉이 보이고 계곡이 흐르는 곳에 있는 야영장', '태그': '가을단풍명소 걷기길 남원시캠핑장 레포츠 봄꽃여행 여름물놀이 전북특별자치도캠핑장'}
,{'place': '거제박물관', 'info': '선사시대부터 6·25전쟁까지의 거제 역사박물관', '태그': '거제문화재단 문화시설 민속실 박물관 아이와 함께 종합박물관 체험학습'}
,{'place': '거제박물관', 'info': '선사시대부터 6·25전쟁까지의 거제 역사박물관', '태그': '거제문화재단 문화시설 민속실 박물관 아이와 함께 종합박물관 체험학습'}
,{'place': '오봉산(양산)', 'info': '영남알프스 끝에 다섯 개의 산봉우리가 만드는 절경', '태그': '경치좋은곳 관광지 양산가볼만한곳 양산오봉산 엽기적인그녀 영남알프스   '}
,{'place': '땅끝황토펜션오토캠핑장', 'info': '포항 바다 바로 앞 소나무 아래에서 즐기는 캠핑감성', '태그': '레포츠 바다낚시 오션뷰 포항캠핑장 해돋이 호미곶인근'}
,{'place': '포로수용소유적박물관', 'info': '포로수용소의 역사를 알 수 있는 박물관', '태그': '거제도가볼만한곳 거제포로수용소 경상권 문화시설 박물관 아이와함께          '}
,{'place': '남천교 청연루', 'info': '시원스레 흐르는 전주천을 한눈에 담아 보자', '태그': '경치좋은곳 관광지 사진찍기 사진찍기좋은곳 사진촬영 야경명소 야경여행      '}
,{'place': '서귀포 치유의 숲', 'info': '제주도 자연과 함께하는 휴식 공간', '태그': '2024_여행가는달과함께하는_발도장여행지 가족휴양 관광지 국내힐링여행               '}
,{'place': '밀모빨강치마캠핑리조트', 'info': '계절마다 운치 있는 풍경을 자랑하는 곳', '태그': '가을단풍명소 레포츠 무주 무주군캠핑장 스키 액티비티 전북특별자치도캠핑장'}
,{'place': '산청지막계곡캠핑장', 'info': '경치가 아름다운 캠핑장', '태그': '경상남도캠핑장 계곡캠핑장 레포츠 뷰가좋은캠핑장 산청캠핑장 지리산계곡캠핑장'}
,{'place': '구례 석주관성', 'info': '왜적과 맞서 호남지방을 지켜낸 역사의 현장', '태그': '가족여행 관광지 구례가볼만한곳 구례고려성 구례석주관성 구례여행       '}
,{'place': '봉좌마을캠핑장', 'info': '다양한 즐길거리와 캠핑', '태그': '나무그늘많은캠핑장 레포츠 봉좌마을캠핑장 아이들과가볼만한곳 체험캠핑장 '}
,{'place': '장흥동학농민혁명기념관', 'info': '반봉건, 반외세 동학농민혁명 최후의 격전지', '태그': '교과서속여행 기념관 문화시설 실내관광지 역사유적지 장흥여행 전라권'}
,{'place': '구례 석주관 칠의사묘', 'info': '정유재란, 왜구에 맞서 싸우던 7명의 의병장을 기리며', '태그': '관광지 구례가볼만한곳 구례역사 사적 석주관성 석주관칠의사묘 아이들과함께                  '}
,{'place': '금강두승산글램핑', 'info': '정읍 시내에서 가까운 글램핑장', '태그': '가족과함께 레포츠 연인과함께 일몰명소 전북특별자치도캠핑장 정읍 '}
,{'place': '하늘정원', 'info': '계절마다 아름다운 꽃밭 풍경을 감상하다', '태그': '가볼래터 관광지 여행구독 인천_여행지_추천 인천가볼만한곳 인천국제공항   '}
,{'place': '성당포구금강체험관야영장', 'info': '성당면 성당리에 있는 성당포구마을', '태그': '걷기길 농촌체험 레포츠 여름물놀이 익산시캠핑장 인기캠핑장2024 '}
,{'place': '정서진', 'info': '서해의 아름다운 일몰을 볼 수 있는 명소', '태그': '가볼래터 관광지 서해 서해여행 여행구독 인천관광 인천명소 인천여행   '}
,{'place': '그린오토캠핑장', 'info': '캠핑과 해수욕을 동시에 즐겨볼 수 있는 곳', '태그': '레포츠 바닷가캠핑 오션뷰캠핑 포항가족여행 포항캠핑장 호미곶캠핑장'}
,{'place': '인천아시아드주경기장', 'info': '스포츠와 문화생활을 위한 공간', '태그': '가족과함께 남녀노소 레포츠 스포츠 아이와함께 육상경기 인천서구   '}
,{'place': '인천어린이과학관', 'info': '어려운 과학 원리도 놀이로 풀어내는 과학 놀이터', '태그': '계양어린이과학관 과학관 문화시설 어린이 어린이과학관 인천가볼만한곳 '}
,{'place': '거제 관광모노레일', 'info': '포로수용소 유적공원부터 계룡산까지', '태그': '거제가볼만한곳 경치좋은곳 관광명소 관광지 모노레일 이색체험 자연환경'}
,{'place': '청운문학도서관', 'info': '한옥에서 독서를 즐길 수 있는 도서관', '태그': '가볼래터 도서관 독서공간 독서교육 문화공간 문화시설 서울가볼만한곳      '}
,{'place': '강릉아트센터', 'info': '예술적 영감이 감동으로 전해지는 문화공간', '태그': '강릉가볼만한곳 강릉공연 강릉아트센터 문화시설 미술전시회 복합문화공간   '}
,{'place': '광덕사(상주)', 'info': '서산에 자리 잡은 사찰', '태그': '경상권 관광지 광덕사 불교 불교문화 사찰 사찰여행 상주관덕사           '}
,{'place': '5·18민주화운동 기록관', 'info': '그날의 역사와 의의를 자세히 알 수 있는 곳', '태그': '5.18 518민주화운동기록관 5·18민주화운동기록관 광주 광주여행 기록관      '}
,{'place': '섬진강(순창)', 'info': '아름다운 푸른 물줄기와 고운 모래가 있는 곳', '태그': '강 관광지 섬진강 자연 자연발생 자연속으로 자연여행 자연좋은곳   '}
,{'place': '경안천 습지생태공원', 'info': '인간과 자연이 공존하는 친환경적인 공원', '태그': '걷기길 경안천습지생태공원 관광지 생태관광 생태관광지 자연 힐링'}
,{'place': '밀양아리랑 우주천문대', 'info': '외계행성과 외계생명에 특화된 천문대', '태그': 'INFP상상여행 가족과함께 관광지 교육프로그램 남녀노소 밀양가볼만한곳           '}
,{'place': '천주교 명례성지', 'info': '경남 지역에서 가장 먼저 설립된 천주교회 본당', '태그': '관광지 역사공부 역사관광지 역사속 역사속으로 역사여행 역사유적      '}
,{'place': '퇴촌식물원', 'info': '자연 속에서 여유, 교육, 체험을 모두 즐길 수 있는 공간', '태그': '경기광주가볼만한곳 관광지 광주퇴촌식물원 산업관광 산업관광지 식물원   '}
,{'place': '영남알프스 얼음골케이블카', 'info': '하늘이 내린 천혜의 자연과 함께하는 케이블카', '태그': '경치여행 경치좋은곳 관광지 얼음골캐이블카 영남알프스 영남알프스얼음골케이블카     '}
,{'place': '경기광주 한옥마을', 'info': '고유의 전통과 역사를 담은 곳', '태그': '관광지 광주한복체험 새오개길39 한옥 한옥마을 한옥스테이'}
,{'place': '풀짚공예박물관', 'info': '자연을 디자인하는 풀짚공예박물관', '태그': '공예체험 공예품 문화시설 복합문화공간 열린문화공간 예술창작 전통공예      '}
,{'place': '용산저수지', 'info': '미르샘 수변길, 수상데크, 분수가 조성된 호수', '태그': '관광지 석산지 용산호 자연 정읍용산저수지 정읍저수지'}
,{'place': '박물관 얼굴', 'info': '옛사람들이 모여서 축제를 꾸미는 공간', '태그': '가족여행 경기광주가볼만한곳 경기광주관광 경기광주박물관 경기광주여행          '}
,{'place': '정읍 신태인시장', 'info': '양질의 고춧가루를 구입할 수 있는 시장', '태그': '가족여행 나들이 나홀로여행 데이트코스 먹거리 쇼핑 시장맛집 아이와함께       '}
,{'place': '굴전여가캠핑장', 'info': '굴 양식장을 체험할 수 있는 캠핑장', '태그': '가족캠핑여행 레포츠 바다뷰캠핑장 바다캠핑장 아이와가기좋은캠핑장 '}
,{'place': '섬진강마실캠핑장', 'info': '자연 속 조용한 힐링처', '태그': '가을단풍명소 걷기길 겨울눈꽃명소 레포츠 봄꽃여행 순창군캠핑장 여름물놀이  '}
,{'place': '화산서원비', 'info': '전북특별자치도에서 가장 오래된 서원을 기념하는 비석', '태그': '관광지 기록유산 역사 역사공부 역사관광지 역사를품은곳 역사문화재         '}
,{'place': '마이산 풍혈냉천 캠핑장', 'info': '힐링하기 좋은 공간', '태그': '가볼래터 레포츠 여행구독 임실치즈마을 전북특별자치도캠핑장 전주한옥마을  '}
,{'place': '마령큰시내들캠핑장', 'info': '공기가 맑아 밤에는 하늘에 펼쳐진 수많은 별을 볼 수 있는 곳', '태그': '걷기길 레포츠 메타세콰이어길 전북특별자치도캠핑장 진안 진안군캠핑장  '}
,{'place': '마이산 풍혈냉천 캠핑장', 'info': '힐링하기 좋은 공간', '태그': '가볼래터 레포츠 여행구독 임실치즈마을 전북특별자치도캠핑장 전주한옥마을  '}
,{'place': '마령큰시내들캠핑장', 'info': '공기가 맑아 밤에는 하늘에 펼쳐진 수많은 별을 볼 수 있는 곳', '태그': '걷기길 레포츠 메타세콰이어길 전북특별자치도캠핑장 진안 진안군캠핑장  '}
,{'place': '무주네버랜드', 'info': '자연 속 행복한 추억을 만들어 나갈 휴식처', '태그': '가을단풍명소 겨울눈꽃명소 계곡물놀이 레포츠 무주군캠핑장 전북특별자치도캠핑장'}
,{'place': '치명자산성지', 'info': '천주교 순교자들이 묻힌 성지', '태그': '관광지 성지 역사 역사공부 역사관광지 역사를품은곳 역사문화재         '}
,{'place': '거제자연예술랜드', 'info': '자연석으로 만든 3천개의 인상석이 있는 곳', '태그': '가족과함께 가족여행 거제가볼만한곳 거제관광 경상권 관광지 아이와함께    '}
,{'place': '가배량진성', 'info': '경상우수영과 삼도수군통제영으로 사용된 수군진성', '태그': '거제가배량성 거제가배량진성 거제역사여행 거제유적지 경상남도지정문화재                   '}
,{'place': '거창 A글램핑', 'info': '편안하게 몸만 와서 즐길 수 있는 글램핑', '태그': '거창A글램핑 거창군 거창여행 글램핑 글램핑장 레포츠'}
,{'place': '일본군 위안부 역사관', 'info': '일본군 위안부 피해자들의 아픔과 역사를 기억하는 공간', '태그': '경기광주가볼만한곳 경기광주역사관 경기도 광주 나눔의 집 나눔의집         '}
,{'place': '산중오토캠핑장', 'info': '덕유산 자락 숲속 안에 자리한 캠핑장', '태그': '거창군 경상남도 레포츠 오토캠핑장 캠핑장'}
,{'place': '거제 구조라관광어촌마을', 'info': '거제에서 해보는 어촌체험', '태그': '가족여행 가족휴양 경상권 관광지 아이와함께 어촌 어촌체험 어촌체험관광마을          '}
,{'place': '섬진강', 'info': '물길로 구비 구비 이어진 전라북도와 전라남도', '태그': '가정역 강변축제 경치여행 경치좋은곳 곡성섬진강 곡성순자강 관광지         '}
,{'place': '오송제', 'info': '자연의 생태가 살아 숨 쉬는 곳', '태그': '관광지 산책하기좋은곳 생태호수공원 자연 전라권 힐링'}
,{'place': '자만마을 벽화갤러리', 'info': '따뜻하고 생동감 있는 벽화가 그려져 있는 마을', '태그': '감성사진 골목투어 관광지 남녀노소 데이트장소추천 사진찍기좋은곳        '}
,{'place': '월곶 생활문화센터', 'info': '생활문화를 향유하고 창조하는 공간', '태그': '남녀노소 문화공간 문화시설 문화예술 열린문화공간 창작공간 창작활동    '}
,{'place': '완산공원', 'info': '한바탕 전주! 전주의 대명사 완산칠봉', '태그': '관광지 반려동물 반려동물동반여행지 사진찍기좋은곳 사진촬영 산책하기좋은곳            '}
,{'place': '도장포어촌체험마을', 'info': '깨끗한 바다와 풍경이 아름다운 마을', '태그': '가족여행 경상권 관광지 도장포어촌체험마을 사진촬영명소 어촌마을     '}
,{'place': '서암생활체육공원', 'info': '김포 시민들의 건강증진을 위한 생활체육 공원', '태그': '게이트볼장 경기도 관광지 김포 김포생활체육공원 운동하기좋은곳 족구장 '}
,{'place': '전주 산성벽화마을', 'info': '아름다운 벽화가 가득한 조용한 마을', '태그': 'SNS감성사진 감성사진 관광지 데이트장소추천 벽화마을 사진여행     '}
,{'place': '외할머니의 부엌', 'info': '따스한 온기가 느껴지는 생활사 박물관', '태그': '가족과함께 문화시설 박물관 아이와함께 연인과함께 열린문화공간 음식만들기체험     '}
,{'place': '임진강 댑싸리정원', 'info': '2만 그루의 댑싸리와 꽃들로 조성된 공원', '태그': '2024_여행가는달과함께하는_발도장여행지 가볼래터 가을추천명소 관광지         '}
,{'place': '임진강 댑싸리정원', 'info': '2만 그루의 댑싸리와 꽃들로 조성된 공원', '태그': '2024_여행가는달과함께하는_발도장여행지 가볼래터 가을추천명소 관광지         '}
,{'place': '피싱파크 진산각', 'info': '동화 같은 정원에서 즐기는 체험', '태그': '가족과함께 관광지 김포 김포가볼만한곳 김포피싱파크 김포피싱파크진산각                   '}
,{'place': '별천지오토캠핑장', 'info': '청계호수 부근, 맑은 계곡물을 채운 수영장이 일품인 캠핑장', '태그': '경상남도캠핑장 레포츠 물놀이캠핑장 산청캠핑장 수영장있는캠핑장 아이들과가볼만한곳'}
,{'place': '비토애', 'info': '파도 소리와 바다 내음을 맡으며', '태그': '가족캠핑여행 경상남도사천글램핑 레포츠 비토섬글램핑 사천가볼만한곳 '}
,{'place': '덕포진교육박물관', 'info': '우리나라 교육의 발자취를 따라가볼 수 있는 곳', '태그': '경기도 김포덕포진교육박물관 김포박물관 덕포진교육박물관 덕포진박물관       '}
,{'place': '로하스캠핑장', 'info': '정읍 산호수마을에 위치한 청정자연 속 캠핑장', '태그': '레포츠 뷰맛집 전북특별자치도캠핑장 정읍카라반 정읍캠핑장 풍경이예쁜'}
,{'place': '불태산 글램핑', 'info': '전남 장성군 진원면에 있는 글램핑장', '태그': '가족캠핑여행 광주시근교글램핑 광주시근교캠핑장 레포츠 물놀이하기좋은캠핑장 '}
,{'place': '팽성생활사박물관', 'info': '팽성의 골목골목에 스며든 성실한 삶의 이야기', '태그': '경기평택시 문화시설 박물관 생활사 생활사전시관 평택 평택가볼만한곳'}
,{'place': '김포 수안산성', 'info': '서해안 지역 방어를 위해 삼국시대에 축조된 성', '태그': '경기도기념물 관광지 김포수안산성 수도권 수안산 수안산성 역사           '}
,{'place': '깊은골캠핑장', 'info': '덕유산 자연환경이 주는 친환경 캠핑장', '태그': '가족캠핑여행 계곡물놀이캠핑장 계곡캠핑장 레포츠 무주캠핑장 아이들과가볼만한곳'}
,{'place': '라온글램핑', 'info': '광주 도심 인근의 깔끔한 글램핑장', '태그': '광주시근교 담양글램핑 라온글램핑 레포츠 에어컨있는 장성글램핑'}
,{'place': '도래마을', 'info': '정갈함과 고즈넉함이 마음을 이끄는 곳', '태그': '관광지 나주가볼만한곳 나주도래마을 나주한옥마을 나주한옥체험 도래전통마을  '}
,{'place': '보은글램핑', 'info': '산 속 계곡 인근에서 즐기는 힐링 캠핑', '태그': '가족캠핑여행 레포츠 보은캠핑 수영장캠핑장 아이들과가볼만한곳 청주근교글램핑'}
,{'place': '문수산산림욕장', 'info': '자연과 역사를 품은 곳', '태그': '관광지 김포 김포가볼만한곳 김포문수산산림욕장 김포산림욕장 김포여행    '}
,{'place': '구만산장오토캠핑장', 'info': '울창한 숲에서의 여유로운 휴식', '태그': '구만계곡캠핑장 구만계곡펜션 구만산숙소 구만산캠핑장 레포츠 밀양오토캠핑장'}
,{'place': '메타파크', 'info': '사계절 다른 매력이 넘치는 캠핑장', '태그': '가족캠핑 금성산캠핑 담양카라반 담양캠핑장 레포츠 숲속캠핑'}
,{'place': '더하루풀빌라펜션', 'info': '심플하고 모던한 야영장', '태그': '겨울눈꽃명소 레포츠 무주 무주군 무주군캠핑장 무주여행 여름물놀이 '}
,{'place': '불회사(나주)', 'info': '화려함보다는 호젓함속에 향기로운 녹차향이 머물다', '태그': '관광지 나주불회사 나주사찰체험 나주템플스테이 덕룡산불회사 불교             '}
,{'place': '명도그오토캠핑장', 'info': '남해도의 맨 위쪽인 설천면 동쪽 해안가에 위치하고 있는 캠핑장', '태그': '남해캠핑장 독일마을인근 레포츠 바다가보이는 전망좋은캠핑장 조용한캠핑장'}
,{'place': '줄포만갯벌생태공원 캠핑장', 'info': '일상을 떠나 자연과 함께하는 여유로운 시간', '태그': '걷기길 겨울눈꽃명소 레포츠 봄꽃여행 부안군캠핑장 수상레저 여름물놀이  '}
,{'place': '지리산천왕봉야영장', 'info': '물 맑은 청정지역에서 힐링하기', '태그': '가을단풍명소 걷기길 남원시캠핑장 레포츠 봄꽃여행 여름물놀이 전북특별자치도캠핑장'}
,{'place': '용화사(김포)', 'info': '한강뷰를 조망할 수 있는 사찰', '태그': '관광지 불교 불교문화 사찰 사찰여행 역사 자연속으로 자연환경       '}
,{'place': '김해생림오토캠핑장', 'info': '볼거리 즐길거리 많은 여행지', '태그': '가족캠핑여행 강변뷰캠핑장 김해오토캠핑장 김해캠핑장 레포츠 산책하기좋은캠핑장'}
,{'place': '대둔산글램핑캠핑장', 'info': '숲과 계곡이 맑고 아름다운 곳', '태그': '대둔산도립공원 레포츠 수락계곡 완주 완주군캠핑장 완주여행 전북특별자치도캠핑장 '}
,{'place': '운흥사(나주)', 'info': '상원주장군과 하원당장군의 인자한 웃음을 간직한 사찰', '태그': '관광지 불교 불교문화 사찰 역사 역사공부 역사를품은곳 역사문화재       '}
,{'place': '대둔산 그린시티 캠핑장', 'info': '넓고 편안한 캠핑장', '태그': '가을단풍명소 걷기길 겨울눈꽃명소 레포츠 봄꽃여행 여름물놀이 완주군캠핑장 '}
,{'place': '미륵사(나주)', 'info': '덕룡산 기슭의 아름다운 사찰', '태그': '가족여행 관광지 불교 불교문화 사찰 사찰&산사 사찰여행 역사 자연속으로          '}
,{'place': '무주홀리데이파크', 'info': '그림 같은 자연 풍경 속 즐겨보는 캠핑', '태그': '겨울눈꽃명소 레포츠 무주 무주군 무주군캠핑장 무주여행 액티비티 '}
,{'place': '함양대봉산휴양밸리', 'info': '아름다운 대봉산 자락에 있는 종합휴양 관광단지', '태그': '관광단지 관광지 대봉산여행 대봉스카이랜드 대봉캠핑랜드 함양관광  '}
,{'place': '무주 솔밭야영장', 'info': '상쾌한 바람과 자연이 머무는 곳', '태그': '걷기길 겨울눈꽃명소 레포츠 무주군캠핑장 스키 여름물놀이 전북특별자치도캠핑장'}
,{'place': '무주 솔밭야영장', 'info': '상쾌한 바람과 자연이 머무는 곳', '태그': '걷기길 겨울눈꽃명소 레포츠 무주군캠핑장 스키 여름물놀이 전북특별자치도캠핑장'}
,{'place': '무주풀빌라&카라반', 'info': '덕유산 국립공원 내 최상단에 위치한 캠핑장', '태그': '겨울눈꽃명소 레포츠 무주 무주군 무주군캠핑장 무주여행 전북특별자치도캠핑장'}
,{'place': '심향사(나주)', 'info': '마음을 찾는 향기로운 도량', '태그': '관광지 불교 불교사찰 사찰 사찰여행 역사 자연속으로 자연좋은곳       '}
,{'place': '김포 아트빌리지', 'info': '전통과 현대의 아름다움이 공존하는 김포의 문화관광공간', '태그': '가족여행 김포아트빌리지 문화관광해설사 문화시설 복합문화공간 아이와함께         '}
,{'place': '무주글램핑', 'info': '계곡 옆 프라이빗한 글램핑을 할 수 있는 곳', '태그': '가을단풍명소 겨울눈꽃명소 레포츠 무주 무주가볼만한곳 무주군캠핑장 '}
,{'place': '치킨체험 테마파크 땅땅치킨랜드', 'info': '보고 만들고 즐기는 치킨 체험 테마파크', '태그': '가볼래터 가족과함께 가족체험 관광지 남녀노소 대구가볼만한곳 땅땅치킨랜드           '}
,{'place': '한수제', 'info': '봄이면 벚꽃이 장관을 이루는 저수지', '태그': '관광지 나주가볼만한곳 나주시유원지 나주시자연생태여행 나주여행 나주치유여행  '}
,{'place': '도덕산 출렁다리', 'info': '초록빛 숲과 인공폭포, 시민이 함께 어우러지는 만남과 화합의 상징', '태그': '가볼래터 관광명소 관광지 광명 광명가볼만한곳 광명도덕산 광명시              '}
,{'place': '빛가람 호수공원', 'info': '아름다운 경관을 자랑하는 호수공원', '태그': '관광지 배메산 빛가람 빛가람전망대 산책 쉼터 음악분수 전망대  '}
,{'place': '도덕산공원', 'info': '도심지 중심에 위치한 자연공원', '태그': '공원 관광지 광명 근린공원 나들이 도덕산 산책 쉼터 인공폭포 '}
,{'place': '문발리헌책방골목 블루박스', 'info': '옛 감성이 풍기는 감성적인 북카페', '태그': '관광지 남녀노소 독서여행 실내데이트 전시/공연 헌책방골목 혼자가도좋은  '}
,{'place': '무주구천동캠핑장', 'info': '구천동 계곡의 물소리를 벗 삼아 즐기는 캠핑', '태그': '가을단풍명소 레포츠 무주 무주군캠핑장 소금강산 여름물놀이 전북특별자치도캠핑장'}
,{'place': '민들레동산힐링센터', 'info': '메타세콰이아 숲 속에서 힐링하는 시간', '태그': '가을단풍명소 겨울눈꽃명소 레포츠 액티비티 완주 완주군캠핑장 완주여행 '}
,{'place': '드들강솔밭유원지', 'info': '청정 자연 속에서 산책하기', '태그': '관광지 나주솔밭유원지 나주시유원지 나주시자연생태여행 나주여행 나주치유여행  '}
,{'place': '회문랜드 캠핑장', 'info': '고요한 산속에 있는 별장과도 같은 곳', '태그': '가을단풍명소 레포츠 순창 순창군캠핑장 야외수영장 여름물놀이 전북특별자치도캠핑장'}
,{'place': '안터생태공원', 'info': '생태계가 복원된 도심 속 내륙습지', '태그': '관광지 광명시 금개구리 나들이 생태공원 연꽃 저수지 휴식'}
,{'place': '무주반딧골캠핑장', 'info': '쉬며 체험하며 힐링하는 시간', '태그': '덕유산국립공원 라제통문 레포츠 무주 무주군캠핑장 전북특별자치도캠핑장 '}
,{'place': '로뎀하우스', 'info': '캠핑장 앞 물가에서 한여름 무더위를 날리다', '태그': '동막해변 레포츠 마니산 완주 완주군캠핑장 완주여행 전등사 전북특별자치도캠핑장'}
,{'place': '정화네 불한증막', 'info': '조용하고 아담한 캠핑장', '태그': '레포츠 아이들이좋아하는 전북특별자치도캠핑장 정읍시캠핑장 한증막 '}
,{'place': '새빛공원', 'info': '광명시민들의 감성과 힐링공간', '태그': '관광지 광명시일직동 새물공원 수변공원 시민휴식공간 자경저류지'}
,{'place': '켄싱턴리조트 남원점', 'info': '남원의 주요 관광지들과 가장 가까운 위치의 리조트', '태그': '관광지 남원 남원가볼만한곳 남원가족과함께 남원숙소 남원호텔 숙박'}
,{'place': '국립운장산자연휴양림 야영장', 'info': '아늑하고 한적한 캠핑', '태그': '가을단풍명소 걷기길 레포츠 용담향교 전북특별자치도캠핑장 진안 진안군캠핑장 '}
,{'place': '마라도', 'info': '대한민국 최남단의 섬', '태그': '관광지 마라도 마라해양도립공원 반려동물 반려동물동반여행지 섬 자연  '}
,{'place': '사천진해변(사천뒷불해수욕장)', 'info': '한적하게 여유를 즐길 수 있는 해변', '태그': '가족여행 강릉해수욕장 강원권 강원도_여행지_추천 관광지 바다풍경            '}
,{'place': '힐링아일랜드글램핑캠핑장', 'info': '주변 관광지가 다양한 자연 속 쉼터', '태그': '남해캠핑장 레포츠 뷰맛집캠핑장 수영장캠핑장 애견동반캠핑장 오션뷰캠핑장'}
,{'place': '에코왕곡마을', 'info': '가족과 함께하는 농촌 힐링 여행지', '태그': '관광지 나주 나주체험마을 농촌체험휴양마을 마을 에코왕곡마을 체험마을'}
,{'place': '산아래글램핑', 'info': '자연과 함께하는 글램핑', '태그': '가을단풍명소 걷기길 겨울눈꽃명소 레포츠 장성군캠핑장 전라남도캠핑장'}
,{'place': '홍길동테마파크오토캠핑장', 'info': '홍길동테마파크 내부에 위치한 야영장', '태그': '가족캠핑여행 광주시근교캠핑장 레포츠 아이와가볼만한곳 장성캠핑장 '}
,{'place': '천황야영장', 'info': '등산과 캠핑을 한번에 즐기다', '태그': '걷기길 국립공원 국립공원캠핑장 레포츠 봄꽃여행 숲속캠핑 영암군캠핑장 '}
,{'place': '지리산내원자동차야영장', 'info': '지리산 공기를 마시며 즐기는 캠핑', '태그': '가을단풍명소 걷기길 겨울눈꽃명소 국립공원야영장 레포츠 봄꽃여행  '}
,{'place': '추월산 골드카라반캠핑장', 'info': '경관이 아름다운 캠핑장', '태그': '낚시 담양 담양관광 담양군캠핑장 레포츠 수상레저 전라남도캠핑장 '}
,{'place': '천일염힐링캠프카라반', 'info': '소금밭 체험장 옆에 있는 캠핑장', '태그': '걷기길 낚시 레포츠 신안군캠핑장 일몰명소 일출명소 전라남도캠핑장'}
,{'place': '호반캠핑장', 'info': '낚시도 즐길 수 있는 캠핑장', '태그': '계곡캠핑장 농장캠핑장 레포츠 물놀이캠핑장 밀양캠핑장 숲속캠핑장'}
,{'place': '선인장', 'info': '빈티지한 분위기가 낭만적인 곳', '태그': '가을단풍명소 담양 담양관광 담양군캠핑장 레포츠 메타세콰이어길 전라남도캠핑장 '}
,{'place': '영남제일관', 'info': '역사적 가치가 있는 대구의 향토 상징 관문', '태그': '가족여행 경상권 고궁 관광지 교과서속여행 대구가볼만한곳 사계절    '}
,{'place': '해오름펜션', 'info': '일상 속 지친 심신을 달래는 곳', '태그': '담양 담양관광 담양군캠핑장 레포츠 봄꽃여행 수상레저 일출명소 '}
,{'place': '초록슈피아캠핑장', 'info': '영남의 알프스라 불리는 밀양 얼음골에 위치해 있는 캠핑장', '태그': '가족캠핑여행 레포츠 물놀이장캠핑장 밀양캠핑장 사과밭캠핑장 아이들과놀기좋은캠핑장'}
,{'place': '진도풍경오토캠핑장', 'info': '한옥이 있는 캠핑장', '태그': '걷기길 레포츠 물놀이장 삼별초공원 액티비티 전라남도캠핑장 진도군캠핑장'}
,{'place': '허브하우스금오산힐링타운', 'info': '만족스러운 캠핑이 가능한 공간', '태그': '금오산캠핑장 김천캠핑장 레포츠 숲속캠핑장 아이와가볼만한곳 체험캠핑장'}
,{'place': '대덕힐링캠핑장', 'info': '해수욕장 인근의 감성 캠핑장', '태그': '가마미해수욕장 레포츠 백제불교도래지 불갑저수지수변공원 영광 영광군캠핑장  '}
,{'place': '해바라기캠핑장', 'info': '쌍룡계곡과 가까운 캠핑장', '태그': '계곡물놀이 낚시 레포츠 문경새재 문경종합온천 문경캠핑장 문경펜션  '}
,{'place': '프로방스 마을', 'info': '정원과 벽화 야간 조명이 조화를 이루는 공간', '태그': '가족여행 관광지 사진찍기 사진찍기좋은곳 산책코스 산책하기좋은곳         '}
,{'place': '무안황토갯벌랜드 캠핑장', 'info': '가족과 함께하는 행복한 캠핑 여행', '태그': '갯벌습지보호지역 걷기길 레포츠 무안군캠핑장 여름물놀이 전라남도캠핑장'}
,{'place': '들음카라반펜션', 'info': '백양사 내장산 국립공원 내에 위치한 캠핑장', '태그': '가을단풍명소 걷기길 겨울눈꽃명소 레포츠 장성군캠핑장 전라남도캠핑장'}
,{'place': '강남축구공원', 'info': '강릉 주민들의 건강을 책임지다', '태그': '강릉관광 강릉여행 관광지 레포츠 체육시설 축구공원'}
,{'place': 'KT&G 상상마당 홍대', 'info': '트렌드를 앞장서는 도심 속 복합문화예술공간', '태그': 'KT&G상상마당 나홀로여행 도심여행 디자인스퀘어 문화시설 문화예술           '}
,{'place': '함평돌머리캠핑장', 'info': '해수욕과 캠핑을 한번에 즐기다', '태그': '돌머리축제 레포츠 어린이날깡통열차 전라남도캠핑장 함평 함평군캠핑장 '}
,{'place': '해밀캠핑장', 'info': '비 온 뒤 맑게 갠 하늘 캠핑장', '태그': '걷기길 덕유산 라제통문 레포츠 무주래프팅 무주리조트 무주캠핑장  '}
,{'place': '라온 캠핑장', 'info': '담양호의 수려한 풍광을 배경으로 펼쳐진 캠핑장', '태그': '담양 담양관광 담양군캠핑장 담양호 레포츠 메타세콰이어길 용마루길 '}
,{'place': '제주플래티늄카라반', 'info': '애월 바닷가 바로 앞에 위치한 캠핑장', '태그': '감성캠핑 레포츠 봄꽃캠핑장 애월바다뷰캠핑장 애월캠핑장 제주도캠핑장'}
,{'place': '제주플래티늄카라반', 'info': '애월 바닷가 바로 앞에 위치한 캠핑장', '태그': '감성캠핑 레포츠 봄꽃캠핑장 애월바다뷰캠핑장 애월캠핑장 제주도캠핑장'}
,{'place': '땅끝오토캠핑장', 'info': '푸른 남해가 한눈에 보이는 캠핑장', '태그': '달마산미황사 땅끝전망대 레포츠 송호리해수욕장 전라남도캠핑장 해남 '}
,{'place': '제주벨리타캠핑장', 'info': '제주도 여행 중 제대로 즐겨보는 캠핑', '태그': '가족캠핑여행 레포츠 사이트주차캠핑장 잔디캠핑장 제주도캠핑장 천연화산석사이트'}
,{'place': '제이원빌리지', 'info': '쾌적한 휴식이 가능한 럭셔리 캠핑장', '태그': '가족캠핑여행 도심캠핑장 레포츠 물놀이캠핑장 수영장캠핑장 진주캠핑장'}
,{'place': '홀통유원지·홀통해변', 'info': '울창한 해송숲에서 즐기는 홍통해변에서의 여유', '태그': '관광지 데이트코스 무안여행 바다풍경 연인과함께 자연 자연좋은곳  '}
,{'place': '영암 프리미엄 키즈글램핑', 'info': '신학분교를 개조해 만든 자연친화적 키즈 글램핑장', '태그': '놀이터 레포츠 아이들이좋아하는 액티비티 영암 영암가볼만한곳 영암군캠핑장 '}
,{'place': '미녀봉', 'info': '봉우리들이 빚어낸 아름다운 산세를 만날 수 있는 곳', '태그': '거창가볼만한곳 거창미녀산 경상권 관광지 등산하기좋은 미녀봉 자연 '}
,{'place': '장수방화동캠핑장', 'info': '푸른 숲과 맑은 계곡이 어우러진 캠핑장', '태그': '가을단풍명소 걷기길 레포츠 여름물놀이 오토캠핑장 장수글램핑 장수캠핑'}
,{'place': '포천 반월성', 'info': '흥망성쇠 1700년 역사 품은 요새', '태그': '관광지 문화재 반월산성 반월성 역사 역사공부 역사관광지 역사를품은곳          '}
,{'place': '자연애품캠핑장', 'info': '자연 속 힐링 공간', '태그': '곡성 곡성캠핑장 레포츠 캠핑 캠핑장'}
,{'place': '침실습지', 'info': '다양한 동·식물이 서식하는 생태의 보고', '태그': '경치좋은곳 관광지 자연 자연관광지 자연체험 휴식여행'}
,{'place': '국립서울현충원', 'info': '국가의 성역이자 국민과 함께하는 호국추모공원', '태그': '가족여행 관광지 교과서속여행 국가유공자 국군묘지 국군의날 국립서울현충원                  '}
,{'place': '용천토리오토캠핑장', 'info': '자연 속에서 채우는 삶의 에너지', '태그': '가족캠핑여행 경주캠핑장 레포츠 사과나무캠핑장 선착순캠핑장 청도캠핑장'}
,{'place': '오토캠핑장봄날처럼', 'info': '도심 속 편안한 휴식 공간', '태그': '경주캠핑장 관리가잘된 놀이시설이다양한 레포츠 아이들이뛰어놀기좋은 '}
,{'place': '휴앤락오토캠핑장', 'info': '경주에 위치한 깔끔한 시설의 캠핑장', '태그': '가을단풍명소 경주캠핑장 레포츠 아이들이좋아하는 야외수영장 여름물놀이  '}
,{'place': '다락원체육공원', 'info': '도봉구민의 건강과 문화를 위한 공간', '태그': '관광지 다락원체육공원 대형체육공원 도봉구게이트볼장 도봉구배드민턴장   '}
,{'place': '용두마을', 'info': '섬진강 물길이 지나던 마을', '태그': '관광지 구례여행 배틀재 섬진강 전남여행 지리산둘레길'}
,{'place': '자연속캠핑장', 'info': '여유롭고 자유로운 휴식', '태그': '가성비좋은캠핑장 가족캠핑여행 레포츠 산책로걷기좋은캠핑장 순천캠핑장 '}
,{'place': '가야진사', 'info': '낙동강 나루터신을 모시고 있는 제당', '태그': '관광지 나루터신제당 낙동강나루터 양산가볼만한곳 양산가야진사 양산기우제   '}
,{'place': '장좌리캠핑장', 'info': '가족과 함께 즐기기 좋은 캠핑장', '태그': '가을단풍명소 걷기길 고성캠핑장 낚시 레포츠 수상레저 여름물놀이 '}
,{'place': '장좌리캠핑장', 'info': '가족과 함께 즐기기 좋은 캠핑장', '태그': '가을단풍명소 걷기길 고성캠핑장 낚시 레포츠 수상레저 여름물놀이 '}
,{'place': '청도캠프원', 'info': '청도 최대 규모 캠핑장', '태그': '가족캠핑여행 경상북도캠핑장 레포츠 수영장캠핑장 아이와가볼만한곳 '}
,{'place': '섬진강변 유원지', 'info': '섬진강변이 선물하는 즐거움의 세계', '태그': '가정역 곡성군 관광지 산책길 산책하기 산책하기좋은곳 유원지 자건거여행       '}
,{'place': '우리두리글램핑', 'info': '산과 계곡 자연과 함께 캠핑을 즐길 수 있는 힐링의 장소', '태그': '강원도캠핑장 낭만글램핑 럭셔리글램핑 레포츠 영월글램핑장 캠핑장걷기좋은곳_법흥사산책길'}
,{'place': '경화역 벚꽃길', 'info': '경남지역 대표적인 벚꽃 명소', '태그': '2024_여행가는달과함께하는_발도장여행지 가족과함께 감성사진 경남권반려동물여행지                    '}
,{'place': '청도그린나래캠핑장', 'info': '아이들과 함께 즐기기 좋은 캠핑장', '태그': '경상북도캠핑장 레포츠 물놀이캠핑장 수영장캠핑장 애견동방캠핑장 청도캠핑장'}
,{'place': '쌍둥이동물농장', 'info': '동물과 함께하는 작은 동물원', '태그': '강릉가족여행 강릉동물원 관광지 놀이기구 동물체험'}
,{'place': '보광사(과천)', 'info': '도심 속에 자리 잡은 고즈넉한 사찰', '태그': '관광지 불교 불교문화 불교사찰 사찰 사찰여행 역사 자연속으로       '}
,{'place': '창동문화체육센터', 'info': '다양한 체육시설과 전문 프로그램을 갖춘 공공 복합문화체육시설', '태그': '교육프로그램 도봉구 레저스포츠 레포츠 문화체육시설 복합레저스포츠     '}
,{'place': '방호정(구례)', 'info': '일제시대의 아픔을 시에 담아 꽃잎에 날리다', '태그': '관광지 구례방호정 구례벚꽃명소 구례산수유마을 구례여행 구례역사                 '}
,{'place': '정산시장', 'info': '오랜전통의 상가주택건물형 중형시장', '태그': '가족여행 사계절 쇼핑 시장맛집 아이와함께 재래시장 전통시장 전통시장투어  '}
,{'place': '몽촌토성', 'info': '한성백제의 국가 터전을 마련했던 거성', '태그': '가족여행 관광지 교과서속여행 몽촌박물관 몽촌역사관 몽촌토성 박물관                   '}
,{'place': '그라운드플래닛', 'info': '아이들을 위한 대형 키즈 어드벤처 놀이터', '태그': '관광지 나들이 서울대공원 서울랜드 실내놀이터 썰매 아이와함께  '}
,{'place': '군지촌정사', 'info': '임금의 뜻을 받아 조선의 영재를 가르치다', '태그': '관광지 교과서속여행 문화재 역사 역사공부 역사관광지 역사를품은곳       '}
,{'place': '서울 풍납동 토성', 'info': '백제시대에 흙으로 쌓은 평지성', '태그': '가족여행 관광지 교과서속여행 당일치기여행 서울근교여행 서울풍납동토성           '}
,{'place': '곡성 함허정', 'info': '섬진강 뷰가 최고인 힐링 산책이 가능한 곳', '태그': '관광지 역사 역사공부 역사관광지 역사를품은곳 역사문화재 역사속       '}
,{'place': '거창향교', 'info': '지방민의 교육을 담당하던 공립 교육기관', '태그': '거창가볼만한곳 거창향교 경상권 관광지 역사 역사관광지 향교 혼자가도좋은'}
,{'place': '안중레포츠공원', 'info': '다양한 스포츠를 즐기기 좋은 공원', '태그': '관광지 궁도장 그라운드골프장 안중레포츠공원 어린이교통공원 축구장 '}
,{'place': '쥬라기랜드', 'info': '공룡이 살아 움직이는 생생한 현장', '태그': '가족과함께 과천가볼만한곳 관광지 남녀노소 놀이공원 레포츠 서울랜드      '}
,{'place': '궷물오름', 'info': '산뜻한 제주도 산행을 할 수 있는 궷물오름', '태그': '가을억새 관광지 궷물오름 오름 자연풍경 제주가볼만한곳 제주도여행  '}
,{'place': '궷물오름', 'info': '산뜻한 제주도 산행을 할 수 있는 궷물오름', '태그': '가을억새 관광지 궷물오름 오름 자연풍경 제주가볼만한곳 제주도여행  '}
,{'place': '서울 석촌동 고분군', 'info': '백제 한성기의 지배층 무덤', '태그': '걷기길 관광지 교과서속여행 서울석촌동고분군 수도권 역사 역사공부          '}
,{'place': '추사박물관(과천)', 'info': '조선 후기 학자 추사 김정희 선생이 학문과 예술혼을 불태운 곳', '태그': '가족체험여행 과천박물관 과천추사박물관 교과서속여행 김정희박물관          '}
,{'place': '서시천체육공원', 'info': '어른들의 힐링과 아이들의 꿈이 자라는 테마공원', '태그': '관광지 구례꽃강 구례놀이터 구례산책길 구례체육공원 벚꽃명소 아이와함께 '}
,{'place': '문원체육공원', 'info': '다양한 체육활동을 즐길 수 있는 공원', '태그': '공원 과천체육공원 관광지 문원매봉골축제 생활체육공원 운동 체육공원 '}
,{'place': '서울 태릉(문정왕후)과 강릉(명종·인순왕후) [유네스코 세계유산]', 'info': '역사적·예술적 가치가 높은 조선왕릉', '태그': '가족여행 관광지 교과서속여행 궁릉 노원태강릉 명종 명종의능 문정왕후                     '}
,{'place': '목포근대역사관 2관', 'info': '나라를 빼앗긴 아픔을 기억하는 공간', '태그': '23_24한국관광100선 25_26한국관광100선 목포가볼만한곳 목포근대거리                  '}
,{'place': '성라공원', 'info': '고양시민들을 위한 힐링 명소', '태그': '가족공원 고양둥이동산 고양시공원 관광지 벚꽃명소 산책코스 성라산 '}
,{'place': '용산재', 'info': '고려 창건의 주역 신숭겸 장군을 기리다', '태그': '곡성역사여행 곡성용산재 관광지 구룡마을 덕양사 신숭겸장군 신숭겸출생지                '}
,{'place': '청계동솔바람야영장', 'info': '계곡의 물과 울창한 소나무 숲 사이에서 즐기는 야영장', '태그': '가족캠핑여행 계곡캠핑장 곡성캠핑장 레포츠 물놀이캠핑장 숲속캠핑장'}
,{'place': '마천목장군과 도깨비살', 'info': '도깨비살 전설이 전해지는 곳', '태그': '관광지 역사 역사공부 역사관광지 역사를품은곳 역사문화재 역사문화재      '}
,{'place': '어청도 등대', 'info': '아름다운 해송과 어우러진 풍경을 자랑하는 어청도의 등대', '태그': '고군산군도 관광지 국가등록문화재 군산여행 등대 등록문화재 문화재  '}
,{'place': '그린스톤캠핑장', 'info': '대자연에 조성한 캠핑장', '태그': '레포츠 문경계곡캠핑장 문경숲속캠핑장 문경오토캠핑장 아이들과가기좋은곳 '}
,{'place': '용도수목원', 'info': '자연 속 힐링여행', '태그': '관광지 나들이 수목원 여행 월간_사진_제보_이벤트 자연체험학습 체험학습  '}
,{'place': '장항습지', 'info': '생명과 평화가 공존하는 장항습지', '태그': '관광지 남녀노소 생태탐방 생태프로그램 자연 자연관광지 자연발생       '}
,{'place': '봉수대오토캠핑장', 'info': '시원한 바다를 보며 즐기는 캠핑장', '태그': '가족캠핑여행 강원도오토캠핑장 고성오토캠핑장 레포츠 바다캠핑장 해수욕장캠핑장'}
,{'place': '구성재', 'info': '드라이브의 재미를 느낄 수 있는 영화 ‘곡성’ 촬영도로', '태그': '경치좋은곳 곡성여행 관광지 드라이브 드라이브여행 드라이브코스 사진찍기       '}
,{'place': '주암오토캠핑장', 'info': '넓고 쾌적한 캠핑을 즐기자', '태그': '가성비좋은캠핑장 가족캠핑여행 레포츠 산책로걷기좋은캠핑장 순천캠핑장 '}
,{'place': '교룡산국민관광지', 'info': '역사 유적과 자연을 함께 즐길 수 있는 곳', '태그': '가족여행 걷기길 걷기여행 걷기좋은곳 관광지 교룡산 교룡산국민관광지            '}
,{'place': '목포근대역사관 1관', 'info': '나라를 빼앗긴 아픔을 기억하는 공간', '태그': '2024_여행가는달과함께하는_발도장여행지 23_24한국관광100선 25_26한국관광100선                        '}
,{'place': '조은자리캠핑장', 'info': '삼국시대에 건립됐다고 전해지는 운문사 인근인 캠핑장', '태그': '가족캠핑여행 레포츠 물놀이캠핑장 수영장캠핑장 어린이놀이터캠핑장 '}
,{'place': '전원일기오토캠핑장', 'info': '취향따라 즐겨보는 여행', '태그': '가족캠핑여행 경주캠핑장 레포츠 물놀이장캠핑장 아이와가볼만한곳'}
,{'place': '일산 어린이천문대', 'info': '반짝이는 별과 함께 꿈과 낭만을 찾을 수 있는 곳', '태그': '관광지 수도권 아이와함께 어린이천문대 자연 천문대 체험'}
,{'place': '테미공원', 'info': '자연경관을 살려 도심 속에 조성한 공원', '태그': '관광지 대전공원 대전데이트코스 대전산책 대전여행 대전힐링여행'}
,{'place': '주노글램핑 완주점', 'info': '비봉산 자락에 위치한 자연 친화적 감성 글램핑장', '태그': '글램핑 레포츠 완주글램핑 완주바베큐 전북글램핑장'}
,{'place': '으능정이문화의거리', 'info': '대전의 명동이라고 불리는 문화예술거리', '태그': '관광지 다양한볼거리 대전가볼만한곳 쇼핑 스카이로드 연중무휴 으능정이문화거리  '}
,{'place': '사근진해변(사근진해수욕장)', 'info': '울창한 송림이 그늘막이 되어주는 아늑한 해변', '태그': '가족과함께 강릉해수욕장 강원권 강원권반려동물여행지 관광지 바다                '}
,{'place': '행땅', 'info': '반려견들을 위한 프라이빗 놀이터', '태그': '경기권반려동물여행지 관광지 반려동물 반려동물놀이터 반려동물동반여행지'}
,{'place': '의마총', 'info': '유팽로장군 충마의 갸륵한 뜻을 기리는 무덤과 비', '태그': '관광지 역사 역사공부 역사관광지 역사를품은곳 역사문화재 역사속       '}
,{'place': '압록유원지', 'info': '스릴과 힐링까지 물놀이의 모든것', '태그': '가족과함께 곡성압록유원지 관광지 산책코스 산책하기좋은곳 섬진강유원지           '}
,{'place': '태평전통시장', 'info': '다양한 볼거리와 먹거리가 가득한 전통시장', '태그': '가족여행 대전가볼만한곳 대전시장 쇼핑 시장맛집 재래시장 전통시장투어   '}
,{'place': '고양 서오릉 [유네스코 세계유산]', 'info': '5기의 능으로 구성되어 있는 조선왕실의 왕릉군', '태그': '가족여행 고양서오릉 관광지 교과서속여행 나들이 당일치기여행 드라이브여행              '}
,{'place': '고양 어울림누리', 'info': '생활 속의 예술가가 되는 곳', '태그': '고양 고양공연 고양어울림누리 공연 문화교육 문화시설 서울근교여행   '}
,{'place': '송정해변', 'info': '울창한 송림과 넓은 백사장을 즐길 수 있는 곳', '태그': '가족여행 강원_가볼만한곳 강원권 관광지 기차여행 나들이 당일치기여행               '}
,{'place': '충의공원', 'info': '구절초 만발하는 호국영령을 모시는 동산', '태그': '가을 곡성 관광지 산책코스 충의공원'}
,{'place': '자인사(포천)', 'info': '궁예의 전설이 깃든 사찰, 산정호수와 명성산은 덤', '태그': '관광지 불교 불교사찰 사찰 사찰여행 수도권 역사 자연속으로 자연좋은곳          '}
,{'place': '덕유대오토캠핑장', 'info': '조용하고 경치가 좋은 캠핑장', '태그': '국립공원캠핑 덕유산야영 덕유산오토캠핑장 덕유산카라반 덕유산캠핑   '}
,{'place': '국립여성사전시관', 'info': '한국의 발전에 기여한 여성들의 업적을 볼 수 있는 곳', '태그': '고양국립여성사전시관 고양박물관 고양전시관 국립여성사전시관 문화시설         '}
,{'place': '래미안밸리', 'info': '아름답고, 편안함이 함께하는 캠핑', '태그': '계곡캠핑 레포츠 완주캠핑장 운주계곡 전주근교'}
,{'place': '배꽃근린공원', 'info': '쉼과 여유를 느낄 수 있는 도심지 근린공원', '태그': '경기도 공원 관광지 근린공원 배꽃근린공원 의정부 의정부가볼만한곳'}
,{'place': '배꽃근린공원', 'info': '쉼과 여유를 느낄 수 있는 도심지 근린공원', '태그': '경기도 공원 관광지 근린공원 배꽃근린공원 의정부 의정부가볼만한곳'}
,{'place': '병지방오토캠프장', 'info': '가족과 함께 여유롭게 캠핑하기 좋은 곳', '태그': '강원도캠핑장 계곡캠핑장 레포츠 물놀이캠핑장 아이들과가볼만한곳 키즈캠핑 '}
,{'place': '마루노리캠핑장', 'info': '다양한 볼거리와 놀거리를 즐길 수 있는 캠핑장', '태그': '강원도캠핑장 계곡캠핑 레포츠 물놀이하기좋은 원주캠핑장 판대천 폐교'}
,{'place': '추동근린공원', 'info': '의정부 중심에 위치한 산지형 근린공원', '태그': '경기도 공원 관광지 근린공원 의정부 의정부가볼만한곳 추동근린공원'}
,{'place': '비로오토파크', 'info': '자연친화적인 캠핑장', '태그': '계곡물놀이캠핑 레포츠 소백산오토캠핑 아이들과가볼만한곳 영주가볼만한곳 '}
,{'place': '블루웨일글램핑', 'info': '북항의 항구와 바다, 부산항 대교의 야경을 볼 수 있는 곳에 위치한 야영장', '태그': '가족물놀이캠핑장 고성글램핑 레포츠 아이들과가볼만한곳 야외수영장있는캠핑장 '}
,{'place': '맹방비치캠핑장', 'info': '민물과 바다를 동시에 즐기는 시간', '태그': '가족캠핑 레포츠 명사십리캠핑 물놀이하기좋은 삼척캠핑장 솔밭캠핑 '}
,{'place': '부산항힐링야영장', 'info': '북항의 항구와 바다, 부산항 대교의 야경을 볼 수 있는 곳에 위치한 야영장', '태그': '가족캠핑여행 레포츠 바다가보이는캠핑장 바다캠핑장 부산시항대교 부산시항캠핑장'}
,{'place': '파주놀이구름', 'info': '온 가족이 함께 즐길 수 있는 놀이문화공간', '태그': 'EBS놀이구름 가족와함께 관광지 놀이시설 문화공간 아이와함께 체험학습  '}
,{'place': '구름아래오토캠핑장', 'info': '숲속의 요새를 연상케하는 캠핑장', '태그': '레포츠 액티비티 여름물놀이 오토캠핑장 유황온천 청도소싸움 청도캠핑장'}
,{'place': '백운호수 음식마을', 'info': '향토음식을 맛 볼 수 있는 음식마을', '태그': '건강식맛집 관광지 맛집 맛집탐방 매운탕 매운탕맛집 백운호수 숨은맛집       '}
,{'place': '강진청자촌오토캠핑장', 'info': '청자도예촌 내에 자리한 곳으로, 전라남도에서 손꼽히는 규모를 자랑하고 있는 캠핑장', '태그': '강진가볼만한곳 강진오토캠핑장 강진캠핑장 공방체험 레포츠 유아놀이방  '}
,{'place': '우농타조농장', 'info': '동물과 자연을 배우고 즐길 수 있는 곳', '태그': '관광지 동물농장 아기동물농장 아이가_좋아하는_여행 아이와함께 이색체험     '}
,{'place': '강나루오토캠핑장', 'info': '평화로운 자연경관을 마주하는 힐링 캠핑', '태그': '가족과함께 강나루캠핑장 레포츠 아이들이좋아하는 칠서강나루오토캠핑장  '}
,{'place': '고성잔디캠핑장', 'info': '즐길 거리가 가득한 캠핑장', '태그': '고성일출명소 고성캠핑장 낚시하는캠핑장 레포츠 캠핑장걷기길'}
,{'place': '블루메 미술관', 'info': '생명을 지향하는 자연과 연결된 미술관', '태그': '관광지 교육프로그램 문화예술공간 문화예술활동 복합문화공간 열린문화공간       '}
,{'place': '경호강캠핑장', 'info': '아름다운 자연 속에서 편안한 휴식을 즐길 수 있는 캠핑장', '태그': '경호강캠핑장 경호강펜션 계곡캠핑장 레포츠 사천캠핑장 수영장이있는캠핑장 '}
,{'place': '그린하우스캠핑장', 'info': '계절마다 다른 재미를 느끼는 캠핑', '태그': '경주캠핑장 계곡캠핑장 그늘이좋은 레포츠 수영장있는캠핑장'}
,{'place': '파주팜랜드', 'info': '스위스 목장에 온 듯한 힐링공간', '태그': '가족나들이 가족여행 관광지 동물체험 아이와함께 파주동물농장 파주시   '}
,{'place': '휴토피아글램핑', 'info': '잊지 못할 추억의 힐링 캠핑 장소', '태그': '감성글램핑 레포츠 홍천가볼만한곳 홍천감성캠핑 홍천럭셔리글램핑 홍천캠핑장'}
,{'place': '휴토피아글램핑', 'info': '잊지 못할 추억의 힐링 캠핑 장소', '태그': '감성글램핑 레포츠 홍천가볼만한곳 홍천감성캠핑 홍천럭셔리글램핑 홍천캠핑장'}
,{'place': '자연인캠핑장', 'info': '아름다운 자연에서의 여유로운 힐링', '태그': '감성캠핑 레포츠 아이들과가볼만한곳 제주도가족캠핑 제주도글램핑 힐링캠프'}
,{'place': '구천계곡오토캠핑장', 'info': '푸르른 자연 속 조용한 캠핑장', '태그': '가을단풍명소 걷기길 구천계곡야영장 구천계곡오토캠핑장 구천계곡캠핑장 '}
,{'place': '라움아트센터', 'info': '다양한 공연과 행사가 열리는 품격 있는 복합문화공간', '태그': '가족과함께 강남라움아트센터 강남예식장 강남플라워클래스 공연 문화시설   '}
,{'place': '양촌여울체험캠프', 'info': '감동과 힐링의 휴가를 보낼 수 있는 캠핑장', '태그': '레포츠 아이들과가볼만한곳 아이들과캠핑 아이들과힐링캠핑 창원가볼만한곳 '}
,{'place': '궁평유원지', 'info': '소나무 밭에서 힐링 산책을', '태그': '관광지 궁평유원지 데이트코스 드라이브코스 서해안드라이브 일몰명소  '}
,{'place': '탄도어촌체험마을', 'info': '관광자원과 더불어 아름다운 풍경을 볼 수 있는 곳', '태그': '가족과함께 갯벌 갯벌체험 관광지 낙조 대부도 대부도갯벌체험 안산가볼만한곳  '}
,{'place': '경주힐링캠프', 'info': '맑은 공기와 푸른 자연 속에서 휴식의 시간을', '태그': '가을단풍명소 경주캠핑장 당일글램핑 레포츠 산골캠핑장 숲속캠핑장'}
,{'place': '구미캠핑장', 'info': '도심 가까이에서 가족들과 즐길 수 있는 힐링', '태그': '가을단풍명소 구미시캠핑장 구미캠핑장 낙동강캠핑장 대구시캠핑장 대규모캠핑장 '}
,{'place': '아웃오브파크', 'info': '다양한 테마의 캠핑을 즐길 수 있는 곳', '태그': '강촌가볼만한곳 강촌글램핑 강촌럭셔리글램핑 레포츠 카라반캠핑장'}
,{'place': '아웃오브파크', 'info': '다양한 테마의 캠핑을 즐길 수 있는 곳', '태그': '강촌가볼만한곳 강촌글램핑 강촌럭셔리글램핑 레포츠 카라반캠핑장'}
,{'place': '고잔저수지', 'info': '자연을 만끽하며 취하는 휴식', '태그': '관광지 자연관광지 저수지걷기 저수지둘레길 평택저수지'}
,{'place': '경주텐타우스글램핑', 'info': '깔끔하고 고급스러운 캠핑을 즐기고 싶다면', '태그': '가을단풍명소 경주글램핑 경주카라반 경주캠핑장 대형수영장 레포츠 '}
,{'place': '경주오류캠핑장', 'info': '경주 해양관광 중심지에서 즐기는 캠핑', '태그': '가족과함께 경주야영장 경주여행 경주캐라반 경주캠핑장 레포츠 바다캠핑장 '}
,{'place': '밀양VIP오토캠핑장', 'info': '깨끗한 시설의 산과 계곡 등 아름다운 자연을 느낄 수 있는 곳', '태그': '계곡물놀이캠핑 레포츠 밀양계곡캠핑장 밀양아이들과가볼만한곳 밀양오토캠핑장 '}
,{'place': '대흥동 문화예술의거리', 'info': '문화 예술의 거리로 다시 태어난 곳', '태그': '2024_여행가는달과함께하는_발도장여행지 관광지 대전가볼만한곳 문화예술거리          '}
,{'place': '가족소풍', 'info': '다양한 체험을 할 수 있는 편리한 캠핑장', '태그': '가족소풍카라반 담양가볼만한곳 담양가족캠핑장 담양캠핑장 레포츠 아이들과가볼만한곳'}
,{'place': '문복산캠핑장', 'info': '연계관광이 쉬운 부산 근교 캠핑장', '태그': '경주캠핑장 데크가좋은 레포츠 문복산 숲속캠핑장 시원한캠핑장'}
,{'place': '당거리나루터', 'info': '가을이면 각종 축제와 행사가 열리는 곳', '태그': '관광지 산책하기좋은곳 유채꽃밭 자전거길 코스모스 평택드라이브길'}
,{'place': '공릉호', 'info': '걷기 좋은 호수 둘레길', '태그': '공릉호국민관광단지 관광지 파주가볼만한곳 파주공릉저수지 파주공릉호  '}
,{'place': '경주별빛마루글램핑장과펜션', 'info': '아늑한 숲 속 낭만과 즐거움이 가득한 곳', '태그': '경주가볼만한곳 경주글램핑장 경주여행 경주캠핑장 경주펜션 레포츠  '}
,{'place': '벽초지수목원', 'info': '동서양의 아름다운 정원을 품은 수목원', '태그': '2024_여행가는달과함께하는_발도장여행지 관광지 나들이 데이트코스                 '}
,{'place': '관음사야영장', 'info': '한라산 속에서 즐기는 청정 캠핑', '태그': '겨울눈꽃명소 관음사야영장 레포츠 봄꽃여행 제주도야영장 한라산야영장'}
,{'place': '가온빌리지', 'info': '캠핑장에서 반려견과의 행복한 추억만들기', '태그': '가온빌리지 레포츠 밀양캠핑장 애견운동장있는글램핑장 캠핑장'}
,{'place': '달천철장유적공원', 'info': '쉬어가기 좋은 울산 북구 달천철장 유적공원', '태그': '관광지 달천철장 쇠부리축제 울산 울산북구축제 울산축제 울산프리마켓'}
,{'place': '가온누리캠핑체험장', 'info': '다양한 이벤트와 즐거움이 가득한 캠핑장', '태그': '경주가족여행 경주물놀이캠핑장 경주캠핑장 레포츠 아이들과가볼만한곳'}
,{'place': '강화 서도 중앙교회', 'info': '전통 목조건물의 가구 형식의 교회', '태그': '강화가볼만한곳 강화서도중앙교회 강화중앙교회 관광지 수도권 역사   '}
,{'place': '국립생물자원관', 'info': '생물자원의 발굴과 보존, 다양한 생물정보를 알 수 있는 곳', '태그': '국립생물자원관 문화시설 미세먼지_피하는_실내여행 생물다양성 서울근교여행           '}
,{'place': '곤달비캠핑장', 'info': '아이들과 함께하기 좋은 놀 거리 가득한 캠핑장', '태그': '경주벚꽃캠핑장 경주캠핑장 그늘이많은 레포츠 수영장있는캠핑장 아이들과함께'}
,{'place': '담양힐링파크', 'info': '자연과 동물과 휴식이 있는 공간', '태그': '가족캠핑여행 가족휴양 담양가족여행 담양글램핑장 담양캠핑 레포츠   '}
,{'place': '담양힐링파크', 'info': '자연과 동물과 휴식이 있는 공간', '태그': '가족캠핑여행 가족휴양 담양가족여행 담양글램핑장 담양캠핑 레포츠   '}
,{'place': '가야랜드달빛야영장', 'info': '아름다운 자연 속 휴식공간', '태그': '김해가볼만한곳 김해가족캠핑여행 김해글램핑장 김해야영장 레포츠 부산시근교캠핑장'}
,{'place': '서울무역전시컨벤션센터(SETEC)', 'info': '다양한 문화행사를 즐길 수 있는 전시컨벤션', '태그': 'setec 국제적인전시 남녀노소 무역전시관 문화공간 문화시설 박람회        '}
,{'place': '가산글램핑', 'info': '칠곡의 맑은 정기를 한껏 느끼는 캠핑', '태그': '경상북도가족글램핑 계곡물놀이캠핑장 대구시근교글램핑 레포츠 칠곡가볼만한곳 '}
,{'place': '월성오토캠핑장', 'info': '아빠 어디가 촬영지 거창 월성오토캠핑장', '태그': '거창군 레포츠 오토캠핑장 캠핑 캠핑장'}
,{'place': '달토 풀빌라 글램핑', 'info': '풀빌라와 글램핑의 만남, 호텔의 편리함과 캠핑 감성이 공존하는 스페셜 글램핑', '태그': '가볼래터 글램핑 레포츠 여행구독 영덕 오션뷰 풀빌라'}
,{'place': '오송호수공원', 'info': '사시사철 계절의 바람을 느끼며 산책하기 좋은 공원', '태그': '관광지 오송호수공원 청주공원 청주데이트코스 청주산책 청주여행 청주포토존'}
,{'place': '표선해수욕장 야영장', 'info': '천연 잔디 위에서 표선해변을 즐길 수 있는 무료 야영장', '태그': '레포츠 아이들과캠핑 제주나홀로캠핑 제주도무료캠핑 제주도캠핑장 표선야영장  '}
,{'place': '청룡사(서울)', 'info': '깊은 역사를 지닌 도심 속 사찰', '태그': '관광지 도심속사찰 불교문화 서울가볼만한곳 수도권 역사 전통사찰   '}
,{'place': '인천가족공원 외국인묘지', 'info': '인천 개항으로 인해 체류하던 외국인을 위한 묘지', '태그': '공동묘지 관광지 역사 역사속으로 역사이야기 외국인묘지 인천가볼만한곳   '}
,{'place': '인천가족공원 외국인묘지', 'info': '인천 개항으로 인해 체류하던 외국인을 위한 묘지', '태그': '공동묘지 관광지 역사 역사속으로 역사이야기 외국인묘지 인천가볼만한곳   '}
,{'place': '부림시장 먹자골목', 'info': '맛있는 먹거리와 인심으로 가득찬 공간', '태그': '가족과함께 관광지 남녀노소 맛집투어 먹자골목 부림시장 숨은맛집     '}
,{'place': '영월키즈캠핑장', 'info': '온가족이 함께 즐기는 산속 캠핑장', '태그': '강원도캠핑장 계곡물놀이 다양한이벤트 레포츠 아이랑가기좋은 영월군캠핑장  '}
,{'place': '황룡사(인천)', 'info': '애국불교를 적극 실천하는 아름다운 사찰', '태그': '가족과함께 관광지 대한불교천태종 불교문화 인천가볼만한곳 인천여행   '}
,{'place': '방태산 솔마루 오토캠핑장', 'info': '도시생활에 지친 현대인들을 위한 휴식처', '태그': '가을단풍명소 강원도캠핑장 걷기길 겨울눈꽃명소 낚시 레포츠 봄꽃여행   '}
,{'place': '반디하우스캠핑장', 'info': '반려견과 놀러갈 수 있는 힐링 공간', '태그': '강원도캠핑장 레포츠 미산계곡 애견동반캠핑장 여름물놀이 인제군캠핑장'}
,{'place': '강릉 선교장', 'info': '300년 전통의 역사가 있는 품위있는 가옥', '태그': '강릉여행 강원권 강원도_여행지_추천 관광지 국가민속문화재 문화관광해설사               '}
,{'place': '강릉문화원', 'info': '지역 문화를 보존하고 지역사회계발을 위해 노력하는 곳', '태그': '강릉문화원 남녀노소 문화강좌 문화공간 문화시설 문화예술교육 복합문화공간     '}
,{'place': '심곡항', 'info': '강릉 바다를 즐길 수 있는 명소', '태그': '관광지'}
,{'place': '비둘기공원', 'info': '모두가 이용할 수 있는 공원', '태그': '관광지 근린공원 배리어프리공원 산책 시흥시공원 장미원 체육공원 '}
,{'place': '비둘기공원', 'info': '모두가 이용할 수 있는 공원', '태그': '관광지 근린공원 배리어프리공원 산책 시흥시공원 장미원 체육공원 '}
,{'place': '진밭골야영장', 'info': '도심 속에서 다양한 형태의 캠핑을 즐길 수 있는 접근성 좋은 야영장', '태그': '가족과함께 대구야영장 레포츠 오토캠핑 카라반'}
,{'place': '흔들전망대', 'info': '갯골의 생태를 한눈에 담을 수 있는 전망대', '태그': '경치여행 경치좋은곳 관광지 남녀노소 생태공원 시흥가볼만한곳 시흥갯골생태공원   '}
,{'place': '시흥 ABC행복학습타운', 'info': '예술과 생명, 문화로 행복을 채우는 공간', '태그': '관광지 문화 문화센터 복합문화공간 산림욕장 소래산 시흥복합문화공간 '}
,{'place': '슬로우캠프', 'info': '영월 무릉도원 100% 황토 전통 펜션과 별빛 계곡 힐링 캠핑', '태그': '강원도캠핑장 계곡물놀이 그늘막사이트 낚시 레포츠 산나물채취 얼음썰매 '}
,{'place': '시흥 그린웨이', 'info': '전원적인 풍경을 간직한 자전거길', '태그': '레포츠 산책로 산책코스 수변산책로 자전거길 자전거코스 커플데이트    '}
,{'place': '은율탈춤전수관', 'info': '은율탈춤을 배우고 즐길 수 있는 곳', '태그': '가족이색체험 문화시설 아이와 함께 이색체험 인천탈춤전수관 전통문화'}
,{'place': '삼미시장', 'info': '정과 웃음이 넘치는 삼미시장', '태그': '가족과함께 남녀노소 삼미시장 쇼핑 쇼핑거리 시장맛집 시흥여행    '}
,{'place': '청라해변공원캠핑장', 'info': '온 가족이 함께하는 힐링 캠핑', '태그': '레포츠 여유있는캠핑장 인기캠핑장2024'}
,{'place': '구학산채관광마을 희망캠핑장', 'info': '시골 폐교를 활용한 자연친화적인 체험형 야영장', '태그': '가족캠핑장 강원도캠핑장 농촌캠핑 둘레길 등산 레포츠 원주시캠핑장'}
,{'place': '첫물내기캠핑장', 'info': '법흥계곡에 위치한 청결한 시설의 가족 친화형 캠핑장', '태그': '가족캠핑 계곡물놀이 레포츠 법흥계곡 아이랑가기좋은 영월캠핑장'}
,{'place': '수도사(평택)', 'info': '원효대사의 깨달음이 있는 사찰', '태그': '관광지 불교 불교문화 사찰 사찰여행 수도사 역사 자연속으로 자연여행         '}
,{'place': '미산분교 캠핑장', 'info': '인제 내린천 상류에 위치한 애견 전용 캠핑장', '태그': '가을단풍명소 강원도캠핑장 레포츠 애견동반캠핑장 여름물놀이 인제군캠핑장'}
,{'place': '정모시쉼터', 'info': '정방폭포로 흐르는 물줄기가 지나가는 도민들만 아는 숨은 명소', '태그': '관광지 서귀포 서귀포시가볼만한곳 여름피서지 제주도'}
,{'place': '춘천숲자연휴양림엔글램핑', 'info': '조용하고 아늑한 자연휴양림', '태그': '가을단풍명소 강원도캠핑장 레포츠 봄꽃여행 스키 춘천시캠핑장'}
,{'place': '전곡항', 'info': '하얀 요트들이 멋진 장관을 이루고 있는 전곡항', '태그': '가족과함께 경기권 경치좋은곳 관광지 마리나요트 바다 바다경치          '}
,{'place': '소제동', 'info': '레트로 감성으로 재탄생한 거리', '태그': '2024_여행가는달과함께하는_발도장여행지 INFP낭만여행 관광지 뉴트로         '}
,{'place': '큐브존캠핑장', 'info': '독특한 컨테이너로 제작된 건물과 함께 이색적인 감성을 느낄 수 있는 캠핑장', '태그': '가을단풍명소 감성캠핑 강원도캠핑장 레포츠 영월군캠핑장 조명이아름다운'}
,{'place': '섬진강향가오토캠핑장', 'info': '자연 속 다양한 체험이 가능한 순창의 종합형 오토캠핑장', '태그': '가족여행 레포츠 섬진강 캠핑장 힐링여행'}
,{'place': '여울소리', 'info': '가족단위로 여름 휴가를 즐기는 캠핑객이 많이 찾는 곳', '태그': '가을단풍명소 가족캠핑 강원도캠핑장 레포츠 여름계곡물놀이 영월군캠핑장 '}
,{'place': '대전역 동광장', 'info': '대전역을 오가는 사람들의 만남의 광장', '태그': '관광지 광장 대전가볼만한곳 대전여행 대전역 대전중앙시장 대전휴식공간   '}
,{'place': '스카이 펜션&캠핑장', 'info': '영월 법흥계곡 옆에 위치한 펜션 겸 캠핑장', '태그': '가을밤줍기 강원도캠핑장 겨울설경 레포츠 애견동반캠핑장 여름계곡물놀이 '}
,{'place': '수암골전망대', 'info': '사방이 트인 청주의 야경 명소', '태그': '가족여행 걷기여행데이트코스 경치좋은곳 관광지 데이트코스 산책하기좋은곳       '}
,{'place': '진두선착장', 'info': '어촌의 생활을 느낄 수 있는 항구', '태그': '관광지 영흥도 인천낚시 인천바다 인천선착장 진두선착장 진두항'}
,{'place': '솔밭공원', 'info': '우거진 소나무 사이 자연 놀이터', '태그': '공원산책 관광지 아이와함게 청주공원 청주데이트코스 청주산책 청주여행 '}
,{'place': '금방아 민박캠핑장', 'info': '계곡 인근의 자연 캠핑장', '태그': '가족캠핑 강원도캠핑장 낚시 레포츠 법흥계곡 애견동반캠핑장 여름물놀이 '}
,{'place': '장미공원', 'info': '아름다움과 향기가 넘치는 공간', '태그': '관광지 문학산 문학장미공원 장미근린공원 장미꽃명소 장미꽃축제 장미여행지 '}
,{'place': '청주 충청도병마절도사영문', 'info': '충청도 병마절도사 영(營)의 출입문', '태그': '관광지 아이와함께 역사 역사공부 역사관광지 역사를품은곳 역사속         '}
,{'place': '궁평항', 'info': '잊을 수 없는 일몰을 선물해 주는 항구', '태그': '2024_여행가는달과함께하는_발도장여행지 경기권 관광지 궁평항 바다          '}
,{'place': '북극곰 야영장', 'info': '계곡과 숲이 있는 캠핑장', '태그': '가을단풍명소 강원도캠핑장 계곡물놀이 레포츠 애견동반캠핑장 평창군캠핑장'}
,{'place': '인천북항다목적부두', 'info': '인천 서구와 동구의 공업단지를 배후로 하는 다목적 부두', '태그': '관광지 무역 부두 서해 인천 인천부두 인천북항 인천북항다목적부두 '}
,{'place': '대전 중앙시장', 'info': '긴 역사를 지닌 대전의 전통시장', '태그': '2024_여행가는달과함께하는_발도장여행지 가족여행 나들이 대전가볼만한곳            '}
,{'place': '명암유원지', 'info': '울창한 산림과 사계절마다 특색있는 명암유원지', '태그': '가족여행 관광지 명암유원지 산책길 산책하기 산책하기좋은곳 생태관광지       '}
,{'place': '꿈엔들캠핑장', 'info': '아름다운 숲속의 넓은 대지 위에 있는 힐링 공간', '태그': '가을단풍명소 강원도캠핑장 고씨굴 레포츠 여름물놀이 영월군캠핑장'}
,{'place': '송대공원', 'info': '호수가를 따라 산책하기 좋은 공원', '태그': '관광지 청주공원 청주데이트코스 청주산책 청주송대공원 청주여행 청주포토존'}
,{'place': '서울 의릉(경종·선의왕후) [유네스코 세계유산]', 'info': '조선 경종과 두 번째 왕비 선의왕후의 무덤', '태그': '관광지 교과서속여행 도심여행 서울의릉 아이와함께 역사 역사공부         '}
,{'place': '대둔사(구미)', 'info': '조선 후기의 건축기법을 알 수 있는 사찰', '태그': '관광지 구미가볼만한곳 불교 불교문화 사찰 역사 전통사찰 한국불교  '}
,{'place': '서울 정릉(신덕황후) [유네스코 세계유산]', 'info': '조선 1대 태조의 두 번째 왕비 신덕고황후의 무덤', '태그': '관광지 서울정릉 세계문화유산 수도권 아이와함께 역사 역사공부            '}
,{'place': '명파해변오토캠핑장', 'info': '파아란 바다를 바라보며', '태그': '강원도캠핑장 걷기길 고성군캠핑장 레포츠 명파해변 수상레저 인기캠핑장2024'}
,{'place': '신항 바다쉼터', 'info': '서해의 아름다움을 보고 휴식을 취할 수 있는 곳', '태그': '관광지 서해바다쉼터 송도가볼만한곳 송도드라이브코스 송도바다산책로    '}
,{'place': '한반도 솔숲 쉼터', 'info': '산세를 즐기며 캠핑 할 수 있는 곳', '태그': '강원도캠핑장 넓은데크사이트 레포츠 별보기좋은 숲속캠핑 신설캠핑장  '}
,{'place': '원각사(구미)', 'info': '민족 정신을 고양하기 위해 지은 사찰', '태그': '관광지 구미가볼만한곳 불교 사찰 역사 전통사찰 한국불교 혼자가도좋은  '}
,{'place': '국립화천숲속야영장', 'info': '반려동물과 함께 깨끗한 공기를 마실 수 있는, 산림청 1호 숲속야영장', '태그': '강원권반려동물여행지 광주시산맥 국립공원캠핑장 레포츠 반려동물 반려동물동반여행지   '}
,{'place': '남강캠프', 'info': '넓은 잔디밭이 있는 캠핑장', '태그': '남강계곡 남강계곡캠핑장 레포츠 어린이가놀기좋은 여름물놀이 캠핑수영장'}
,{'place': '학마을캠프펜션', 'info': '호수와 산으로 둘러싸인 자연친화적 캠핑장', '태그': '레포츠 물멍불멍하기좋은 민물낚시 뷰맛집 철원캠핑장 펜션가능 학저수지'}
,{'place': '산여울캠프장', 'info': '깨끗한 계곡과 함께', '태그': '계곡물놀이캠핑장 레포츠 벚꽃캠핑장 숲속의캠핑장 영월캠핑장 조용한캠핑장'}
,{'place': '장호비치캠핑장', 'info': '장호해변 언덕에 위치해 있는 장호비치캠핑장', '태그': '레포츠 물놀이캠핑장 삼척캠핑장 아이들과가볼만한곳 장호항캠핑장 해양체험캠핑장'}
,{'place': '원주키즈캠핑장', 'info': '아이들이 즐길 수 있는 공간이 많은 캠핑장', '태그': '가족캠핑 다양한놀이시설 대형수영장 레포츠 아이랑가기좋은 원주캠핑장'}
,{'place': 'SPA 1899', 'info': '홍삼의 힘을 전하는 SPA 1899', '태그': '건강여행 관광지 남녀노소 부모님과함께 뷰티.스파 연인과함께 우수웰니스관광지        '}
,{'place': '상당공원', 'info': '청주시의 중심부, 상당공원', '태그': '관광지 청주공원 청주데이트코스 청주산책 청주상당공원 청주여행 청주포토존'}
,{'place': '한반도 솔숲 쉼터', 'info': '산세를 즐기며 캠핑 할 수 있는 곳', '태그': '강원도캠핑장 넓은데크사이트 레포츠 별보기좋은 숲속캠핑 신설캠핑장  '}
,{'place': '원각사(구미)', 'info': '민족 정신을 고양하기 위해 지은 사찰', '태그': '관광지 구미가볼만한곳 불교 사찰 역사 전통사찰 한국불교 혼자가도좋은  '}
,{'place': '국립화천숲속야영장', 'info': '반려동물과 함께 깨끗한 공기를 마실 수 있는, 산림청 1호 숲속야영장', '태그': '강원권반려동물여행지 광주시산맥 국립공원캠핑장 레포츠 반려동물 반려동물동반여행지   '}
,{'place': '남강캠프', 'info': '넓은 잔디밭이 있는 캠핑장', '태그': '남강계곡 남강계곡캠핑장 레포츠 어린이가놀기좋은 여름물놀이 캠핑수영장'}
,{'place': '학마을캠프펜션', 'info': '호수와 산으로 둘러싸인 자연친화적 캠핑장', '태그': '레포츠 물멍불멍하기좋은 민물낚시 뷰맛집 철원캠핑장 펜션가능 학저수지'}
,{'place': '산여울캠프장', 'info': '깨끗한 계곡과 함께', '태그': '계곡물놀이캠핑장 레포츠 벚꽃캠핑장 숲속의캠핑장 영월캠핑장 조용한캠핑장'}
,{'place': '장호비치캠핑장', 'info': '장호해변 언덕에 위치해 있는 장호비치캠핑장', '태그': '레포츠 물놀이캠핑장 삼척캠핑장 아이들과가볼만한곳 장호항캠핑장 해양체험캠핑장'}
,{'place': '원주키즈캠핑장', 'info': '아이들이 즐길 수 있는 공간이 많은 캠핑장', '태그': '가족캠핑 다양한놀이시설 대형수영장 레포츠 아이랑가기좋은 원주캠핑장'}
,{'place': 'SPA 1899', 'info': '홍삼의 힘을 전하는 SPA 1899', '태그': '건강여행 관광지 남녀노소 부모님과함께 뷰티.스파 연인과함께 우수웰니스관광지        '}
,{'place': '상당공원', 'info': '청주시의 중심부, 상당공원', '태그': '관광지 청주공원 청주데이트코스 청주산책 청주상당공원 청주여행 청주포토존'}
,{'place': '청계산', 'info': '청정한 산림의 청계산', '태그': '과천산 관광지 관악산 서울대공원 서울청계산 자연 청룡산'}
,{'place': '상무화훼단지', 'info': '봄을 먼저 맞이하는 곳', '태그': '광주가볼만한곳 광주꽃구입 광주꽃단지 광주꽃시장 광주나무구입 광주여행  '}
,{'place': '삼남매와뭉치네캠핑장', 'info': '자작나무 숲길이 있는 캠핑장', '태그': '감성카라반 강원도캠핑장 레포츠 아이와가볼만한곳 아이와캠핑 평창캠핑장'}
,{'place': '사랑나무캠핑장', 'info': '울창한 나무 아래에서 선선한 바람을 느끼며', '태그': '계곡물놀이캠핑장 내리계곡캠핑장 레포츠 물놀이하기좋은캠핑장 아이들과가볼만한곳 '}
,{'place': '구룡산(서울)', 'info': '부담 없이 즐기는 등산 명소', '태그': '관광지 구룡산 산 수도권 자연 자연좋은곳 자연환경 트레킹 휴식여행 '}
,{'place': '서부농수산물도매시장', 'info': '생산자와 소비자를 보호하는 호남 최대의 도매시장', '태그': '농수산물 농수산물검사소 농수산물경매 도매시장 서부농수산물도매시장  '}
,{'place': '별마로빌리지캠핑장', 'info': '맑은 공기와 청정계곡으로 유명한 별마로캠핑장', '태그': '김삿갓계곡 레포츠 물멍하기좋은 신서유기촬영지 애견동반캠핑장 여름물놀이  '}
,{'place': '목포 대중음악의 전당', 'info': '현대의 디지털 세계에서 만나는 레트로', '태그': '구호남은행 대중음악의전당 목포가볼만한곳 목포박물관 목포실내관광지    '}
,{'place': '산동교친수공원', 'info': '코스모스와 억새 물결이 일렁이는 공원', '태그': '관광지 광주가볼만한곳 광주코스모스 산동교 산동교친수공원 유채꽃명소  '}
,{'place': '마가리캠핑장', 'info': '숲과 계곡이 쾌적한 캠핑장', '태그': '강원도캠핑장 계곡캠핑 레포츠 봉평캠핑장 여름계곡 평창캠핑장'}
,{'place': '푸른길분수공원', 'info': '우리나라 최초의 폐선부지 공원', '태그': '2022친환경추천여행지 관광지 광주 광주_여행지_추천 광주가볼만한곳      '}
,{'place': '리버힐즈오토캠핑장', 'info': '1만 평 이상의 넓은 강변 부지 캠핑장', '태그': '강원도캠핑장 계곡캠핑 나무가많은 레포츠 영월캠핑 예약없는캠핑장'}
,{'place': '서울 헌릉(태종·원경왕후)과 인릉(순조·순원황후) [유네스코 세계유산]', 'info': '조선 태종과 순조, 그리고 왕비들의 무덤', '태그': '가족여행 관광지 교과서속여행 서울헌릉과인릉 아이와함께 역사 역사공부                  '}
,{'place': '계방산오토캠핑장', 'info': '청정지역과 함께하는 캠핑', '태그': '레포츠 청정지역 캠핑 평창 평창캠핑장'}
,{'place': '상남시장', 'info': '정이 느껴지는 현대식 건물의 도시형 전통시장', '태그': '가족여행 경상권 나들이 나홀로여행 먹거리 상남시장 쇼핑 시장맛집     '}
,{'place': '밀양의열기념관', 'info': '항일독립운동에 앞장섰던 독립투사들의 정신을 배우자!', '태그': '교육여행 문화시설 밀양의열기념관 아이와함께 역사공부 역사를품은곳        '}
,{'place': '샛도리물', 'info': '삼양해수욕장 해안 길을 걷다 보면 만날 수 있는 작은 샘', '태그': '관광지 물놀이 여름 제주 제주가볼만한곳'}
,{'place': '갈천오토캠핑장', 'info': '쭉쭉 뻗은 나무사이로 흐르는 맑은 공기와 시원한 바람들', '태그': '계곡물놀이 나무가많은캠핑장 레포츠 속초인근 양양캠핑장 자연속힐링'}
,{'place': '도란도란', 'info': '프라이빗한 공간에서 휴식을 즐길 수 있는 카라반 캠핑', '태그': '걷기길 레포츠 봄꽃여행 스파글램핑 스파캠핑 여름물놀이 전주글램핑 '}
,{'place': '은평한옥마을', 'info': '북한산의 아름다운 자연환경과 현대식 한옥이 어우러진 명소', '태그': '관광지 서울 은평구 한옥마을 한옥체험'}
,{'place': '감악산캠핑숲', 'info': '자연 속 깨끗하고 아담한 캠핑장', '태그': '강원도캠핑장 계곡캠핑장 그늘좋은캠핑장 레포츠 아이들과놀기좋은캠핑장 '}
,{'place': '서울 살곶이 다리', 'info': '조선 전기 다리 중 규모가 가장 큰 다리', '태그': '걷기길 관광지 교과서속여행 산책하기 산책하기좋은곳 살곶이다리 역사관광지   '}
,{'place': '칠포오토캠핑장', 'info': '넓고 푸른 잔디밭 사이트가 특징인 캠핑장', '태그': '가족캠핑여행 레포츠 물놀이장캠핑장 아이와가볼만한곳 자연학습장캠핑장 '}
,{'place': '상원사(원주)', 'info': '치악산 남대봉 기슭에 위치한 유서 깊은 사찰', '태그': '강원권 관광지 교과서속여행 불교 불교문화 사찰여행 역사 자연환경      '}
,{'place': '어반슬로우시티', 'info': '힐링 가득한 감성 캠핑과 수상레저를 동시에!', '태그': '가족캠핑여행 레포츠 아이와가볼만한곳 춘천 춘천캠핑장'}
,{'place': '대변시장', 'info': '멸치와 다양한 해산물을 구매할 수 있는 시장', '태그': '기장멸치축제 기장여행 대변시장 대변항 멸치시장 부산가볼만한곳 부산바다여행 '}
,{'place': '의령벽계야영장', 'info': '의령 자연에 녹아드는 캠핑', '태그': '가족캠핑여행 레포츠 물놀이장캠핑장 물놀이캠핑장 아이들과가볼만한곳 '}
,{'place': '서울 선릉과 정릉', 'info': '성종과 정현왕후 윤씨 그리고 중종의 왕릉', '태그': '2024_여행가는달과함께하는_발도장여행지 가족여행 관광지 교과서속여행          '}
,{'place': '힐링스토리캠핑장', 'info': '고즈넉한 외딴 산속, 하늘에 떠다니는 구름을 쳐다보며 여유를 즐겨보자', '태그': '가족캠핑여행 강원도캠핑장 레포츠 숲속캠핑장 조용한캠핑장 회성캠핑장'}
,{'place': '오도1리해수욕장', 'info': '주상절리의 비경을 감상할 수 있는 간이 해수욕장', '태그': '관광지 오도1리해수욕장 포항가볼만한곳 포항여행 포항주상절리 포항해수욕장 '}
,{'place': '신창 간이 해수욕장', 'info': '민물과 바닷물이 만나는 일출이 아름다운 해수욕장', '태그': '관광지 민물과바닷물이만나는 신창간이해수욕장 포항가볼만한곳 포항여행  '}
,{'place': '측도', 'info': '눈앞에 펼쳐지는 모세의 기적', '태그': '관광지 대한민국섬 서해바다 인천유인도 측도 측도둘레길'}
,{'place': '양포항', 'info': '해양휴식공간 속 힐링여행', '태그': '관광지 양포항 포항가볼만한곳 포항낚시 포항바다낚시 포항여행 포항여행지추천'}
,{'place': '송도솔밭도시숲', 'info': '숲에서 맨발 걷기 체험이 가능한 곳', '태그': '관광지 근린공원 맨발로둘레길 어린이물놀이 포항도시속피크닉 포항송도솔밭  '}
,{'place': '당너머해수욕장', 'info': '이국적인 인천의 해변', '태그': '관광지 당너머해수욕장 목섬 선재도해수욕장 옹진군해수욕장 인천해수욕장'}
,{'place': '해간도캠핑장', 'info': '자연 친화적인 풍경 속에서 캠핑', '태그': '낚시 레포츠 수상레저 캠핑장 통영 통영캠핑'}
,{'place': '용한 1리 해수욕장', 'info': '영일만항과 맞닿은 곳에 펼쳐져 있는 해수욕장', '태그': '관광지 영일만항 용한1리해수욕장 포항서핑 포항여행 포항해수욕장 '}
,{'place': '호불사', 'info': '수려한 경관이 어우러진 사찰', '태그': '관광지 불교 불교문화 사찰여행 수도권 역사 역사유적지 연중무휴       '}
,{'place': '칠포해오름전망대', 'info': '한반도에서 해가 가장 먼저 뜨는 곳', '태그': '관광지 영일만북파랑길 포항바다 포항산책 포항여행 포항전망대 포항투어 '}
,{'place': '하동 고소성', 'info': '섬진강과 동정호가 내려다 보이는 산성', '태그': '경상권 고소성 관광지 사적 역사 역사관광지 역사유적지 자연풍경   '}
,{'place': '용골송어와캠핑', 'info': '강원도 평창 송어회로 유명한 용골캠핑장', '태그': '계곡캠핑 다양한체험활동 레포츠 물멍하기좋은 청정계곡물놀이 키즈캠핑 '}
,{'place': '칠포항', 'info': '고즈넉한 분위기의 어항', '태그': '관광지 칠포항 포항가볼만한곳 포항낚시 포항여행 포항항구 항구'}
,{'place': '모포항', 'info': '사진 명소로 유명한 항구', '태그': '관광지 모포항 포항가볼만한곳 포항낚시 포항낚시여행 포항시 포항여행'}
,{'place': '궁농오토캠핑장', 'info': '소중한 사람들과 특별한 경험과 추억을 쌓는 캠핑', '태그': '거제도가볼만한곳 거제도오토캠핑장 낚시랑캠핑이랑 레포츠 바다보이는캠핑장'}
,{'place': '하선대', 'info': '절벽 옆 바다 위를 거닐 수 있는 곳', '태그': '관광지 산책명소 산책하기좋은곳 자연 자연환경 포항가볼만한곳 하선대'}
,{'place': '충주 남산성', 'info': '몽고군의 침입을 꿋꿋이 막아낸 산성', '태그': '관광지 역사 역사유적지 자연풍경 충주가볼만한곳 충주남산성 충청권  '}
,{'place': '거장산오토캠핑장', 'info': '자연과 함께 즐기는 힐링 캠핑장', '태그': '가족과함께 거장산오토캠핑장 거장산캠핑장 레포츠 연인과함께 의령캠핑장 '}
,{'place': '포항 북부시장', 'info': '싱싱한 자연산 생물로 유명한 곳', '태그': '가족여행 경상권 먹거리 사계절 쇼핑 아이와함께 이색체험 전통시장      '}
,{'place': '장기장 (4, 9일)', 'info': '포항의 특산물을 만날 수 있는 전통 시장', '태그': '가족여행 경상권 기차여행 나들이 나홀로여행 먹거리 쇼핑 시장맛집       '}
,{'place': '산골캠핑장', 'info': '산골의 정취를 느끼는 힐링 공간', '태그': '가족캠핑여행 계곡물놀이캠핑 레포츠 법흥계곡캠핑장 얼음장같은수영장 '}
,{'place': '산골캠핑장', 'info': '산골의 정취를 느끼는 힐링 공간', '태그': '가족캠핑여행 계곡물놀이캠핑 레포츠 법흥계곡캠핑장 얼음장같은수영장 '}
,{'place': '황산문화체육공원 국민여가캠핑', 'info': '즐길 거리가 가득하고 쾌적한 양산 캠핑장', '태그': '가족과함께 레포츠 미니기차 양산시민우선예약 오토캠핑장 친구와함께 '}
,{'place': '대학공원', 'info': '인천 연수구의 벚꽃 명소로 알려진 공원', '태그': '관광지 대학공원 연수구공원 연수구배드민턴장 연수구산책하기좋은공원'}
,{'place': '송도 한옥마을', 'info': '가장 현대적인 도시속 전통적인 한옥을 즐길 수 있는 곳', '태그': '관광지 도깨비촬영지 송도데이트코스 송도한옥단지 송도한옥마을 야왕촬영지    '}
,{'place': '행화촌자연테마파크', 'info': '아이들과 함께 할 수 있는 체험거리가 가득한', '태그': '구미캠핑장 대구시캠핑장 레포츠 아이들이좋아하는 야외수영장 여름물놀이  '}
,{'place': '국립일제강제동원역사관', 'info': '인권과 세계평화에 대한 국민 교육의 장', '태그': '문화시설 부산가볼만한곳 부산여행 아이와함께 역사공부 역사관광지   '}
,{'place': '욕지도청보리오토캠프', 'info': '낚시와 바다체험 캠핑을 한 곳에서', '태그': '낚시 노을맛집 레포츠 바다뷰 섬캠핑 수상레저 여름물놀이 욕지도캠핑장 '}
,{'place': '요카카캠핑장', 'info': '사계절 운영하는 충주의 캠핑장', '태그': '남한강뷰 레포츠 물놀이하기좋은 수영장이큰 애견동반캠핑장 충주캠핑장 '}
,{'place': '건지공원', 'info': '자연 속에 자리한 주민들의 휴식처', '태그': '건지공원 건지어린이공원 관광지 서구배드민턴장 서구산책로 서구어린이공원'}
,{'place': '완주소양캠핑장', 'info': '전주 한옥마을과 가까운 자연 속 캠핑장', '태그': '다양한사이트 레포츠 불멍하기좋은 산림욕 완주캠핑장 전주한옥마을인근  '}
,{'place': '완도명사십리오토캠핑장', 'info': '미네랄이 가장 풍부한 해수욕장에서 즐기는 캠핑', '태그': '가성비캠핑장 가족캠핑 데크가넓은 레포츠 명사십리해수욕장 신지도 '}
,{'place': '지리산반내골오토캠핑장', 'info': '지리산 상류의 맑은 정기와 함께하는 캠핑', '태그': '레포츠 봄꽃여행 아이들이좋아하는 야외수영장 여름물놀이 오토캠핑장  '}
,{'place': '작은동산', 'info': '자연이 만들어낸 청풍호의 풍경을 감상할 수 있는 곳', '태그': '경치좋은곳 관광지 외솔봉 자연관광지 자연풍경 작은동산 제천 제천가볼만한곳 '}
,{'place': '옥샘캠프촌', 'info': '어린이들이 놀기 좋은 자연 속 캠핑장', '태그': '가족캠핑 그늘이많은 당일캠핑가능 레포츠 밀양캠핑장 숲속캠핑'}
,{'place': '힐링오토캠핑장', 'info': '아름다운 청도를 대표하는 캠핑 쉼터', '태그': '가성비좋은캠핑장 경상북도캠핑장 놀이방캠핑장 레포츠 아이와가볼만한곳 '}
,{'place': '연희공원', 'info': '자연과 사람이 어우러진 생태 휴식공간', '태그': '관광지 연못정원 연희공원 연희근린공원 인천수변공원 인천아이와함께가볼만한곳    '}
,{'place': '횡성초보캠핑장', 'info': '설경이 아름다운 캠핑장', '태그': '가을단풍명소 레포츠 봄꽃여행 스키 여름물놀이 조용한캠핑 횡성캠핑장'}
,{'place': '독립문', 'info': '민족의 독립과 자유를 위한 국민의 상징인 기념물', '태그': '관광지 교과서속여행 독립문 독립협회 서대문독립공원 서울가볼만한곳    '}
,{'place': '상우농장야영장', 'info': '다양하고 즐거운 이벤트가 가득한 캠핑장', '태그': '가족캠핑여행 계곡물놀이캠핑장 레포츠 서울시근교캠핑장 아이와가볼만한곳 '}
,{'place': '치악산한다리골캠핑장', 'info': '청정 계곡이 흐르는 자연 속 휴식처', '태그': '레포츠 숲속캠핑 원주캠핑장 조용한캠핑 치악산 횡성한우마을인근'}
,{'place': '평화누리 캠핑장', 'info': 'DMZ 관광과 글램핑을 함께 즐길 수 있는 캠핑장', '태그': '가족과함께 레포츠 오토캠핑장 임진각국민관광지 파주가볼만한곳 파주여행    '}
,{'place': '폭포가있는캠핑장', 'info': '도시에서 벗어나 그림 같은 풍경 속으로 !', '태그': '가을단풍명소 레포츠 여름물놀이 인제캠핑장 캠핑장'}
,{'place': '취병조형유허비', 'info': '일본통신사 행차를 기록한 조형선생을 기리기 위한 비', '태그': '관광지 광주취병조형유허비 교과서속여행 문화재 역사 역사공부 역사를품은곳             '}
,{'place': '속초밤하늘글램핑', 'info': '편안하고 감성 가득한 글램핑의 추억', '태그': '가족캠핑여행 감성글램핑 낭만글램핑 레포츠 분위기좋은글램핑장 속초글램핑'}
,{'place': '월계동장고분', 'info': '우리나라 무덤의 발달사와 한일관계의 역사를 보여주는 삼국시대 고분', '태그': '관광지 광주고분 광주월계동장고분 신라고분 역사 역사공부 역사관광지           '}
,{'place': '강남고속터미널 혼수상가', 'info': '차별화된 혼수와 커튼 전문상가', '태그': '강남고속버스터미널 강남고속터미널혼수상가 강남혼수상가 고속버스터미널혼수상가     '}
,{'place': '프라임캠핑장', 'info': '용화산의 맑은 기운과 즐길 거리가 가득한 캠핑장', '태그': '가을단풍명소 레포츠 봄꽃여행 여름물놀이'}
,{'place': '엘림캠핑장', 'info': '소나무 지붕 밑에서 즐기는 캠핑', '태그': '계곡물놀이캠핑장 내평계곡캠핑장 레포츠 벚꽃캠핑장 삼척캠핑장 주변여행지많은캠핑장'}
,{'place': '티피캠핑장', 'info': '바다와 산을 동시에 즐길 수 있는 곳', '태그': '겨울눈꽃명소 고성카라반 고성캠핑 능파대 레포츠 바다캠핑 수상레저낚시   '}
,{'place': '신룡동오층석탑', 'info': '석불입상과 함께 무심히 서있는 신라계 고려 석탑', '태그': '고려5층탑 고려오층탑 관광지 광주신룡동오층석탑 역사 역사공부 역사관광지         '}
,{'place': '회동솔향캠핑장', 'info': '솔향기가 가득한 캠핑장', '태그': '가리왕산자연휴양림 가성비캠핑장 레포츠 소나무숲캠핑 정선캠핑장 조용한캠핑장'}
,{'place': '서울고속버스터미널 의류도매상가', 'info': '다양한 상품을 만날 수 있는 곳', '태그': '강남고속버스터미널 강남고속터미널 고속터미널역옷쇼핑 고터옷쇼핑 고터의류도매상가    '}
,{'place': '원적산체육공원', 'info': '인천시민의 건강한 여가활동을 위한 체육공원', '태그': '공원생태연못 관광지 부평구공원농구장 부평구공원배드민턴장 부평구공원족구장      '}
,{'place': '칠송정', 'info': '초야에 묻힌 선비의 효를 말해주는 조선시대 기효증의 정자', '태그': '관광지 광산칠송정 광주칠송정 기대승 역사 역사공부 역사관광지              '}
,{'place': '수목화송정마을오토캠핑장', 'info': '마을체험시설과 향토음식체험장도 함께 운영되는 곳', '태그': 'DMZ DMZ여행 고성캠핑장 레포츠 물놀이가능캠핑장 아이들과가볼만한곳  '}
,{'place': '삼거동 고인돌', 'info': '청동기 시대의 대표적 무덤을 보여주는 곳', '태그': '관광지 역사 역사공부 역사관광지 역사를품은곳 역사문화재 역사속      '}
,{'place': '경암근린공원', 'info': '가족 나들이하기 좋은 도심 공원', '태그': '경암근린공원 관광지 도심속숲 도심속휴식 무장애나눔길 어린이와가볼만한곳 '}
,{'place': '어라운드폴리', 'info': '제주의 자연을 몸소 느끼다', '태그': '레포츠 문화공연전시캠핑장 서귀포캠핑장 수영장캠핑장 아름다운캠핑장 '}
,{'place': '흥정계곡캠핑700', 'info': '흥정계곡 상류에 위치한 다이빙 가능한 계곡캠핑장', '태그': '계곡캠핑 레포츠 여름물놀이 철저한매너타임 평창캠핑장 해발700고지 '}
,{'place': '인천 아트플랫폼', 'info': '끊임없이 변화하는 새로운 문화예술의 창작공간', '태그': '공연&전시 남녀노소 문화시설 문화예술 문화예술공간 복합문화공간    '}
,{'place': '천사섬캠핑&글램핑', 'info': '탁 트인 바다와 아름다운 일출', '태그': '가족캠핑여행 레포츠 바닷가캠핑장 솔숲캠핑장 신안캠핑장 자은도글램핑장'}
,{'place': '비아5일시장', 'info': '규모는 작지만 없는 게 없는 알찬 시장', '태그': '공설시장 광주오일장 비아5일시장 쇼핑 시장 오일장 정이넘치는시장'}
,{'place': '강원도힐링캠핑장', 'info': '눈·비에도 쾌적하게 캠핑을 즐기다', '태그': '가족캠핑 놀이시설많은 다양한체험프로그램 레포츠 산속캠핑 아이가좋아하는 '}
,{'place': '칠포해수욕장', 'info': '여름에는 해수욕장으로, 가을에는 재즈공연장으로 변하는 해변', '태그': '가족여행 관광지 바다풍경 아이와함께 여름방학가볼만한곳 여름여행      '}
,{'place': '서울고속버스터미널 꽃도매상가', 'info': '향기 가득한 꽃들의 집결지', '태그': '고터꽃도매상가 고터꽃쇼핑 고터꽃시장 고터쇼핑 꽃 꽃도매상가 쇼핑 '}
,{'place': '영천아우노펠리스', 'info': '아이와 가기 참 좋은 곳', '태그': '럭셔리글램핑 레포츠 물놀이하기좋은 영천글램핑장 쾌적하고편리한 호텔시설'}
,{'place': '구룡포항', 'info': '질 좋은 과메기와 싱싱한 대게를 맛볼 수 있는 곳', '태그': '가족과함께 관광지 구룡포 구룡포근대역사관 구룡포시장 구룡포일본인가옥거리         '}
,{'place': '굴포천 생태하천', 'info': '자연과 이야기하면서 걷고 싶은 주민의 휴식공간', '태그': '관광지 부평굴포천생태하천 부평생태하천 인천굴포천생태하천 인천생태하천 '}
,{'place': '산노리캠핑장', 'info': '원주시 신림면 황둔리 감악산 계곡과 어우러진 자연속에 캠핑장', '태그': '가족캠핑여행 계곡물놀이캠핑장 레포츠 수영장캠핑장 아이들과가볼만한곳 '}
,{'place': '선성현문화단지 한옥체험관', 'info': '옛 관아를 복원해 전통 문화를 이해하고 체험 할 수 있는 공간', '태그': '관광지 선성현관아 선성현한옥체험 안동전통숙소 안동한옥 안동한옥체험관 '}
,{'place': '파주 임진각(평화누리공원)', 'info': '화해와 상생, 평화와 통일을 상징하는 복합문화공간', '태그': '2024_여행가는달과함께하는_발도장여행지 21_22한국관광100선 23_24한국관광100선                   '}
,{'place': '동호해수욕장', 'info': '청정해변의 부드러운 모래를 느끼며 산책할 수 있는 곳', '태그': '가볼래터 가족여행 관광지 동호해수욕장 여름가볼만한곳 여름여행 여름피서지       '}
,{'place': '군위 아미타여래삼존 석굴', 'info': '제2의 석굴암으로 유명한 신라의 석굴사원', '태그': '관광지 군위관광지 대구가볼만한곳 석굴사원 석굴암 아미타여래삼존석굴    '}
,{'place': '포항 발산리 모감주나무와 병아리꽃나무 군락', 'info': '세계적 희귀종인 모감주나무의 국내 최대 군락지', '태그': '경치여행 경치좋은곳 관광지 역사속으로 역사이야기 자연 자연속으로    '}
,{'place': '별밤캠핑장', 'info': '깔끔함이 돋보이는 낭만적인 공간', '태그': '나주글램핑 나주글램핑 나주캠핑장 나주캠핑장 레포츠 물놀이하기좋은캠핑장     '}
,{'place': '예담숲캠핑장', 'info': '아이들이 뛰어놀기 좋은 캠핑장', '태그': '다양한놀거리 단체가능 대형슬라이드 레포츠 범버카타기 생일파티하기좋은  '}
,{'place': '아빠의캠핑장', 'info': '사계절 아름다운 캠핑 스팟', '태그': '가족캠핑여행 계곡캠핑장 레포츠 물놀이캠핑장 아이들과가볼만한곳 원주캠핑장'}
,{'place': '흥환간이해수욕장', 'info': '구룡포로 가는 길목에 자리한 조용한 해수욕장', '태그': '관광지 여름피서지 자연 포항여행 포항해변 포항해수욕장 호미반도해안둘레길'}
,{'place': '부곡체육공원', 'info': '각종 스포츠 시설을 갖춘 종합 체육공원', '태그': '가족과함께 관광지 나들이 산책길 스포츠시설 운동하기좋은 의왕가볼만한곳  '}
,{'place': '초원숲속캠핑장', 'info': '넓은 초원 위 아늑한 휴식처', '태그': '기장캠핑장 돌물농장캠핑장 레포츠 물놀이장캠핑장 부산시캠핑장 아이와가볼만한곳'}
,{'place': '초원숲속캠핑장', 'info': '넓은 초원 위 아늑한 휴식처', '태그': '기장캠핑장 돌물농장캠핑장 레포츠 물놀이장캠핑장 부산시캠핑장 아이와가볼만한곳'}
,{'place': '사비나미술관', 'info': '한국 현대미술문화를 이끄는 선봉', '태그': '2022_추석_문화여행 가볼래터 기획전시 단체관람 도심여행 문화시설           '}
,{'place': '예그리나숲', 'info': '사계절을 즐길 수 있는 아름다운 풍경', '태그': '가을단풍명소 나무그늘이많은 레포츠 별구경할수있는 불멍물멍하기좋은  '}
,{'place': '초의선사탄생지', 'info': '우리나라의 다도문화를 중흥시킨 다성', '태그': '가볼래터 관광지 다도 무안가볼만한곳 여행구독 역사 역사관광지   '}
,{'place': '용천수캠핑장', 'info': '경주 유적지를 함께 탐방하기 좋은 캠핑장', '태그': '가족캠핑 경주캠핑장 레포츠 불국사인근 애견동반캠핑장 주변관광지가많은'}
,{'place': '삼정해수욕장', 'info': '깨끗한 바닷물과 완만한 경사가 조화를 이룬 해수욕장', '태그': '관광지 여름피서지 자연 자연속으로 자연여행 자연좋은곳 자연풍경    '}
,{'place': '드림파크 주민체육공원', 'info': '지역주민들의 건강과 여가를 위한 열린 공간', '태그': '관광지 드림파크 주민체육공원 인천생태습지공원 인천서구인라인트랙   '}
,{'place': '용궐산하늘길', 'info': '섬진강 굽어보는 짜릿한 잔도', '태그': '2024_여행가는달과함께하는_발도장여행지 가볼래터 레포츠 섬진강 순창가볼만한곳     '}
,{'place': '용궐산', 'info': '수려한 자연경관을 바라보며 걷는 하늘길 트래킹', '태그': '관광지 등산 순창등산 순창여행 순창트래킹 용궐산 용궐산하늘길 '}
,{'place': '달콤한캠핑장', 'info': '낚시와 물놀이를 동시에', '태그': '강원캠핑 레포츠 백암산 여름물놀이 풍암5일장 홍천맛국수 홍천찰옥수수축제 '}
,{'place': '요기는캠핑장', 'info': '아이들과 함께 신나게 뛰어놀 수 있는 곳', '태그': '가을단풍명소 레포츠 여름물놀이 영월 영월캠핑장'}
,{'place': '웅천친수공원', 'info': '휴식을 취할 수 있는 시민들의 근린공원', '태그': '걷기길 공원 관광지 바다산책 산책코스 산책하기좋은곳 여수가볼만한곳      '}
,{'place': '늘푸른공원', 'info': '공촌천을 따라 길게 조성된 공원', '태그': '관광지 늘푸른공원 청라게이트볼장 청라물놀이공원 청라산책공원 청라어린이물놀이장 '}
,{'place': '모인화랑', 'info': '현대미술 작품 소통의 장', '태그': '기획전시 남녀노소 모인화랑 문화공간 문화시설 문화예술 미술관데이트          '}
,{'place': '김씨네마카롱공장(주식회사필링)', 'info': '좋은 재료와 최고의 맛, 합리적인 가격의 수제 마카롱', '태그': '가자미 관광두레 관광두레_주민사업체 관광지 대왕암공원 디저트 로컬여행        '}
,{'place': '낙화담협동조합', 'info': '추억을 담은 소풍 같은 힐링 수변 캠핑', '태그': 'MTB 경북 경북_관광두레 경북_레포츠 관광두레 관광두레_주민사업체         '}
,{'place': '더숲캠핑장', 'info': '아이들이 맘껏 뛰어놀기 좋은 캠핑장', '태그': '경주오토캠핑장 경주장박 경주캠핑장 레포츠 잔디밭캠핑'}
,{'place': '농업회사법인삼산도가주식회사', 'info': '농사부터 양조까지 건강한 술을 빚는 청년공동체', '태그': '관광두레 관광두레_주민사업체 관광지 로컬여행 로컬푸드 막걸리 양조장        '}
,{'place': '인천 경서동 녹청자 요지', 'info': '녹청자 도요기술의 역사가 담겨 있는 곳', '태그': '가족여행 관광지 서울근교여행 역사 역사공부 역사를품은곳 역사문화재         '}
,{'place': '농업회사법인주식회사꿀벌인', 'info': '꿀벌의 생활을 체험하는 칠곡 대표 양봉 체험 공간', '태그': '경북 경북_관광두레 경북_체험 공예 관광두레 관광두레_주민사업체         '}
,{'place': '장길리 복합 낚시공원', 'info': '아름다운 바다의 풍경과 풍부한 해산물이 가득한 곳', '태그': '관광지 바다낚시 산책하기좋은곳 아이와함께 친구와함께 포항낚시 포항여행    '}
,{'place': '농업회사법인주식회사보령은행빵', 'info': '지역 청년들이 설립한 농촌 체험형 디저트 브랜드', '태그': '관광두레 관광두레_주민사업체 농촌체험 디저트 로컬여행 로컬푸드          '}
,{'place': '캄파슬로우', 'info': '울창한 나무 그늘과 계곡바람을 만끽할 수 있는 캠핑', '태그': '글램핑가능 깨끗하고관리가잘된 레포츠 백운산인근 숲속캠핑 원주캠핑장 '}
,{'place': '밋추로드(주식회사소중한모든것)', 'info': '문학산을 즐기는 N가지 방법', '태그': '관광두레 관광두레_주민사업체 관광지 로컬여행 문학산 비류 설화        '}
,{'place': '이탈리아마을 피노키오와 다빈치', 'info': '여권 없이 떠나는 이탈리아 여행', '태그': 'INFP핫플여행 가볼래터 관광지 다빈치 사진찍기좋은곳 아이와함께      '}
,{'place': '능내근린공원', 'info': '아름다운 자연의 변화를 감상할 수 있는 공간', '태그': '공원잔디축구장 관광지 능내공원 능내근린공원 만수산공원 인조잔디축구장'}
,{'place': '수달궁(문화살롱주식회사)', 'info': '전주 문화예술 기반의 창작·교육 프로젝트 진행', '태그': '관광두레 관광두레_주민사업체 관광지 기념품 로컬여행 수공예 수달      '}
,{'place': '연희자연마당', 'info': '자연과 사람이 어우러진 생태공원', '태그': '가볼래터 공원 관광지 생태공원 아이와함께 여행구독 연희공원'}
,{'place': '영양펜션 산적 (청년협동조합별빛고은)', 'info': '외씨버선길의 가장 아름다운 구간과 예쁜 초가집 펜션', '태그': '경북 경북_관광두레 경북_숙박 관광두레 관광두레_주민사업체 로컬여행        '}
,{'place': '구룡포 주상절리', 'info': '구룡포해수욕장에서 만날 수 있는 주상절리', '태그': '관광지 구룡포가볼만한곳 구룡포여행 구룡포주상절리 여행지추천 포항가볼만한곳 '}
,{'place': '오래실', 'info': '원예체험, 전통공예, 캠핑의 조화', '태그': '공방 관광두레 관광두레_주민사업체 레포츠 로컬여행 보은 식물원       '}
,{'place': '유니의바다카라반캠핑장', 'info': '바다와 함께하는 캠핑장', '태그': '구룡포캠핑장 럭셔리글램핑 레포츠 바다낚시 바다뷰 일출맛집 포항카라반'}
,{'place': '유한회사더담희', 'info': '남도의례음식 이수자의 향토음식 기반 체험프로그램', '태그': '관광두레 관광두레_주민사업체 관광지 광주 광주_관광두레 남도의례음식          '}
,{'place': '일상화글램핑(주식회사일상화)', 'info': '숙박과 문화 체험이 가능한 폐교를 활용한 글램핑장', '태그': '관광두레 관광두레_주민사업체 글램핑 노천스파 레포츠 로컬여행 문화체험         '}
,{'place': '전라감영양조장(전라감영주식회사)', 'info': '전주의 전통주 체험이 가능한 전통주 판매장', '태그': '가양주 관광두레 관광두레_주민사업체 관광지 로컬여행 양조장 이색체험     '}
,{'place': '청라호수공원 환상의 숲 놀이터', 'info': '어린이들의 꿈과 건강이 자라나는 즐거운 도심속 놀이공간', '태그': '도깨비촬영지 레포츠 서구어린이놀이터 음악분수 인천어린이놀이터 청라호수공원  '}
,{'place': '주식회사 놀핏', 'info': '놀면서 피트니스, 노르딕워킹과 로컬 도보투어의 협업', '태그': '관광두레 관광두레_주민사업체 관광지 노르딕워킹 로컬도보여행 로컬여행      '}
,{'place': '주식회사감천아울', 'info': '환경도 살리고 감천문화마을을 소개하는 업사이클링 굿즈', '태그': '감천문화마을 관광두레 관광두레_주민사업체 관광지 기념품 로컬여행        '}
,{'place': '주식회사꽃초린두레', 'info': '자연에서 치유하며 경험할 수 있는 힐링 공간', '태그': '경남 경남_관광두레 경남_체험 관광두레 관광두레_주민사업체 관광지       '}
,{'place': '주식회사더정원의뜰', 'info': '지친 마음을 치유할 다양한 재밋거리를 만들어내는 사람들', '태그': '경기 경기_체험 경기도_관광두레 관광두레 관광두레_주민사업체 관광지          '}
,{'place': '주식회사도시나무', 'info': '커피 찌꺼기를 활용한 체험 프로그램과 유니크한 기념품', '태그': 'DIY체험 관광두레 관광두레_주민사업체 관광지 기념품 로컬여행 부울_관광두레       '}
,{'place': '정서진중앙시장', 'info': '1989년 개장한 테마가 있는 문화관광형 전통시장', '태그': '쇼핑 수산물 전통시장 정서진 정서진시장 정서진중앙시장'}
,{'place': '주식회사라온문화창작소', 'info': '쉼, 여유 그리고 전통의 멋이 있는 목공 체험 공간', '태그': '경북 경북_관광두레 경북_체험 관광두레 관광두레_주민사업체 관광지     '}
,{'place': '주식회사뷰티홀릭', 'info': '뷰티를 플레이 하다, 퍼스널 뷰티 여행', '태그': 'K-뷰티 관광두레 관광두레_주민사업체 관광지 광안리 네일체험 로컬여행       '}
,{'place': '주식회사아래모래', 'info': '감양이와 대롱이, 감천문화마을을 소개하는 캐릭터 굿즈', '태그': '감양이 감천문화마을 관광두레 관광두레_주민사업체 기념품 대롱이       '}
,{'place': '주식회사어반힐링', 'info': 'Color trip 컬러를 찾아 떠나는 로컬여행사', '태그': 'K-뷰티 관광두레 관광두레_주민사업체 관광지 로컬여행 로컬여행사      '}
,{'place': '주식회사영리캐롤', 'info': '칠곡 미군 캠프 ‘캠프캐롤’의 시크릿 BBQ 투어', '태그': '경북 경북_관광두레 경북_식음 관광두레_주민사업체 관광지 디저트      '}
,{'place': '주식회사조아지구', 'info': '버려지는 식물성 부산물을 활용하여 만든 ‘비건점토’', '태그': '관광두레 관광두레_주민사업체 관광지 로컬여행 비건점토 완주 이색체험      '}
,{'place': '주식회사피알아이피', 'info': '독립책방과 여행을 주제로 한 체험형 프로그램', '태그': '관광두레 관광두레_주민사업체 관광지 기념품 독립책방 디지털카메라     '}
,{'place': '하얀민들레마을(주식회사미진)', 'info': '마음의 울림과 치유를 전하는 체험 공간', '태그': '관광두레 관광두레_주민사업체 관광지 로컬여행 민들레차 보은 소반체험      '}
,{'place': '트릭아트스토리', 'info': '착시 효과로 재미난 전시를 만나볼 수 있는 곳', '태그': '관광지 아이와함께 인천가볼만한곳 전시시설 착시효과 친구와함께 트릭아트 '}
,{'place': '학수고대협동조합', 'info': '’마을을 명상하다’ 치유 명상 농촌관광', '태그': '경북 경북_관광두레 경북_체험 계절밥상 관광두레_주민사업체 관광지      '}
,{'place': '농업회사법인주식회사 월출산꽃따리농장', 'info': '영암에서 경험하는 상큼한 감귤 체험', '태그': '가족과함께 관광두레 관광두레_주민사업체 관광지 농촌체험 이색체험      '}
,{'place': '대해불빛시장', 'info': '따뜻한 인심과 맛있는 먹거리가 가득한 시장', '태그': '다양한먹거리 다양한식자재 대해종합시장 쇼핑 포항가볼만한곳 포항쇼핑 '}
,{'place': '달달구리(주식회사달달구리)', 'info': '완주를 닮은 달달구리', '태그': '관광두레 관광두레_주민사업체 관광지 연인과함께 전북_관광두레 주말추천여행지  '}
,{'place': '지리산둘레길 6코스 수철-성심원', 'info': '자연속의 산과 강을 어우르는 가벼운 둘레길', '태그': '도보여행 레포츠 지리산 지리산둘레길 지리산둘레길6코스'}
,{'place': '양학시장', 'info': '상가주택 건물형의 소규모 시장', '태그': '다양한식자재 먹거리 쇼핑 양학시장 포항가볼만한곳 포항소형시장 포항시장'}
,{'place': '지리산둘레길 7코스 성심원-운리', 'info': '길따라 걸어보는 시간', '태그': '도보여행 레포츠 지리산 지리산둘레길 지리산둘레길7코스'}
,{'place': '현대프리미엄아울렛 SPACE1', 'info': '문화와 쇼핑을 한꺼번에 즐길 수 있는 국내 첫 갤러리형 아울렛', '태그': '반려동물 반려동물동반여행지 쇼핑'}
,{'place': '상대종합시장', 'info': '아날로그 감성을 물씬 느낄 수 있는 시장', '태그': '다양한먹거리 다양한식자재 상대종합시장 쇼핑 포항가볼만한곳 포항소형시장 '}
,{'place': '상대종합시장', 'info': '아날로그 감성을 물씬 느낄 수 있는 시장', '태그': '다양한먹거리 다양한식자재 상대종합시장 쇼핑 포항가볼만한곳 포항소형시장 '}
,{'place': '삼산건강공원', 'info': '벚꽃길이 아름다운 걷기 좋은 공원', '태그': '관광지 부평산책로 부평테니스장 삼산건강공원 삼산공원 인천테니스장'}
,{'place': '포항 남부종합시장', 'info': '따뜻한 정을 체험하다', '태그': '남부종합시장 다양한먹거리 쇼핑 식자재 포항가볼만한곳 포항소형시장 '}
,{'place': '지리산둘레길 8코스 운리-덕산', 'info': '경남 산청군 단성면 운리부터 산청군 시천면 덕산까지 이어지는 코스', '태그': '도보여행 레포츠 지리산 지리산둘레길 지리산둘레길8코스'}
,{'place': '지리산둘레길 9코스 덕산-위태', 'info': '자연 경관 만끽하며 걷는 길', '태그': '도보여행 레포츠 지리산 지리산둘레길 지리산둘레길9코스'}
,{'place': '한중문화관', 'info': '한국과 중국의 문화교류의 장', '태그': '가족여행 나들이 문화시설 복합문화공간 실내여행지 이색체험 인천가볼만한곳   '}
,{'place': '강릉 모래내 한과마을(갈골한과)', 'info': '한과의 역사와 전통을 이어가는 마을', '태그': '갈골한과 강릉가볼만한곳 과자 관광지 농촌마을 농촌여행 명절음식              '}
,{'place': '공무원연금공단 김해상록골프클럽', 'info': '상록의 명성, 옛 가야의 이야기를 담은 김해상록cc', '태그': '경남골프클럽 공무원연금공단 공무원연금공단골프장 김해18홀골프장    '}
,{'place': '열우물전통시장', 'info': '새롭게 변화하는 시장', '태그': '부평과일가게 부평먹거리 부평전통시장 부평채소가게 쇼핑 십정시장 '}
,{'place': '부평시장', 'info': '부평의 최대 상권지', '태그': '부평깡시장 부평먹거리 부평시장 부평전통시장 부평종합시장 쇼핑 인천전통시장'}
,{'place': '창원산촌생태마을', 'info': '풍요롭고 넉넉한 농심의 지리산 산촌마을', '태그': '관광지 다랭이논 산촌마을 오도재 창고마을 함양여행'}
,{'place': '사천해변(사천해수욕장)', 'info': '반려동물과 걷기 좋은 한적하고 푸르른 해변', '태그': '가족여행 강릉여행 강원권 강원권반려동물여행지 강원도_여행지_추천                      '}
,{'place': '청라호수도서관', 'info': '행복과 지식을 함께 가꾸어 가는 도서관', '태그': '남녀노소 도서관 독서공간 독서여행 문화공간 문화시설 복합문화공간     '}
,{'place': '만경촌마을', 'info': '다양한 먹거리 프로그램이 있는 농촌체험마을', '태그': '경북농촌체험마을 관광지 만경촌먹거리 만경촌먹거리체험관 의성농촌단체체험  '}
,{'place': '신세동 벽화마을', 'info': '골목골목 누비며 만나는 벽화가 아름다운 마을', '태그': '관광지 벽화마을 안동벽화마을 안동산책로 안동여행 안동투어 안동포토존'}
,{'place': '전고령가야왕릉', 'info': '고령가야의 도읍이었던 함창에서 발견된 왕릉', '태그': '관광지 역사 역사공부 역사관광지 역사를품은곳 역사문화재 역사속      '}
,{'place': '하평해변(강릉)', 'info': '하얀 모래밭과 해송이 넓게 펼쳐져 있는 해변', '태그': '가족과함께 강원권 경치여행 경치좋은곳 관광지 바다 바다경치 바다내음   '}
,{'place': '미술관아래협동조합', 'info': '지역 문화예술가가 만든 기념품과 체험이 있는 카페', '태그': '2024으뜸두레 관광두레 관광두레_주민사업체 관광지 미술관 복합문화공간      '}
,{'place': '애니멀스토리', 'info': '동물들을 체험하면서 즐거운 시간을 보낼수 있는 곳', '태그': '강릉기니피그 강릉이색체험 강릉체험장 관광지 다양한동물체험 동물농장  '}
,{'place': '보현사(강릉)', 'info': '천년의 쉼터이자 마음의 휴식처', '태그': '관광지 불교 불교사찰 사찰 사찰여행 역사 자연속으로 자연좋은곳      '}
,{'place': '영진항', 'info': '연어가 회귀하고 향좋은 은어가 많이 잡히는 연곡천 하류 북쪽에 있는 곳', '태그': '강릉가볼만한곳 관광지 영진항 자연'}
,{'place': '순긋해변', 'info': '잔잔한 파도 소리와 아늑하고 조용한 분위기가 지친 일상을 치유하는 해변', '태그': '가볼래터 가족과함께 강원_가볼만한곳 강원권 강원도_여행지_추천 관광지     '}
,{'place': '밀양아리랑길3코스', 'info': '나들이 코스', '태그': '레포츠 밀양가볼만한곳 밀양걷는길 밀양아리랑길 밀양아리랑길1코스    '}
,{'place': '밀양아리랑길2코스', 'info': '경남 밀양의 걷고 싶은 길로 손꼽히는 밀양아리랑길', '태그': '레포츠 밀양가볼만한곳 밀양걷는길 밀양아리랑길 밀양아리랑길1코스    '}
,{'place': '야미야미팜(야미야미팜농업회사법인(주))', 'info': '전국 최고 품질의 멜론과 수박', '태그': '가족과함께 관광두레 관광두레_주민사업체 관광지 농산물 미식 미식여행        '}
,{'place': '송금체험마을영농조합법인', 'info': '풀드포크, 지역특산 홍시 음료를 제공하는 농촌체험마을', '태그': '경북_관광두레 관광두레 관광지 농촌체험 아이와함께 염색체험 이색체험  '}
,{'place': '예담은(사단법인예담은규방문화원)', 'info': '전통방식 그대로 자연의 빛깔을 담아내는 예담은', '태그': '2024으뜸두레 관광두레 관광두레_주민사업체 관광지 굿즈 염색체험        '}
,{'place': '오로시예술공방(주식회사오로시프로젝트)', 'info': '쉼과 예술을 제공하는 작은 여행, 오로시프로젝트', '태그': '공예 관광두레 관광두레_주민사업체 관광지 도예 도자기체험 실내관광지     '}
,{'place': '우리술과힐링(주식회사우리술과힐링)', 'info': '영암 월출산에서 우리술과 함께 힐링 가능', '태그': '관광두레 관광두레_주민사업체 관광지 미식여행 전남_관광두레 전통주    '}
,{'place': '현덕사(강릉)', 'info': '달밤이 운치를 더하는 소박한 사찰', '태그': '관광지 불교 불교문화 사찰 사찰여행 역사 자연좋은곳 자연환경       '}
,{'place': '밀양아리랑길1코스', 'info': '숲속에서 걸어보는 시간', '태그': '레포츠 밀양가볼만한곳 밀양걷는길 밀양아리랑길 밀양아리랑길1코스    '}
,{'place': '유한회사복합문화공간 서리울', 'info': '자연 속 작은 배움터, 내가 주인공이 되는 체험 공간', '태그': '가족과함께 관광두레 관광두레_주민사업체 관광지 아이와함께 이색체험  '}
,{'place': '착한공작소(착한공작소협동조합)', 'info': '로컬의 다양한 자원으로 전주 굿즈와 식음 개발', '태그': '공예 관광두레 관광두레_주민사업체 관광지 실내관광지 전북_관광두레   '}
,{'place': '한복입고 유유자적 (주식회사고영)', 'info': '영암 구림마을에서 한복 입고 체험하는 관광 콘텐츠', '태그': '관광두레 관광두레_주민사업체 관광지 문화체험 원데이클래스 이색체험        '}
,{'place': '독무재', 'info': '조선 후기의 학자 하시찬을 기리는 추모 재실', '태그': '관광지 대구가볼만한곳 대구독무재 역사 역사관광지 역사속으로 역사유적지  '}
,{'place': '이현종합시장', 'info': '상가주택건물형의 종합시장', '태그': '쇼핑 이현종합상가 정글인 진주가구 진주가구단지 진주가구용품 진주골프   '}
,{'place': '여수 선소유적', 'info': '이순신 장군이 거북선을 만든 장소', '태그': '가족과함께 거북선 관광지 선소유적 여수가볼만한곳 여수여행 역사   '}
,{'place': '동상시장', 'info': '전통과 이국의 다양한 글로벌 문화가 사이좋게 공존하는 시장', '태그': '김해가볼만한곳 김해농산물 김해쇼핑 김해여행지 김해재래시장 김해전통시     '}
,{'place': '영진해변', 'info': '가족단위로 피서를 즐기기에 적합한 곳', '태그': '강원권 관광지 바다경치 바다내음 바다풍경 여름여행 여름휴가 연중무휴      '}
,{'place': '김해농수산물종합유통센터', 'info': '우리 농수산물을 보호하고 농어민의 소득 안정과 유통구조 혁신을 위하다', '태그': '김해농수산물종합유통센터 김해시 농협하나로마트 생필품 쇼핑 식자재마트'}
,{'place': '부경축산물주촌도매시장', 'info': '싱싱하고 저렴하게 고기를 구매할 수 있는 큰 규모의 도축장', '태그': '김해관광지 김해쇠고기 김해쇼핑 김해여행 김해여행지 김해축산물 부경축산물주촌도매시장 '}
,{'place': '양산북부시장', 'info': '광장구조 양산북부시장', '태그': '쇼핑 양산농산물 양산상설시장 양산쇼핑 양산시 양산시장여행 양산재래시장 '}
,{'place': '자유시장 (진주)', 'info': '진주의 역사가 있는 전통시장', '태그': '경남특산물 경상남도시장 쇼핑 자유시장 진주가볼만한곳 진주시장 진주여행  '}
,{'place': '진주농산물도매시장', 'info': '유기농 농산물도매시장', '태그': '쇼핑 진주가볼만한곳 진주농산물 진주농산물도매시장 진주도매시장 진주식재료시장 '}
,{'place': '덕산마을오일장', 'info': '사람과 사람의 온정이 오고가는 전통 재래시장', '태그': '가볼만한곳 경남 덕산마을 마을 산청 쇼핑 오일장 재래시장 지리산 '}
,{'place': '청송구과상유문암', 'info': '마그마가 관입되어 형성된 꽃모양 암석', '태그': '관광지 구과상유문암 독특한암석 마그마 청송꽃돌'}
,{'place': '송강리 습곡구조', 'info': '지각변동의 특성을 관찰 할 수 있는 곳', '태그': '관광지 습곡구조 지질명소 청송가볼만한곳 청송볼거리'}
,{'place': '용두목', 'info': '일상에서 벗어나 잠시 휴식을 취하다', '태그': '레포츠 밀양가볼만한곳 밀양강 밀양아리랑길 밀양여행 밀양여행지추천  '}
,{'place': '강릉 오죽헌·시립박물관', 'info': '오죽헌과 강릉의 역사와 문화 이야기를 전하는 곳', '태그': '강릉시시립박물관 강릉시오죽헌 강원권 교과서속여행 문화관광해설사                '}
,{'place': '덕진공원', 'info': '세대별 추억이 공존하는 도시공원', '태그': '가볼래터 가족여행 걷기여행 관광지 나들이 덕진공원 덕진호 사계절               '}
,{'place': '풍각나무망치 (나무이야기 영농조합법인)', 'info': '따뜻하고 낭만적인 목공 체험 공방', '태그': '가볼래터 경북_관광두레 관광두레 관광두레_주민사업체 관광지 아이와함께    '}
,{'place': '정촌유기농원', 'info': '아름다운 농촌을 널리 알리고 있는 힐링 치유 농장', '태그': '가볼래터 가족과함께 관광두레 관광두레_주민사업체 관광지 베이킹     '}
,{'place': '헤이리 예술마을', 'info': '다양한 문화적 특성을 갖춘 아름다운 마을', '태그': '15_16한국관광100선 17_18한국관광100선 2024_여행가는달과함께하는_발도장여행지                  '}
,{'place': '용천사(청도)', 'info': '비슬산 동쪽 기슭에 자리하고 있는 사찰', '태그': '경상권 관광지 불교 불교문화 사찰 사찰여행 역사 자연속으로 자연환경         '}
,{'place': '아이아이 연남', 'info': '서로의 마음을 알아볼 수 있는 심리 전시 공간', '태그': '관광지 데이트하기좋은곳 마음에세이 서울가볼만한곳 심리전시 연인과함께  '}
,{'place': '파주 덕은리 주거지와 지석묘군', 'info': '선사시대 사람들의 삶의 흔적을 볼 수 있는 곳', '태그': '관광지 덕은리지석묘 역사 역사공부 역사관광지 역사를품은곳 역사문화            '}
,{'place': '호미곶 해안단구 (경북 동해안 국가지질공원)', 'info': '한반도에서 가장 먼저 해가 떠오르는 일출 명소', '태그': '경북_동해안_국가지질공원 경북동해안 경치좋은곳 관광지 국가지질공원                '}
,{'place': '오류', 'info': '', '태그': ''}
,{'place': '구룡소 돌개구멍 (경북 동해안 국가지질공원)', 'info': '바닷가에 위치한 연못과 같은 지형', '태그': '경북_동해안_국가지질공원 경북동해안 관광지 국가지질공원 바다경치            '}
,{'place': '골굴암 타포니 (경북 동해안 국가지질공원)', 'info': '자연적으로 발달된 타포니와 신라인들의 불교문화가 조화를 이룬 가치 있는 명소', '태그': '경북_동해안_국가지질공원 경북동해안 관광지 국가지질공원 동굴 불교               '}
,{'place': '골굴암 타포니 (경북 동해안 국가지질공원)', 'info': '자연적으로 발달된 타포니와 신라인들의 불교문화가 조화를 이룬 가치 있는 명소', '태그': '경북_동해안_국가지질공원 경북동해안 관광지 국가지질공원 동굴 불교               '}
,{'place': '시화호', 'info': '경기도 안산, 시흥, 화성에 걸쳐있는 인공호수', '태그': '관광지 드라이브 서울근교여행지 수도권 시화방조제 시흥 안산 안산드라이브     '}
,{'place': '자인전통시장', 'info': '오랜 전통을 가지고 있는 경산의 시장', '태그': '가족여행 경상권 기차여행 나들이 나홀로여행 먹거리 쇼핑 시장맛집         '}
,{'place': '탄도바닷길', 'info': '바다가 갈라지는 신비한 현상을 볼 수 있는 곳', '태그': '갯벌체험 경치좋은곳 관광지 바다풍경 사진찍기 사진찍기좋은곳 산책로   '}
,{'place': '탄도선착장', 'info': '대부도의 낙조가 아름다운 선착장', '태그': '갯벌체험 관광지 낙조 대부도 안산가볼만한곳 어촌마을 일몰 탄도항  '}
,{'place': '애플글램핑', 'info': '사과 체험이 가능한 글램핑장', '태그': '걷기길 레포츠 백양사 장성군캠핑장 전라남도캠핑장 캠핑 편백숲'}
,{'place': '프로방스펜션', 'info': '아름다운 노을,신기한 기암절벽과 함께', '태그': '가족과함께 레포츠 백수해안도로 어린이수영장 영광군캠핑장 전라남도캠핑장'}
,{'place': '워렉스 캠핑랜드', 'info': '온 가족이 여유롭게 캠핑을', '태그': '가을단풍명소 레포츠 봄꽃여행 여름물놀이 전라남도캠핑장 캠핑 화순군캠핑장'}
,{'place': '타임앤블레이드 박물관', 'info': '다양한 시계와 칼을 구경할 수 있는 박물관', '태그': '가족과함께 관광지 미세먼지_피하는_실내여행 아이와함께 안보관광        '}
,{'place': '제물포구락부', 'info': '시민들의 근대 국제문화 교류의 현장', '태그': '건축사 건축여행 관광지 교과서속여행 근대건축 근대건축물 수도권     '}
,{'place': '강릉 경포대', 'info': '다섯 개의 달이 뜨는 낭만적인 달맞이 명소', '태그': '2024_여행가는달과함께하는_발도장여행지 강원권 경포대 관광지 문화관광해설사            '}
,{'place': '고성 자란만', 'info': '맑은 남해 바다가 아름다운 해안 드라이브코스', '태그': '관광지 바다향기 자란도 자연 참장어 포교마을 해안드라이브'}
,{'place': '웅천친수공원 야영장', 'info': '가성비와 주변 환경 모두 완벽한 최고의 캠핑장', '태그': '레포츠 수상레저 여름물놀이 여수시캠핑장 인기캠핑장2024 전라남도캠핑장 '}
,{'place': '캠핑아이', 'info': '아름다운 자연 속에서 즐기는 휴식', '태그': '가마미해수욕장 가볼래터 레포츠 숲쟁이공원 여행구독 영광군캠핑장  '}
,{'place': '성주사(창원)', 'info': '불모산 자락의 성인이 머무는 사찰', '태그': '가족여행 관광지 불교 불교사찰 사찰&산사 아이와함께 역사 역사유적지         '}
,{'place': '약산해안 치유의숲', 'info': '다도해의 바닷바람과 숲길이 선사하는 힐링', '태그': '관광지 완도여행 완도여행추천 완도치유의숲 전라남도여행 전라남도여행지추천 '}
,{'place': '영종대교휴게소 기념관', 'info': '인천대교의 경관을 조망할 수 있는 기념관', '태그': '가족여행 기념관 드라이브여행 문화시설 바다풍경 서울근교여행 아이와함께    '}
,{'place': '송도해안볼레길', 'info': '자연 속 트래킹', '태그': '레포츠 부산가볼만한곳 부산해안산책 부산해안산책로 송도포구 송도해수욕장 '}
,{'place': '파라다이스시티', 'info': '문화예술의 감동과 다채로운 즐거움이 공존하다', '태그': '관광지 리조트 문화예술 스파 씨메르 인천 인천관광 인천당일코스    '}
,{'place': '스탈릿성수', 'info': '성수동 핫플 도심형 복합상업시설', '태그': 'MZ세대 모나미스토어 문화시설 복합문화공간 복합상업시설 성수동핫플레이스  '}
,{'place': '성산포JC공원', 'info': '봄이면 유채꽃이 만발하는 곳', '태그': '관광지'}
,{'place': '이사부광장', 'info': '삼척항 옆 이사부 테마 레저공원', '태그': '관광지 광장 삼척 삼척관광 삼척시 삼척여행 이사부광장 해변산책'}
,{'place': '뿅뿅다리', 'info': '회룡포마을로 이어지는 이색적인 다리', '태그': '관광지 다리 뿅뿅다리 예천관광 예천군 예천여행'}
,{'place': '남애1리해수욕장', 'info': '남애항 남쪽 작은 어촌과 레트로 감성의 해변', '태그': '관광지 남애1리해수욕장 남애항 양양관광 양양군 양양여행 해수욕장'}
,{'place': '쉬리공원', 'info': '김화읍 화강변에 조성된 생태 및 레저공원', '태그': '관광지 쉬리공원 철원관광 철원군 철원여행 화강생태공원'}
,{'place': '솔바다캠핑장', 'info': '바다가 보이는 오션뷰 캠핑장', '태그': '노을이아름다운캠핑장 레포츠 바다캠핑장 영흥도캠핑장 인천 인천숙박  '}
,{'place': '선재오토캠핑장', 'info': '서해 바다의 아름다운 낙조를 감상할 수 있는 곳', '태그': '레포츠 물놀이할수있는캠핑장 바다가보이는캠핑장 바다캠핑장 수영장캠핑장   '}
,{'place': '죽천해수욕장', 'info': '차박과 낚시 명소로 이름 높은 해수욕장', '태그': '관광지 죽천해수욕장 포항 포항관광 포항바다 포항여행 해수욕장'}
,{'place': '선바위공원', 'info': '울산 12경 중 하나인 선바위가 있는 공원', '태그': '관광지 선바위공원 선바위휴게소 울산가볼만한곳 울산선바위 울주 울주군가볼만한곳'}
,{'place': '인천여성가족재단', 'info': '다양한 교양강좌를 운영하는 문화 공간', '태그': '문화공간 문화행사 부평교양강좌 수도권 여성문화 열린문화공간 이색체험    '}
,{'place': '승학체육공원', 'info': '지역주민들 건강한 삶을 위한 체육활동공원', '태그': '관광지 미추홀구농구장 미추홀구인라인스케이트장 미추홀구인조잔디축구장   '}
,{'place': '구영공원', 'info': '주민들을 위한 여가선용과 휴식공간으로 조성된 공원', '태그': '관광지 구영공원 구영리강아지산책 구영리산책 울산명소 울산산책'}
,{'place': '황희선생묘', 'info': '조선시대 명망 있는 재상으로 칭송되었던 황희 선생의 묘', '태그': '관광지 교과서속여행 수도권 아이와함께 역사 역사공부 역사관광지          '}
,{'place': '간절곶소망우체통', 'info': '반드시 사진을 찍어야 하는 명물 소망우체통', '태그': '간절곶소망길 관광지 느린우체통 소망우체통 울산가볼만한곳 울산간절곶'}
,{'place': '발포역사전시체험관', 'info': '임진왜란의 해전 속 주인공이 되어보자!', '태그': '고흥가볼만한곳 고흥여행 고흥체험학습 문화시설 발포역사전시체험관   '}
,{'place': '인천광역시교육청부평도서관', 'info': '지역민을 위한 평생학습과 발명에 특화된 도서관', '태그': '도서관 독서 독서공간 문화시설 부평도서관 아이와 함께 인천가볼만한곳 '}
,{'place': '하이커 그라운드', 'info': '실감체험이 가능한 멀티버스 한국관광 랜.드.마.크!', '태그': '2022_추석_문화여행 2024_여행가는달과함께하는_발도장여행지 가볼래터         '}
,{'place': '충무아트센터', 'info': '서울의 대표 공공문화예술회관', '태그': '문화생활 문화시설 수도권 아이와함께 연인과함께 예술 음악회 전시관   '}
,{'place': '서울 신당동 박정희 가옥', 'info': '박정희 전 대통령이 살았던 집', '태그': '가옥관람 관광지 국가등록문화재 문화재가옥 박정희 박정희대통령가옥   '}
,{'place': '성혼선생묘', 'info': '조선 중기 성리학의 대가 성혼 선생의 묘', '태그': '가족여행 경기도 관광지 성혼선생묘 역사 역사공부 역사관광지 역사를품은곳        '}
,{'place': '피크닉', 'info': '도심 속에서 누리는 소풍 같은 휴식', '태그': '갤러리카페 문화시설 복합문화공간 중구갤러리카페 카페피크닉 피크닉 '}
,{'place': '전주한벽문화관', 'info': '전통문화 체험과 공연이 있는 문화공간', '태그': '문화공간 문화시설 전시실 전주여행 전주한벽문화관 전통문화 전통혼례 '}
,{'place': '광안리회센터(민락어민활어직판장,씨랜드회센터)', 'info': '싱싱한 회를 맛볼 수 있는 곳', '태그': '광안대교 광안리맛집 광안리해수욕장 광안리회센터 민락동맛집 부산가볼만한곳      '}
,{'place': '키자니아 서울', 'info': '국내 최초의 직업체험 테마파크', '태그': '관광지 도심여행 미세먼지_피하는_실내여행 실내여행지 아이가_열광하는_여행         '}
,{'place': '전포 삼거리', 'info': '레트로한 부산 거리', '태그': '문화시설 부산 부산진구 부산포토존 아이유 영화촬영장소 포토존'}
,{'place': '인천광역시교육청화도진도서관', 'info': '인천의 향토역사를 볼 수 있는 도서관', '태그': '공공도서관 도서관 독서공간 독서여행 동구화도진도서관 문화시설 수도권     '}
,{'place': '추자도 등대', 'info': '선박들의 안전한 밤길을 인도하는 등대', '태그': '가족과함께 관광지 등대 자연 자연풍경 제주가볼만한곳 추자도 추자도등대 '}
,{'place': '백천계곡', 'info': '맑은 물 속, 열목어가 사는 계곡', '태그': '경상권 계곡 관광지 등산 등산로 봉화계곡 봉화단풍명소 여름물놀이    '}
,{'place': '남이섬', 'info': '상상의 즐거움이 가득한 나미나라공화국', '태그': '2024_여행가는달과함께하는_발도장여행지 21_22한국관광100선 23_24한국관광100선              '}
,{'place': '동해 종합시장', 'info': '바다보다 큰 정이 넘치는 시장', '태그': '가족여행 경상권 나들이 나홀로여행 데이트코스 동해종합시장 먹거리        '}
,{'place': '범바위 전망대', 'info': '낙동강과 어우러지는 아름다운 풍광을 만날 수 있는 곳', '태그': '관광지'}
,{'place': '주인근린공원', 'info': '주인선을 따라 조성된 공원', '태그': '관광지 미추홀구공원 주인근린공원 주인선'}
,{'place': '선달산(봉화)', 'info': '조선 최고의 약수인 오전약수가 있는 산', '태그': '경치여행 경치좋은곳 관광지 등산 등산객 자연 자연속으로 자연좋은곳    '}
,{'place': '달성 현풍 석빙고', 'info': '조선시대의 화강석제 얼음창고 석빙고', '태그': '경상권 관광지 역사 역사공부 역사관광지 역사를품은곳 역사속 역사속으로      '}
,{'place': '키자니아 부산', 'info': '아이들의 무궁무진한 미래를 위한 테마파크', '태그': '가족여행 경상권 관광지 미세먼지_피하는_실내여행 실내관광지 실내놀이터        '}
,{'place': '선달산(영월)', 'info': '아름다운 계곡과 우아한 산세가 인상적인 산', '태그': '관광지 선달산 영월 영월군 영월여행 자연'}
,{'place': '테디베어팜', 'info': '아기자기한 테디베어가 기다리는 농장', '태그': '곰인형 문화시설 속초가볼만한곳 속초여행 속초전시관 아기자기한 아이와함께  '}
,{'place': '드림파크CC', 'info': '2014년 인천아시안게임 골프 경기를 진행한 드림파크 골프장', '태그': '드림파크CC 드림파크골프장 레포츠 서구야외골프장 서구필드 인천드림파크'}
,{'place': '동아서점', 'info': '종이냄새가 물씬 풍기는 아늑한 서점', '태그': '독서하기좋은 문화시설 서점 속초가볼만한곳 조용한분위기 책읽기좋은곳   '}
,{'place': '청원사(안성)', 'info': '천덕산 깊은 곳에 자리한 조용한 사찰', '태그': '관광지 불교 안성가볼만한곳 안성청원사 역사 전통사찰 한국불교   '}
,{'place': '청원사(안성)', 'info': '천덕산 깊은 곳에 자리한 조용한 사찰', '태그': '관광지 불교 안성가볼만한곳 안성청원사 역사 전통사찰 한국불교   '}
,{'place': '유엔군 초전기념관', 'info': '죽미령고개 전투의 역사적 의미를 되새기는 곳', '태그': '가족여행 교과서속여행 기념관 문화시설 사계절 아이와함께 역사관광지     '}
,{'place': '노작홍사용문학관', 'info': '노작홍사용의 문학업적을 계승하는 공간', '태그': '교과서속여행 노작홍사용문학관 문화시설 수도권 아이와함께 전시관   '}
,{'place': '인천광역시 미추홀도서관', 'info': '인천시민의 지식과 지헤의 보고', '태그': '도서관 독서공간 독서교육 문화시설 사계절 아이와함께 인천가볼만한곳    '}
,{'place': '학익시장', 'info': '60여 년의 전통을 간직한 인천의 재래시장', '태그': '미추홀구전통시장 쇼핑 인천전통시장 재래시장 전통시장 학익동시장 '}
,{'place': '칠곡평화전망대', 'info': '역사의 현장, 칠곡을 바라보며', '태그': '관광지 낙동강 전망대 칠곡 칠곡가볼만한곳 칠곡평화전망대 평화'}
,{'place': '송현배수지제수변실', 'info': '인천 최초의 상수도시설이자 도시계획시설', '태그': '관광지 역사 역사공부 역사관광지 역사를품은곳 역사문화재 역사속        '}
,{'place': '한국항공대학교 항공우주박물관', 'info': '항공우주의 모든 것을 만날 수 있는 공간', '태그': '관광지 문화콘텐츠 박물관 산업관광 산업관광지 수도권 아이와함께       '}
,{'place': '강화유수부이방청', 'info': '수령을 보좌하던 지방행정 실무공간', '태그': '강화가볼만한곳 강화유수부이방청 관광지 역사 역사관광지 역사유적지  '}
,{'place': '인천 화평동 세숫대야 냉면거리', 'info': '세숫대야 냉면의 진수를 보여주는 맛집 거리', '태그': '가족과함께 관광지 냉면맛집 맛집탐방 비빔냉면 숨은맛집 음식거리        '}
,{'place': '강화유수부 동헌', 'info': '군청의 역할을 했던 조선시대의 관아', '태그': '강화가볼만한곳 강화유수부동헌 관광지 수도권 역사 역사관광지 역사유적지   '}
,{'place': '천년숲', 'info': '자연과 사람이 공존하는 도심 속 쉼터', '태그': '가족과함께 걷기좋은곳 경북권 레포츠 맨발걷기 산책로 안동가볼만한곳      '}
,{'place': '안동호', 'info': '낚시터와 휴양지로 인기 좋은 곳', '태그': '가족여행 경상권 레포츠 안동가볼만한곳 안동호 자연경관 호수'}
,{'place': '배후령고개', 'info': '자전거 라이딩 명소가 된 해발 600미터의 산길', '태그': '관광지 배후령고개 자전거라이딩 춘천 춘천관광 춘천시 춘천여행 '}
,{'place': '강화향교', 'info': '고려시대에 세워진 강화의 국립교육기관', '태그': '강화가볼만한곳 강화향교 관광지 역사 역사관광지 역사유적지 인천여행  '}
,{'place': '상주천주교신앙고백비', 'info': '천주교 순교자를 기리는 비', '태그': '경상권 관광지 신앙고백비 역사 역사공부 역사관광지 역사를품은곳         '}
,{'place': '포항중앙상가', 'info': '포항 시내 중심 상권을 대표하는 공간', '태그': '쇼핑 포항 포항관광 포항쇼핑 포항시 포항여행 포항중앙상가'}
,{'place': '태평성대경상감영공원', 'info': '옛 상주의 영광을 느껴볼 수 있는 상주의 대표 문화공간 복합단지', '태그': '관광지 교육여행 사진찍기좋은곳 아이와함께 역사공부 역사속으로 역사이야기    '}
,{'place': '경포생태저류지', 'info': '넓은 호수와 푸른 하늘이 어우러진 생태 저류지', '태그': '강릉관광 강릉여행 강릉유채꽃 경포생태 경포호관광 관광지'}
,{'place': '이가리간이해변', 'info': '남녀노소 누구나 해수욕하기 좋은 곳', '태그': '관광지 이가리간이해변 포항관광 포항시 포항시해변 포항여행 해변'}
,{'place': '후포근린공원', 'info': '울진 앞바다의 수려한 풍경이 펼쳐지는 곳', '태그': '경상북도울진 관광지 근린공원 울진관광 울진군 울진여행 후포근린공원'}
,{'place': '운달산', 'info': '천혜의 자연 속 천년 사찰 김룡사를 품고 있는 문경의 명산', '태그': '관광지 문경관광 문경명소 문경여행 문경운달산 용암산 운달산 자연'}
,{'place': '대도', 'info': '경남 하동 유일의 유인섬', '태그': '관광명소 관광지 다양한체험프로그램 유인섬 자연 하동가볼만한곳 하동군'}
,{'place': '함평 향교리 느티나무·팽나무·개서어나무 숲', 'info': '자연재해로부터 마을을 지켜온 숲', '태그': '관광지 자연 전라권 천연기념물 함평개서어나무 함평느티나무 함평팽나무      '}
,{'place': '예천박물관', 'info': '예천의 역사를 보고 느낄 수 있는 공간', '태그': '경상북도예천군 문화시설 예천관광 예천군 예천박물관 예천여행 예천역사'}
,{'place': '강화 화문석 문화관', 'info': '화문석의 우수성을 알리고 발전시키는 곳', '태그': '강화가볼만한곳 강화도여행 강화화문석 강화화문석문화관 문화강좌 문화시설    '}
,{'place': '장척호와 번개늪', 'info': '수백종의 식물과 조류 및 어류가 서식하는 곳', '태그': '경남권 경남창녕군 관광명소 레포츠 창녕 창녕가볼만한곳 창녕바다'}
,{'place': '영천역 광장', 'info': '여행자의 휴식 공간이자 만남의 장소', '태그': '경상북도영천시 관광지 영천관광 영천광장 영천시 영천역광장'}
,{'place': '월등도', 'info': '바닷길이 열리는 작고 작은 섬', '태그': '2022찾아가고싶은섬 2022찾아가고싶은섬_가을섬 관광지 비토섬 사천가볼만한곳  '}
,{'place': '금산인삼약령시장', 'info': '저렴하게 인삼과 한약재를 유통하는 금산의 시장', '태그': '가볼래터 가족여행 금산시장 금산약령시장 금산인삼 금산인삼시장 금산인삼약령시장                 '}
,{'place': '쌍산재', 'info': '역사를 품은 고택과 자연이 어우러진 한옥', '태그': '가볼래터 가족여행 관광지 구례 구례가볼만한곳 구례여행 부모님과함께  '}
,{'place': '부흥해변', 'info': '서핑과 당일 캠핑을 즐기기 좋은 해변', '태그': '관광지 부흥해변 서핑 영덕관광 영덕군 영덕여행 캠핑'}
,{'place': '토성산맹꽁이작은도서관', 'info': '책 읽는 즐거움을 알아가는 곳', '태그': '가볼래터 가족과함께 나들이 도서관 문화시설 서산 서산가볼만한곳    '}
,{'place': '방화수류정(동북각루)', 'info': '반려동물과 함께 즐기는 아름다운 경관', '태그': '가볼래터 경기권반려동물여행지 관광지 반려동물 반려동물동반여행지     '}
,{'place': '길안천지생태공원', 'info': '차박 명소로 산자수명한 공원', '태그': '공원 관광지 길안천지생태공원 생태공원 안동공원 안동관광 안동생태공원 '}
,{'place': '청량폭포', 'info': '아름다운 풍경과 어우러지는 청량폭포', '태그': '관광지 봉화여행 봉화폭포 청량산도립공원 청량폭포'}
,{'place': '오성한옥마을', 'info': '자연과 문화가 조화를 이루어 살아가는 마을', '태그': '25_26한국관광100선 BTS bts여행 BTS촬영지 bts투어 BTS힐링성지           '}
,{'place': '주암정', 'info': '연꽃과 능소화가 장관을 이루는 정자', '태그': '관광지 문경관광 문경시 문경여행 주암정 주암채익하선생'}
,{'place': '비토섬 가는길', 'info': '별주부전의 전설을 품고 있는 비토섬으로 가는 드라이브코스', '태그': '경상남도여행 경치여행 경치좋은곳 관광지 드라이브여행 드라이브코스           '}
,{'place': '봉천사', 'info': '연보랏빛 개미취가 장관을 이루는 곳', '태그': '가볼래터 관광지 문경여행 사찰 여행구독 역사'}
,{'place': '봉천사', 'info': '연보랏빛 개미취가 장관을 이루는 곳', '태그': '가볼래터 관광지 문경여행 사찰 여행구독 역사'}
,{'place': '망산공원', 'info': '탁 트인 전망과 다양한 놀이시설이 구비되어 있는 공원', '태그': '걷기좋은길 걷기좋은길 관광지 근린휴식공원 사천공원 산책길 산책코스            '}
,{'place': '청학시장', 'info': '좋은 물건 싸게 파는 전통시장', '태그': '부산 부산가볼만한곳 부산여행 부산전통시장 상가주택복합형시장 쇼핑  '}
,{'place': '미포정거장', 'info': '아름다운 해안 절경을 마주하는 곳', '태그': '레포츠 미포정거장 미포철길 부산가볼만한곳 산책하기좋은곳 해안산책로 '}
,{'place': '반송공원', 'info': '아이와 함께 놀기 좋은 도심 속 공원', '태그': '관광지 도심공원 반송공원 부산광역시 부산반송공원 해운대 해운대공원 '}
,{'place': '거제씨월드', 'info': '국내 최고의 돌고래 테마파크', '태그': '가족여행 가족체험 거제씨월드 경상권 기념품판매 동물원 문화시설       '}
,{'place': '표선세화해안도로', 'info': '제주의 자연 풍경을 볼 수 있는 해안도로', '태그': '드라이브코스 레포츠 서귀포가볼만한곳 서귀포시 연인과함께 자연풍경   '}
,{'place': '하늘누리 경비행기', 'info': '비행으로 하늘에서 펼쳐보는 꿈', '태그': '경비행기교육 관광지 레포츠 비행교육 비행체험 전국투어비행 청혼비행  '}
,{'place': '부산새벽시장', 'info': '부산의 대표 전통재래시장', '태그': '부산 부산가볼만한곳 부산관광명소 부산여행 부산여행추천 새벽시장 '}
,{'place': '청량사(봉화)', 'info': '청량함과 고귀함을 간직한 사찰', '태그': '경상권 관광지 봉화청량사 불교 불교사찰 사찰 사찰여행 역사 자연속으로          '}
,{'place': '뽀로로파크 세종점', 'info': '뽀로로와 친구들을 만나는 공간', '태그': '가족여행 관광지 뽀로로 뽀로로파크 뽀로로파크세종점 세종키즈카페 '}
,{'place': '제주과학탐구체험관', 'info': '어린이와 함께 가기 좋은 과학 체험 실내 전시실', '태그': '문화시설 제주관광 제주도 제주융합과학연구원 제주융합과학연구원체험관   '}
,{'place': '오백나한', 'info': '기암괴석으로 둘러싸인 계곡', '태그': '계곡 관광지 기암괴석 영실기암 자연 자연풍경 자연환경 제주가볼만한곳 '}
,{'place': '보문사(강화)', 'info': '서해를 바라보고 있는 3대 해상 관음기도 도량', '태그': '2024_여행가는달과함께하는_발도장여행지 강화가볼만한곳 강화도관광지         '}
,{'place': '부산청과', 'info': '농민의 꿈을 소중히 키우다', '태그': '부산가볼만한곳 부산관광명소 부산도매시장 부산시장 부산여행추천 부산청과 '}
,{'place': '와흘메밀마을', 'info': '마을에서 운영하는 체험장과 메밀꽃 사진 촬영지', '태그': '관광지 마을체험 제주관광 제주메밀꽃촬영지 제주여행 제주와흘메밀마을'}
,{'place': '덕포시장', 'info': '부산 가볼 만한 전통 시장', '태그': '덕포시장 부산가볼만한곳 부산관광명소 부산명승지 부산시장 부산여행추천 '}
,{'place': '헌마공신김만일기념관', 'info': '김만일의 업적과 다양한 말 체험, 정보가 있는 전시관', '태그': '문화시설 서귀포 제주 제주관광 제주김만일기념관 제주서귀포 제주서귀포관광 '}
,{'place': '봉화시장', 'info': '활기와 정이 넘치는 봉화시장', '태그': '가족여행 경상권 나들이 나홀로여행 먹거리 봉화시장 쇼핑 시장맛집      '}
,{'place': '봉곡테마공원', 'info': '한적하게 산책하기 좋은 자연 친화적인 공간', '태그': '경상북도 관광지 구미공원 구미관광 구미시 구미여행 봉곡테마공원'}
,{'place': '하서항', 'info': '자물쇠 모양의 포토존이 있는 명소', '태그': '경주관광 경주시 경주여행 레포츠 하서항'}
,{'place': '아끈다랑쉬 오름', 'info': '바람에 흩날리는 억새를 보며 산책하기 좋은 오름', '태그': '가을여행 관광지 다랑쉬오름 아끈다랑쉬 억새 오름여행 오름추천       '}
,{'place': '강릉 3·1운동 기념공원', 'info': '경포대 아래 있는 3·1독립 만세 운동 기념공원', '태그': '강릉 강릉3.1운동기념공원 강릉공원 강릉관광 강릉시 경포 관광지 '}
,{'place': '인제라이딩센터', 'info': '숙박,정비,트레이닝 시설을 갖춘 자전거 라이더 특화시설', '태그': '라이딩체험 레포츠 설악그란폰도 인제관광 인제군 인제라이딩센터'}
,{'place': '영주산', 'info': '제주에서 낭만 가득한 산책을 할 수 있는 명소', '태그': '걷기좋은길 관광지 영주산 자연풍경 제주 제주가볼만한곳 풍경맛집 '}
,{'place': '남동공단근린공원', 'info': '산책하기 좋은 아늑한 공원', '태그': '관광지 남동공단근린공원 남동구공원 남동구산책 남동구축구장 남동구테니스'}
,{'place': '다대씨파크', 'info': '신선한 회와 요리를 맛볼 수 있는 활어 전문 수산시장', '태그': '다대씨파크 다대포항 다대포해수욕장 부산수산시장 부산시장탐방 부산해산물맛집 '}
,{'place': '철원승일공원', 'info': '승일교와 한탄대교를 동시에 볼 수 있는 강변공원', '태그': '공원 관광지 철원관광 철원관광지 철원군 철원승일공원 한탄강관광지'}
,{'place': '천호지·단대호수', 'info': '벚꽃과 야경이 아름다운 단국대 근처 호수', '태그': '걷기여행 걷기좋은길 관광지 단대호수 단대호수공원 데이트코스 사계절                  '}
,{'place': '마정저수지', 'info': '평화로운 풍경을 볼 수 있는 마정저수지', '태그': '관광지 저수지산책 천안낚시 천안데이트코스 천안산책 천안여행 천안저수지'}
,{'place': '송정서핑학교', 'info': '우리나라 최초의 서핑교육기관', '태그': '딩기요트 레포츠 부산액티비티 서핑강습 서핑교육 서핑체험 송정서핑학교 '}
,{'place': '경주세계자동차박물관', 'info': '자동차의 역사를 엿볼 수 있는 박물관', '태그': '경주 경주가볼만한곳 경주관광 경주데이트코스 경주세계자동차박물관  '}
,{'place': '정글의 법칙 미디어파크', 'info': '미디어아트로 만나는 정글의 법칙', '태그': '경주가볼만한곳 경주여행 관광지'}
,{'place': '볼음도', 'info': '백합 캐기와 낚시를 즐길 수 있는 자연경관 좋은 곳', '태그': '2022친환경추천여행지 강화도여행 갯벌 갯벌체험 관광지 볼음도 섬                     '}
,{'place': '물초울공원', 'info': '아이들이 좋아하는 놀이터와 물놀이장이 있는 공원', '태그': '관광지 진주물놀이장 진주아이들놀이터'}
,{'place': '원효대사 깨달음 체험관', 'info': '원효대사의 깨달음을 배울 수 있는 곳', '태그': '문화관광해설사 문화시설 사찰음식 수도사 원효대사 이색체험 친구와함께  '}
,{'place': '숭실대학교 한국기독교박물관', 'info': '한국기독교와 역사문화유산을 함께 전시하는 박물관', '태그': '문화시설 박물관 수도권 숭실대학교 숭실대학교한국기독교박물관 아이와함께     '}
,{'place': '팔용근린공원', 'info': '산책하기 좋은 공원', '태그': '관광지 마산공원 마산아이와가볼만한곳 창원공원 창원산책하기좋은공원 '}
,{'place': '진동시장', 'info': '사람 냄새 풍기는 정겨운 시장', '태그': '경남시장 쇼핑 진동리유적시장 창원시장 창원장서는날 창원전통시장'}
,{'place': '겸재정선미술관', 'info': '진경산수화의 대가 겸재 정선의 예술세계를 만날 수 있는 공간', '태그': '강서구미술관 겨울방학가볼만한곳 겸재미술관 겸재정선 구립미술관 당일치기여행              '}
,{'place': '간성향교', 'info': '조선시대 성현들의 정신이 살아있는 교육기관', '태그': '고성가볼만한곳 관광지 교과서속여행 분향참관 역사 역사관광지 향교 '}
,{'place': '진해삼포방파제', 'info': '잔잔하고 조용한 어촌 풍경을 볼 수 있는 방파제', '태그': '레포츠 진해감성돔 진해낚시 진해낚시포인트 진해방파제 진해생활낚시'}
,{'place': '서서울컨트리클럽', 'info': '자연지형을 최대한 살린 코스가 매력적인 곳', '태그': '골프장 레포츠 서서울컨트리클럽 클럽하우스 파주'}
,{'place': '만석부두', 'info': '서울로 향하는 곡물을 만석이나 쌓아놓았던 곳', '태그': '공업단지 관광지 만석부두 만석전망대 만석포구 서해바다 웨이브데크   '}
,{'place': '자매공원', 'info': '서울과 터키 앙카라의 우정을 상징하는 테마형 공원', '태그': '걷기길 관광지 나들이 남녀노소 도심공원 산책길 산책하기 산책하기좋은곳        '}
,{'place': '가포해안변공원', 'info': '마창대교와 일출을 동시에 감상할 수 있는 공원', '태그': '관광지 마산가볼만한곳 마산데이트코스 마산산책하기좋은곳 마산해변공원 '}
,{'place': '자라나는 숲(광진숲 나루 전망대)', 'info': '기둥과 덩굴식물이 숲과 같은 공간을 형성한 전망대', '태그': '관광지 광진숲나루전망대 광진자라나는숲 서울가볼만한곳 서울명소 서울여행'}
,{'place': '키튼드림센터', 'info': '아린이를 위한 스포츠 놀이 공간', '태그': '레포츠 아이와가볼만한곳 진주관광 진주수영장 진주여행 키튼드림센터'}
,{'place': '강릉 해운정', 'info': '강릉에서 오죽헌 다음으로 오래된 건축물', '태그': '관광지 역사 역사공부 역사관광지 역사를품은곳 역사문화재 역사속      '}
,{'place': '반송공원', 'info': '다양한 산책 코스를 갖춘 도심 속 힐링 공원', '태그': '관광지 창원공원 창원데이트코스 창원산책하기좋은공원 창원아이와함께가기좋은공원 '}
,{'place': '동인천역 북광장', 'info': '인천 근대로 발자취 여행의 시작점', '태그': '관광지 동인천역 동인천역북광장 동인천역행사 인천문화행사 인천페스티벌'}
,{'place': '솔바람다리', 'info': '아름다운 야간 조명과 바닷바람이 좋은 곳', '태그': '걷기여행 걷기좋은길 경치좋은곳 관광지 바다경치 바다산책 바다풍경          '}
,{'place': '부용체육공원', 'info': '제주도 부럽지 않은 세종의 유채꽃 명소', '태그': '관광지 세종가볼만한곳 유채꽃명소 한적한관광지 힐링'}
,{'place': '소죽도공원', 'info': '바다를 끼고 산책할 수 있고 소죽도 공원', '태그': '관광지 진해가족공원 진해공원 진해바다나들이 진해캠핑공원 진해피크닉'}
,{'place': '경포 아쿠아리움', 'info': '강릉의 석호 생태관인 대형 아쿠아리움', '태그': '가족과함께 강릉여행 관광지 교육여행 남녀노소 아이와함께 아쿠아리움   '}
,{'place': '인천 동부시장', 'info': '인천 동구청 송림로타리 부근에 소재한 전통시장', '태그': '가족여행 나들이 나홀로여행 드라이브여행 마을축제 먹거리 서울근교여행           '}
,{'place': '방축천 음악분수', 'info': '수변공원과 음악분수가 어우러진 도심 속 하천', '태그': '관광지 방축천 방축천음악분수 세종가볼만한곳 세종데이트코스 세종음악분수'}
,{'place': '오동동문화광장', 'info': '문화 예술 공연을 즐기는 문화 공간', '태그': '관광지 마산공원 마산문화거리 마산문화공연 마산볼거리 창원공원'}
,{'place': '문래창작촌', 'info': '구석구석 예술적 감성이 묻어나는 공간', '태그': '2024_여행가는달과함께하는_발도장여행지 관광지 문래동예술촌 문래동창작촌               '}
,{'place': '진해중앙시장', 'info': '다양한 먹거리가 풍부한 전통시장', '태그': '쇼핑 진해시장 진해전통시장 창원시장 창원시장맛집 창원전통시장'}
,{'place': '문래창작촌', 'info': '구석구석 예술적 감성이 묻어나는 공간', '태그': '2024_여행가는달과함께하는_발도장여행지 관광지 문래동예술촌 문래동창작촌               '}
,{'place': '진해중앙시장', 'info': '다양한 먹거리가 풍부한 전통시장', '태그': '쇼핑 진해시장 진해전통시장 창원시장 창원시장맛집 창원전통시장'}
,{'place': '자라섬캠핑장', 'info': '자라섬에서 신나는 캠핑 즐기기', '태그': '가족여행 레포츠 서울근교캠핑장 아이와함께 연인과함께 오토캠핑장      '}
,{'place': '천연제작소', 'info': '환경을 생각하는 제로웨이스트 잡화점', '태그': '무포장가게 부산_여행지_추천 부산가볼만한곳 부산권 쇼핑 제로웨이스트샵 '}
,{'place': '책방오월', 'info': '다양한 서적을 보유한 아늑한 쉼터', '태그': '관광지 독립서점 부산 부산_여행지_추천 부산가볼만한곳 부산권 부산여행 '}
,{'place': '책방봄봄', 'info': '그림책을 보고, 시를 보고, 차와 함께 마주 보고, 세상을 바라본다', '태그': '관광지 그림책방 독립서점 부산_여행지_추천 부산가볼만한곳 부산권  '}
,{'place': '강릉커피박물관', 'info': '전 세계의 다양하고 진귀한 커피유물을 전시하는 박물관', '태그': '가족과함께 강릉커피 강릉커피거리 강릉커피박물관 문화시설 미세먼지_피하는_실내여행        '}
,{'place': '대림동 차이나타운', 'info': '중국 현지 음식과 분위기를 느껴보고 싶다면!', '태그': '관광지 맛집골목 맛집여행 맛집투어 먹자골목 반려동물 반려동물동반여행지         '}
,{'place': '정동심곡 바다부채길', 'info': '정동진 해변의 파도가 만들어내는 해안단구', '태그': '25_26한국관광100선 가족과함께 강릉여행 강원권 동해바다 레포츠           '}
,{'place': '오금공원', 'info': '다양한 운동시설과 자연학습장을 갖춘 녹지 공간', '태그': '관광지 데이트코스 산책길 산책하기 산책하기좋은곳 아이와함께 연인과함께      '}
,{'place': '아르떼뮤지엄 강릉', 'info': '시각적 강렬함과 감각적인 사운드가 어우러진 체험을 제공', '태그': '강릉가볼만한곳 강릉여행 남녀노소 문화공간 문화시설 문화예술 미디어아트   '}
,{'place': '창원시 어린이교통공원', 'info': '놀이터와 레포츠 시설을 갖춘 어린이 교통 공원', '태그': '관광지 도로교통안전체험 창원가족공원 창원공원 창원아이와가볼만한곳  '}
,{'place': '성대시장', 'info': '정이 넘치는 친환경 시장', '태그': '동작구전통시장 상도동시장 성대시장 성대전통시장 쇼핑 재래시장 전통상업보존지구 '}
,{'place': '진영전통시장', 'info': '진영 단감으로 유명한 진영읍 전통시장', '태그': '경남시장맛집 경남시장투어 경남여행시장 김해가볼만한곳 김해전통시장 '}
,{'place': '하슬라아트월드', 'info': '반려동물과 함께, 자연과 예술을 감상하는 복합문화 예술공간', '태그': '2024_여행가는달과함께하는_발도장여행지 가족과함께 강릉여행 강원권                  '}
,{'place': '제주 중앙지하상가', 'info': '새롭게 단장한 쇼핑 상점가 중앙지하상가', '태그': '다양한볼거리 상점가 쇼핑 쇼핑하기좋은곳 제주가볼만한곳 제주여행   '}
,{'place': '국립대관령치유의숲', 'info': '명품 금강소나무숲에서 산림치유 프로그램을 체험하는 곳', '태그': '가족과함께 관광지 남녀노소 자연 자연관광지 자연속으로 자연여행     '}
,{'place': '남성사계시장', 'info': '오랜 세월을 자랑하는 전통시장', '태그': '남성사계시장 남성시장 사당동시장 쇼핑 재래시장 전통시장'}
,{'place': '수학체험공원', 'info': '어린이와 함께 가기 좋은 공원', '태그': '관광지 양산공원 양산어린이와가볼만한곳'}
,{'place': '김동명문학관', 'info': '김동명 시인의 업적과 생애를 기념하기 위한 곳', '태그': '강릉 강릉가볼만한곳 강릉여행 김동명문학관 문화시설 시인김동명 전시시설  '}
,{'place': '다랑쉬오름(월랑봉)', 'info': '아름다운 자연풍경을 볼 수 있는 오름', '태그': '관광지 다랑쉬오름 월랑봉 자연 자연풍경 제주가볼만한곳 제주여행 '}
,{'place': '포도뮤지엄', 'info': '다양성을 존중하고 서로의 생각을 공유하는 열린 문화공간', '태그': '문화시설 제주가볼만한곳 제주가볼만한곳 제주서귀포실내전시관 제주안덕전시관  '}
,{'place': '선화랑', 'info': '한국 현대미술의 성장과 발전을 견인해 온 명문화랑', '태그': '문화공간 문화시설 문화예술 문화예술공간 미술관데이트 복합문화공간         '}
,{'place': '타오르는 강 문학관', 'info': '영산강을 배경으로 한 대하소설 테마의 문학관', '태그': '관광지 역사 역사공부 역사관광지 역사를품은곳 역사문화재 역사속      '}
,{'place': 'e스포츠 명예의 전당', 'info': 'e스포츠의 역사가 있는 곳', '태그': '가족과함께 당일치기여행 문화시설 전시관 친구와함께'}
,{'place': '바라캇 서울', 'info': '세계적인 유물 컬렉션을 보유한 바라캇 갤러리 서울 본관', '태그': '2022_추석_문화여행 가볼래터 갤러리 문화시설 미술품 실내데이트   '}
,{'place': '중문관광단지', 'info': '볼거리와 체험이 가득한 중문관광단지', '태그': '2024_여행가는달과함께하는_발도장여행지 가족과함께 가족휴양지 관광지       '}
,{'place': '매월당 김시습 기념관', 'info': '김시습의 고매한 얼을 기리기 위해 건립된 기념관', '태그': '강릉가볼만한곳 강릉여행 기념관 매월당 문화시설 작가김시습 전시시설 '}
,{'place': '장유전통시장', 'info': '훈훈한 인심을 느낄 수 있는 시장', '태그': '김해오일장 김해전통시장 먹거리 쇼핑 시장 장유재래시장 장유전통시장 '}
,{'place': '늘솔길공원', 'info': '도심에서 양떼를 볼 수 있는 자연친화공원', '태그': '관광지 남동공원 남동늘솔길공원 인천공원 인천남동늘솔길공원 인천늘솔길공원 '}
,{'place': '경동시장', 'info': '청량리역 인근에 형성된 대규모 한약·먹거리 시장', '태그': '가볼래터 가족여행 나들이 나홀로여행 데이트코스 먹거리 밤에도_빛나는_야시장               '}
,{'place': '168계단', 'info': '초량동의 산 윗동네와 아랫동네를 바로 연결하는 유일한 길', '태그': '168계단 168계단모노레일 경상권 관광지 부산가볼만한곳 역사공부          '}
,{'place': '강릉아기동물농장', 'info': '강릉가볼만한곳 중 한 곳인 강릉아기동물농장이다.', '태그': '강릉 강릉아기동물농장 관광지 동물과의교감 아이와함께 이색체험 체험여행'}
,{'place': '늘솔길공원양떼목장', 'info': '시민들이 건강과 휴식을 위해 많이 찾는 공원', '태그': '관광지 늘솔길공원 늘솔길공원양떼목장 동물체험학습 아이와함께가볼만한곳   '}
,{'place': '강릉자수박물관', 'info': '자수에 새겨진 미적세계를 느낄 수 있는 공간', '태그': '강릉가볼만한곳 강원권 기념품판매 문화시설 박물관 아이와함께 이색체험  '}
,{'place': '인천중앙공원', 'info': '도심의 중심축을 이루고 있는 공원', '태그': '간석동중앙공원 관광지 남동구중앙공원 인천중앙공원 중앙공원'}
,{'place': '청량리수산시장', 'info': '신선한 수산물을 구입하기 위한 사람들로 북적이는 곳', '태그': '쇼핑 수산물도매시장 수산물새벽시장 수산물시장 수산시장 청량리수산시장'}
,{'place': '현대시장', 'info': '답십리역 인근 골목에 위치한 전통시장', '태그': '구암현대시장 쇼핑 창원가볼만한곳 창원관광 창원장보기 창원재래시장  '}
,{'place': '청량종합도매시장', 'info': '공산품들을 저렴하게 살 수 있는 상설시장', '태그': '깡통시장 생필품도매시장 쇼핑 종합시장 청량도매시장 청량종합도매시장'}
,{'place': '합천호', 'info': '주변 명산과 어우러져 멋진 풍광을 자아내는 호수', '태그': '가족여행 관광지 아이와함께 연인과함께 인생샷 자연 자연좋은곳    '}
,{'place': '전곡시장', 'info': '동대문구 장안동의 소형 종합 전통시장', '태그': '동대문구전통시장 서울전통시장 쇼핑 재래시장 전곡시장 전통시장'}
,{'place': '한향림옹기박물관', 'info': '한국 전통의 멋을 담고 있는 옹기박물관', '태그': '당일치기여행 문화시설 박물관 서울근교여행 아이와함께 옹기박물관  '}
,{'place': '합천 영상테마파크', 'info': '깨끗한 자연이 어우러진 시대물 오픈세트장', '태그': '2024_여행가는달과함께하는_발도장여행지 TV속촬영지 가족여행 경상권                '}
,{'place': '서울 약령시장', 'info': '전국에서 가장 큰 한약재 시장', '태그': '가족여행 나들이 나홀로여행 데이트코스 도심여행 먹거리 서울약령시장         '}
,{'place': '소래포구 종합어시장', 'info': '어촌마을의 낭만을 즐길 수 있는 소래포구 종합어시장', '태그': '2024_여행가는달과함께하는_발도장여행지 가족여행 나들이 나홀로여행          '}
,{'place': '유앤아이센터', 'info': '여성과 청소년이 가진 소중한 자원과 역량을 발굴하는 곳', '태그': '경기화성여행 김연아훈련장 레포츠 복합문화공간 빙상장 아이스링크  '}
,{'place': '상록해수욕장', 'info': '깨끗한 바다와 잔잔한 파도가 휴양하기 좋은 해변', '태그': '가족여행 관광지 바다풍경 상록해수욕장 아이와함께 여름방학가볼만한곳      '}
,{'place': '비단길 현대시장', 'info': '현대와 전통이 어우러진 복합문화 시장터', '태그': '금천구전통시장 독산동현대시장 비단길현대시장 쇼핑 재래시장 전통시장'}
,{'place': '부평도호부청사', 'info': '인천 도심 속 옛 관아의 흔적을 찾아볼 수 있는 곳', '태그': '관광지 역사 역사공부 역사관광지 역사를품은곳 역사문화재 역사속       '}
,{'place': '보은 법주사 [유네스코 세계유산]', 'info': '문화재의 보물창고이자 미륵신앙의 중심인 사찰', '태그': '2024_여행가는달과함께하는_발도장여행지 23_24한국관광100선 25_26한국관광100선                       '}
,{'place': '보은 법주사 [유네스코 세계유산]', 'info': '문화재의 보물창고이자 미륵신앙의 중심인 사찰', '태그': '2024_여행가는달과함께하는_발도장여행지 23_24한국관광100선 25_26한국관광100선                       '}
,{'place': '봉일사지삼층석탑', 'info': '인천지역에서 유일하게 원위치에 보존되어 있는 석탑', '태그': '계양봉일사지삼층석탑 관광지 역사 역사공부 역사관광지 역사를품은곳          '}
,{'place': '경남관광기념품점 (구, 경남관광공예명품관)', 'info': '경남지역의 우수하고 품질 좋은 기념품을 만날 수 있는 곳', '태그': '가족여행 경남관광 경상권 기념품점 기념품판매 기차여행 나들이        '}
,{'place': '복합생활스포츠타운', 'info': '누구나 즐길 수 있는 복합생활스포츠타운', '태그': '공원 군포국민체육센터 군포체육공원 레포츠 스포츠 운동 체육공원 '}
,{'place': '경인아라뱃길 다남공원', 'info': '가족 나들이 하기 좋은 공원', '태그': '경인아라뱃길 경인아라뱃길 다남공원 관광지 다남공원 인천가볼만한곳 '}
,{'place': '계양산전통시장', 'info': '계양산 둘레길의 통로, 인천 재래시장의 대표주자', '태그': '계양산둘레길 계양산시장 계양산전통시장 계양시장 계양전통시장 꽈배기달인      '}
,{'place': '수리사(군포)', 'info': '수리산 자락에 자리한 고즈넉한 천년고찰', '태그': '관광지 군포제2경 불교 불교문화 불교사찰 사찰 사찰여행 수리산           '}
,{'place': '괴시리 전통마을', 'info': '영남 유학의 전형적인 조선시대 전통 마을', '태그': '가족여행 경상권 고택마을 관광지 괴시리 민속문화재 역사 역사공부            '}
,{'place': '고래불해수욕장', 'info': '끝없이 펼쳐지는 명사 20리', '태그': '가족여행 경북여름휴가지 고래불국민야영장 고래불해수욕장 관광지 대진해수욕장                   '}
,{'place': '청련사(강화)', 'info': '푸른색 연꽃이 떨어진 자리에 지은 절', '태그': '강화청련사 관광지 불교 불교문화 사찰 수도권 역사 연중무휴 인천강화청련사     '}
,{'place': '마장호수', 'info': '나들이 장소로 제격인 마장호수', '태그': '2024_여행가는달과함께하는_발도장여행지 경기권반려동물여행지 관광지   '}
,{'place': '별빛 남문시장', 'info': '금천구 독산동에 위치한 아케이드형 골목시장', '태그': '남문시장 독산동시장 별빛남문시장 쇼핑 재래시장 전통시장'}
,{'place': '월랑저수지', 'info': '버드나무 데크길 따라 걸어보는 휴식공간', '태그': '관광지 산책하기좋은곳 아산 아산산책 아산저수지 월랑저수지 저수지'}
,{'place': '춘천 애니메이션박물관&토이로봇관', 'info': '꿈이 현실이 되는 곳', '태그': '가족과함께 가족여행 문화시설 아이와함께 이색체험 체험학습 춘천가볼만한곳'}
,{'place': '함덕민속오일시장', 'info': '현지인과 관광객이 어우러지는 시장', '태그': '가족과함께 쇼핑 오일장 전통시장 제주가볼만한곳 제주시장 제주여행 '}
,{'place': '춘천 삼악산 호수케이블카', 'info': '반려견 동반 가능한 국내 최장 케이블카', '태그': '2024_여행가는달과함께하는_발도장여행지 23_24한국관광100선 강원권           '}
,{'place': '강촌레일파크 (김유정레일바이크)', 'info': '반려동물과 함께 옛 철길따라 추억 싣고 달리는 레일바이크', '태그': '가족여행 강원권 강원권반려동물여행지 강촌레일파크 레포츠 반려동물동반여행지'}
,{'place': '창원의 집', 'info': '도심 속 창원시의 역사가 살아 숨 쉬는 공간', '태그': '경상권 기차여행 문화관광해설사 문화시설 아이와함께 전통역사문화체험  '}
,{'place': '쥬라리움 파주점', 'info': '동물을 만나는 새로운 방법', '태그': '관광지 먹이주기 아이와함께 어린이동물체험장 쥬라리움 쥬라리움파주점  '}
,{'place': '염치저수지', 'info': '잔잔한 저수지를 바라보며 산책을 즐겨보자', '태그': '관광지 산책하기좋은곳 아산데이트코스 아산산책 아산여행 아산저수지 '}
,{'place': '여좌천(벚꽃명소)', 'info': '하천을 따라 걷는 벚꽃 터널길', '태그': '2024_여행가는달과함께하는_발도장여행지 23_24한국관광100선 감성사진                   '}
,{'place': '한국도로공사 전주수목원', 'info': '국내 유일 도로전문 수목원', '태그': '2024_여행가는달과함께하는_발도장여행지 가족과함께 가족여행 관광지           '}
,{'place': '대왕암공원', 'info': '푸른바다, 우리의 꿈 대왕암', '태그': '2024_여행가는달과함께하는_발도장여행지 21_22한국관광100선 23_24한국관광100선                   '}
,{'place': '천안아산역 동광장', 'info': '천안시민들의 만남의 광장', '태그': '관광지 아산광장 아산데이트코스 아산산책 아산여행 아산역동광장 천안아산만남의광장 '}
,{'place': '월영교', 'info': '아름다운 호수와 달빛을 품은 사랑의 다리', '태그': '2022_추석_별빛_가득_야간관광 2024_여행가는달과함께하는_발도장여행지                    '}
,{'place': '삼의악오름', 'info': '사방의 전망이 아름다운 샘이 있는 오름', '태그': '경치좋은곳 관광지 산책하기좋은곳 오름 자연 자연속으로 자연풍경   '}
,{'place': '한원미술관', 'info': '한국화의 대중화와 현대화를 이끄는 미술관', '태그': '남부터미널미술관 문화공간 문화시설 문화예술 미술품 복합문화공간       '}
,{'place': '맨발의 청춘길', 'info': '그때 그 시절이 떠오르는 곳', '태그': '관광지 맨발의청춘길 문화관광거리 울산가볼만한곳 울산도심여행 울산여행  '}
,{'place': '구미에코랜드', 'info': '나무 곰이 반겨주는 구미 에코랜드', '태그': '가족여행 가족체험여행 걷기길 경상권 관광지 구미가볼만한곳 구미에코랜드       '}
,{'place': '장수승마체험장/장수승마레저파크', 'info': '승마 체험과 휴양, 문화, 레저 활동을 함께 즐길 수 있는 곳', '태그': '게르 레포츠 말먹이주기 말역사박물관 스테이크 · 카페 · 와인 548 장수가볼만한곳  '}
,{'place': '울산 중앙전통시장', 'info': '낮에도 밤에도 즐길 수 있는 울산중앙시장', '태그': '2024_여행가는달과함께하는_발도장여행지 가족여행 경상권 기차여행                 '}
,{'place': '신안정사', 'info': '조선시대에 지어진 신안정사', '태그': '관광지 산청가볼만한곳 송시열 수정궁 추자'}
,{'place': '강화 장정리 석조여래입상', 'info': '온화한 미소를 머금은 고려시대 석조불상', '태그': '강화장정리석조여래입상 관광지 보물 수도권 역사 역사공부 역사관광지            '}
,{'place': '강동오토캠핑장', 'info': '울산 바다뷰의 캠핑장', '태그': '그늘이 맑은 깨끗한 레포츠 물맑은 바다가 보이는 사이트 간격이 넓은    '}
,{'place': '둔지봉', 'info': '다양한 나무와 함께 자연을 느낄 수 있는 오름', '태그': '관광지 구름의언덕 오름 자연 자연풍경 제주가볼만한곳 제주오름  '}
,{'place': '삼천포 팔포항', 'info': '어촌마을의 정이 살아 숨 쉬는 항구', '태그': '관광지 사량도도행배편 사량도행카페리 사천가볼만한곳 사천관광명소   '}
,{'place': '강화 장정리 오층석탑', 'info': '고려시대에 세워진 5층 석탑', '태그': '강화오층석탑 강화장정리오층석탑 관광지 보물 수도권 역사 역사공부            '}
,{'place': '유어마인드', 'info': '독립출판물을 만날 수 있는 곳', '태그': '관광지 다락방 독립출판물 서점 쇼핑 유어마인드'}
,{'place': '올림픽공원', 'info': '88올림픽을 치른 서울의 대표 공원', '태그': '2024_여행가는달과함께하는_발도장여행지 88올림픽공원 sk핸드볼경기장                    '}
,{'place': '삼매봉', 'info': '세 개의 봉우리가 매화를 닮은 곳', '태그': '경치좋은곳 관광지 등산로 산책로 산책코스 산책하기좋은곳 서귀포가볼만한곳     '}
,{'place': '강화 참성단', 'info': '단군부터 조선시대까지 제사를 올리던 제단', '태그': '강화참성단 관광지 사적 수도권 역사 역사공부 역사관광 역사관광지           '}
,{'place': '구미농수산물도매시장', 'info': '역사와 함께하는 공간', '태그': '과일도매 구미가볼만한곳 구미농수산물 구미시장투어 농수산물도매시장  '}
,{'place': '슬도등대', 'info': '도심 속 무인도 여행이 가능한 곳', '태그': '관광지 노을명소 바위섬 방어진항 소리체험관 슬도등대 울산등대  '}
,{'place': '남목전통시장', 'info': '정겨운 울산의 재래시장', '태그': '가족여행 경상권 기차여행 나들이 나홀로여행 남목시장 데이트코스          '}
,{'place': '송도해변(포항)', 'info': '산책하기 좋은 송림테마거리가 있는 해변', '태그': '가족여행 관광지 바다풍경 송도해변 아이와함께 여름방학가볼만한곳      '}
,{'place': '큰노꼬메오름', 'info': '억새와 일몰의 경관을 함께 볼 수 있는 오름', '태그': '관광지 애월오름 억새명소 작은노꼬메오름 전망명소 제주도오름 제주등반  '}
,{'place': '영일 칠포리 암각화군', 'info': '곤륜산 일대에 있는 다산과 풍요를 기원하는 바위그림', '태그': '관광지 역사 역사공부 역사관광지 역사를품은곳 역사문화재 역사속     '}
,{'place': '화성해양아카데미연구회', 'info': '멋진 풍광을 즐기며 다양한 해양체험하기', '태그': '레포츠 요트체험 전곡항 해양레저체험 해양레포츠체험 해양안전교육  '}
,{'place': '수촌교회(화성)', 'info': '수촌리에서 3·1 운동을 주도한 교회', '태그': '31운동 관광지 역사여행 유적지 향토유적 화성가볼만한곳 화성여행'}
,{'place': '백련사(강화)', 'info': '하얀 연꽃에서 피어난 천년고찰', '태그': '강화가볼만한곳 강화백련사 강화사찰 관광지 불교 불교문화 불교사찰            '}
,{'place': '포항 해상스카이워크', 'info': '바다 위를 걷는 것 같은 스릴을 느껴볼 수 있는 곳', '태그': '관광지 포항가볼만한곳 포항바다위의길 포항스카이워크 포항야경명소  '}
,{'place': '사강스마트팜캠핑장', 'info': '편하게 캠핑을 즐기는 곳', '태그': '경기화성여행 레포츠 서울시당일치기캠핑 캠핑 프라이빗캠핑 화성 화성가볼만한곳'}
,{'place': '엘리캠핑', 'info': '신비로운 계곡, 낭만적인 캠핑', '태그': '가족 경기화성여행 레포츠 서울시당일치기캠핑 캠핑 프라이빗캠핑 화성'}
,{'place': '풀내음캠핑장', 'info': '시내에서 접근성이 좋은 캠핑장', '태그': '가족 경기화성여행 레포츠 아이들 놀기 좋은 친절한 캠핑장 화성'}
,{'place': '울산 문화의거리', 'info': '예술가들의 혼이 담긴 문화거리', '태그': '관광지 데이트장소추천 문화의거리 산책길 산책하기 산책하기좋은곳       '}
,{'place': '제부마리나', 'info': '경기도 최대 규모의 마리나', '태그': '경기 경기화성여행 관광지 보트 제부도 제부마리나 화성'}
,{'place': '안양천 물놀이장', 'info': '도심 속에서 시원한 여름을 즐길 수 있는 가족 친화형 피서지', '태그': '레포츠 물놀이장 안양천 안양천물놀이장 여름피서지'}
,{'place': '위미동백나무군락', 'info': '제주의 정취를 물씬 풍기는 사철 아름다운 동백숲', '태그': '관광지 남녀노소 동백꽃 사진찍기좋은곳 산책하기좋은곳 서귀포가볼만한곳      '}
,{'place': '롯데월드타워 서울스카이', 'info': '하늘 위 새로운 세상', '태그': '2024_여행가는달과함께하는_발도장여행지 23_24한국관광100선 25_26한국관광100선                       '}
,{'place': '두현저수지', 'info': '한적하고 고즈넉하여 산책하기 좋은 저수지', '태그': '관광지 두현저수지 산책코스 산책하기좋은 울산숨은명소 울주산책명소 '}
,{'place': '두현저수지', 'info': '한적하고 고즈넉하여 산책하기 좋은 저수지', '태그': '관광지 두현저수지 산책코스 산책하기좋은 울산숨은명소 울주산책명소 '}
,{'place': '구로 중앙유통단지', 'info': '동양 최대 규모의 산업용품 전문 상가단지', '태그': '도소매 무역 산업용품 서울가볼만한곳 쇼핑 오프라인매장 유통단지   '}
,{'place': '강동사랑길', 'info': '산과 들, 바다를 모두 만끽할 수 있는 사랑을 간직한 7가지 녹색명품길', '태그': '가족과함께 걷기여행 관광지 당일치기여행 둘레길 레포츠 산책로     '}
,{'place': '제지기오름', 'info': '숲에 덮인 서귀포의 숨은 풍경 명소', '태그': '관광지 자연 자연관광지 자연풍경 제주가볼만한곳 제주숨은명소 제주오름  '}
,{'place': '면곡시장', 'info': '옛 재래시장의 정취를 간직한 소규모 골목시장', '태그': '골목시장 광진구시장 광진구전통시장 면곡시장 쇼핑 재래시장 전통시장'}
,{'place': '어물동 마애여래좌상', 'info': '거대한 자연 암벽면에 돋을새김으로 조각한 불상', '태그': '경상권 관광지 어물동마애여래좌상 역사 역사공부 역사를품은곳 역사문화재        '}
,{'place': '별도봉', 'info': '제주항과 푸른 바다가 내려다보이는 봉우리', '태그': '관광지 자연 자연관광지 자연풍경 제주가볼만한곳 제주별도봉 제주오름  '}
,{'place': '서울 신림동 굴참나무', 'info': '역사와 전설이 깃든 천연기념물로, 지역의 생태·민속적 가치를 지닌 보호수', '태그': '강감찬장군 관광지 관악산노거수 굴참나무 서울신림동굴참나무 수도권     '}
,{'place': '황매산오토캠핑장 제1캠핑', 'info': '하늘과 땅과 자연이 함께하는 공간', '태그': '걷기길 레포츠 봄꽃여행 수상레저 일몰명소 일출명소 합천캠핑장 '}
,{'place': '서울 신림동 민속순대타운', 'info': '독특한 순대 요리로 유명한 곳', '태그': '골목식당 관광지 남녀노소 맛집 맛집골목 먹자골목 서울맛집 수도권          '}
,{'place': '황매산오토캠핑장 제1캠핑', 'info': '하늘과 땅과 자연이 함께하는 공간', '태그': '걷기길 레포츠 봄꽃여행 수상레저 일몰명소 일출명소 합천캠핑장 '}
,{'place': '서울 신림동 민속순대타운', 'info': '독특한 순대 요리로 유명한 곳', '태그': '골목식당 관광지 남녀노소 맛집 맛집골목 먹자골목 서울맛집 수도권          '}
,{'place': '소머리오름(우도봉)', 'info': '아름다운 풍경이 보이는 우도에서 가장 높은 봉우리', '태그': '관광지 섬여행 아름다운풍경 우도 우도봉 자연 제주가볼만한곳 제주여행   '}
,{'place': '오치골공원', 'info': '아이들이 뛰어 놀기 좋은 공원', '태그': '관광지 오치골공원 울산가볼만한곳 울산놀이시설 울산돔플레이 울산여행  '}
,{'place': '고사포야영장', 'info': '고사포해수욕장 인근 야영장', '태그': '고사포야영장 고사포일몰명소 국립공원야영장 레포츠 부안야영장 여름물놀이  '}
,{'place': '우가산유포봉수대', 'info': '옛날 군사통신 수단의 하나', '태그': '관광지 역사 역사공부 역사를품은곳 역사문화재 역사속 역사속으로      '}
,{'place': '관음사(서울)', 'info': '신라시대에 창건된 유서 깊은 비보사찰', '태그': '관광지 관악능선 관악산 관악산등산코스 관악산사찰 불교문화 사찰여행            '}
,{'place': '망상오토캠핑리조트', 'info': '자연과 어우러진 자연친화적 오토캠핑리조트', '태그': '강원도캠핑장 동해시캠핑장 레포츠 수상레저 액티비티 일출명소'}
,{'place': '용연구름다리', 'info': '호수와 불빛 숲들이 어우러진 다리', '태그': '관광지 야경 연인과함께 올레길17코스 용두암 용연계곡 용연구름다리   '}
,{'place': '하이키수상레저', 'info': '물 위에서 신나는 하루, 남한강 물놀이의 끝판왕', '태그': '단체수상레저 레포츠 복합수상레저 수상레저 여주 여주가볼만한곳 여주수상레저'}
,{'place': '삼성산성지', 'info': '자연 속 신앙 선조들의 숨결을 느끼다', '태그': '2024_여행가는달과함께하는_발도장여행지 가톨릭순례길 관광지 반려동물                    '}
,{'place': '탄도인공습지공원', 'info': '일몰 명소', '태그': '갈대밭 관광지 생태학습 일몰명소 탄도인공습지공원 화성가볼만한곳 '}
,{'place': '화봉종합시장', 'info': '저렴하고 다양한 식자재를 만날 수 있는 곳', '태그': '쇼핑 수산시장 울산동구 울산동구맛집 울산수산물 울산수산물판매센터   '}
,{'place': '성불오름', 'info': '삼나무와 편백나무가 우거진 오름', '태그': '경치좋은곳 관광지 성불오름 오름 자연관광지 자연풍경 제주가볼만한곳  '}
,{'place': '서울 양천고성지', 'info': '한강 하류를 방어하던 전략적 요충지로서 역사적 가치가 큰 테뫼식 산성', '태그': '고성지 관광지 궁산근린공원 궁산둘레길 궁산역사문화둘레길 궁산유적지               '}
,{'place': '전곡수산시장', 'info': '전곡항 맞은편에 위치한 종합수산시장', '태그': '서해가볼만한곳 서해여행 서해횟집 쇼핑 전곡종합수산시장 전곡항횟집 '}
,{'place': '윗세오름', 'info': '봄철에 피어나는 꽃들이 가득한 오름', '태그': '관광지 봄에가기좋은곳 자연 자연관광지 자연풍경 제주오름 제주웃세오름  '}
,{'place': '높은오름', 'info': '정상에서 바라보는 조망이 아름다운 오름', '태그': '경치좋은곳 관광지 오름 자연 자연관광지 자연속으로 자연풍경 제주가볼만한곳  '}
,{'place': '옛수원가든', 'info': '노을과 연꽃이 아름다운 데이트, 드라이브 코스', '태그': '데이트코스 드라이브코스 반려견동반 보통리저수지맛집 옛수원가든 오리구이  '}
,{'place': '네이처스파', 'info': '지하 1,300m 천연암반수를 이용한 스파, 찜질방', '태그': '관광지 남녀노소 노천탕 스파 연인과함께 찜질방 파주가볼만한곳     '}
,{'place': '연산군유배지', 'info': '연산군이 유배되면서 머물던 초가집', '태그': '강화가볼만한곳 강화유적지 관광지 교동가볼만한곳 역사교육 연산군'}
,{'place': '천아오름', 'info': '작은 말굽형 화구를 이루고 있는 오름', '태그': '경치좋은곳 관광지 오름 자연 자연풍경 제주가볼만한곳 제주오름  '}
,{'place': '석모도 칠면초 군락지', 'info': '넓은 갯벌 위에 자줏빛 칠면초가 무리 지어 피는 명소', '태그': '2024_여행가는달과함께하는_발도장여행지 관광지 석도모칠면초군락지    '}
,{'place': '무태돈대', 'info': '해안의 철통방어를 책임진 돈대', '태그': '강화무태돈대 관광지 무태돈대 수도권 역사 역사공부 역사관광지            '}
,{'place': '김상용순절비', 'info': '김상용 선생의 충절을 기린 순절비', '태그': '강화김상용순절비 관광지 김상용순절비 수도권 역사 역사공부 역사관광지          '}
,{'place': '강화도동부해안도로', 'info': '드라이브하며 강화도 관광하기!', '태그': '갑곶돈대 강화 강화도 유적 강화동부해안도로 관광지 광성보 덕진진      '}
,{'place': '형제섬', 'info': '바다 한 가운데 마주 보고 있는 경관이 아름다운 섬', '태그': '가족과함께 관광지 낚시 사계해변일몰 서귀포가볼만한곳 섬여행 일몰명소   '}
,{'place': '슬도', 'info': '파도소리가 거문고 연주처럼 들려오는 바위섬', '태그': '가족과함께 경치여행 경치좋은곳 관광지 남녀노소 바다경치 바다내음      '}
,{'place': '과개안(너븐개) 해안', 'info': '과거에 고래를 포획하던 몽돌해변', '태그': '경상권 과개안해안 관광지 너븐개해안 몽돌해변 바다풍경 울산 울산과개안       '}
,{'place': '대송농수산물시장', 'info': '싱싱하고 질 좋은 농수산물을 저렴한 가격에 제공하는 시장', '태그': '농수산물시장 대송농수산물시장 대송시장 대송시장맛집 쇼핑 시장먹거리    '}
,{'place': '강화 갑곶리 탱자나무', 'info': '오랜 세월 동안 억센 가시로 성벽을 지켰던 탱자나무', '태그': '갑곶리탱자나무 강화탱자나무 관광지 수도권 인천갑곶리탱자나무 인천탱자나무  '}
,{'place': '양헌수승전비', 'info': '병인양요 때 프랑스군을 물리친 양헌수 장군 공적비', '태그': '강화양헌수승전비 관광지 문화재 수도권 양헌수승전비 역사 역사공부           '}
,{'place': '강서습지생태공원', 'info': '생태 보전과 여가 활동이 조화를 이루는 복합 테마형 공원', '태그': '가족여행 강서습지생태공원 강서한강공원 걷기길 관광지 나들이 도심여행         '}
,{'place': '링크나인GC', 'info': '누구나 골프를 쉽게 접할 수 있는 곳', '태그': '대중제9홀 레포츠 링크나인GC 벤트그라스 중지 켄터키블루 퍼블릭CC '}
,{'place': '서신바다낚시터', 'info': '바다 낚시를 즐기고 싶다면', '태그': '궁평항바다낚시터 낚시체험 레포츠 바다낚시 바다좌대낚시 서신낚시터  '}
,{'place': '강화 홍릉', 'info': '팔만대장경으로 승리를 염원했던 고려왕의 능', '태그': '관광지 교과서속여행 문화재 수도권 역사 역사공부 역사관광지 역사를품은곳            '}
,{'place': '노진낚시터', 'info': '붕어들이 입질하는 토종 붕어 잡이터', '태그': '노지낚시 노진낚시터 레포츠 붕어낚시 야경명소 잡이터 토종붕어 '}
,{'place': '고잔낚시터', 'info': '풍부한 어종을 보유한 낚시터', '태그': '고잔낚시터 고잔저수지 노지낚시 레포츠 상생국민지원금 손맛터 잡이터 '}
,{'place': '화성상록GC', 'info': '자연미와 스릴을 선사하는 골프장', '태그': '공무원연금공단 레포츠 상록골프 퍼블릭CC 퍼블릭골프장 화성골프장 '}
,{'place': '화성GC', 'info': '자연과 하나 된 아름다운 레이아웃', '태그': '9홀 대중제골프장 레포츠 퍼블릭CC 퍼블릭골프장 화성GC 화성골프클럽'}
,{'place': '석모도', 'info': '수도권 시민들이 체험과 휴식을 즐길 수 있는 섬', '태그': '강화석모도 관광지 석모도 섬 수도권 인천강화석모도 인천석모도 '}
,{'place': '뒤굽은이오름', 'info': '제주 동부의 풍경을 한눈에 담을 수 있는 오름', '태그': '관광지 뒤굽은이오름 뒷굽은이오름 자연 자연풍경 전망좋은 제주가볼만한곳  '}
,{'place': '오어사(포항)', 'info': '원효대사의 삿갓이 보관되어 있는 사찰', '태그': '2024_여행가는달과함께하는_발도장여행지 경상권 관광지 문화재 불교              '}
,{'place': '장곶돈대', 'info': '강화군 화도면 장화리에 있는 조선 후기의 돈대', '태그': '강화장곶돈대 관광지 수도권 역사 역사공부 역사관광지 역사를품은곳            '}
,{'place': '개오름', 'info': '봉긋한 모양으로 부풀어 올라있는 오름', '태그': '관광지 사진찍기좋은 서귀포가볼만한곳 자연 자연풍경 제주 제주사진명소  '}
,{'place': '종해루', 'info': '전등사의 남문이자 외국군을 몰아낸 전승지', '태그': '강화종해루 관광지 역사 역사공부 역사관광지 역사를품은곳 역사문화재         '}
,{'place': '영일대전망대', 'info': '전국 유일의 바다 위에 떠 있는 해상 누각', '태그': '관광지 영일교 영일대해수욕장 포항방파제 포항여행 포항영일대 포항영일대전망대 '}
,{'place': '망향대', 'info': '실향민들의 고향에 대한 그리움이 담겨있는 장소', '태그': '강화군 강화도가볼만한곳 강화도여행 강화망향대 관광지 교육여행 망향대            '}
,{'place': '24시 마린낚시', 'info': '바다 낚시를 즐길 수 있는 곳', '태그': '24시마린낚시 낚시체험 레포츠 민물낚시 바다낚시 선상출조 전곡항바다낚시'}
,{'place': '포항 초롱구비마을', 'info': '농촌전통 테마마을 초롱구비마을', '태그': '관광지 농촌 농촌길 농촌여행코스 농촌체험 농촌체험마을 농촌체험여행          '}
,{'place': '포항 초롱구비마을', 'info': '농촌전통 테마마을 초롱구비마을', '태그': '관광지 농촌 농촌길 농촌여행코스 농촌체험 농촌체험마을 농촌체험여행          '}
,{'place': '안산대부광산퇴적암층', 'info': '공룡의 흔적이 발견된 대부광산의 채석장', '태그': '공룡화석 관광지 비경 사진명소 식물화석 안산가볼만한곳 자연관광지    '}
,{'place': '법광사(포항)', 'info': '비학산 기슭에 있었던 신라의 대규모 사찰', '태그': '관광지 불교 불교사찰 사찰 사찰여행 역사 자연속으로 자연좋은곳       '}
,{'place': '장화리일몰조망지', 'info': '서울 근교 드라이브코스 겸 당일치기로 좋은 여행지', '태그': '강화도낙조 강화도일몰 관광지 서해 서해낙조 서해일몰 장화리일몰조망지'}
,{'place': '황청포구', 'info': '주변의 자연경관을 감상하며 여유로운 시간을 보낼 수 있는 곳', '태그': '관광지 석모도 차박캠핑 차박캠핑명소 차크닉 황청리선착장 황청포구'}
,{'place': '모던화랑', 'info': '오랜시간 자리를 지킨 원로 중진 작가 전문 갤러리', '태그': '문화시설 문화예술공간 미술품 서울가볼만한곳 예술공간 인사동화랑  '}
,{'place': '마실 휴양농원', 'info': '인테리어까지 개성이 곳곳에 묻어있는 체험농원', '태그': '경기화성마실휴양농원 레포츠 체험 체험농원 체험농장 캠핑장 펜션 '}
,{'place': '천상병 귀천(歸天) 공원', 'info': '건평항의 하늘을 구경할 수 있는 공원', '태그': '경치좋은곳 관광지 바다경치 바다내음 산책하기 산책하기좋은곳 시인공원    '}
,{'place': '부천로보파크', 'info': '국내·외 최초의 로봇상설전시장', '태그': '로보파크 로봇 로봇전시장 문화시설 부천로보파크 아이들과함께 체험활동'}
,{'place': '매향리캠핑장', 'info': '석양을 바라보며 낚시와 갯별체험을 즐길 수 있는 곳', '태그': '경기화성여행 레포츠 매향리캠핑장 바다뷰캠핑장 서해캠핑장 화성 화성매향리캠핑장 '}
,{'place': '무봉산', 'info': '사계절이 아름다운 도심 근교에 위치한 무봉산', '태그': '경기화성여행 관광지 만의사 무봉산 무봉산일출 화성 화성무봉산'}
,{'place': '조사리간이해수욕장', 'info': '조용한 해변, 완벽한 휴식', '태그': '관광지 조사리간이해수욕장 포항여행 포항피서지 포항해변 포항해수욕장 '}
,{'place': '운제산산림욕장', 'info': '자연과 함께 하는 시민생활공간', '태그': '관광지 운제산산림욕장 포항 포항가볼만한곳 포항나들이 포항산림욕 '}
,{'place': '화성ICT 생활문화센터', 'info': '예술과 기술의 융합을 통한 새로운 가능성을 발견하는 공간', '태그': '경기화성에이씨티그라운드화성ICT생활문화센타 경기화성여행 문화시설     '}
,{'place': '서해랑 제부도 해상케이블카', 'info': '전곡항과 제부도를 잇는 새로운 하늘길', '태그': '25_26한국관광100선 가볼래터 관광지 서해랑케이블카 수도권 여행구독     '}
,{'place': '글램비 글램핑', 'info': '편안한 캠핑을 즐길 수 있는 캠핑장', '태그': '경기화성여행 낚시 레포츠 수상레저 액티비티 여름물놀이 일몰명소 '}
,{'place': '수성구민운동장', 'info': '구민 건강 지킴이 역할을 하는 운동장', '태그': '가족과함께 대구 대구가볼만한곳 레포츠 수성구민운동장 운동장 운동하기좋은  '}
,{'place': '은빛초원캠핑장', 'info': '송정산업단지 옆 서울근교 가까운 캠핑장', '태그': '경기화성여행 관리가잘된캠핑장 레포츠 불멍하기좋은 서울시인근 친구랑캠핑  '}
,{'place': '고모역복합문화공간', 'info': '미래를 이야기하는 문화 플랫폼역', '태그': '문화공간 고모역 고모역복합문화공간 뉴트로_감성여행 대구 대구가볼만한곳    '}
,{'place': '구룡포장 (3, 8일)', 'info': '싱싱한 바다 음식의 장터', '태그': '가족여행 경상권 구룡포장 기차여행 나들이 나홀로여행 먹거리 쇼핑         '}
,{'place': '화성시매송오토캠핑장', 'info': '매송인근 어천저수지 옆 화성시 직영 캠핑장', '태그': '경기화성여행 깨끗한캠핑장 레포츠 서울근교캠핑장 캠핑 화성 화성가볼만한곳'}
,{'place': '탄도호캠핑장', 'info': '탄도호가 멀리 보이는 노을이 아름다운 캠핑장', '태그': '경기화성여행 레포츠 일몰명소 캠핑 캠핑장 화성 화성가볼만한곳'}
,{'place': '동탄 북광장', 'info': '반석산 근린공원 근처 상업지구 광장', '태그': '경기화성여행 공원 관광지 반석산근린공원 상업지구광장 화성 화성가볼만한곳'}
,{'place': '포항시농산물도매시장', 'info': '경북 농산물 유통의 중심', '태그': '다양한먹거리 도매시장 쇼핑 포항가볼만한곳 포항도매시장 포항시농산물도매시장 '}
,{'place': '기배역사공원', 'info': '융건릉 근처 산책과 휴식하기 좋은 공원', '태그': '경기화성여행 공원 관광지 융건릉 홍재원 화성 화성가볼만한곳'}
,{'place': '흥해장 (2, 7일)', 'info': '조선시대부터 생겨난 전통 깊은 장터', '태그': '가족여행 경상권 쇼핑 이색체험 전통시장 체험학습 친환경여행 특산물 '}
,{'place': '황계동 정조마을', 'info': '조선 정조대왕이 능행차 때 쉬어 가던 마을', '태그': '관광지 능행차 성황대제 정조대왕 정조마을 화성시도시재생마을'}
,{'place': '아중호수도서관', 'info': '아중호수 경관을 조망할 수 있는 음악특화 공공도서관', '태그': 'LP음반 가볼래터 가족과함께 도서관 문화시설 아이와함께 여행구독     '}
,{'place': '화개사', 'info': '고즈넉하고 한적한 고려 시대 사찰', '태그': '강화나들길9코스 강화도사찰 강화도유적지 관광지 교동8경 조용한사찰   '}
,{'place': '고려천도공원', 'info': '강화도에 흩어져 있는 역사문화 유적지들을 소개하다', '태그': '강화가볼만한곳 강화도역사문화유적지 강화도천도 고려시대대몽항쟁 고려천도공원        '}
,{'place': '용마폭포공원', 'info': '인공폭포가 있는 용마산 자락의 휴식공간', '태그': '걷기길 관광지 나들이 면목동공원 산책길 산책하기 산책하기좋은곳              '}
,{'place': '포항 장기읍성', 'info': '왜적의 침략으로부터 민가를 보호하기 위한 고려시대 토성', '태그': '고려시대토성 관광지 동악산토성 역사 역사공부 역사관광지 역사를품은곳             '}
,{'place': '교동도선착장대합실', 'info': '조용하고 아기자기한 선착장', '태그': '관광지 교동도선착장 교동도선착장대합실 교동도추억여행 월선포 월선포구  '}
,{'place': '장성중앙시장', 'info': '태백의 역사와 인심이 살아 숨 쉬는 전통시장', '태그': '가족여행 강원권 기차여행 나들이 나홀로여행 데이트코스 먹거리           '}
,{'place': '육송정 홍교', 'info': '아름다운 조형미를 보여주는 조선시대 석조 다리', '태그': 'DMZ DMZ여행 강원권 강원권여행 고성관광지 관광지 석교 역사관광지          '}
,{'place': '남산포 선착장', 'info': '강화 교동도 남쪽의 외지고 조용한 작은 선착장', '태그': '강화도차박 강화선착장 관광지 낚시 남산포선착장 선착장 차박캠핑 '}
,{'place': '경의선숲길(연트럴파크)', 'info': '철길 따라 다른 풍경, 경의선숲길', '태그': '경의선숲길 데이트명소 데이트코스 도심산책 레포츠 마포구 산책코스    '}
,{'place': '대구 구암동 고분군', 'info': '신라시대 칠곡지역의 수장층 무덤', '태그': '관광지 구암동고분군 대구가볼만한곳 대구여행 수장층무덤 자연풍경  '}
,{'place': '딜라이트 담양', 'info': '담양의 생태와 인문학을 재해석한 체험형 미디어 전시관', '태그': '담양 담양가볼만한곳 담양관광 담양관광명소 담양나들이 담양데이트          '}
,{'place': '동검선착장', 'info': '강화도 일출 명소', '태그': '강화도 강화도캠핑 관광지 동검도 동검선착장 서해 서해바다 인천캠핑'}
,{'place': '동검선착장', 'info': '강화도 일출 명소', '태그': '강화도 강화도캠핑 관광지 동검도 동검선착장 서해 서해바다 인천캠핑'}
,{'place': '전주 영화의 거리', 'info': '관람객으로 붐비는 활기찬 거리', '태그': '관광지 영화의거리 전주 전주가볼만한곳 전주국제영화제 전주영화의거리'}
,{'place': '창후리선착장', 'info': '교동대교가 보이는 선착장', '태그': '강화도 관광지 교동 군락지 서해 창후리선착장 창후리포구 칠면초'}
,{'place': '하늘공원', 'info': '하늘과 초원이 맞닿은 공원', '태그': '2024_여행가는달과함께하는_발도장여행지 관광지 남녀노소 산책하기                '}
,{'place': '황산선착장', 'info': '산책로를 따라 걷기 좋은 선착장', '태그': '강화나들코스 강화도 강화도나들길 관광지 황산도 황산선착장'}
,{'place': '장구너머항', 'info': '드넓은 바다 수평선위로 파란하늘을 감상하기 좋은 작은 항구', '태그': '강화도낚시 강화도항구 관광지 석모도항구 장곳항 장구너머항'}
,{'place': '영일대 호수공원', 'info': '파란 하늘과 호수를 바라보며 힐링하는 시간', '태그': '관광지 포항공원 포항벚꽃 포항산책 포항여행 포항투어 포항호수여행 '}
,{'place': '엑스코(EXCO)', 'info': '사람과 정보 서비스가 모이는 국제 전시컨벤션센터', '태그': 'exco 대구가볼만한곳 대구광역시 문화시설 북구 전시관 전시컨벤션센터 '}
,{'place': '새만금홍보관', 'info': '새만금의 과거와 미래를 한눈에 볼 수 있는 전시관', '태그': 'BTS 문화시설 부안 부안가볼만한곳 새만금홍보관 전북부안 전시'}
,{'place': '영일만항', 'info': '포항의 유명한 항구', '태그': '관광지 영일만항 영일만항낚시 포항가볼만한곳 포항낚시 포항방파제 '}
,{'place': '국립검마산자연휴양림', 'info': '반려동물과 함께 느끼는 아름다운 소나무 숲', '태그': '가족과함께 가족여행 걷기길 검마산 경상권 경상권반려동물여행지 관광지           '}
,{'place': '양포항복합공원', 'info': '푸르른 바다를 바라보며 즐기는 여유로운 휴식', '태그': '공원 관광지 양포항 양포항근린공원 양포항복합공원 포항가볼만한곳 '}
,{'place': '포항 마북리 느티나무', 'info': '권 씨 할배나무라 불리는 포항의 느티나무 노거수', '태그': '경상권 경상북도보호수 관광지 무자천손나무 보호수 자연관광지 자연속으로      '}
,{'place': '적석사 낙조대', 'info': '그림같은 낙조를 감상할 수 있는 곳', '태그': '강화8경 강화가볼만한곳 강화도낙조봉 관광지 낙조대 낙조명소 서해안일몰  '}
,{'place': '용연저수지', 'info': '천혜 자연환경을 자랑하는 포항 최대 저수지', '태그': '경북권 레포츠 신광지 용연지 자연좋은곳 자연풍경 자연환경 포항저수지 '}
,{'place': '그라운드시소 서촌', 'info': '예술과 문화가 어우러진 복합 문화 공간', '태그': '2024_여행가는달과함께하는_발도장여행지 그라운드시소 문화시설 복합문화시설   '}
,{'place': '전주디지털독립영화관', 'info': '독립영화 매니아들의 아지트', '태그': '독립영화관 문화시설 영화관 전주 전주디지털독립영화관'}
,{'place': '부산 차이나타운(상해문,상해거리)', 'info': '부산 속 작은 중국, 이국적인 분위기의 거리', '태그': '가족과함께 걷기길 경상권 관광지 맛집투어 부산차이나타운 부산차이나타운특구              '}
,{'place': '하리선착장', 'info': '아름다운 풍경 속 평화로운 여행', '태그': '관광지 미법도 서검도 서해 석모도 인천선착장 하리선착장'}
,{'place': '김현주 갤러리', 'info': '현대문화예술의 꽃을 피우는 갤러리', '태그': '문화공간 문화시설 문화예술 미술관 미술인 복합문화공간 삼청동미술관       '}
,{'place': '책마루전망대', 'info': '동구도서관 옥상에 자리한 전망대', '태그': '관광지 도서관전망대 동구도서관 동구전망대 부산동구도서관 부산전망대 '}
,{'place': '낙빈서원', 'info': '사육신을 배향하기 위해 세운 조선 후기 서원', '태그': '관광지 달성가볼만한곳 대구여행 묘골마을 사육신기념관 서원 역사  '}
,{'place': '인흥서원', 'info': '명심보감 판본을 품은 조선시대의 서원', '태그': '관광지 노당추적사당 달성가볼만한곳 대구여행 명심보감판본 역사 역사관광지  '}
,{'place': '육신사', 'info': '묘골마을 육신사 사육신을 기리는 사당', '태그': '2024_여행가는달과함께하는_발도장여행지 관광지 달성가볼만한곳 대구여행      '}
,{'place': '[서울둘레길 21코스] 북한산 도봉코스', 'info': '깊은 역사의 울림이 있는 길', '태그': '걷기길 관광지 둘레길 북한산 북한산도봉코스 서울둘레길 서울둘레길21코스'}
,{'place': '[서울둘레길 20코스] 북한산 강북코스', 'info': '묘지 앞에 나라를 생각하는 길', '태그': '걷기길 관광지 둘레길 북한산 북한산강북코스 서울둘레길 서울둘레길20코스'}
,{'place': '아중체련공원', 'info': '인조잔디로 다양한 체육 활동을 즐길 수 있는 곳', '태그': '관광지'}
,{'place': '달성 삼가헌 고택', 'info': '아름다운 연꽃잎 정자 하엽정을 품은 고택', '태그': '가족여행 관광지 달성삼가헌고택 묘골고택 역사 역사관광지 연꽃   '}
,{'place': '[서울둘레길 19코스] 북한산 성북코스', 'info': '사색에 잠겨 걷는 호젓한 숲길', '태그': '걷기길 관광지 둘레길 북한산 북한산성북코스 서울둘레길 서울둘레길19코스'}
,{'place': '[서울둘레길 18코스] 북한산 종로코스', 'info': '계곡과 산이 만난 도심길', '태그': '걷기길 관광지 둘레길 북한산종로코스 서울둘레길 서울둘레길18코스'}
,{'place': '사효자굴', 'info': '사형제의 깊은 효심이 담긴 전설이 전해지는 동굴', '태그': '관광지 달성가볼만한곳 대구여행 비슬산사효자굴 사형제굴 사효자굴    '}
,{'place': '[서울둘레길 17코스] 북한산 은평코스', 'info': '하늘과 맞닿은 길', '태그': '걷기길 관광지 둘레길 북한산은평코스 서울둘레길 서울둘레길17코스'}
,{'place': '[서울둘레길 16코스] 봉산·앵봉산코스', 'info': '능선 따라 고즈넉함을 느끼는 산길', '태그': '걷기여행 걷기여행길 걷기좋은길 관광지 둘레길 봉산·앵봉산코스 서울둘레길 '}
,{'place': '남지장사', 'info': '최정산 중턱의 숲길이 아름다운 사찰', '태그': '관광지 남지장사 남지장사벚나무 달성가볼만한곳 백련암 불교문화 석조석가여래삼존좌상     '}
,{'place': '[서울둘레길 15코스] 노을·하늘공원코스', 'info': '복원된 자연길', '태그': '걷기여행 걷기여행길 걷기좋은길 관광지 노을·하늘공원코스 둘레길  '}
,{'place': '[서울둘레길 14코스] 안양천 하류코스', 'info': '안양천 옛 추억길', '태그': '걷기여행 걷기여행길 걷기좋은길 관광지 둘레길 서울둘레길 서울둘레길14코스 '}
,{'place': '[서울둘레길 12코스] 호암산코스', 'info': '설화와 풍수, 역사를 담은 길', '태그': '걷기여행 걷기여행길 걷기좋은길 관광지 둘레길 서울둘레길 서울둘레길12코스  '}
,{'place': '무안낙지골목', 'info': '무안의 명물 낙지를 한자리에서 만나볼 수 있는 명소', '태그': '관광지 무안낙지골목'}
,{'place': '진주강씨모열각', 'info': '진주 강씨의 열행을 표창하여 건립한 비각', '태그': '관광지 달성가볼만한곳 대구여행 모열각 역사 역사관광지 진주강씨부인  '}
,{'place': '광양불고기특화거리', 'info': '광양불고기의 처음과 끝을 만나볼 수 있는 거리', '태그': '관광지 광양불고기특화거리'}
,{'place': '[서울둘레길 11코스] 관악산코스', 'info': '민속신앙과 불교가 만나는 길', '태그': '걷기여행 걷기여행길 걷기좋은길 관광지 관악산코스 둘레길 서울둘레길 '}
,{'place': '[서울둘레길 10코스] 우면산코스', 'info': '산속에서 보는 서울길', '태그': '걷기좋은길 관광지 둘레길 서울둘레길 서울둘레길10코스 우면산코스'}
,{'place': '도성암(달성)', 'info': '경상북도 수행도량 중 하나인 도성암', '태그': '관광지 달성가볼만한곳 달성도성암 대구여행 역사 자연풍경 자연환경    '}
,{'place': '[서울둘레길 9코스] 대모·구룡산코스', 'info': '자연생태 유람길', '태그': '걷기여행길 걷기좋은길 관광지 대모·구룡산코스 둘레길 서울둘레길  '}
,{'place': '잇다스페이스', 'info': '다양한 문화예술 기록의 공간', '태그': '갤러리 데이트코스 문화시설 복합문화공간 재생건축 전시'}
,{'place': '노사평화의 전당', 'info': '노사상생을 위한 문화공간', '태그': '가족단위나들이 노동교육체험관 노동산업역사관 노동을알기쉽게 노사관계를올바르게     '}
,{'place': '[서울둘레길 3코스] 불암산코스', 'info': '하늘을 품은 포근한 숲길', '태그': '걷기길 걷기여행길 걷기좋은길 관광지 둘레길 불암산코스 서울둘레길 '}
,{'place': '달성 조길방 고택', 'info': '주택 발달사 연구 자료인 삼량 구조로 결구 된 초가', '태그': '고택 관광지 교과서속여행 달성가볼만한곳 대구여행 역사 역사관광지  '}
,{'place': '두타몰', 'info': '심야쇼핑으로 유명한 동대문 랜드마크 복합 쇼핑몰', '태그': '서울 서울여행코스 쇼핑 외국인친구에게추천 중구 중구쇼핑 패션'}
,{'place': '축서사(봉화)', 'info': '문수산 기슭의 천년고찰 축서사', '태그': '관광지 불교 불교문화 불교사찰 사찰 사찰여행 역사 자연속으로       '}
,{'place': '오천시장', 'info': '1954년, 공설시장으로 지정된 시장', '태그': '다양한먹거리 쇼핑 시장 오천시장 포항가볼만한곳 포항시장 포항오일장'}
,{'place': '봉화 닭실마을', 'info': '역사와 전통이 살아 숨 쉬는 곳', '태그': '경상권 관광지 닭실마을 봉화닭실마을 역사 역사속 역사속으로 역사탐험       '}
,{'place': '봉화목재문화체험장', 'info': '다양한 목재 체험을 경험해 볼 수 있는 곳', '태그': '관광지 목재오토마틱만들기 봉화목재도서관 봉화목재체험학습 봉화춘향목  '}
,{'place': '송포항', 'info': '부안의 작은 항구', '태그': '관광지 마실길 변산마실길 변산마실길2코스 변산반도 변산해수욕장    '}
,{'place': '주산지(청송 국가지질공원)', 'info': '사계절 아름다운 경치와 자연 속의 조화로움', '태그': '2024_여행가는달과함께하는_발도장여행지 23_24한국관광100선 25_26한국관광100선                       '}
,{'place': '효자시장', 'info': '다양한 먹거리가 가득한 포항의 명물 시장', '태그': '쇼핑 시장 포항가볼만한곳 포항시장 포항장보기 포항효자시장 효자시장'}
,{'place': '구미시승마장', 'info': '전국 제일의 명품 구미시승마장', '태그': '가족과함께 레포츠 승마 승마레슨 승마장 승마체험 승마캠프 승마트레킹 '}
,{'place': '궁평항광장', 'info': '낙조가 아름다운 서해 항구', '태그': '갯벌체험 경기화성여행 관광지 궁평항광장 낙조 서해 수산물직판장 '}
,{'place': '지천지', 'info': '낚시객들 사이에서 유명한 공간', '태그': '관광지 칠곡가족여행 칠곡군 칠곡산책 칠곡여행 칠곡저수지 칠곡투어'}
,{'place': '창녕 인양사 조성비', 'info': '통일신라 전성기의 사찰조성 과정을 기록한 비석', '태그': '경남인양사조성비 경남조성비 경남창녕인양사 경남창녕조성비 경상권                  '}
,{'place': '창녕 술정리 서 삼층석탑', 'info': '보물로 지정된 통일신라시대 석탑', '태그': '가족여행 경남삼층석탑 경남술정리삼층석탑 경상권 관광지 역사 역사공부               '}
,{'place': '향남오토캠핑장', 'info': '화성 종합경기타운 옆에 자리한 향남 오토캠핑장', '태그': '경기화성여행 레포츠 서울시인근캠핑 아이랑가기좋은 조용하고한적한   '}
,{'place': '패션주얼리전문타운', 'info': '주얼리 구입은 물론 직접 제작도 해 볼 수 있는 이색 체험 공간', '태그': '기획전시 남녀노소 대구귀금속가공타운 대구주얼리타운 대구쥬얼리쇼핑              '}
,{'place': '노을진 캠핑장', 'info': '저녁노을이 아름다운, 경인아라뱃길 옆 수도권 힐링 캠핑장', '태그': '2024_여행가는달과함께하는_발도장여행지 가족 깨끗한 레포츠 사이트 간격이 넓은       '}
,{'place': '배다리생태공원', 'info': '자연을 즐길 수 있는 평택시의 공원', '태그': '관광지 배다리공원 배다리생태공원 생태공원 평택가볼만한곳 평택나들이 '}
,{'place': '운암지 수변공원', 'info': '대구 도심 속 함지산 아래 펼쳐진 아름다운 수변공원', '태그': '걷기길 경상권 관광지 대구공원 대구북구수국명소 대구수변공원 산책길           '}
,{'place': '격포해수욕장', 'info': '얕은 수심과 잔잔한 파도로 아이에서 어른까지 함께 즐길 수 있는 격포해수욕장', '태그': '가족여행 격포해변 관광지 바다풍경 바다풍경여행 아이와함께 여름방학가볼만한곳      '}
,{'place': '고사포해수욕장', 'info': '은빛 백사장과 송림이 아름다운 해변', '태그': '가족과함께 가족여행 관광지 바다풍경 부안여행 여름피서지 자연         '}
,{'place': '[서울둘레길 13코스] 안양천 상류코스', 'info': '기찻길 따라 걷는 벚꽃길', '태그': '둘레길 레포츠 서울둘레길 서울둘레길13코스 수도권 안양 안양천 '}
,{'place': '국동항', 'info': '여수의 대표적인 항구, 국동항', '태그': '관광지 국동항 국동항수변공원 여수가볼만한곳 여수가족여행 여수바다여행  '}
,{'place': '기계장 (1, 6일)', 'info': '온정이 가득한 전통시장', '태그': '가족여행 경상권 기계장 기차여행 나들이 나홀로여행 먹거리 쇼핑       '}
,{'place': '풍류해수욕장', 'info': '봄이 오면 벚꽃을 눈에 담고, 여름이 오면 바다에 발을 담고', '태그': '가족여행 갯벌 관광지 바다경치 바다풍경 아이와함께 여름피서지     '}
,{'place': '단종대왕비', 'info': '태백산 자락에 자리한 단종을 기리는 역사 유적', '태그': '강원 강원태백 관광지 단종대왕비 역사유적'}
,{'place': '[서울둘레길 7코스] 일자산코스', 'info': '도심 속에서 만나는 작은 숲길', '태그': '노을공원 둘레길 레포츠 서울둘레길 서울둘레길7코스 서울트래킹 수국사   '}
,{'place': '망경사', 'info': '천년의 시간을 품은 고산 사찰 망경사', '태그': '관광지'}
,{'place': '[서울둘레길 6코스] 고덕산코스', 'info': '한강을 따라 펼쳐지는 역사길', '태그': '고덕산 고덕산코스 레포츠 서울둘레길6코스 서울트래킹 숲길 자연속으로     '}
,{'place': '금곡사(칠곡)', 'info': '풍수지리적 요건을 갖춘 곳에 자리한 전통사찰', '태그': '경상권 관광지 불교 불교문화 사찰 사찰여행 역사 자연속으로 자연환경        '}
,{'place': '사내골소나무', 'info': '500년 세월을 품은 태백의 명품 노거수', '태그': '경치여행 경치좋은곳 관광지 자연 자연관광지 자연마을 자연발생    '}
,{'place': '[서울둘레길 5코스] 아차산코스', 'info': '사람과 역사를 잇는 길', '태그': '걷기좋은곳 관악산 둘레길5코스 레포츠 삼성산 서울둘레길 서울여행   '}
,{'place': '백단사', 'info': '자연과 문화가 어우러진 산사, 태백의 백단사', '태그': '강원 관광지 사찰 태백 태백산'}
,{'place': '[서울둘레길 5코스] 아차산코스', 'info': '사람과 역사를 잇는 길', '태그': '걷기좋은곳 관악산 둘레길5코스 레포츠 삼성산 서울둘레길 서울여행   '}
,{'place': '백단사', 'info': '자연과 문화가 어우러진 산사, 태백의 백단사', '태그': '강원 관광지 사찰 태백 태백산'}
,{'place': '덕동산근린공원', 'info': '아이들이 뛰어놀기 좋은 도심 속 공원', '태그': '공원데이트 관광지 산책하기좋은 평택근린공원 평택어린이공원 평택어린이숲체험 '}
,{'place': '배다리지', 'info': '배다리 호수를 끼고 있는 도심 속 저수지', '태그': '관광지 배다리저수지 배다리지 평택가볼만한곳 평택배다리생태공원 평택배다리지'}
,{'place': '황매산 억새군락지', 'info': '자연 그대로의 경관을 품고 있는 억새 군락지', '태그': '가을여행 가을여행코스 가을축제/행사 가족과함께 관광지 노을 등산        '}
,{'place': '연천 경순왕릉', 'info': '천년 신라의 마지막 왕이 잠든 곳', '태그': '경순왕릉 고려시대왕릉 관광지 교과서속여행 수도권 신라시대 신라시대_마지막왕                 '}
,{'place': '송림사(칠곡)', 'info': '승려 명관이 불사리를 봉안하기 위해 창건한 절', '태그': '가족여행 경상권 관광지 불교 불교문화 사찰 사찰여행 역사 연중무휴         '}
,{'place': '금화계곡', 'info': '여름철을 시원하게 보내는 계곡', '태그': '경북계곡 관광지 여름물놀이 칠곡계곡 칠곡물놀이 칠곡산책 칠곡여행 '}
,{'place': '태백 철암역두 선탄시설', 'info': '국내 최초 무연탄 선탄시설로 근대 산업사의 흔적을 간직한 곳', '태그': '가족과함께 관광지 교육여행 아이와함께 역사문화재 역사속으로 역사이야기   '}
,{'place': '군산 영화의 거리', 'info': '영화속으로 시간여행, 그리고 인생샷 한컷', '태그': '관광지 군산여행 군산영화촬영지 군산초원사진관 영화촬영지 초원사진관  '}
,{'place': '삼학도', 'info': '세 마리 학이 내려앉은 전설의 섬', '태그': '가족과함께 관광지 목포의눈물 바다경치 바다내음 삼학도 섬 섬여행         '}
,{'place': '예술의 섬 장도', 'info': '자연과 함께 하는 복합문화예술 공원', '태그': '가족과함께 관광지 문화공간 복합문화공간 아이와함께 여수여행 열린문화공간       '}
,{'place': '의열체험관', 'info': '의열단이 되어보는 체험의 현장', '태그': '가족과함께 관광지 역사 역사관광지'}
,{'place': '반포종합운동장', 'info': '서초구민의 스포츠를 위해 열린 체육시설', '태그': '게이트볼교실테니스장 게이트볼장 농구 눈썰매장 레포츠 반포종합운동장         '}
,{'place': '목포 갓바위', 'info': '아름다운 조명과 전설이 어우러진 바위', '태그': '2024_여행가는달과함께하는_발도장여행지 걷기길 걷기여행 관광지 데이트코스               '}
,{'place': '가산산성마을', 'info': '청정자연과 여유로움이 있는 곳', '태그': '관광지 농촌마을 농촌여행 농촌체험 농촌체험마을 농촌체험여행 아이와함께       '}
,{'place': '하동 천년차밭길', 'info': '하동에서 봄을 만나자, 향기 그윽한 산책길 !', '태그': '남녀노소 녹차밭 레포츠 붉은오름자연휴양림 산책코스 산책하기좋은곳       '}
,{'place': '금산인삼관', 'info': '고려 인삼의 우수성을 널리 알리는 곳', '태그': '가족여행 관광지 금산인삼관 박물관 산업관광 산업관광지 아이와함께     '}
,{'place': '늘미곡', 'info': '필환경 시대에 걸맞은 전주 최초의 제로 웨이스트 숍', '태그': '곡물 남녀노소 늘미곡 쇼핑 연인과함께 전주가볼만한곳 전주여행      '}
,{'place': '황매산 군립공원', 'info': '사계절이 뚜렷하게 보이는 아름다운 산', '태그': '21_22한국관광100선 23_24한국관광100선 경상권 관광지 사계절       '}
,{'place': '분주령야생화 트레킹', 'info': '야생화 군락과 백두대간 능선을 따라 걷는 트레킹 코스', '태그': '두문동재 레포츠 야생화군락지 야생화천국 자연여행 트레킹 하늘다람쥐  '}
,{'place': '[서울둘레길 8코스] 장지·탄천코스', 'info': '자연생태가 복원된 도시 하천길', '태그': '둘레길코스 레포츠 서울둘레길 서울둘레길8코스 서울둘레길코스 서울트래킹   '}
,{'place': '수종사', 'info': '500년 수령의 은행나무가 있는 전망 명소 사찰', '태그': '가족여행 경기권 관광지 불교 불교문화 불교사찰 사찰 서울근교여행        '}
,{'place': '르인천구락부 포디움126', 'info': '동인천 마을호텔 르인천구락부의 체크인센터이자, 컨시어지샵 컨셉의 로컬 카페', '태그': '관광지 동인천 로컬 르인천구락부 인천 카페 포디움126'}
,{'place': '더드림핑 야영장', 'info': '휴식과 즐거움이 살아 숨 쉬는 공간', '태그': '경기도캠핑장 남양주시캠핑장 럭셔리글램핑 레포츠 수상레저 실내서핑  '}
,{'place': '대둔사(칠곡)', 'info': '신라 화랑들의 수행도량', '태그': '경상권 관광지 불교 불교문화 사찰 사찰 사찰여행 역사 자연속으로         '}
,{'place': '황부자 며느리 친정가는 둘레길', 'info': '황지연못의 전설을 테마로 조성된 이야기 풍경길', '태그': '강원권 강원권여행 레포츠 본적산 태백 태백트래킹 트래킹 트래킹코스'}
,{'place': '[서울둘레길 4코스] 망우·용마산코스', 'info': '도시를 벗어나 만나는 길', '태그': '걷기좋은길 둘레길 레포츠 서울둘레길 서울둘레길4코스 서울여행 서울트래킹  '}
,{'place': '동명장 (4, 9일)', 'info': '경북 칠곡군 전통시장', '태그': '가족여행 경상권 기차여행 나들이 나홀로여행 동명장 먹거리 쇼핑         '}
,{'place': '소극장 산울림', 'info': '고전과 현대를 아우르는 무대를 보여주는 소극장', '태그': '공연 데이트 문화시설 문화예술 산울림소극장 서울가볼만한곳 소극장   '}
,{'place': '영도 마리노 오토캠핑장', 'info': '바다와 바람, 그리고 불빛이 하나 되는 곳', '태그': '가족 가족과함께 레포츠 부산 부산가볼만한곳 부산여행 오토캠핑장   '}
,{'place': '[서울둘레길 2코스] 덕릉고개코스', 'info': '도시와 자연을 잇는 생태', '태그': '덕릉고개코스 둘레길 레포츠 서울둘레길 서울둘레길2코스 서울산책  '}
,{'place': '춘천 의암호 자전거길(물레길)', 'info': '의암호를 따라 공지천까지 달리는 자전거길', '태그': '강원권 공지천 꿀잼_액티비티 레포츠 의암호 자전거길 휴양여행 힐링'}
,{'place': '금대봉야생화 군락지', 'info': '희귀 생태자원의 보고이자 한강의 발원지를 품은 태백의 생태 명소', '태그': '강원권 관광지 금대봉 금대봉야생화군락지 생태탐방 자연 자연발생   '}
,{'place': '청룡사(상주)', 'info': '비봉산 중턱에 위치한 아름다운 사찰', '태그': '경상권 경천섬 관광지 낙동강조망대 불교 불교문화 비봉산 비봉산고찰                '}
,{'place': '산호공원(용마공원) 시의거리', 'info': '마산을 대표하는 시인들의 시비가 있는 거리', '태그': '가족여행 걷기길 경남산호공원 경남용마공원 경상권 관광지 산책길           '}
,{'place': '[서울둘레길 1코스] 수락산코스', 'info': '속세를 떠나지 않은 옛사람의 길', '태그': '경치여행 경치좋은곳 레포츠 서울둘레길 서울둘레길1코스 수도권 수락산등산       '}
,{'place': '콰이강의다리', 'info': '야경이 아름다운 창원의 핫플레이스', '태그': '2024_여행가는달과함께하는_발도장여행지 가족과함께 걷기 경치좋은곳              '}
,{'place': '부안누에타운', 'info': '누에와 뽕나무를 테마로 한 전시 체험관', '태그': '가족과함께 곤충박물관 교육여행 문화시설 민속품 부안가볼만한곳 부안군      '}
,{'place': '실리도', 'info': '수려한 해안경관과 갯벌이 잘 발달되어 있는 곳', '태그': '가족여행 관광지 낚시 섬 섬여행 자연 창원시'}
,{'place': '송도 센트럴파크', 'info': '국내 최초로 바닷물을 이용한 해수공원', '태그': '19_20한국관광100선 2024_여행가는달과함께하는_발도장여행지 21_22한국관광100선                           '}
,{'place': '연화산둘레길', 'info': '연화산 자락에 위치한 자연 친화적인 힐링 산책 코스', '태그': '강원도둘레길 걷기여행 둘레길 레포츠 연화산 자연 자연여행 태백여행'}
,{'place': '소쿠리섬', 'info': '아름다운 자연속에서 즐기는 캠핑 명소', '태그': '관광지 자연관광지 자연여행 자연좋은곳 자연환경 휴식공간 휴식여행  '}
,{'place': '발안만세시장', 'info': '정겹고 친근한 화성시 재래시장', '태그': '5일장 발안만세벽화거리 발안만세시장 발안천벚꽃축제 쇼핑 월문온천관광휴양지  '}
,{'place': '남양시장', 'info': '넉넉한 인심이 가득한 화성의 5일장', '태그': '5일장 남양성모성지 남양시장 남양읍내장터 쇼핑 전통시장 홍난파생가 '}
,{'place': '장어구이거리', 'info': '지친 몸과 마음의 원기를 채워주는 장어구이', '태그': '가족과함께 갯장어 관광지 남녀노소 민물장어 보양식 부모님과함께         '}
,{'place': '백미리희망캠핑장', 'info': '바다 풍경이 멋진 캠핑장', '태그': '갯벌체험 경기화성여행 레포츠 바다가보이는뷰 서울시근교 일몰명소  '}
,{'place': '청와대 앞길', 'info': '경복궁 담장길과 청와대 사이에 난 산책길', '태그': '23_24한국관광100선 25_26한국관광100선 가족여행 관광지 나홀로여행                '}
,{'place': '인천시송도국제캠핑장', 'info': '자연과 가까운 서울 근교 힐링 캠핑장', '태그': '2024_여행가는달과함께하는_발도장여행지 가족과함께 도심캠핑 레포츠   '}
,{'place': '마포대교', 'info': '아름다운 야경을 감상할 수 있는 마포와 여의도를 잇는 다리', '태그': '걷기 관광지 데이트코스 도심여행 드라이브여행 마포대교 마포대교야경           '}
,{'place': '오동동 통술거리', 'info': '식당마다 각색 있는 다양한 안주를 즐길 수 있는 곳', '태그': '관광지 남녀노소 맛집여행 맛집탐방 술안주 술집 숨은맛집 연인과함께     '}
,{'place': '자연과별천문대', 'info': '자연 속에서 쏟아지는 별을 찾는 천문대', '태그': '가족과함께 가평가볼만한곳 가평여행 관광지 교육여행 수도권 아이와함께    '}
,{'place': '화성 은행나무마을', 'info': '오래된 은행나무가 지켜주는 요당마을', '태그': '관광지 수도권 요당리마을 자연마을 자연속으로 자연여행 자연좋은곳         '}
,{'place': '녹동항 바다정원', 'info': '바다 한가운데 떠 있는 바다정원', '태그': '고흥가볼만한곳 관광지 녹동항 바다경치 바다내음 산책하기 산책하기좋은곳      '}
,{'place': '너른공원', 'info': '화성시 동탄 일반산업단지 미세먼지 차단 숲', '태그': '경기화성여행 관광지 미세먼지차단숲 화성 화성너른공원 화성방아다리공원 '}
,{'place': '양구백토 (강원평화지역 국가지질공원)', 'info': '조선시대 백자의 백토 공급지', '태그': '경치좋은곳 관광지 국가지질공원 자연 자연관광지 자연발생 자연속으로       '}
,{'place': '남양체육공원', 'info': '걷고 뛰기 좋은 운동시설이 잘 갖춰진 푸른 건강 뜰 공원', '태그': '관광지 남양도서관 남양도호부오리정 남양역사문화공원 남양체육공원  '}
,{'place': '제부랜드', 'info': '바다가 갈라지는 신비의 섬 제부도에서 관광과 레저를 한 번에', '태그': '레포츠 서해랑케이블카 제부도놀이공원 제부도테마파크 제부도통통열차  '}
,{'place': '서봉산산림욕장', 'info': '걷는 내내 초록의 시원함이 함께하는 서봉산산림욕장', '태그': '경기화성여행 관광지 산림욕 서봉산 화성 화성등산코스 화성서봉산산림욕장'}
,{'place': '창녕 신라 진흥왕 척경비', 'info': '국보로 지정된 신라시대 비석', '태그': '관광지 삼국시대비석 순수비 역사 역사공부 역사관광지 역사를품은곳           '}
,{'place': '주니어천문스쿨 동탄점', 'info': '별을 보며 무한한 상상력을 키워나가는 곳', '태그': '경기화성여행 관광지 동탄천문대 별관측 주니어천문스쿨 천문대체험  '}
,{'place': '다람산공원', 'info': '사계절 자연을 만날 수 있는 곳', '태그': '관광지 다람공원 다람산공원물농이장 다람산근린공원 병점다람산공원    '}
,{'place': '제부리어촌체험마을', 'info': '바닷길이 열리는 아름다운 제부도에서 가능한 갯벌체험', '태그': '경기화성제부리어촌체험휴양마을 관광지 제부도갯벌체험 제부도갯벌체험장    '}
,{'place': '동탄복합문화센터반석아트홀', 'info': '동탄복합문화센터 내 위치한 모든 예술인의 열정이 함께하는 공연장', '태그': '공연장 동탄복합문화센터 문화시설 반석아트홀 복합문화센터 아트홀  '}
,{'place': '월명공원', 'info': '군산의 중심에 위치한 역사와 자연이 어우러진 휴식처', '태그': '가족여행 관광지 남녀노소 산책 산책길 산책로 산책코스 산책하기         '}
,{'place': '금장대 수변공원', 'info': '여유롭게 산책하며 쉬어갈 수 있는 공간', '태그': '경북권 경상북도_여행지_추천 경주가볼만한곳 관광지 봄꽃여행지 사진명소'}
,{'place': '센트럴곤충생태농장', 'info': '동물을 통한 정서함양과 관찰력, 책임감을 키워주는 곳', '태그': '경기화성센트럴곤충생태농장 경기화성여행 곤충체험 관광지 동물곤충체험농장  '}
,{'place': '국립장성치유의숲', 'info': '아름다운 자연이 마음을 치유해주는 곳', '태그': '2024_여행가는달과함께하는_발도장여행지 관광지 산책길 산책하기 산책하기좋은곳            '}
,{'place': '행복텃밭', 'info': '일 년 내내 여러 가지 농작물을 체험할 수 있는 교육농장', '태그': '감자캐기체험 고구마캐기체험 관광지 귤체험 딸기체험농장 블루베리체험   '}
,{'place': '어달해변', 'info': '고운 모래와 잔잔한 파도가 일품인 해변', '태그': '가족여행 강원권 관광지 기차여행 나들이 당일치기여행 데이트코스              '}
,{'place': '석류공원', 'info': '진주를 상징하는 꽃, 석류공원', '태그': '관광지 근린공원 남강 석류공원 여행지 전망명소 진주시 폭포'}
,{'place': '주렁주렁 동물원 동탄점', 'info': '도심 속, 생명이 살아 숨쉬는 동물을 실내에서 만날 수 있는 곳', '태그': '경기도아이와갈만한곳 경기실내동물원 경기아쿠아가든 경기화성여행 관광지   '}
,{'place': '충익사 의병탑', 'info': '곽재우 장군과 휘하의 장수 17인의 위업을 기리는 곳', '태그': '관광지 기념탑 남산천변 아이와함께 역사 역사공부 역사관광지 역사를품은곳             '}
,{'place': '구례수목원', 'info': '숲속 교실에서 만나는 숲속의 친구들', '태그': '관광지 구례숲체험 구례여행 구례힐링명소 숲속힐링 지리산정원 힐링구례'}
,{'place': '매향리평화생태공원', 'info': '미공군 사격훈련장 부지에 조성한 생태공원', '태그': '경기화성여행 관광지 매향리가볼만한곳 매향리평화생태공원 서울근교생태공원   '}
,{'place': '제부도 매바위', 'info': '일몰이 아름다운 매의 부리를 닮은 바위섬', '태그': '경기화성여행 경기화성제부도매바위 관광지 서해섬여행 제부도매바위  '}
,{'place': '당골낚시터', 'info': '자연의 아름다움과 낚시를 즐길 수 있는 최적의 장소', '태그': '100수 경기낚시터 경기도낚시터 낚시체험 당골낚시터 레저스포츠 레포츠     '}
,{'place': '대구 도동 측백나무 숲', 'info': '향산 절벽에 군락을 이루고 있는 천연기념물 측백나무 숲', '태그': '관광지 달성측백수림 대구10경 대구천연기념물 도동측백수림 문화관광해설사    '}
,{'place': '거금해양낚시공원', 'info': '자연이 준 선물을 만끽할 수 있는 곳', '태그': '거금해양낚시공원 고흥관광 낚시터 레포츠 체험장 해상공원 해상펜션 '}
,{'place': '관광역사공원', 'info': '보문관광단지 조성의 기록을 담은 공원', '태그': '경주벚꽃 경주여행 경주핑크뮬리 관광지 보문호 선덕여왕공원'}
,{'place': '남양호(남양황라)', 'info': '철새와 낚시를 즐길 수 있는 이색적인 공간', '태그': '겨울철새 경기화성남양호 경기화성남양황라 관광지 남양호 남양황라  '}
,{'place': '양산디자인공원', 'info': '양산시민들의 휴식공간', '태그': '관광지 디자인공원 석가산폭포 양산가볼만한곳 양산가족공원 양산디자인공원   '}
,{'place': '금화지', 'info': '자연의 아름다움을 그대로 간직한 저수지', '태그': '관광지 칠곡가볼만한곳 칠곡가족여행 칠곡금화지 칠곡산책 칠곡여행  '}
,{'place': '동탄여울공원', 'info': '동탄시민의 아늑한 휴식 공간', '태그': '경기화성여행 공원 관광지 동탄 동탄가볼만한곳 오산천 화성'}
,{'place': '동탄센트럴파크', 'info': '동탄 신도시 도심속 공원', '태그': '경기화성여행 공원 관광지 도심속공원 동탄신도시 화성 화성가볼만한곳'}
,{'place': '흥무공원', 'info': '나들이하기 좋은 공원', '태그': '걷기좋은산책길 경주공원 경주여행 공원나들이 관광지 김유신장군묘 '}
,{'place': '동탄호수공원', 'info': '도심 속 자연과 야경이 아름다운 공원', '태그': '2024_여행가는달과함께하는_발도장여행지 가족과함께 관광지 댕댕이와밤밤곡곡                '}
,{'place': '신시도 어촌체험마을', 'info': '바지락 캐기와 낚시를 할 수 있는 체험 마을', '태그': '가족여행 관광지 바다풍경 섬여행 아이와함께 연인과함께 자연 친구와함께 '}
,{'place': '임피향교', 'info': '조선 시대 교육과 제례 전통을 간직한 유교 건축물', '태그': '관광지 군산관광지 군산임피향교 군산향교 역사 역사관광지 역사를품은곳            '}
,{'place': '비응도', 'info': '해양관광과 어업이 어우러진 서해안의 미래형 복합 해양도시', '태그': '군산바다 군산바다낚시 레포츠 바다낚시 전북권'}
,{'place': '비응도', 'info': '해양관광과 어업이 어우러진 서해안의 미래형 복합 해양도시', '태그': '군산바다 군산바다낚시 레포츠 바다낚시 전북권'}
,{'place': '군산 해망굴', 'info': '역사가 깃든 군산 월명산의 터널', '태그': '가족여행 관광지 군산여행 근대문화유산 등록문화재 문화재 여름피서지          '}
,{'place': '어청도', 'info': '맑은 바다와 전설, 아름다운 등대가 어우러진 서해 최서단의 섬', '태그': '관광지 군산가볼만한곳 바다경치 바다내음 섬여행 어청도 자연 자연여행   '}
,{'place': '태백 상장동 벽화마을', 'info': '탄광마을의 이야기를 그려낸 아름다운 벽화마을', '태그': '가족여행 감성사진 강원권 관광지 데이트장소 벽화마을 사진찍기좋은곳       '}
,{'place': '호반전망대', 'info': '진주 시민의 생명수라 불리는 청정 호수', '태그': '관광지 노을 노을명소 놀이시설 동물원 여행지 진양호 진주시 호반전망대 '}
,{'place': '염포해수욕장', 'info': '검은빛을 띈 자갈과 울창한 해송이 아름다운 해수욕장', '태그': '경치좋은곳 관광지 바다낚시 바다풍경 여름여행 염포해수욕장 자연    '}
,{'place': '익금해수욕장', 'info': '남해안 최남단의 맑고 푸른 바다와 은빛 모래사장', '태그': '가족여행 경치좋은곳 관광지 바다낚시 아이와함께 여름여행 연인과함께      '}
,{'place': '비와야폭포', 'info': '비가 와야만 흐르는 태백의 특별한 비경', '태그': '관광지 여름여행지 태백가볼만한곳 폭포'}
,{'place': '연소해수욕장', 'info': '갯벌체험하기 좋은 이곳!', '태그': '고흥군 관광지 바다 자연 전라남도'}
,{'place': '나로도항', 'info': '나로도에 있는 연안항', '태그': '고흥가볼만한곳 고흥바다여행 고흥여행 관광지 나로도항 낚시데이트  '}
,{'place': '남열마을', 'info': '해돋이와 푸른 파도가 일품인 마을', '태그': '경치좋은곳 관광지 숙박 자연 자연마을 자연좋은곳 휴식공간 휴식여행  '}
,{'place': '소록도', 'info': '작은 사슴을 닮아 예쁜 섬 소록도', '태그': '고흥관광명소 관광지 국립소록도병원 어린사슴 자연 자연관광지'}
,{'place': '거금대교', 'info': '금빛으로 반짝이는 바다 위의 산책로', '태그': '거금대교 고흥 고흥가볼만한곳 관광지 다리'}
,{'place': '최초석탄발견지탑', 'info': '우리나라 석탄 산업의 시작을 기념하는 태백의 상징적 유적', '태그': '가족과함께 공터 관광지 무연탄 비석 석탄 소풍 최초발견지'}
,{'place': '팔현생태공원', 'info': '대구의 숨은 유채꽃 명소', '태그': '관광지 대구_여행지_추천 대구가볼만한곳 대구권 봄꽃여행지 유채꽃'}
,{'place': '진밭골산림공원', 'info': '도심 속 힐링과 휴양', '태그': '관광지 대구공원 대구나들이 대구등산 대구산책 대구여행 산책로 '}
,{'place': '삼수령', 'info': '하나의 빗물이 세 방향으로 갈라지는 강원 태백의 분수령', '태그': '강원권 관광지 산책코스 산책하기좋은곳 삼수령 자연 자연환경 풍경좋은곳   '}
,{'place': '함지근린공원', 'info': '북구에서 가장 큰 규모의 생활공원', '태그': '관광지 대구가족여행 대구공원 대구나들이 대구산책 대구여행'}
,{'place': '공항교강변공원', 'info': '아름다운 풍경을 만끽할 수 있는 공원', '태그': '관광지 도심산책 산책길 산책코스 산책하기 산책하기좋은곳 자연좋은곳   '}
,{'place': '연암공원', 'info': '걸으며 힐링하기 좋은 공원', '태그': '관광지 대구가족여행 대구공원 대구등산 대구산책 대구여행'}
,{'place': '옻골동산', 'info': '산책하기 좋은 도심 속 공원', '태그': '관광지 대구공원 대구산책 대구여행 옻골공원 옻골동산'}
]

In [3]:
import pandas as pd

pd.DataFrame(tour).to_csv('추가데이터2.csv',index=False)